In [1447]:
import base64
import datetime
import pandas as pd
import pytesseract
from PIL import Image
import re
import json
import io
import requests
from glob import glob
import os
import mysql.connector
from sqlalchemy import create_engine
import random
import errno
from word2number import w2n

In [1448]:
from PIL import Image
import pytesseract
from pytesseract import image_to_string, image_to_osd

In [1449]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files (x86)/Tesseract-OCR/tesseract'
TESSDATA_PREFIX = 'C:/Program Files (x86)/Tesseract-OCR'

In [20]:
master_dir = str(input(" Enter Path of Documents: "))
all_images = glob(os.path.join(master_dir, '*.jpg'))
df_images = pd.DataFrame({'path': all_images})
df_images.head()

 Enter Path of Documents: D:\VScan\1 July\New folder


,path
0,D:\VScan\1 July\New folder\14309_NPSE.jpg
1,D:\VScan\1 July\New folder\EM000005_AMOL CHOUD...
2,D:\VScan\1 July\New folder\EM000005_AMOL CHOUD...
3,D:\VScan\1 July\New folder\EM000005_AMOL CHOUD...
4,D:\VScan\1 July\New folder\EM000005_AMOL CHOUD...


In [21]:
start_time = datetime.datetime.now().strftime('%H:%M:%S')

In [22]:
def encode_to_string(row):
    
    with open(row.path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
    return encoded_string

In [23]:
df_images['Data']= df_images.apply(encode_to_string,axis=1)

In [24]:
df_images['file_name'] = df_images['path'].map(lambda in_path: in_path.split(os.sep)[-1])
df_images['file_type'] = df_images['path'].map(lambda in_path: os.path.splitext(in_path)[1][1:])  # [2]
df_images.head()

,path,Data,file_name,file_type
0,D:\VScan\1 July\New folder\14309_NPSE.jpg,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,14309_NPSE.jpg,jpg
1,D:\VScan\1 July\New folder\EM000005_AMOL CHOUD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/4RDqRXhpZgAATU0A...,EM000005_AMOL CHOUDHARY_1952_NPS.pdf_(1.jpg).jpg,jpg
2,D:\VScan\1 July\New folder\EM000005_AMOL CHOUD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/4RDqRXhpZgAATU0A...,EM000005_AMOL CHOUDHARY_1953_NPS.pdf_(2.jpg).jpg,jpg
3,D:\VScan\1 July\New folder\EM000005_AMOL CHOUD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/4RDqRXhpZgAATU0A...,EM000005_AMOL CHOUDHARY_1954_NPS.pdf_(3.jpg).jpg,jpg
4,D:\VScan\1 July\New folder\EM000005_AMOL CHOUD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/4RDqRXhpZgAATU0A...,EM000005_AMOL CHOUDHARY_1955_NPS.pdf_(4.jpg).jpg,jpg


In [25]:
def data_extraction(row):
    content = base64.decodebytes(row.Data)
    dataBytesIO = io.BytesIO(content)
    im = Image.open(dataBytesIO)
    text = pytesseract.image_to_string(im, lang = 'eng')
    return text

In [26]:
%%time
df_images['corpus'] = df_images.apply(data_extraction,axis=1)

Wall time: 48.6 s


In [27]:
df_images.head()

,path,Data,file_name,file_type,corpus
0,D:\VScan\1 July\New folder\14309_NPSE.jpg,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,14309_NPSE.jpg,jpg,Electronic Fund Transfer\n\neNPS\n\n \n\nONLIN...
1,D:\VScan\1 July\New folder\EM000005_AMOL CHOUD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/4RDqRXhpZgAATU0A...,EM000005_AMOL CHOUDHARY_1952_NPS.pdf_(1.jpg).jpg,jpg,91001 L-1yled MeN ‘e=uy |EUOIIMNSU) GeIND'Y/7|...
2,D:\VScan\1 July\New folder\EM000005_AMOL CHOUD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/4RDqRXhpZgAATU0A...,EM000005_AMOL CHOUDHARY_1953_NPS.pdf_(2.jpg).jpg,jpg,91001 L-1ul8G MeN'Baly JeUONMASU| GeIND'y/7L-q...
3,D:\VScan\1 July\New folder\EM000005_AMOL CHOUD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/4RDqRXhpZgAATU0A...,EM000005_AMOL CHOUDHARY_1954_NPS.pdf_(3.jpg).jpg,jpg,SLOOL1-1uled MaN'Bavy jeuonNyASU| GeIND'‘y/y|-...
4,D:\VScan\1 July\New folder\EM000005_AMOL CHOUD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/4RDqRXhpZgAATU0A...,EM000005_AMOL CHOUDHARY_1955_NPS.pdf_(4.jpg).jpg,jpg,QLOOLL-Huled MeN'ealy jeuORNIASU] GeIND ‘y/pL-...


In [28]:
df_result = df_images

In [20]:
df_result

,path,Data,file_name,file_type,corpus
0,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2184_Rent_Agreement...,jpg,i\nThe LESSOR hereby acknowledges having recei...
1,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2185_Rent_Agreement...,jpg,10.\n\n41.\n\n12.\n\n13.\n\n14.\n\n15.\n\n16\n...
2,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2186_Rent_Agreement...,jpg,ik\n\n18.\n\n19.\n\n20.\n\n21.\n\n22.\n\n23.\n...
3,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2187_Rent_Agreement...,jpg,"A : my\nor Incurred, pald or sustained by the ..."
4,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2188_Rent_Agreement...,jpg,Annexure-!\n\n \n\n \n\n \n\n \n\n \n\n \n\n \...
5,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2189_Rent_Agreement...,jpg,"4\n\n\\n\nIN WITNESS WHEREOF, THE LESSEE AND T..."
6,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2190_Rent_Agreement...,jpg,LEASE AGREEMENT\n\nLEASE ASSESS\n\nTHIS DEED O...
7,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2191_Rent_Agreement...,jpg,f\nWHEREAS: :\n\nA. The LESSOR declares that h...
8,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2192_Rent Receipt-D...,jpg,Weesopcay Diode\n\n——_—_—————_—_—_\n\n \n\nv_ ...
9,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2193_Rent Receipt_B...,jpg,ac eCooe\non ahs\n\nWe understand your world\n...


In [21]:
def rendered(row):
    out = []
    buff = []
    a = row.corpus
    for c in a:
        if c == '\n':
            out.append(''.join(buff))
            buff = []
        else:
            buff.append(c)
    else:
        if buff:
            out.append(''.join(buff))
    return out

In [22]:
def rendered_score(row):
    out = []
    buff = []
    a = row
    for c in a:
        if c == '\n':

            out.append(''.join(buff))
            buff = []

        else:
            buff.append(c)
    else:
        if buff:
            out.append(''.join(buff))
    return out

In [23]:
def rendered_list_t(rendered_text_crop):
    out = []
    buff = []
    a = rendered_text_crop
    for c in a:
        if c == '\t':

            out.append(''.join(buff))
            buff = []

        else:
            buff.append(c)
    else:
        if buff:
            out.append(''.join(buff))
    return out

In [24]:
def per_doc_confidence(row):
    content = base64.decodebytes(row.Data)
    dataBytesIO = io.BytesIO(content)
    im = Image.open(dataBytesIO)
    
    text = pytesseract.image_to_data(im)
    datalist = rendered_score(text)
    data = {}
    for i in range(0, len(datalist)):
        if len(rendered_list_t(datalist[i])) > 11:
            data['a%d' % i] = rendered_list_t(datalist[i])
  
    data_conf = pd.DataFrame.from_dict(data)
    data_conf = data_conf.transpose()
    header = data_conf.iloc[0]
    data_conf = data_conf[1:]
    data_conf.columns = header

    data_conf = data_conf.reset_index(drop=True)
    data_conf.conf = pd.to_numeric(data_conf.conf, errors='coerce')
    ConfidencePerDoc = data_conf.conf.mean()
    return ConfidencePerDoc

In [25]:
df_result['rendered_text'] = df_result.apply(rendered,axis=1)

In [26]:
words = ['\n'] 
for i in words:
    df_result = df_result.replace(to_replace=i,value=" ",regex=True)

In [27]:
def rendered_text_blank(row):
    rendered_text = row.rendered_text
    rendered_text = [x for x in rendered_text if len(x.strip()) > 0]
    return rendered_text

In [28]:
df_result['rendered_text'] = df_result.apply(rendered_text_blank,axis=1)

In [29]:
df_result

,path,Data,file_name,file_type,corpus,rendered_text
0,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2184_Rent_Agreement...,jpg,i The LESSOR hereby acknowledges having receiv...,"[i, The LESSOR hereby acknowledges having rece..."
1,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2185_Rent_Agreement...,jpg,10. 41. 12. 13. 14. 15. 16 The LESSOR s...,"[10., 41., 12., 13., 14., 15., 16, The LESSOR ..."
2,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2186_Rent_Agreement...,jpg,ik 18. 19. 20. 21. 22. 23. 24. 25. Th...,"[ik, 18., 19., 20., 21., 22., 23., 24., 25., T..."
3,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2187_Rent_Agreement...,jpg,"A : my or Incurred, pald or sustained by the o...","[A : my, or Incurred, pald or sustained by the..."
4,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2188_Rent_Agreement...,jpg,Annexure-! ...,"[Annexure-!, ‘THE LESSOR -, $. No. Description..."
5,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2189_Rent_Agreement...,jpg,"4 \ IN WITNESS WHEREOF, THE LESSEE AND THE L...","[4, \, IN WITNESS WHEREOF, THE LESSEE AND THE ..."
6,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2190_Rent_Agreement...,jpg,LEASE AGREEMENT LEASE ASSESS THIS DEED OF LE...,"[LEASE AGREEMENT, LEASE ASSESS, THIS DEED OF L..."
7,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2191_Rent_Agreement...,jpg,f WHEREAS: : A. The LESSOR declares that he i...,"[f, WHEREAS: :, A. The LESSOR declares that he..."
8,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2192_Rent Receipt-D...,jpg,Weesopcay Diode ——_—_—————_—_—_ v_ ainjeu...,"[Weesopcay Diode, ——_—_—————_—_—_, v_ ainjeubi..."
9,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAO...,EM000002_RAVINDRA JOGDAND _2193_Rent Receipt_B...,jpg,ac eCooe on ahs We understand your world Pag...,"[ac eCooe, on ahs, We understand your world, P..."


In [30]:
# df_result.to_excel('TATA_Corpus.xlsx',index = False)

In [4]:
##################################################################################

In [1450]:
# df_result = pd.read_excel('ALL_CORPUS.xlsx',index = False)
df_result = pd.read_excel('TATA_Corpus.xlsx',index = False)

for i in range(0,len(df_result)):
    df_result.loc[i,'corpus'] = str(df_result.loc[i,'corpus'])
    df_result.loc[i,'rendered_text'] = eval(df_result.loc[i,'rendered_text'])

In [1451]:
def mediclaim_type(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(mediclaim|mediciaim|mediciame|medictaim|meuiclaim|mediprime|health insurance|medical|health suraksha|star health|new india assurance|health policy|ealth guard|health guard|medicare|health forever|happy family floater|health medisure|individualhealthpolicy|medicare policy|happy family .*?floater|icici lombard|parivar mediclaim|parivar meuiclaim|deduction under section 80(|.|..)d|provisions of section 80(|.|..)d|section 80(|.|..)d of income tax act|tax exemption under sec 80d|mediciaim insurance|mediclaim insurance|sereligare health|religare health|customerfirst@religarehealthinsurance.com|www.religarehealthinsurance.com|chola healthline|www.cholsingurance.com|weew.cholsingurance.com|cholamandalam ms general insurance company|cholamandalam ws general insurance company|customerservice@apollomunichinsurance.com|customerservice@apoiomunichinsurance.com|www.apollomunichinsurance.com|www.apalismunichinsurance.com|www.apoliomunichinsurance.com|apollo munich health insurance|anotio munich health lasurance|oriental insurance company ltd|oriental insurance company|hospitalization cover|parivar medictaim|ndividualfamilymedicarepolicy|ndividualfamilymedicarepoliey|individualfamilymedicarepolicy|individualfamily medicarepolicy|lindividuall lealthpolicy|individual lealthpaliey|individualhleath policy)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("MEDICLAIM")
    return None
    
def ulip_type(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(ulip)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("ULIP")
    return None

def npse_type(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(national pension system|permanent retirement account number|pran|pension|pension|npse|nps|atal pension yojana)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("NPSE")
    return None

def mutual_fund_type(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(hdfc equity fund-growth|hdfc equity fund.growth|birla sun life equity advantage fund|equity fund.growth|equity advantage fund|equity hybrid|mutual fund|elss|folio|reliance tax saver|mutual.{0,100}?fund|sip(.|..|...|)instalment)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("MUTUAL FUND")
    return None

def childedu_type(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(fee receipt|manipal university|exam fees|collage exam fees|student copy|school fee|name of the student|student name|student's copy|tuition fe|student/parent|computer fees|roll no|admission fee|tuffion fee|tuition fee|tuition fees|tuition ree|term fee|school fees|tution fee|annual fee|bus fees|bus fee|bus\\\\van no|bus\\van no|tuitionof|fee payment receipt|regular tuition|for the academic year|ninme of the student|institute of technology|term tuition|termtuition fees|termtuition fee|termtuition)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("CHILDREN EDUCATION")
    return None

def home_loan_type(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(housing finance|housing|housing loan|home/mortgage loan|mortgage loan|home loan|icici home finance|home finance|provisional certificate for claiming deduction under section 24\(b|section 24\(b|gruh|pradhan mantri aavas yojana|pradhan maniri aavas yojana|aavas yojana|ashiana hsg fl|provisional interest certificate for the period.*?(principal|interest)|provisional interest certificate.*?(principal|interest)|provisional intrest certificate.*?(principal|interest)|retail tl(|.|..)ashiana hsg fl|home(|.|..)loan|indiashelter|lndiashelter|ome loan)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("HOUSING LOAN")
    return None

def fd_type(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(fixed deposit interest certificate|fixed deposits|fixed deposit|5yl8|deposit confirmation advice|fd-5-yrs|60(|.|..)months)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("FD")
    return None


def handicap_type(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(handicapped|handicaped|handicapped dependent|handicaped dependent|mental retardation)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("Handicap")
    return None

def special_disease_type(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(femto cataract surgery|cataract surgery|special disease)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("Special disease")
    return None   

def rent_type(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(rent receipt|rent of house|rent|tenant|leave and licence agreement|leave and license agreement|licence agreement|witness of execution|itness of execution|license agreement|licensor|licensee|inhabitant|licensees|occupancy of my house)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("RENTAL AGREEMENT")
    return None

def ppf_type(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(public provident fund|provident fund|ppt tran|ppf account number|ppf deposit|ppf)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("PPF")
    return None

def nsc_type(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(nsc|savings institute|saving institute|savings bank passbook|saving bank passbook|indiapost|department of posts|department of post)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("NSC")
    return None

def eduloan_type(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(education loan|educational loan)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("EDUCATION LOAN")
    return None

def lic_type(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(life insurance corporation of india|hife insurance corporation.of india|lic|ilic|life insurance|life insurence|.life(.|..)insurance|ite insurance|lic’s|ww(.|..|...)iciciprulife(.|..|...)com|lichelp|lic help|national swasthya bima policy|swasthya bima policy|icici pru protect|icici pru|icic! pru|.c.c. pru|frenewal premium receipt|renewal premium receipt|l.i.c. of india|l.1.c. of india|life irwursice|life inturance|absli digi shield plan|absli dig shield plan|term assurance death benefit|term assurance plan|term assurance pian|bsli dream plan|bsl! dream plan|mex life|max life online term|tox benefit under section 30c|uc of india|slife(|.|..)insurance|life(|.|..)insurance|max life online term plan|absli life shield|receipt of payment made by cheque is valid subject to realisation of the cheque|realisation of the cheque|renewal premium acknowledgement|https://www.iciciprulife.com|https://www.iciciprulife.comy|lifeline@iciciprufife(.|..)com|lfeline@iciciprufife-com)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("LIFE INSURANCE")
    return None

def invalid_type1(corpus):
    corpus_split = corpus.lower()
    regex=r"(statement of transactions in savings account|statement of transat.{0,10}?ns in savings account|transaction details for your account)"
#     regex=r"Hindustan Petroleum|LAXMI TYRES AUTO SERVICE|PARK GARAGE|GARAGE|PETROL SUPPLY|Petrol Suppliers|BPCL fetrol|BPCL petrol|PETROL|etrol pumps|petrol pumps|fetrol|GASOHOL|Tubricants|MOTOR OIL|lubricants|DIESEL|Petroleum Service Station|SAVE FUEL|Thanks for fuelling with us|Bharat Petroleum|Petroleum|Shreeji Automobiles|Automobiles|Clutch Release Bearing|Clutch Cable|Clutch|Gear Liver Bush|Gear Shifter Cable|Indian Oil Dealers|Indian Otl Dealers|Next Oil Change|Oil Change|Nozzle No|Distance Travelled|Spare Parts|Magnet Coil|AC Compressor|Wiring Repair|Fly Wheel Skimming|Plate & Pressure|Gas Top UP|GARUDA AUTO SERVICE|PRESET TYPE|NOZZLE NO|LPG OIL|RATE\/LTR|INR\/Ltr|Pipe \“O\" Ring|NOTARY  GOVT. OF INDIA|NOTARY PUBLIC|NOTARY|LEASE AGREEMENT|india non judicial|METRO PHARMA|MAA ANKAPURNA ENTERPRISE|REFER and EARN cash rewards|Refer & Earn up to|referral mobile app|Google Play|Refer a friend"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("Invalid Document")
    return None

def invalid_type2(corpus):
    corpus_split = corpus.lower()
    regex=r"(statement of transactions in savings account|statement of transat.{0,10}?ns in savings account|transaction details for your account|hindustan petroleum|laxmi tyres auto service|mahalaxmi automotives|mahalaxmlautnmotives.com|park garage|garage|petrol supply|petrol suppliers|bpcl fetrol|bpcl petrol|petrol|etrol pumps|petrol pumps|fetrol|gasohol|tubricants|motor oil|lubricants|diesel|petroleum service station|save fuel|thanks for fuelling with us|bharat petroleum|petroleum|shreeji automobiles|automobiles|clutch release bearing|clutch cable|clutch|gear liver bush|gear shifter cable|indian oil dealers|indian otl dealers|next oil change|oil change|nozzle no|distance travelled|spare parts|magnet coil|ac compressor|wiring repair|fly wheel skimming|plate & pressure|gas top up|garuda auto service|preset type|nozzle no|lpg oil|rate\/ltr|inr\/ltr|pipe \“o\" ring|notary  govt. of india|notary public|notary|lease agreement|india non judicial|metro pharma|maa ankapurna enterprise|refer and earn cash rewards|refer & earn up to|referral mobile app|google play|refer a friend|leave and licenses|tube lights|waterjet panel|celling fans|water purifying machine|wooden sofa|wooden tepoy|dinning table|wooden book shelf|wodden plank|wooden bed|chappal stand|sewing machine stand|car package policy|bill of supply|doctor fees|billno & date|bill no|bill date|invoice no|invoice|driving services|challan mtr form|challan mtr|challan|op bill|ip-final bill|bill prepared by|bill prepared date|doctor consultation charges|inspector general of registration|door lach normal noise|key damage|front bumper damage|distress relief fund|working as a driver|service charges for driving|electricity distribution|energy charges|wheeling charges|capacitor penalty|janak pharmacy|janak fharpacy|haemoglobin|oncology|neutrophils|lymphocytes|monocytes|eosinophils|hypochromiat|microcytes|polychromatophili|anisocytosis|ultrasonography|abdomen and pelvis|imaging clinic|laboratory test report|taboratory test report|cholesterol|triglycerides|haematocrit|blood count|haematology & immunology|haematologist & immunologist|serum ferritin levels|donation to.{0,50}?trust)"
#     regex=r"Hindustan Petroleum|LAXMI TYRES AUTO SERVICE|PARK GARAGE|GARAGE|PETROL SUPPLY|Petrol Suppliers|BPCL fetrol|BPCL petrol|PETROL|etrol pumps|petrol pumps|fetrol|GASOHOL|Tubricants|MOTOR OIL|lubricants|DIESEL|Petroleum Service Station|SAVE FUEL|Thanks for fuelling with us|Bharat Petroleum|Petroleum|Shreeji Automobiles|Automobiles|Clutch Release Bearing|Clutch Cable|Clutch|Gear Liver Bush|Gear Shifter Cable|Indian Oil Dealers|Indian Otl Dealers|Next Oil Change|Oil Change|Nozzle No|Distance Travelled|Spare Parts|Magnet Coil|AC Compressor|Wiring Repair|Fly Wheel Skimming|Plate & Pressure|Gas Top UP|GARUDA AUTO SERVICE|PRESET TYPE|NOZZLE NO|LPG OIL|RATE\/LTR|INR\/Ltr|Pipe \“O\" Ring|NOTARY  GOVT. OF INDIA|NOTARY PUBLIC|NOTARY|LEASE AGREEMENT|india non judicial|METRO PHARMA|MAA ANKAPURNA ENTERPRISE|REFER and EARN cash rewards|Refer & Earn up to|referral mobile app|Google Play|Refer a friend"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("Invalid Document")
    return None

In [1452]:
def inv_type(row):
    corpus = row.corpus
    
    inv = invalid_type1(corpus)
    if inv is None:    
        inv = mediclaim_type(corpus)
        if inv is None:
            inv = ulip_type(corpus)
            if inv is None:
                inv = npse_type(corpus)
                if inv is None:
                    inv = mutual_fund_type(corpus)
                    if inv is None:
                        inv = home_loan_type(corpus)
                        if inv is None:
                            inv = childedu_type(corpus)
                            if inv is None:
                                inv = fd_type(corpus)
                                if inv is None:
                                    inv = handicap_type(corpus)
                                    if inv is None:
                                        inv = rent_type(corpus)
                                        if inv is None:
                                            inv = ppf_type(corpus)
                                            if inv is None:
                                                inv = nsc_type(corpus)
                                                if inv is None:
                                                    inv  = eduloan_type(corpus)
                                                    if inv is None:
                                                        inv = lic_type(corpus)
                                                        if inv is None:
                                                            inv = special_disease_type(corpus)
                                                            if inv is None:
                                                                inv = invalid_type2(corpus)
                                                                
    return(inv)

In [1453]:
df_result['INVESTMENT_TYPE'] = df_result.apply(inv_type,axis = 1)

In [1454]:
def find_tier(corpus):
    regex=r"(Tier I|Tier \||Tier 1|tier I|tier \||tier 1)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

In [1455]:
def lic_verification(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(transaction no|policyholder|premium receipt|premium certificate|renewal premium receipt|wrenewal premium receipt|received premium|renewal premium|premium paid statement|premium paid certificate|annual premium statement|receipt of payment|received with thanks|roceived with thanks|premium certificate|premium is received|total amount received|annual premium statement|life assured|insurance policies|life insurance policies|premium paid|policy no|policy number|swasthya bima policy|payment made by cheque|receipt of payment made by cheque|this is an electronically generated statement|this receipt is electronically generated|payment acknowledgement|this document is electronically generated|This réceipt is.electronically generated|this is a computer generated receipt|deposit receipt for advance premium|premium as per the applicable tax rates|stamp duty has been paid by demand draft|amount under consideration|amount in words|payment received from|Har Pal Aapke Saath|Modal Premium with Service Tax|Modal Premium Amount|Modal Premium|realisation of the cheque|zindagi ke saath bhi(.|..|...|)zindagi ke baad bhi|zindagi ke saath bhi|zindagi ke baad bhi|policy holder’s|gst borne by lic of india|gst (.|..|...|....|.....) by lic of india)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

In [1456]:
for i in range(0,len(df_result)):
    if df_result.loc[i,'INVESTMENT_TYPE'] == 'NPSE':
        corpus = df_result.loc[i,'corpus']
        tier = find_tier(corpus)
        if tier is None:
            df_result.loc[i,'INVESTMENT_TYPE'] = 'NPSE_Tier2'
        else:
            df_result.loc[i,'INVESTMENT_TYPE'] = 'NPSE_Tier1'

In [1457]:
for i in range(0,len(df_result)):
    if df_result.loc[i,'INVESTMENT_TYPE'] == 'LIFE INSURANCE':
        corpus = df_result.loc[i,'corpus']
        lic = lic_verification(corpus)
        if lic is None:
            df_result.loc[i,'INVESTMENT_TYPE'] = ''
        else:
            df_result.loc[i,'INVESTMENT_TYPE'] = 'LIFE INSURANCE'

In [1458]:
df_result

,path,Data,file_name,file_type,corpus,rendered_text,INVESTMENT_TYPE
0,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2184_Rent_Agreement...,jpg,i The LESSOR hereby acknowledges having receiv...,"[i, The LESSOR hereby acknowledges having rece...",None
1,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2185_Rent_Agreement...,jpg,10. 41. 12. 13. 14. 15. 16 The LESSOR s...,"[10., 41., 12., 13., 14., 15., 16, The LESSOR ...",RENTAL AGREEMENT
2,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2186_Rent_Agreement...,jpg,ik 18. 19. 20. 21. 22. 23. 24. 25. Th...,"[ik, 18., 19., 20., 21., 22., 23., 24., 25., T...",RENTAL AGREEMENT
3,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2187_Rent_Agreement...,jpg,"A : my or Incurred, pald or sustained by the o...","[A : my, or Incurred, pald or sustained by the...",None
4,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2188_Rent_Agreement...,jpg,Annexure-! ...,"[Annexure-!, ‘THE LESSOR -, $. No. Description...",None
5,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2189_Rent_Agreement...,jpg,"4 \ IN WITNESS WHEREOF, THE LESSEE AND THE L...","[4, \, IN WITNESS WHEREOF, THE LESSEE AND THE ...",None
6,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2190_Rent_Agreement...,jpg,LEASE AGREEMENT LEASE ASSESS THIS DEED OF LE...,"[LEASE AGREEMENT, LEASE ASSESS, THIS DEED OF L...",Invalid Document
7,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2191_Rent_Agreement...,jpg,f WHEREAS: : A. The LESSOR declares that he i...,"[f, WHEREAS: :, A. The LESSOR declares that he...",RENTAL AGREEMENT
8,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2192_Rent Receipt-D...,jpg,Weesopcay Diode ——_—_—————_—_—_ v_ ainjeu...,"[Weesopcay Diode, ——_—_—————_—_—_, v_ ainjeubi...",None
9,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2193_Rent Receipt_B...,jpg,ac eCooe on ahs We understand your world Pag...,"[ac eCooe, on ahs, We understand your world, P...",RENTAL AGREEMENT


### LIC Provider-wise

In [1459]:
def lic_type_lic(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(life insuance corporation|life insurance corporation|lic of india|l.t.c. of india|lichelp|lic|ilic|uc of India|l.i.c. of india|lai(.|..|...)of lndia|l.t.c. of thdia|bo_\w+.{0,2}licindia.com|zindagi ke saath bhi(.|..|...|)zindagi ke baad bhi|zindagi ke saath bhi|zindagi ke baad bhi)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('LIFE INSURANCE_LIC')
    return None

def lic_type_hdfc(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(hdfc standard life insurance|hdfc standaro life insurance|hofc term assurance|https://onlinepayments.hdfclife.com/hdfclife/quick_pay.html|https://cp.hdfclife.com/cpweb/login.aspx|https://cp.hdfclife.com/cpweb/premiumreceiptsdisplay.aspx|hdfc term assurance|nriservice@hdfclife.com|service@hdfclife.com|serviceohdfclife.com|service@hdiclite.com|service@hdfelife.com|nriservice@hdtclite.com|nkiservice@hdfclife.com|support@hdfelife.com|wwwhdtclife.com|www.hdfclife.com|www.hdfctife.com|wwwhdfelife.com|wewhdfclife.com|hdfc life|hdfc)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('LIFE INSURANCE_HDFC')
    return None

def lic_type_aviva(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(aviva life insurance|aviva life|aviva)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('LIFE INSURANCE_AVIVA')
    return None

def lic_type_birla(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(aditya birla sunlife insurance|aditya birla sun life insurance|aditya birla sunlite insurance|aditya birla sunkfe insurance|care.lifeinsurance@adityabirlacapital.com|lifeinsurance@adityabirlacapital.com|aditybirlasuniifeinsurance.com|aditya birla sun life|aditya bile sun life insurance|bsli flexi secure life|birla sun life insurance|aditya birla life insurance|aditya birla suniiie insurance|birla sun life|birla sun|aditya birla capital|aditya birla|birla sunkfe insurance|birla sur life|birla|absl|absli digi shield plan|absli dig shield plan|aaitya bis sur life irwursice|adityabirlacapital|bsli dream plan|bsl! dream plan)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('LIFE INSURANCE_BIRLA')
    return None

def lic_type_sbi(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(sbi life insurance company|sbi life insurance|sbi life  insurance|www.onlinesbilife.com|www.sbilife.ce.in|www.sbilife.co.in|sbi life|sbilife|s21 life|osbi|sbi)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('LIFE INSURANCE_SBI')
    return None

def lic_type_aegon(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(aegon life|wegon life|www.aegonlife.com|wwwaegonlfe.com|customer.care@aegonlife.com)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('LIFE INSURANCE_AEGON')
    return None

def lic_type_exide(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(exidelife’  insurance|exide life insurance|care@exidelife.in|exidelife.in|exide life|exipe life|exide)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('LIFE INSURANCE_EXIDE')
    return None

def lic_type_max_life(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(max life insurance|max life comprehensive|max life|max slife|mix life|helpdesk @maxlifeinsurance.com|maxlifeinsurance.com|www.maxlifeinsurance.com|amaxlifeinsurance.com|moxlifeinsurance.com|maxhfeinsurance.com|facebook.com\/maxlife|facebook.com\/maxiila|maxliteinsurance.com|maxlifeinsurance|max financial services|maxiifeinsurance|mazlifeinsarance|max otife)"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('LIFE INSURANCE_MAX LIFE')
    return None

def lic_type_tata_aia(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(tata aia life insurance company limited|tata aia life insurance|tata ala life insurance|ata aia life insurance|tata aia  tiee  life insurance|taw ala life insuanee|tain ata life insurance|tata ata life insurance|tata aia life|tata aia|tata ala|tata ata|ata aia|http://www tataaia.com|http://www.tataaia.com|customercare@tataaia.com)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('LIFE INSURANCE_TATA AIA')
    return None

def lic_type_bajaj(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(bajaj allianz life insurance|bajaj allianz future_wealth_gain|bajaj allianz future gain|www.bajajallianz.com|life@bajajallianzlife.co.in|bajaj allianz|baja allianz|baja 1\|allianz)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('LIFE INSURANCE_BAJAJ')
    return None



def lic_type_future_generali(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(future generali india life insurance|future generali|future generall|tuture generali|www.futuregenerali.in|www.futuregenerall.in|www.tuturegenerall.in|care@futuregenerali.in|care@tuturegenerall.in|care@futuregenerall.in)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('LIFE INSURANCE_FUTURE GENERALI')
    return None


def lic_type_icici(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(http://www.iciciprulife.com/public/others/call-us|icici prudential life insurance|communication@iciciprulife.com|icici prodential life insurance|icici prudential|icich prudential|spicich predential|icici pru lifetime|icici prldential|icici pru iprotect|icici pru health saver|icici pru savings|prudential life|estatements@iciciprulife.com|lifeline@iciciprulife.com|lifeline@iciciprulife.co|www.iciciprulil@.com|ciciprulife.com|lifelinegiciciprulife.com|lweline@eciprulife.com|www.iciciprulife.com|www.iciciprulfe.com|gciciprulite.com|iciciprulife.com|iciciprulite|ficici prudential|icici pru life|icici pru|iciciprulife|prudential|icici)"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('LIFE INSURANCE_ICICI')
    return None

def lic_type_kotak(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(kotak mahindra life insurance company|kotak mahindra life insurance|kotak mahindra|kotak life insurance|http://insurance.kotak.com|insurance.kotak.com|emailclientservicedesk@kotak.com|clientservicedesk@kotak.com|kotak preferred term|kotak super advantage|kotak capital multiplier|kotak term plan|kotak assured savings|kotak e-torm|kotak life family|kotak life|kotak)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('LIFE INSURANCE_KOTAK')
    return None

def lic_type_reliance(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(reliance nippon life insurance company|reliance life insurance company|reliance capital company|reliance center|www reliancenipponlite.com|customerservice@relianceada.com|www rellancenipponife.com|nippon life relian insurance|reliance)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('LIFE INSURANCE_RELIANCE')
    return None

def lic_type_bharti_axa(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(bharti axa life insurance|jeevan suraksha ka / naya nazariya|bharti axa)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('LIFE INSURANCE_BHARTI AXA')
    return ('LIFE INSURANCE_DEFAULT')


In [1460]:
for i in range(0,len(df_result)):
    if df_result.loc[i,'INVESTMENT_TYPE'] == 'LIFE INSURANCE':
        corpus = df_result.loc[i,'corpus']
        lic = lic_type_lic(corpus)
        df_result.loc[i,'INVESTMENT_TYPE'] = lic
        if lic is None:
            lic = lic_type_hdfc(corpus)
            df_result.loc[i,'INVESTMENT_TYPE'] = lic
            if lic is None:
                lic = lic_type_aviva(corpus)
                df_result.loc[i,'INVESTMENT_TYPE'] = lic
                if lic is None:
                    lic = lic_type_birla(corpus)
                    df_result.loc[i,'INVESTMENT_TYPE'] = lic
                    if lic is None:
                        lic = lic_type_sbi(corpus)
                        df_result.loc[i,'INVESTMENT_TYPE'] = lic
                        if lic is None:
                            lic = lic_type_aegon(corpus)
                            df_result.loc[i,'INVESTMENT_TYPE'] = lic
                            if lic is None:
                                lic = lic_type_exide(corpus)
                                df_result.loc[i,'INVESTMENT_TYPE'] = lic
                                if lic is None:
                                    lic = lic_type_max_life(corpus)
                                    df_result.loc[i,'INVESTMENT_TYPE'] = lic
                                    if lic is None:
                                        lic = lic_type_tata_aia(corpus)
                                        df_result.loc[i,'INVESTMENT_TYPE'] = lic
                                        if lic is None:
                                            lic = lic_type_bajaj(corpus)
                                            df_result.loc[i,'INVESTMENT_TYPE'] = lic
                                            if lic is None:
                                                lic = lic_type_icici(corpus)
                                                df_result.loc[i,'INVESTMENT_TYPE'] = lic
                                                if lic is None:
                                                    lic = lic_type_future_generali(corpus)
                                                    df_result.loc[i,'INVESTMENT_TYPE'] = lic
                                                    if lic is None:
                                                        lic = lic_type_kotak(corpus)
                                                        df_result.loc[i,'INVESTMENT_TYPE'] = lic                                                      
                                                        if lic is None:
                                                            lic = lic_type_reliance(corpus)
                                                            df_result.loc[i,'INVESTMENT_TYPE'] = lic                                                
                                                            if lic is None:
                                                                lic = lic_type_bharti_axa(corpus)
                                                                df_result.loc[i,'INVESTMENT_TYPE'] = lic

### Mediclaim Provider-wise

In [1461]:
def mediclaim_type_aditya_birla(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(adityabirlahealthinsurance|adityabirlacapital|aditya birla health insurance|birla sun life insurance|birla sun life|birla sun|birla|absl)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('MEDICLAIM_ADITYA_BIRLA')
    return None

def mediclaim_type_apollo(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(apollo munich health insurance|apollomunichinsurance|apollomunich|apoliomunich|apollo munich|apollo|customerservice@apollomunichinsurance.com|customerservice@apoiomunichinsurance.com|www.apollomunichinsurance.com|www.apalismunichinsurance.com|www.apoliomunichinsurance.com|apollo munich health insurance|anotio munich health lasurance)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('MEDICLAIM_APOLLO')
    return None

def mediclaim_type_bajaj(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(bajaj allianz general insurance|bajajallianz|bajaj allianz)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('MEDICLAIM_BAJAJ')
    return None

def mediclaim_type_cigna(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(cignattkinsurance|cignattk|cigna ttk|cigna)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('MEDICLAIM_CIGNA')
    return None

def mediclaim_type_hdfc_ergo(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(hdfc ergo general insurance|hdfc ergo|hdfc standard life|hdfc life|hofc life|hdfc|hofc)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('MEDICLAIM_HDFC_ERGO')
    return None

def mediclaim_type_icici(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(icici lombard general insurance company|icici prudential|icici lombard|icici)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('MEDICLAIM_ICICI')
    return ('MEDICLAIM_DEFAULT')

def mediclaim_type_iffco(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(iffcotokio|iffco tokio|iffco)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('MEDICLAIM_IFFCO')
    return None

def mediclaim_type_max(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(maxhealthcare|max life|max bupa health insurance|max bupa|max healthcare)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('MEDICLAIM_MAX')
    return None

def mediclaim_type_national_mediclaim(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(national insurance company limited|www.nationalinsuranceindia.com|(.|..|...|....|.....)nationalinsuranceindia(.....|....|...|..|.)|national insurance|national mediclaim policy|national mediclaim|national medictaim|national \|nsuranéé|parivar mediclaim.*?customer name)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('MEDICLAIM_NATIONAL_MEDICLAIM')
    return None

def mediclaim_type_new_india(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(the new india assurance|new india assurance|new india mediclaim policy|new india mediclaim|new india)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('MEDICLAIM_NEW_INDIA')
    return None

def mediclaim_type_oriental(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(orientalinsurance|oriental insurance)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('MEDICLAIM_ORIENTAL')
    return None

def mediclaim_type_religare(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(religarehealthinsurance|religare health|religare)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('MEDICLAIM_RELIGARE')
    return None

def mediclaim_type_royal_sundaram(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(royal sundaram general|royalsundaram|royal sundaram)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('MEDICLAIM_ROYAL_SUNDARAM')
    return None

def mediclaim_type_star_health(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(star health and allied insurance|star comprehensive insurance|starhealth|star health)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('MEDICLAIM_STAR_HEALTH')
    return None

def mediclaim_type_united_india(corpus):
    corpus_split = corpus.lower()
    
    regex = r"\b(united india insurance company limited|united india insurance|uneted india insurance|united india|http://gccore\.uiic\.in|http://gocore\.uiic\.in|gccore\.uiic\.in|gocore\.uiic\.in)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return ('MEDICLAIM_UNITED_INDIA')
    return None


In [1462]:
for i in range(0,len(df_result)):
    if df_result.loc[i,'INVESTMENT_TYPE'] == 'MEDICLAIM':
        corpus = df_result.loc[i,'corpus']
        mediclaim = mediclaim_type_aditya_birla(corpus)
        df_result.loc[i,'INVESTMENT_TYPE'] = mediclaim
        if mediclaim is None:
            mediclaim = mediclaim_type_apollo(corpus)
            df_result.loc[i,'INVESTMENT_TYPE'] = mediclaim
            if mediclaim is None:
                mediclaim = mediclaim_type_bajaj(corpus)
                df_result.loc[i,'INVESTMENT_TYPE'] = mediclaim
                if mediclaim is None:
                    mediclaim = mediclaim_type_cigna(corpus)
                    df_result.loc[i,'INVESTMENT_TYPE'] = mediclaim                   
                    if mediclaim is None:
                        mediclaim = mediclaim_type_iffco(corpus)
                        df_result.loc[i,'INVESTMENT_TYPE'] = mediclaim
                        if mediclaim is None:
                            mediclaim = mediclaim_type_max(corpus)
                            df_result.loc[i,'INVESTMENT_TYPE'] = mediclaim
                            if mediclaim is None:
                                mediclaim = mediclaim_type_national_mediclaim(corpus)
                                df_result.loc[i,'INVESTMENT_TYPE'] = mediclaim
                                if mediclaim is None:
                                    mediclaim = mediclaim_type_new_india(corpus)
                                    df_result.loc[i,'INVESTMENT_TYPE'] = mediclaim
                                    if mediclaim is None:
                                        mediclaim = mediclaim_type_oriental(corpus)
                                        df_result.loc[i,'INVESTMENT_TYPE'] = mediclaim
                                        if mediclaim is None:
                                            mediclaim = mediclaim_type_religare(corpus)
                                            df_result.loc[i,'INVESTMENT_TYPE'] = mediclaim
                                            if mediclaim is None:
                                                mediclaim = mediclaim_type_royal_sundaram(corpus)
                                                df_result.loc[i,'INVESTMENT_TYPE'] = mediclaim
                                                if mediclaim is None:
                                                    mediclaim = mediclaim_type_star_health(corpus)
                                                    df_result.loc[i,'INVESTMENT_TYPE'] = mediclaim
                                                    if mediclaim is None:
                                                        mediclaim = mediclaim_type_united_india(corpus)
                                                        df_result.loc[i,'INVESTMENT_TYPE'] = mediclaim
                                                        if mediclaim is None:
                                                            mediclaim = mediclaim_type_hdfc_ergo(corpus)
                                                            df_result.loc[i,'INVESTMENT_TYPE'] = mediclaim
                                                            if mediclaim is None:
                                                                mediclaim = mediclaim_type_icici(corpus)
                                                                df_result.loc[i,'INVESTMENT_TYPE'] = mediclaim
                                                            

In [1463]:
df_result

,path,Data,file_name,file_type,corpus,rendered_text,INVESTMENT_TYPE
0,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2184_Rent_Agreement...,jpg,i The LESSOR hereby acknowledges having receiv...,"[i, The LESSOR hereby acknowledges having rece...",None
1,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2185_Rent_Agreement...,jpg,10. 41. 12. 13. 14. 15. 16 The LESSOR s...,"[10., 41., 12., 13., 14., 15., 16, The LESSOR ...",RENTAL AGREEMENT
2,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2186_Rent_Agreement...,jpg,ik 18. 19. 20. 21. 22. 23. 24. 25. Th...,"[ik, 18., 19., 20., 21., 22., 23., 24., 25., T...",RENTAL AGREEMENT
3,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2187_Rent_Agreement...,jpg,"A : my or Incurred, pald or sustained by the o...","[A : my, or Incurred, pald or sustained by the...",None
4,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2188_Rent_Agreement...,jpg,Annexure-! ...,"[Annexure-!, ‘THE LESSOR -, $. No. Description...",None
5,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2189_Rent_Agreement...,jpg,"4 \ IN WITNESS WHEREOF, THE LESSEE AND THE L...","[4, \, IN WITNESS WHEREOF, THE LESSEE AND THE ...",None
6,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2190_Rent_Agreement...,jpg,LEASE AGREEMENT LEASE ASSESS THIS DEED OF LE...,"[LEASE AGREEMENT, LEASE ASSESS, THIS DEED OF L...",Invalid Document
7,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2191_Rent_Agreement...,jpg,f WHEREAS: : A. The LESSOR declares that he i...,"[f, WHEREAS: :, A. The LESSOR declares that he...",RENTAL AGREEMENT
8,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2192_Rent Receipt-D...,jpg,Weesopcay Diode ——_—_—————_—_—_ v_ ainjeu...,"[Weesopcay Diode, ——_—_—————_—_—_, v_ ainjeubi...",None
9,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDABALDA4MChAODQ...,EM000002_RAVINDRA JOGDAND _2193_Rent Receipt_B...,jpg,ac eCooe on ahs We understand your world Pag...,"[ac eCooe, on ahs, We understand your world, P...",RENTAL AGREEMENT


In [1464]:
# df_result.to_excel('correction_corpus.xlsx',index = False)

## Helper Functions

### Default Category

In [1465]:
###################################################################PROVIDER

def provider_def(corpus):
    regex=r"\b(TATA AIG|Tata AIG|STAR HEALTH AND ALLIED INSURANCE|STAR COMPREHENSIVE INSURANCE|Star Health|STAR HEALTH|Apollo Munich|APOLLO MUNICH|Apollo|APOLLO|STATE BANK OF INDIA|State Bank of India|SBI Life|SBI|sbi|EXIDE Life|Exide Life|Exide|EXIDE|ICICI PRUDENTIAL|ICICI Lombard|ICICI LOMBARD|ICICI|icici|Max Life|MAX LIFE|max life|HDFC LIFE|HDFC Life|hdfc life|HDFC ERGO|hdfc ergo|HDFC Standard Life|HDFC ERGO General Insurance|HOFC Life|HDFC Mutual Fund|HDFC|hdfc|LIC of India|LIC OF INDIA|Life Insurance Corporation|LIFE INSURANCE CORPORATION|LIC|lic|kotak life|KOTAK LIFE|Kotak Life|Kotak Mahindra Life|KOTAK LIFE|KOTAK|kotak|NEW INDIA ASSURANCE|New India Assurance|NEW INDIA|Aegon Life|Aegon life|aegon life|aegon|AEGON|Bajaj Allianz|BAJAJ ALLIANZ|BAJAJ|bajaj|Birla Sun Life Mutual Fund|Birla Sun Life Insurance|Birla Sun Life|ABSL|BIRLA SUN LIFE|birla sun life|BIRLA SUN|Birla Sun|PUNJAB NATIONAL BANK|Punjab National Bank|Punjab National|punjab national|PNB MetLife India Insurance|PNB MetLife India|PNB MetLife|pnb metlife|PNB METLIFE|PNB|pnb|AVIVA|Aviva|aviva|Bharti axa|Bharti AXA|bharti axa|MAX BUPA|max bupa|Max Bupa|UTI|uti|Bank of Baroda|DHFL|Dhfl|IDBI|GIC|Mumbai District Central bank|Mumbai District Central Co-op. Bank|Mumbai District Central|Mumbai.*?Co-op. Bank|LIC HFL|LIC Housing Finance Ltd|SARASWAT Bank|Saraswat Bank|SARASWAT|Saraswat|Royal Sundaram General|royal sundaram|Royal Sundaram|UNITED INDIA INSURANCE|United India|united india|ORIENTAL INSURANCE|Oriental Insurance|oriental insurance|National Mediclaim|L&T Mutual Fund|L&T|Axis Mutual Fund|Axis|axis|Motilal Oswal Mutual Fund|Motilal Oswal|RELIANCE Mutual fund|RELIANCE|Mahindra Mutual Fund|Mahindra|Franklin Templeton|Franklin India|DSP BLACKROCK|OSP BLACKROCK|DSP BlackRock|DSP|Dewan Housing Finance Corporation|Dewan Housing Finance|Dewan Housing)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_provider_lower(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(indiabulls housing finance limited|www.indiabulls.com|homeloans@indiabulls.com|indiabulls|bank of india|standard chartered|petrolium|auto service|automobiles|allahabad bank|tata aia|star health and allied insurance|star comprehensive insurance|star health|apollo munich|apollo|state bank of india|sbi life|sbi|exide life|exide|icici prudential|icici lombard|icici|max life|hdfc ergo general insurance|hdfc ergo|hdfc standard life|hdfc life|hofc life|hdfc|hofc|life insuance corporation|lic of india|lic|kotak mahindra life|kotak life|kotak|new india assurance|new india|aegon life|aegon|bajaj allianz|bajaj|birla sun life insurance|birla sun life|birla sun|birla|absl|punjab national bank|punjab national|pnb metlife india insurance|pnb metlife india|pnb metlife|pnb|aviva|bharti axa|max bupa|uti|bank of baroda|dhfl|idbi|gic|mumbai district central bank|lic hfl|lic housing finance ltd|lic housing finance|saraswat bank|saraswat|royal sundaram general|united india insurance|united india|oriental insurance|national mediclaim|l&t mutual fund|l&t|axis mutual fund|axis|motilal oswal mutual fund|motilal oswal|reliance mutual fund|reliance|mahindra mutual fund|mahindra|franklin templeton|franklin india|dsp blackrock|osp balckrock|dsp|osp|magma|liberty|generali|signa|raheja|cholamandalam|ecgc|iffco|indiafirst|peerless|sahara|religare|hsbc|edelweiss|aia|dai-ichi|dewan housing finance|dewan housing|dewan|osbi|gicici)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_provider_For_DCB(corpus):
    regex = r"\b(DCB Bank|DCB|dcb bank|DCR RANK|DCB BANK)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return ('DCB BANK')
    return None

################################################################### INSURED NAME

def name_def(corpus):
    regex=r"\b(Smt/Me /Sr iba|Smt/Ms./Shri(|.|..|...|....)SHRI|Smt/Ms\.J/Shri|SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mr/Ms|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Policy Owner Name|Name & Father Name of Investor|Name & Father Name of Investor 1|This is to certify that|THIS IS TO CERTIFY THAT|Name of the Subscriber)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Ali|ALI|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Joy|JOY|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Rks|RKS|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s\s|\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Ali|ALI|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Joy|JOY|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Rks|RKS|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
#     regex=r"(\b(Mrs\.|MRS\.|Mrs|MRS|M/s|Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|MR |Policy Holder Name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|Name & Father Name of Investor|Name & Father Name of Investor 1)(.|..|...|....)(\w+(.|\s)\w+(.|\s)\w{3,}|\w \w+|\w+ \w\. \w+|\w+ \w+))|\b(Dear)\b(.|..)(\w+ \w+ \w{3,}|\w+ \w+)"
#     regex=r"((M/s|Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|MR |Policy Holder Name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|Name & Father Name of Investor|Name & Father Name of Investor 1)(.|..|...|....)((\w+ \w+ \w{3,})|(\w \w+)|(\w+ \w+)))|\b(Dear)\b(.|..)(\w+ \w+ \w{3,}|\w+ \w+)"

    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

###################################################################DATE
    
def date_def1(corpus):
    regex = r"(Date|Date:|DATE|date|Date of Receipt|Date of issue|DATE OF ISSUE|Date of Issue|Date of Issuc|Date and Time :|Receipt Date|INTEREST CERTIFICATE|Date of Receipt|DATE OF RECEIPT|Date and Time|DATE AND TIME|INSURANCE Date)(\s|)(.|..|...|....|.....)\b(((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4})(\b\s|-)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def date_def2(corpus):
    regex = r"\b(((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4})(\b\s|-|,)"
#     regex = r"(\d{1,4}|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||,|,\s|\s)(\d{1,4}|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||,|,\s|\s)\d{1,4}"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################POLICY NUMBER

def policy_def1(corpus):
    regex=r"\b(POLICY|Policy|policy|Poticy|PoticyNo|P\.|p\.|P|p|Pol|pol|Pol\.|pol\.|Details|Proposal/Policy|Policy)+\s(NO|No|Num|No\.|no\.|num|Number|number|NUMBER|)(\s|)(|.|..|...|....|.....)((\d\w\d+|\d+\w+\d+)(/|\\|\||)(\d+|\w+))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[4])
    else:
        return None

def policy_def2(corpus):
    
    regex = r"\b(Policy|policy|Pol|pol|Pol\.|pol\.|Proposal/Policy|Policy|Plicy|plicy)\b(\s)(No|Num|No\.|no\.|num|Number|number|NUMBER).*?\s(\d\w\d+|\d+\w+\d+)(/|\\|\||-|)(\d+|\w[a-zA-Z]|)(\s)"
#     regex = r"(\b(Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Details|Proposal/Policy|Policy)+\s(No|Num|No.|no.|num|Number|number).*?((\d\w\d+|\d+\w+\d+)(/|\\|\||-|)(\d+|\w[a-zA-Z]|))(\s))|\b(Policy|policy|Plicy|plicy)\b.*?((\d\w\d+|\d+\w+\d+)(/|\\|\||-|)(\d+|\w[a-zA-Z]|))(\s)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[3])
    else:
        return None

def policy_def3(corpus):
    corpus_split = corpus.lower()
    regex = r"\b(receipt no|document no|folio no|folio number|foliono|your dcb educational loan account|dcb educational loan account|educational loan account|your dcb education loan account|dcb education loan account|education loan account|loan account number|account number|account no(\.|)|acc(\.|\s|\.\s)no(\.|)|a/c(\.|\s|\.\s)no(\.|)|chief minister's distress relief fund kerala  b no|loan account statement)\b(.|..|...|....|.....)((\d+\W+\d+|\w+(/)\d+|\w+\d+).*?\s)"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[7])
    else:
        return None    
    
###################################################################AMOUNT

def amt_def(corpus):
    corpus1 = corpus.lower()
    regex = r"\b(principal amt|principal amount|sum of|amount paid|total repayment received|total for the year|total amount rs|total amount|amount of rs|amount of|amt. collected|amount deposited|rent of rs|total|amount|amt|cleared balance|amount paid towards premium|towards premium is|towards oremiun is|received with thanks|ceived with thanks|paid by cheque|regular premium)\b(|.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|\$|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(|.|..|...|....|.....)(\s|:|\.)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus1, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):        
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[6])
    else:
        return None
    
# def amt_def1(corpus):   
#     regex = r"(\b(Premium.A.|Principal Amt|Premium Amount|Premium amount|Total Premium Amount|Amount Paid|Total Premium Paid|Total|Total Amount Rs. |amount of|Amount paid|premium amount of Rs.|Total Premium :Rs|Amt. Collected|Total Premium :Rs.)\b...((\d+.\d+)|(: \d+.\d\d)|(\d+,\d+.\d\d)))"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

# def amt_def2(corpus):
#     regex = r"((Amount|amount|amt|Amount paid|Total Premium Paid|Amount Deposited)(.|)(.|)\w+(.|)\w+(.|)(Rs|rs).*?((\d+|\w+)|(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|)(\s|)\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)))"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

# def amt_def3(corpus):
#     regex = r"(Total Repayment Received|total repayment received).*?(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)\d+(,|\.|\d)\d+(,|\.)\d+(,|\.|)(\d+|)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

# def amt_def4(corpus):
#     regex = r"(Amount paid|amount paid|Amount Deposited|AMOUNT|amount|Amount|Total Amount|total amount)(.|..|...|....)(\d+\,\d+\.\d+|\d+\.\d+|\d+)"
# #     regex = r"(Amount paid|amount paid|Amount Deposited|Total Amount|total amount)(.|..|...|....)\d+"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

### Life Insurance

In [1466]:
###################################################################PROVIDER


def find_provider_reg(corpus):
    regex=r"\b(National Insurance Company Limited|TATA AIG|Tata AIG|STAR HEALTH AND ALLIED INSURANCE|STAR COMPREHENSIVE INSURANCE|Star Health|STAR HEALTH|Apollo Munich|APOLLO MUNICH|Apollo|APOLLO|STATE BANK OF INDIA|State Bank of India|SBI Life|SBI|sbi|EXIDE Life|Exide Life|Exide|EXIDE|ICICI PRUDENTIAL|ICICI Lombard|ICICI LOMBARD|ICICI|icici|Max Life|MAX LIFE|max life|HDFC LIFE|HDFC Life|hdfc life|HDFC ERGO|hdfc ergo|HDFC Standard Life|HDFC ERGO General Insurance|HOFC Life|HDFC Mutual Fund|HDFC|hdfc|LIC of India|LIC OF INDIA|Life Insurance Corporation|LIFE INSURANCE CORPORATION|LIC|lic|kotak life|KOTAK LIFE|Kotak Life|Kotak Mahindra Life|KOTAK LIFE|KOTAK|kotak|NEW INDIA ASSURANCE|New India Assurance|NEW INDIA|Aegon Life|Aegon life|aegon life|aegon|AEGON|Bajaj Allianz|BAJAJ ALLIANZ|BAJAJ|bajaj|Birla Sun Life Mutual Fund|Birla Sun Life Insurance|Birla Sun Life|ABSL|BIRLA SUN LIFE|birla sun life|BIRLA SUN|Birla Sun|PUNJAB NATIONAL BANK|Punjab National Bank|Punjab National|punjab national|PNB MetLife India Insurance|PNB MetLife India|PNB MetLife|pnb metlife|PNB METLIFE|PNB|pnb|AVIVA|Aviva|aviva|Bharti axa|Bharti AXA|bharti axa|MAX BUPA|max bupa|Max Bupa|UTI|uti|Bank of Baroda|DHFL|Dhfl|IDBI|GIC|Mumbai District Central bank|Mumbai District Central Co-op. Bank|Mumbai District Central|Mumbai.*?Co-op. Bank|LIC HFL|LIC Housing Finance Ltd|SARASWAT Bank|Saraswat Bank|SARASWAT|Saraswat|Royal Sundaram General|royal sundaram|Royal Sundaram|UNITED INDIA INSURANCE|United India|united india|ORIENTAL INSURANCE|Oriental Insurance|oriental insurance|National Mediclaim|L&T Mutual Fund|L&T|Axis Mutual Fund|Axis|axis|Motilal Oswal Mutual Fund|Motilal Oswal|RELIANCE Mutual fund|RELIANCE|Mahindra Mutual Fund|Mahindra|Franklin Templeton|Franklin India|DSP BLACKROCK|OSP BLACKROCK|DSP BlackRock|DSP|Dewan Housing Finance Corporation|Dewan Housing Finance|Dewan Housing)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_provider_lower(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(national insurance company limited|national insurance|allahabad bank|tata aia|tata aig|star health and allied insurance|star comprehensive insurance|star health|apollo munich|apollo|state bank of india|sbi life|sbi|exide life|exide|icici prudential|icici lombard|icici|max life|hdfc ergo general insurance|hdfc ergo|hdfc standard life|hdfc life|hofc life|hdfc|hofc|life insuance corporation|lic of india|lic|kotak mahindra life|kotak life|kotak|new india assurance|new india|aegon life|aegon|bajaj allianz|bajaj|birla sun life insurance|birla sun life|birla sun|birla|absl|punjab national bank|punjab national|pnb metlife india insurance|pnb metlife india|pnb metlife|pnb|aviva|bharti axa|max bupa|uti|bank of baroda|dhfl|idbi|gic|mumbai district central bank|lic hfl|lic housing finance ltd|lic housing finance|saraswat bank|saraswat|royal sundaram general|united india insurance|united india|oriental insurance|national mediclaim|l&t mutual fund|l&t|axis mutual fund|axis|motilal oswal mutual fund|motilal oswal|reliance mutual fund|reliance|mahindra mutual fund|mahindra|franklin templeton|franklin india|dsp blackrock|osp balckrock|dsp|osp|magma|liberty|generali|signa|raheja|cholamandalam|ecgc|iffco|indiafirst|peerless|sahara|religare|hsbc|edelweiss|aia|dai-ichi|dewan housing finance|dewan housing|dewan|osbi|gicici)\b"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_provider_For_DCB(corpus):
    regex = r"\b(DCB Bank|DCB|dcb bank|DCR RANK|DCB BANK)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return ('DCB BANK')
    return None

###################################################################INSURED NAME

def find_name(rendered_text):
    regex = r"\b(Smt\./Ms\./Shri|SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
#     regex = r"(\b(Smt/MsJ/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|\bMr\b|\bMR\b|\bMS\b|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Smt.|Ms.|Shri|Sri|Sri.|MR |Policy Holder Name|Premium Payor Name|Policyholder's Name|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|Received with thanks from|Name of the Life Assured)(.|..|...|)((\w+ \w+ \w+)|(\w+ \w. \w+)|(\w\w+)+|(\w \w+ \w+)))|\b(Dear)\b(.|..)(\w+ \w+ \w{2,}|\w+ \w+)"
#     regex = r"((M/s|Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|MR |Policy Holder Name|Premium Payor Name|Policyholder's Name|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of)(.|..|...|)((\w+ \w+ \w+)|(\w\w+)+|(\w \w+ \w+)))|\b(Dear)\b(.|..)(\w+ \w+ \w{2,}|\w+ \w+)"
    for text in rendered_text:
        matches = re.match(regex, text)
        if matches:
            return text
            break 
            
    return None

    
def find_name2(corpus):
    regex=r"((Received with thanks from|Insured Name|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Name of Nominee|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Life Assured)(.|..|...|....)|)\b(Sh\./Smt\./ M/s|Smt/Ms/Shri|SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Mr/Ms|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Policyholder Name|Name of the Proposer|ime of the Proposer)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Ali|ALI|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Joy|JOY|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Ali|ALI|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Joy|JOY|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

def find_name3(corpus):
    regex=r"\b(Smt/Me /Sr iba|Smt/Ms./Shri(|.|..|...|....)SHRI|Smt/Ms.J/Shri|Mr\./Mrs\. MISS|Smit/Ms/Shri|Sh\./Smt\./ M/s|Smt\./Ms/Shri|Smt/Ms/Shri|SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|This is to certify that|Thisis to certify that|THIS IS TO CERTIFY THAT|this is to certify that|To Whom It May Concern|To Whomsoever It May Concern|To whomsoever it may concern|Insured's Name|Insured Name|insured Name)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Ali|ALI|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Joy|JOY|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Rks|RKS|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s\s|\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Ali|ALI|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Joy|JOY|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Rks|RKS|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
#     regex=r"(\b(Smt/Ms/Sr|Smt/MsJ/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Smt/MsJ/Shri|Smt\./Ms\./Shri|Shri/Smt\.Smt\.|Mrs|mrs|Mr|mr|Smt\./Ms/Sri\. Sree WATT|Smt./Ms/Sri. Sree|M/s|Mr./Mrs.|\bMr\b|\bMR\b|\bMS\b|\bMs\b|\bMiss\b|\bMrs\b|\bProf\b|Mr./Mrs.|Name|Dear MR.|\bSmt\b|\bShri\b|\bSri\b|MR )\b)(.|..|...|....)((\w+|\w)(.|)(\w+|\w)|(\w+|\w)(.|)(\w+|\w)(.|)(\w+|\w))"
#     regex=r"(((insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name|\s)(.|..|...|....)|)(Smt\./Ms/Sri\. Sree WATT|Smt./Ms/Sri. Sree|M/s|Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|MR |insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of)(.|..|...|....|)((\w+ \w+ \w+)|(\w+ \w+)|(\w\w+)+|(\w \w+ \w+)))|\b(Dear)\b(.|..)(\w+ \w+ \w{2,}|\w+ \w+)|(\w[a-zA-Z]+ \w[a-zA-Z]+ \w[a-zA-Z]+|\w+ \w \w+|\w+ \w\. \w+|\w\. \w\. \w+|\w \w \w+)+(.|..|...)(Address|Flat|Room|Road)|(Name(.|insured Name|Insured Name|Insured Name|)(.|..|...|....)((Mrs|mrs|Mr|mr|Smt\./Ms/Sri\. Sree WATT|Smt./Ms/Sri. Sree|M/s|Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|MR )|)(.|)((\w+|\w)(.|)(\w+|\w)|(\w+|\w)(.|)(\w+|\w)(.|)(\w+|\w)))"
#     regex=r"((Smt\./Ms/Sri\. Sree WATT|Smt./Ms/Sri. Sree|M/s|Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|MR |insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of)(.|..|...|)((\w+ \w+ \w+)|(\w+ \w+)|(\w\w+)+|(\w \w+ \w+)))|\b(Dear)\b(.|..)(\w+ \w+ \w{2,}|\w+ \w+)|(\w[a-zA-Z]+ \w[a-zA-Z]+ \w[a-zA-Z]+|\w+ \w \w+|\w+ \w\. \w+|\w\. \w\. \w+|\w \w \w+)+(.|..|...)(Address|Flat|Room|Road)|(Name(.|insured Name|Insured Name|Insured Name|)(.|..|...|....)((Mrs|mrs|Mr|mr)|)(.|)((\w+|\w)(.|)(\w+|\w)|(\w+|\w)(.|)(\w+|\w)(.|)(\w+|\w)))"
#     regex=r"((M/s|Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|MR |Policy Holder Name|Premium Payor Name|Policyholder's Name|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of)(.|..|...|)((\w+ \w+ \w+)|(\w\w+)+|(\w \w+ \w+)))|\b(Dear)\b(.|..)(\w+ \w+ \w{2,}|\w+ \w+)|(\w[a-zA-Z]+ \w[a-zA-Z]+ \w[a-zA-Z]+|\w+ \w \w+|\w+ \w\. \w+|\w\. \w\. \w+|\w \w \w+)+(.|..|...)(Address|Flat|Room|Road)"
#     regex=r"((M/s|Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|MR |Policy Holder Name|Premium Payor Name|Policyholder's Name|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of)(.|..|...|)((\w+ \w+ \w+)|(\w\w+)+|(\w \w+ \w+)))|\b(Dear)\b(.|..)(\w+ \w+ \w{2,}|\w+ \w+)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    
###################################################################POLICY NUMBER

def find_policyno_table(corpus):
    #regex = r" (Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Master olicy)+\s(No|Num|No.|no.|num|Number|number) \d+"
    regex=r"\b(Policy|policy) (No\.|no\.) (Name|name).{1,}mode \(2\) \d+"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

def find_policy(corpus):
    #regex = r" (Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Master olicy)+\s(No|Num|No.|no.|num|Number|number) \d+"
    regex=r"\b(POLICY|Policy|policy|Poticy|PoticyNo|P\.|p\.|P|p|Pol|pol|Pol\.|pol\.|Details|Proposal/Policy|Policy)+\s(NO|No|Num|No\.|no\.|num|Number|number|)(\s|\s+|.|)(:|,|;|-|=|\s|)(.|..|...|....)(\d\w\d+|\d+\w+\d+)(/|\\|\||)(\d+|\w+)|((?<=Section 80C).*?(\d+))|((Remark(?:s)|remark(?:s)|REMARK(?:S))).*?\d+"
#     regex=r"\b(POLICY|Policy|policy|P\.|p\.|P|p|Pol|pol|Pol\.|pol\.|Details|Proposal/Policy|Policy)+\s(NO|No|Num|No\.|no\.|num|Number|number|)(\s|\s+|.|)(:|,|;|-|=|\s|) (\d\w\d+|\d+\w+\d+)(/|\\|\||)(\d+|\w+)|((?<=Section 80C).*?(\d+))|((Remark(?:s)|remark(?:s)|REMARK(?:S))).*?\d+"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

def find_policy_slash(corpus):
    
    regex = r"\b(Policy|policy|Pol|pol|Pol\.|pol\.|Details|Proposal/Policy|Policy|Plicy|plicy|towards the following)\b(\s)(No|Num|No\.|no\.|num|Number|number|).*?((\d\w\d+|\d+\w+\d+)(/|\\|\||-|)(\d+|\w[a-zA-Z]|))(\s)"
#     regex = r"(\b(Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Details|Proposal/Policy|Policy)+\s(No|Num|No.|no.|num|Number|number).*?((\d\w\d+|\d+\w+\d+)(/|\\|\||-|)(\d+|\w[a-zA-Z]|))(\s))|\b(Policy|policy|Plicy|plicy)\b.*?((\d\w\d+|\d+\w+\d+)(/|\\|\||-|)(\d+|\w[a-zA-Z]|))(\s)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def find_policy_slash1(corpus):
#     regex = r"(Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Details|Proposal/Policy|Policy)+\s(No|Num|No.|no.|num|Number|number).*?((\d\w\d+|\d+\w+\d+)(/|\\|\||)(\d+|\w+|))(|.)(\w+)(|.)(\d+)(|.)(\d+)(|.)(\d+)" 
#     regex = r"((Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Details|Proposal/Policy|Policy)+\s(No|Num|No.|no.|num|Number|number).*?((\d\w\d+|\d+\w+\d+)(/|\\|\||-|)(\d+|\w[a-zA-Z]|))(\s))|\b(Policy|policy|Plicy|plicy)\b.*?((\d\w\d+|\d+\w+\d+)(/|\\|\||-|)(\d+|\w[a-zA-Z]|))(\s)" 
#     #regex=r"(Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Details|Proposal/Policy|Policy)+\s(No|Num|No.|no.|num|Number|number).*?(\d\w\d+|\d+\w+\d+)(/|\\|\||)(\d+|\w+|)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
    regex = r"\b(Policy|policy|Pol|pol|Pol\.|pol\.|Details|Proposal/Policy|Policy|Plicy|plicy|towards the following)\b(\s)(No|Num|No\.|no\.|num|Number|number).*?((\d\w\d+|\d+\w+\d+)(/|\\|\||-|)(\d+|\w[a-zA-Z]|))(\s)"
#     regex = r"\b(Policy|policy|Pol|pol|Pol\.|pol\.|Details|Proposal/Policy|Policy|Plicy|plicy|towards the following)\b(\s)(No|Num|No\.|no\.|num|Number|number|).*?((\d\w\d+|\d+\w+\d+)(/|\\|\||-|)(\d+|\w[a-zA-Z]|))(\s)"
#     regex = r"(\b(Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Details|Proposal/Policy|Policy)+\s(No|Num|No.|no.|num|Number|number).*?((\d\w\d+|\d+\w+\d+)(/|\\|\||-|)(\d+|\w[a-zA-Z]|))(\s))|\b(Policy|policy|Plicy|plicy)\b.*?((\d\w\d+|\d+\w+\d+)(/|\\|\||-|)(\d+|\w[a-zA-Z]|))(\s)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[4])
    else:
        return None    

def find_policy_slash2(corpus):
    regex=r"\b(Policy|policy|Pol|pol|Pol\.|pol\.|Details|Proposal/Policy|Policy|Plicy|plicy|ay|cy|poli)\b(\s)(No|Num|No\.|no\.|num|Number|number|yer).*?((\d\w\d+|\d+\w+\d+)(/|\\|\||-|)(\d+|\w[a-zA-Z]|))(\s)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group() 



def find_policy_until_number(corpus):
    regex = r"\b(PolicyNo|Policy|policy|Pol|pol|Pol\.|pol\.|Details|Proposal/Policy|Policy)(\s|)(No|Num|No\.|no\.|num|Number|number|)\b.*?\d+"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None


    
###################################################################DATE


def find_if_date_and_time(corpus):
    regex=r"((Receipt Date & Time|Date of Receipt|DATE OF RECEIPT|Date of Issue|DATE OF ISSUE|Date of Issuc|Date and Time|DATE AND TIME|Date...Time.|Date..Time...|Date...Time....|Receipt Date|INSURANCE Date:|Payment Date and Time :|Date|DATE|Dale|TNO|Tr\. No\.)(\s|).{0,10}?\b(((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4})(\b\s|-|,))"
#     regex=r"((Date of Receipt|DATE OF ISSUE:|DATE OF ISSUE :|Date and Time|Date...Time.|Date..Time...|Date...Time....|Receipt Date|INSURANCE Date:|Payment Date and Time :|Date|DATE|TNO|Tr\. No\.)(\s|)(.|..|...|....|.....)\b((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4}\b\s)"
#     regex=r"((Date of Receipt|DATE OF ISSUE:|DATE OF ISSUE :|Date...Time.|Date..Time...|Date...Time....|Receipt Date|INSURANCE Date:|Payment Date and Time :|Date :|Date:|Date|TNO|Tr\. No\.)(\s|)(|:|)+.(\d+|\w+|\w)(-|/|\|.|,|\s)(\d+|\w+)(-|/|\|.|,|\s)(\d+|\w+))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[3])
    else:
        return None



###################################################################AMOUNT

def find_premium(rendered_text):
#     regex = r"(Premium.A.|Premium Amount|Premium amount|Premium|Total Premium Amount|Amount paid|Total Premium Paid|Gross Premium .Rs.|Gross)..(?: \d\d+)+ (?=\()|\d+,\d+.\d\d"
#     regex = r"(Premium.A.|Premium Amount|Premium amount|PREMIUM AMOUNT|Premium|Total Premium Amount|Amount paid|Total Premium Paid|Gross Premium .Rs.|Gross|)(\s)(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)\d+(,|\.|\d)\d+(,|\.)\d+(,|\.|)(\d+|)"
    regex = r"(Premium.A.|Premium Amount|Premium amount|PREMIUM AMOUNT|Premium|Total Premium Amount|Amount paid|Total Premium Paid|Gross Premium .Rs.|Gross|)(\s)(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)\d+(,|\.|\d)\d+(,|\.)\d+(,|\.|)(\d+|)|((?<=Section 80C).*?(\d+))"
    for text in rendered_text:
        matches = re.match(regex, text)
        if matches:
            return text
            break
    return None


def find_premium_cor1(corpus):
    regex = r"((Premium.A.|Premium Amount|Premium amount|Premium|Total Premium Amount|Amount paid|Total Premium Paid|Gross Premium .Rs.|Gross|Premium Payable|Received with thanks|Premium Paid|Premium Paid \(Rs\.\))(.|..|...|....)(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|¥|%|\(USD\) \$|£|€|=)(\s|)\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))|(Premium Paid|Premium Paid \(Rs\.\)).*?\d+"
#     regex = r"(Premium.A.|Premium Amount|Premium amount|Premium|Total Premium Amount|Amount paid|Total Premium Paid|Gross Premium .Rs.|Gross|Premium Payable|Received with thanks)(.|..|...|....)(?: \d\d+)+ (?=\()|(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|¥|%|\(USD\) \$|£|€|=)(\s|)\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)"
#     regex = r"(Premium.A.|Premium Amount|Premium amount|Premium|Total Premium Amount|Amount paid|Total Premium Paid|Gross Premium .Rs.|Gross)..(?: \d\d+)+ (?=\()|\d+,\d+.\d\d"
#     regex = r"(Premium.A.|Premium Amount|Premium amount|Premium|Total Premium Amount|Amount paid|Total Premium Paid|Gross Premium .Rs.|Gross|Premium Payable)..(?: \d\d+)+ (?=\()|(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|¥|\(USD\) \$|£|€|=)(\s|)\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_premium_cor1_1(corpus):
    regex = r"(Premium.A.|Premium Amount|Premium amount|Premium|Total Premium Amount|Amount paid|Total Premium Paid|Gross Premium .Rs.|Gross|Premium Payable)(.|..|...|....|.....|)(\d+(,|\.|)\d+(,|\.|)\d+)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_premium_cor2(corpus):
#     regex = r"(Premium.A.|Premium Amount|Premium amount|Premium|Total Premium Amount|Amount paid|Total Premium Paid|Gross Premium .Rs.|Gross)..(?: \d\d+)+ (?=\()|\d+,\d+.\d\d"
    regex = r"\b(Prem|prem|Premium Details|Premium.A.|Premium Amount|Premium amount|PREMIUM AMOUNT|Total Premium Amount|Amount paid|Total Premium Paid|Gross Premium .Rs.|Gross|Premium Payable)\b((\s)|.*?)(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|¥|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|\d)\d+(,|\.)\d+(,|\.|)(\d+|)|(\d+(.|..)\d+))"
#     regex = r"(Premium Details|Premium.A.|Premium Amount|Premium amount|PREMIUM AMOUNT|Total Premium Amount|Amount paid|Total Premium Paid|Gross Premium .Rs.|Gross|Premium Payable)((\s)|.*?)(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|¥|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)\d+(,|\.|\d)\d+(,|\.)\d+(,|\.|)(\d+|)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def prem_amt(corpus):
    regex = r"(\b(Premium.A.|Premium Amount|Premium amount|Total Premium Amount|Amount Paid|Total Premium Paid|Total|Total Amount Rs. |amount of|Amount paid|premium amount of Rs.|Total Premium :Rs|Amt. Collected|Total Premium :Rs.)\b...((\d+.\d+)|(: \d+.\d\d)|(\d+,\d+.\d\d)))"
    
    test_str = corpus
    matches = re.finditer(regex, test_str, re.MULTILINE)

    for matchNum, match in enumerate(matches):
        matchNum = matchNum + 1
        if match:
            return match.group()
    return None

def prem_amt1(corpus):
    regex = r"(Premium\.A\.|Premium Amount|Premium amount|PREMIUM AMOUNT|premium amount|Total Premium Amount|TOTAL PREMIUM AMOUNT|total premium amount|AMOUNT PAID|amount paid|Amount Paid|Total Premium Paid|Total Amount Rs|amount of|Amount paid|premium amount of Rs|Amt\. Collected|Total Premium :Rs|paid|Total Premium :Rs|Total Received|total received|TOTAL RECEIVED)(.|..|...|....)((\d+,\d+.\d\d)|(\d+.\d\d)|(\d+))"
    
    test_str = corpus
    matches = re.finditer(regex, test_str, re.MULTILINE)

    for matchNum, match in enumerate(matches):
        matchNum = matchNum + 1
        if match:
            return match.group()
    return None


def amt_lic_1(corpus):
    regex=r"\b(We acknowledge with thanks|We acknowledge receipt of an amount|We acknowledge receipt of|Renewal Premium to be Paid|Received with thanks|received with thanks|ceived with thanks|RECEIVED WITH THANKS|Premium Received|Total Received|Grand|Total Amount|Total|total).*?(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||).*?(\d+(,|\.|\d+)\d+(,|\.)\d+(,|.)(\d+|))"
#     regex=r"\b(We acknowledge with thanks|We acknowledge receipt of an amount|We acknowledge receipt of|Renewal Premium to be Paid|Received with thanks|received with thanks|ceived with thanks|RECEIVED WITH THANKS|Premium Received|Total Received|Grand|Total Amount|Total|total).*?(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||).*?(\d+(,|.)\d+(,|.)\d+(,|.)(\d+|))"
#     regex=r"\b(We acknowledge with thanks|We acknowledge receipt of an amount|We acknowledge receipt of|Renewal Premium to be Paid|Received with thanks|received with thanks|ceived with thanks|RECEIVED WITH THANKS|Premium Received|Total Received|Grand|Total|total).*?(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||).*?(\d+(,|.)\d+(,|.)\d+(,|.)(\d+|))"
#     regex=r"\b(Received with thanks|received with thanks|ceived with thanks|RECEIVED WITH THANKS|Premium Received|Total Received|Grand|Total|total).*?(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||).*?(\d+(,|\.)\d+(,|\.)\d+(,|.)(\d+|))"
#     regex=r"(Grand(.|)Total(.|)(.|)Rs(.|)(.|)(\d+.\d+.\d+|\d+.\d+))|Premium Received.*?(INR|Rs\.|RS\.|Rs|RS|=).*? (\d+(,|.|)\d+(,|.|)\d+(,|.)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[3])
    else:
        return None
    
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None



def amt_lic_2(corpus):
    regex=r"\b(We acknowledge with thanks|We acknowledge receipt of an amount|We acknowledge receipt of|Renewal Premium to be Paid|Received with thanks|received with thanks|ceived with thanks|RECEIVED WITH THANKS|Premium Received|Total Received|Grand|Total|total).*?(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||).*?(\d+(,|.)\d+(,|.)\d+(,|.)(\d+|))"
#     regex=r"(Grand(.|)Total(.|)(.|)Rs(.|)(.|)(\d+.\d+.\d+|\d+.\d+))|Premium Received.*?(INR|Rs\.|RS\.|Rs|RS|=).*? (\d+(,|.|)\d+(,|.|)\d+(,|.)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def lic_amount1(corpus):
    regex = r"(Premium Paid|Premium amount|Premium \(A\)|Target Amount|Amount Received|Amount)(\s|).*?(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|¥|\(USD\) \$|£|€|=|)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
###################################################################PAY MODE    

def pay_mode(corpus):
    regex=r"(Payment(.|)Frequency(.|)(.|)(\w+)|payment(.|)frequency(.|)(.|)(\w+)|Modal(.|)Premium(.|)and(.|)Mode(.|)(.|)Rs(.|)(.|)(\d+.\d+|\d+.\d|\d+)(.|)(.|)(\w+))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def pay_mode1(corpus):
    regex=r"\b(Quarterly|YLY|Yearly|Semi-Annual|YLy|Yly|QLy|QLY|Qly|Hly|Monthly|Half Yearly|Half yearly|half-yearly)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################GST



# def lic_CGST0(corpus):
#     regex=r"(sGsT/UTGST|CGST|cGsT|CGST|cGST|cest|C@ST|SGST|sGsT|SGsT|S@ST|SGST|sGST|SQST_|SQST|CaST_|CaST)\b(.|..|...|....|.....)(INR|Inr|Rs\.|RS\.|Rs|RS|R8\.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
#     #regex=r"\b(CGST|cGsT|CGST|cGST|GST|IGST)\b(.....|....|...|..|.)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\|)(.|..|...|....|.....)(\d+(,|\.|)\d+(,|\.|)\d+)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     result = []
#     for matchNum, match in enumerate(matches, start=1):
#         for groupNum in range(0, len(match.groups())):
#             groupNum = groupNum + 1
#             result.append(match.group(groupNum))           
#     if len(result)!=0:
#         return(result[5])
#     else:
#         return None
    
# def lic_CGST1(corpus):
#     regex=r"(CGST\(9\.00%\)|CGST @9.00% 2|CGST @9.00%|CGST @9%|GST\(9\.00%\)|GST @9%|Central Tax 9%|State Tax 9%)(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
#     #regex=r"\b(CGST|cGsT|CGST|cGST|GST|IGST)\b(.....|....|...|..|.)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\|)(.|..|...|....|.....)(\d+(,|\.|)\d+(,|\.|)\d+)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     result = []
#     for matchNum, match in enumerate(matches, start=1):
#         for groupNum in range(0, len(match.groups())):
#             groupNum = groupNum + 1
#             result.append(match.group(groupNum))           
#     if len(result)!=0:
#         return(result[5])
#     else:
#         return None
    

# def lic_CGST3(corpus):
#     regex = r"(Integrated Goods and Service Tax \(18\.00 %\)|Service Tax \(18\.00 \%\)|GST\(18\.00%\)|Goods & Services Tax|Goods & Service Tax|Service Tax|GST)(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
# #     regex = r"(\b(CGST|cGsT|CGST|cGST|GST|Goods & Services Tax|Goods & Service Tax|Service Tax)\b(.|)(\W\d\W|).(\W|)(.|)\d(.|)(\d+|\d))|(GST(.|)as(.|)applicable(.|)(\W|)\w+(\W|)(.|)(\d+|\d)(.|)(\d+|\d))|(?<=GST\(9.00%\))(.+?\d+.\d{2})|(?<=CGST\(9.00%\))(.+?\d+.\d{2})|(?<=GST @9%)(.+?\d+.\d{2})|(?<=CGST @9%)(.+?\d+.\d{2})"
    
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     result = []
#     for matchNum, match in enumerate(matches, start=1):
#         for groupNum in range(0, len(match.groups())):
#             groupNum = groupNum + 1
#             result.append(match.group(groupNum))        
#     if len(result)!=0:
#         return(result[5])
#     else:
#         return None


# def lic_SGST1(corpus):
#     regex=r"(sGsT/UTGST|UTGST|SGST|sGsT|SGsT|S@ST|SGST|sGST|SBC|SQST_|SQST|KKC|CGST|cGsT|CGST|cGST|cest|CaST_|CaST|Goods & Services Tax|Goods & Service Tax|Service Tax)\b(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     result = []
#     for matchNum, match in enumerate(matches, start=1):
#         for groupNum in range(0, len(match.groups())):
#             groupNum = groupNum + 1
#             result.append(match.group(groupNum))
#     if len(result)!=0:
#         return(result[5])
#     else:
#         return None

# def lic_SGST2(corpus):    
#     regex = r"(SGST/UTGST @9%|SGST\(9.00%\)|SGST @9.00% 2|SGST @9.00%|SGST @9%|Service Tax \(18\.00 \%\)|CGST\(9\.00%\)|CGST @9.00% 2|CGST @9.00%|CGST @9%|GST\(9\.00%\)|GST @9%|GST\(18\.00%\)|Central Tax 9%|State Tax 9%)(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     result = []
#     for matchNum, match in enumerate(matches, start=1):
#         for groupNum in range(0, len(match.groups())):
#             groupNum = groupNum + 1
#             result.append(match.group(groupNum))    
#     if len(result)!=0:
#         return(result[5])
#     else:
#         return None
    
# ###################################################################REMARK

# def lic_remark1(corpus):
#     regex=r"(CGST|cGsT|CGST|cGST|GsT|GST|Integrated Goods and Service Tax|Central Goods and Service Tax|Goods & Services Tax|Goods & Service Tax|Service Tax)\b"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

# def lic_remark2(corpus):
#     regex=r"(State/Union Territory Goods and Service Tax|UTGST|SGST|sGsT|SGST|sGST|SBC|KKC)\b"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None
############################################################################GST

def CGST0(corpus):
    regex=r"(sGsT/UTGST|CGST|cGsT|CGST|cGST|cest|C@ST|SGST|sGsT|SGsT|S@ST|SGST|sGST|SQST_|SQST|CaST_|CaST)\b(.|..|...|....|.....)(INR|Inr|Rs\.|RS\.|Rs|RS|R8\.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
    #regex=r"\b(CGST|cGsT|CGST|cGST|GST|IGST)\b(.....|....|...|..|.)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\|)(.|..|...|....|.....)(\d+(,|\.|)\d+(,|\.|)\d+)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[5])
    else:
        return None
      
    
def CGST1(corpus):
    regex=r"(Central Goods and Service Tax \(9.00 %\)|CGST\(9\.00%\)|State/Union Territory Goods and Service Tax \(9.00 %\)|CGST @9.00% 2|CGST @9.00%|CGST @9%|GST\(9\.00%\)|GST @9%|Central Tax 9%|State Tax 9%)(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
    #regex=r"\b(CGST|cGsT|CGST|cGST|GST|IGST)\b(.....|....|...|..|.)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\|)(.|..|...|....|.....)(\d+(,|\.|)\d+(,|\.|)\d+)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))           
    if len(result)!=0:
        return(result[5])
    else:
        return None
    
def CGST2(corpus):
    regex = r"(Integrated Goods and Service Tax \(18\.00 %\)|Service Tax \(18\.00 \%\)|GST\(18\.00%\)|Goods & Services Tax|Goods & Service Tax|Service Tax|GST)(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
#     regex = r"(\b(CGST|cGsT|CGST|cGST|GST|Goods & Services Tax|Goods & Service Tax|Service Tax)\b(.|)(\W\d\W|).(\W|)(.|)\d(.|)(\d+|\d))|(GST(.|)as(.|)applicable(.|)(\W|)\w+(\W|)(.|)(\d+|\d)(.|)(\d+|\d))|(?<=GST\(9.00%\))(.+?\d+.\d{2})|(?<=CGST\(9.00%\))(.+?\d+.\d{2})|(?<=GST @9%)(.+?\d+.\d{2})|(?<=CGST @9%)(.+?\d+.\d{2})"
    
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))        
    if len(result)!=0:
        return(result[5])
    else:
        return None


def SGST0(corpus):
    regex=r"(sGsT/UTGST|UTGST|SGST|sGsT|SGsT|S@ST|SGST|sGST|SBC|SQST_|SQST|KKC|CGST|cGsT|CGST|cGST|cest|CaST_|CaST)\b(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[5])
    else:
        return None

def SGST1(corpus):
    regex=r"(Central Goods and Service Tax \(9.00 %\)|CGST\(9\.00%\)|State/Union Territory Goods and Service Tax \(9.00 %\)|CGST @9.00% 2|CGST @9.00%|CGST @9%|GST\(9\.00%\)|GST @9%|Central Tax 9%|State Tax 9%)(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
    #regex=r"\b(CGST|cGsT|CGST|cGST|GST|IGST)\b(.....|....|...|..|.)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\|)(.|..|...|....|.....)(\d+(,|\.|)\d+(,|\.|)\d+)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))           
    if len(result)!=0:
        return(result[5])
    else:
        return None
    
# def SGST2(corpus):
#     regex = r"(Integrated Goods and Service Tax \(18\.00 %\)|Service Tax \(18\.00 \%\)|GST\(18\.00%\)|Goods & Services Tax|Goods & Service Tax|Service Tax|GST)(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
# #     regex = r"(\b(CGST|cGsT|CGST|cGST|GST|Goods & Services Tax|Goods & Service Tax|Service Tax)\b(.|)(\W\d\W|).(\W|)(.|)\d(.|)(\d+|\d))|(GST(.|)as(.|)applicable(.|)(\W|)\w+(\W|)(.|)(\d+|\d)(.|)(\d+|\d))|(?<=GST\(9.00%\))(.+?\d+.\d{2})|(?<=CGST\(9.00%\))(.+?\d+.\d{2})|(?<=GST @9%)(.+?\d+.\d{2})|(?<=CGST @9%)(.+?\d+.\d{2})"
    
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     result = []
#     for matchNum, match in enumerate(matches, start=1):
#         for groupNum in range(0, len(match.groups())):
#             groupNum = groupNum + 1
#             result.append(match.group(groupNum))        
#     if len(result)!=0:
#         return(result[5])
#     else:
#         return None
    
###################################################################REMARK

def cgst_remark1(corpus):
    corpus1 = corpus.lower()
    regex=r"(central goods and services tax|central goods and service tax|state goods and services tax|state goods and service tax|central tax|state tax|cgst|sgst|utgst|sgst/utgst|c@st|s@st|sqst(|.|..))\b"
    matches = re.finditer(regex, corpus1, re.MULTILINE)
    for match in matches:
        return ("CGST")
    return None

def cgst_remark2(corpus):
    corpus1 = corpus.lower()
    regex=r"(igst|gst|integrated goods and service tax|goods & services tax|goods & service tax|service tax)\b"
    matches = re.finditer(regex, corpus1, re.MULTILINE)
    for match in matches:
        return ("GST")
    return None

def sgst_remark1(corpus):
    corpus1 = corpus.lower()
    regex=r"(central goods and services tax|central goods and service tax|state goods and services tax|state goods and service tax|central tax|state tax|cgst|sgst|utgst|sgst/utgst|c@st|s@st|sqst(|.|..))\b"
    matches = re.finditer(regex, corpus1, re.MULTILINE)
    for match in matches:
        return ("SGST")
    return None

def sgst_remark2(corpus):
    corpus1 = corpus.lower()
    regex=r"(utgst|sbc|kkc)\b"
    matches = re.finditer(regex, corpus1, re.MULTILINE)
    for match in matches:
        return match.group()
    return None


In [1467]:
def information_extract_LIC_def(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus

    provider = find_provider_reg(corpus) 
    if provider is None:
        provider = find_provider_lower(corpus)
        if provider is None:
            provider = find_provider_For_DCB(corpus)
    
    policy_number = find_policyno_table(corpus)
    if policy_number is None:    
        policy_number = find_policy(corpus)
        if policy_number is None:
            policy_number = find_policy_slash(corpus)
            if policy_number is None:
                policy_number = find_policy_until_number(corpus)

    
    premium_amt = amt_lic_1(corpus)
    if premium_amt is None:
        premium_amt = amt_lic_2(corpus)
        if premium_amt is None:
            premium_amt = find_premium_cor1(corpus)
            premium_amt = premium_amt
            if premium_amt is None:
                premium_amt = find_premium_cor1_1(corpus)
                if premium_amt is None:
                    premium_amt = find_premium_cor2(corpus)
                    premium_amt = premium_amt
                    if premium_amt is None:
                        premium_amt = med_prem_amt(corpus)
                        premium_amt = premium_amt
                        if premium_amt is None:
                #             premium_amt = find_premium_ex(corpus)
                #             premium_amt=premium_amt
                #             if premium_amt is None:
                            premium_amt = prem_amt(corpus)
                            premium_amt=premium_amt
                            if premium_amt is None:
                                premium_amt = prem_amt1(corpus)
                                premium_amt=premium_amt
                                if premium_amt is None:
                #                     premium_amt=find_premium_back(rendered)
                #                     premium_amt=premium_amt
                #                     if premium_amt is None:
                                    premium_amt = lic_amount1(corpus)


    insured_name = find_name2(corpus)
    if insured_name is None:
        insured_name = find_name(rendered)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)   
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)
    
#     CGST = lic_CGST0(corpus)
#     if CGST is None:
#         CGST = lic_CGST1(corpus)
#         if CGST is None:
#             CGST = lic_CGST2(corpus)
#             if CGST is None:
#                 CGST = lic_CGST3(corpus)            
        
#     SGST = lic_SGST1(corpus)
#     if SGST is None:
#         SGST = lic_SGST2(corpus)
        
    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)         
    
#     remark1 = lic_remark1(corpus)    
#     remark2 = lic_remark2(corpus)
    
    payment_mode = pay_mode1(corpus)
    if payment_mode is None:
        payment_mode = pay_mode(corpus)
             
    #return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)
    return (provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### LIFE INSURANCE_AEGON

In [1468]:
def find_name_cor_aegon(corpus):
    regex = r"((Received with thanks from|Insured Name|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Name of Nominee|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Dear)(.|..|...|....)|)\b(Smt/Ms/Shri|SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Dear)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_policy_cor_aegon1(corpus):
    regex = r"\b(Policy|policy|Pol|pol|Pol\.|pol\.|Details|Proposal/Policy|Policy|Plicy|plicy|towards the following)\b(\s)(Number|number|No|Num|No\.|no\.|num).{0,100}?\s(\d{12})\s"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[3])
    else:
        return None

def find_premium_cor_aegon1(corpus):
    regex = r"(Total Premiums Paid|total premiums paid|Premium Amount|Total Premiums Payable|premium amount).*?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||).*?\s(\d+(,|\.|\d+)\d+(,|\.|)\d+(,|\.|)(\d+|))\s"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[3])
    else:
        return None
    
def find_premium_cor_aegon2(corpus):
    regex = r"(Total Premium Rs\.\(in figures\)|Total Premiums Paid|total premiums paid|Premium Amount|Total Premiums Payable|premium amount).*?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||).*?\s(\d+(,|\.|\d+)\d+(,|\.|)\d+(,|\.|)(\d+|))\s"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[3])
    else:
        return None    

In [1469]:
def information_extract_AEGON(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus
    
    provider = "AEGON LIFE"
    
    policy_number = find_policy_cor_aegon1(corpus)

    
    premium_amt = find_premium_cor_aegon1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_cor_aegon2(corpus)


    insured_name = find_name_cor_aegon(corpus)
    if insured_name is None:
        insured_name = find_name(rendered)
        if insured_name is None:
            insured_name = find_name2(corpus)
            if insured_name is None:
                insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)   
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)
    
    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)   
    
    payment_mode = pay_mode1(corpus)
    if payment_mode is None:
        payment_mode = pay_mode(corpus)
             
    #return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)
    return (provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### LIFE INSURANCE_AVIVA

In [1470]:
def find_premium_cor_aviva(corpus):
    regex = r"((Premium received(.|..|...|....)in figures|premium received(.|..|...|....)in figures|Premium Recieved(.|..|...|....)in figure|Premium Received(.|..|...|....)in figure|Total Amount|Total amount|total amount))(.|..|...|....)(INR|Inr|Rs.|RS.|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|)(\s|)(.*?)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_name_cor_aviva(corpus):
    regex = r"((Received with thanks from|Insured Name|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Name of Nominee|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Name|Payment received from|Payment Received from)(.|..|...|....)|)\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Policyholder Name|Payment received from|Payment Received from)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

In [1471]:
def information_extract_AVIVA(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus

    provider = "AVIVA LIFE INSURANCE"
    
    policy_number = find_policyno_table(corpus)
    if policy_number is None:    
        policy_number = find_policy(corpus)
        if policy_number is None:
            policy_number = find_policy_slash(corpus)
            if policy_number is None:
                policy_number = find_policy_until_number(corpus)

    
    premium_amt = find_premium_cor_aviva(corpus)


    insured_name = find_name_cor_aviva(corpus)
    if insured_name is None:
        insured_name = find_name(rendered)
        if insured_name is None:
            insured_name = find_name2(corpus)
            if insured_name is None:
                insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    

    premium_date = date_def2(corpus)
    
    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)   
    
    payment_mode = pay_mode1(corpus)
    if payment_mode is None:
        payment_mode = pay_mode(corpus)
             
    #return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)
    return (provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### LIFE INSURANCE_BAJAJ

In [1472]:
def find_premium_cor_bajaj(corpus):
    regex = r"((Received with thanks|Total|TOTAL PREMIUM PAY ABLE FOR THE SELECTED MODE|TOTAL PREMIUM PAYABLE FOR THE SELECTED MODE|Regular Premium|Premium Amount|Gross Amt|Gross amt|Gross Amount|Gross amount|Amount of(.|..|...)in figures))(.|..|...|....)(INR|Inr|Rs.|RS.|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|)(\s|)(.|..|...|....|.....)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_date_cor_bajaj(corpus):
    regex = r"((Date of Receipt|DATE OF RECEIPT|Date of Issue|DATE OF ISSUE|Date and Time|DATE AND TIME|Date...Time.|Date..Time...|Date...Time....|Receipt Date|INSURANCE Date:|Payment Date and Time :|Date|DATE|TNO|Tr\. No\.)(\s|)(.|..|...|....|.....)\b(((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4}\b\s))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))

    if len(result)!=0:
        return(result[4])
    else:
        return None

In [1473]:
def information_extract_BAJAJ(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus

    provider = "BAJAJ ALLIANZ"
    
    policy_number = find_policyno_table(corpus)
    if policy_number is None:    
        policy_number = find_policy(corpus)
        if policy_number is None:
            policy_number = find_policy_slash(corpus)
            if policy_number is None:
                policy_number = find_policy_until_number(corpus)

    
    premium_amt = find_premium_cor_bajaj(corpus)


    insured_name = find_name(rendered)
    if insured_name is None:
        insured_name = find_name2(corpus)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    premium_date = find_date_cor_bajaj(corpus)  
    if premium_date is None:
        premium_date = find_if_date_and_time(corpus)   
        if premium_date is None:
            premium_date = date_def2(corpus)
    
    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)   
    
    payment_mode = pay_mode1(corpus)
    if payment_mode is None:
        payment_mode = pay_mode(corpus)
             
    #return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)
    return (provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### Life Insurace_BHARTI AXA

In [1474]:
def find_premium_cor_bharti_axa(corpus):
    regex = r"(Amount|amount)(INR|Inr|Rs.|RS.|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|)(\s|)(.|..|...|....|.....)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_policy_cor_bharti_axa(corpus):
    #regex = r" (Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Master olicy)+\s(No|Num|No.|no.|num|Number|number) \d+"
    regex=r"\b(Policy|policy|Pol|pol|Pol\.|pol\.|Details|Proposal/Policy|Policy|Plicy|plicy|towards the following)\b(\s)(No|Num|No\.|no\.|num|Number|number|).*?((\d\w\d+|\d+\w+\d+)(/|\\|\||-|)(\d+|\w[a-zA-Z]|))(\s)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    
def find_name_cor_bharti_axa(corpus):
    regex=r"((Received with thanks from|Insured Name|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Name of Nominee|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Dear)(.|..|...|....)|)\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Policyholder Name|Dear)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

In [1475]:
def information_extract_BHARTI_AXA(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus

    provider = "BHARTI AXA"
    
    policy_number = find_policy_cor_bharti_axa(corpus)
    if policy_number is None:
        policy_number = find_policyno_table(corpus)
        if policy_number is None:    
            policy_number = find_policy(corpus)
            if policy_number is None:
                policy_number = find_policy_slash(corpus)
                if policy_number is None:
                    policy_number = find_policy_until_number(corpus)

    premium_amt = find_premium_cor_bharti_axa(corpus)

    insured_name = find_name_cor_bharti_axa(corpus)
    if insured_name is None:
        insured_name = find_name(rendered)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)   
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)
    
    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
            
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)   
    
    payment_mode = pay_mode1(corpus)
    if payment_mode is None:
        payment_mode = pay_mode(corpus)
             
    #return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)
    return (provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)


### LIFE INSURANCE_BIRLA

In [1476]:
def find_premium_cor_birla1(corpus):
    regex = r"((Your premium amount of|Your Premium amount of|Your preimum ane nt of|Your premium aount of Rs|Your hremium amount of Rs|premium amount of Rs|Total Amount|total amount|total Amount|Tota\) Am@iitt|Total Received|total received))(.|..|...|....)(INR|Inr|Rs.|RS.|Re.|Bs|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|¢|%|\(USD\) \$|£|€|=|)(.*?)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[5])
    else:
        return None
    
def find_premium_cor_birla2(corpus):
    regex = r"((Installment Premium\(Total\)|Installment Premium\(Tetal\)|Instalimant Premium\(Total\)|Instalimant Premium\(Tetal\)|Installment premium|Installment Premium))(INR|Inr|Rs.|RS.|Re.|Bs|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|¢|%|\(USD\) \$|£|€|=|)(.|..|...|....|.....|......|.......|............)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def find_premium_cor_birla3(corpus):
    regex = r"(Modal Premium with Service Tax)(INR|Inr|Rs.|RS.|Re.|Bs|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|¢|%|\(USD\) \$|£|€|=|)(.|..|...|....|.....|......|.......|............)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[3])
    else:
        return None

def find_name_cor_birla(corpus):
    regex = r"((Received with thanks from|Insured Name|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Name of Nominee|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Dear)(.|..|...|....)|)\b(Smt/Ms/Shri|SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Policyholder Name|Dear)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None
    
def find_policy_cor_birla(corpus):
    regex=r"\b(Policy|policy|PolicyNo|Proposal/Policy|Policy|Plicy|plicy|PolicyNumber|Poticy)\b(.|..|...|....)(No|Num|No\.|no\.|num|Number|number).*?(\b\d{9}\b)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[3])
    else:
        return None
    
def find_date_cor_birla(corpus):
    regex=r"((Date of Receipt|DATE OF RECEIPT|Date of Issue|DATE OF ISSUE|Date of Issuc|Date and Time|DATE AND TIME|Date...Time.|Date..Time...|Date...Time....|Receipt Date|INSURANCE Date:|Payment Date and Time :|Date|DATE|TNO|Tr\. No\.)(\s|)(.|..|...|....|.....)\b(((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4})(\b\s|-))(.|..|...|....|.....|)(\d{2}:\d{2})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[4])
    else:
        return None

In [1477]:
def information_extract_BIRLA(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus

    provider = "BIRLA SUN LIFE"
    
    policy_number = find_policy_cor_birla(corpus)

    
    premium_amt = find_premium_cor_birla1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_cor_birla2(corpus)
        if premium_amt is None:
            premium_amt = find_premium_cor_birla3(corpus)

    insured_name = find_name_cor_birla(corpus)
    if insured_name is None:
        insured_name = find_name(rendered)
        if insured_name is None:
            insured_name = find_name2(corpus)
            if insured_name is None:
                insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    premium_date = find_date_cor_birla(corpus)
    if premium_date is None:
        premium_date = find_if_date_and_time(corpus)    
        if premium_date is None:
            premium_date = date_def2(corpus)
    
    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)  
    
    payment_mode = pay_mode1(corpus)
    if payment_mode is None:
        payment_mode = pay_mode(corpus)
             
    #return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)
    return (provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### LIFE INSURANCE_EXIDE

In [1478]:
def find_premium_cor_exide(corpus):
    regex=r"((Total Premium Paid|This is to acknowledge the receipt of|receipt of f|receipt of|Amount paid|amount paid|Receipt Amount))(.|..|...|....)(INR|Inr|Rs.|RS.|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|)(\s|)(.*?)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[6])
    else:
        return None
    
def find_policy_cor_exide(corpus):
    regex = r"\b(POLICY|Policy|policy|Poticy|PoticyNo|Pol|pol|Pol\.|pol\.|Details|Proposal/Policy|Policy|Exide Life|life Plan)+\s(NO|No|Num|No\.|no\.|num|Number|number|)(\s|\s+|.|)(:|,|;|-|=|\s|)(|.|..|...|....)(\b\d{8}\b)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[5])
    else:
        return None

def find_name_cor_exide(corpus):
    regex = r"((Received with thanks from|Insured Name|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Name of Nominee|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Ufe Assured Name|Dear)(.|..|...|....)|)\b(Smt/Ms/Shri|SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Policyholder Name|Ufe Assured Name|Dear)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_date_cor_exide(corpus):
    regex = r"((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4}\b\s"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

In [1479]:
def information_extract_EXIDE(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus

    provider = "EXIDE LIFE"
    
     
    policy_number = find_policy_cor_exide(corpus)
#     if policy_number is None:   
#         policy_number = find_policyno_table(corpus)
#         if policy_number is None:    
#             policy_number = find_policy(corpus)
#             if policy_number is None:
#                 policy_number = find_policy_slash(corpus)
#                 if policy_number is None:
#                     policy_number = find_policy_until_number(corpus)

    
    premium_amt = find_premium_cor_exide(corpus)


    insured_name = find_name(rendered)
    if insured_name is None:
        insured_name = find_name2(corpus)
        if insured_name is None:
            insured_name = find_name3(corpus)
            if insured_name is None:
                insured_name = find_name_cor_exide(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)   
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)
        if premium_date is None:
            premium_date = find_date_cor_exide(corpus)
        
    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)  
    
    payment_mode = pay_mode1(corpus)
    if payment_mode is None:
        payment_mode = pay_mode(corpus)
             
    #return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)
    return (provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### LIFE INSURANCE_FUTURE GENERALI

In [1480]:
def find_policy_future_generali(corpus):
    regex = r"\b(Policy|POLICY|policy|Policy|Plicy|plicy|Poticy)\b(.|..|...|....|.....)(No|Num|No\.|no\.|num|Number|number).*?(\b\d{8}\b)"

    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[3])
    else:
        return None

def find_premium_cor_future_generali(corpus):
    regex = r"(\b\d{8}\b).*?(\d+\.\d+).*?(stands for Critical)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[1])
    else:
        return None
    
def find_date_cor_future_generali(corpus):
    regex=r"\b((Date|Date:|DATE|date|Date of Receipt|Date of issue|DATE OF ISSUE|Date and Time :|Receipt Date|INTEREST CERTIFICATE))\b( {0,1})( |:|)(( {0,1})(\d+|\w{3,})(-|/|\||\.|,|\s)(\d+|\w{3,})(-|/|\||\.|,|\s)(\d+|\w{3,})|(\d+(-)((?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?))(-)\d+)|\d{1,2}-\d{1,2}-\d{2,4}|((?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?))( |)\d{1,2}(, |,| ,| , )\d{2,4})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[4])
    else:
        return None

In [1481]:
def information_extract_FUTURE_GENERALI(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus

    provider = "FUTURE GENERALI"
    
    policy_number = find_policy_future_generali(corpus)

    premium_amt = find_premium_cor_future_generali(corpus)

    insured_name = find_name2(corpus)
    if insured_name is None:
        insured_name = find_name(rendered)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_date_cor_future_generali(corpus)
    if premium_date is None:
        premium_date = find_if_date_and_time(corpus)   
        if premium_date is None:
            premium_date = date_def2(corpus)
    
    
    payment_mode = pay_mode1(corpus)
    if payment_mode is None:
        payment_mode = pay_mode(corpus)
             
    #return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)
    return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)


### Life Insurance_HDFC

In [1482]:
def find_name_cor_hdfc(corpus):
    regex=r"((Received with thanks from|Insured Name|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Name of Nominee|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|life insurance policy held by|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Dear)(.|..|...|....)|)\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Dear|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|life insurance policy held by|life insurance policy hel by|life insurance policy he! by|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Policyholder Name)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

def find_premium_cor_hdfc1(corpus):
    regex=r"((Renewal Premium to be Paid|Premlum to be Paid|Your premium payment of INR))(INR|Inr|Rs.|RS.|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|)(.|..|...|....|.....|......|.......|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def find_premium_cor_hdfc2(corpus):
    regex=r"((Installment premium for insured|Premium Amount(.|..|...)Including Taxes|Premium Amount))(INR|Inr|Rs.|RS.|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|)(.|..|...|....|.....|......|.......|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[5])
    else:
        return None
    
def find_premium_cor_hdfc3(corpus):
    regex=r"((Section 80|Section 80|Section B0C|Section co))(INR|Inr|Rs.|RS.|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|)(.|..|...|....|.....|......|.......)(\b\d{2,6}\b)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def find_premium_cor_hdfc4(corpus):
    regex=r"((Premium))(.|..|...|....|.....|)(INR|Inr|Rs.|RS.|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|).{0,15}?(\d+(,|\.|)\d+(,|\.|)\d+(,|\.)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[4])
    else:
        return None    
    
def find_policy_cor_hdfc(corpus):
    regex=r"\b(Policy|policy|Pol|pol|Pol\.|pol\.|Details|Proposal/Policy|Proposal/Palicy|Proposal/Poli¢y|Policy|Plicy|plicy|towards the following)\b(\s)(No|Num|No\.|no\.|num|Number|number|).*?(\b\d{8,9}\b)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[3])
    else:
        return None
    

In [1483]:
def information_extract_HDFC(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus

    provider = "HDFC LIFE"
    
    policy_number = find_policy_cor_hdfc(corpus)

    premium_amt = find_premium_cor_hdfc1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_cor_hdfc2(corpus)
        if premium_amt is None:
            premium_amt = find_premium_cor_hdfc3(corpus)
            if premium_amt is None:
                premium_amt = find_premium_cor_hdfc4(corpus)            
                                

    insured_name = find_name_cor_hdfc(corpus)
    if insured_name is None:
        insured_name = find_name(rendered)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)   
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)
    
    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)  
    
    payment_mode = pay_mode1(corpus)
    if payment_mode is None:
        payment_mode = pay_mode(corpus)
             
    #return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)
    return (provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### LIFE INSURANCE_ICICI

In [1484]:
def find_name_cor_icici(corpus):
    regex=r"((Received with thanks from|Insured Name|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Name of Nominee|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Dear)(.|..|...|....)|)\b(Smt/Ms/Shri|SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Policyholder Name|Dear)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

def find_premium_cor_icici1(corpus):
    regex = r"((Total Premium Paid|Totel Premium Paid|Net Premium Paid|Net premium payable|Policy Total))(.|..|...|....)(INR|Inr|Rs.|RS.|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|)(.|..|...|....)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)|\d+)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_premium_cor_icici2(corpus):
    regex = r"\b(\d{8})(.|..|...|....)(\d+(,|\.|)\d+(,|\.)\d+(,|\.|)\d+)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))

    if len(result)!=0:
        return(result[2])
    else:
        return None

def find_policy_cor_icici1(corpus):
    regex = r"\b(Policy|POLICY|policy|Pol|pol|Pol\.|pol\.|Details|Proposal/Policy|Policy|Plicy|plicy|Product/Rider|ICICI Pru)\b(\s)(No|Num|No\.|no\.|num|Number|number|).*?(\d{8})\s"
#     regex = r"(\b(Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Details|Proposal/Policy|Policy)+\s(No|Num|No.|no.|num|Number|number).*?((\d\w\d+|\d+\w+\d+)(/|\\|\||-|)(\d+|\w[a-zA-Z]|))(\s))|\b(Policy|policy|Plicy|plicy)\b.*?((\d\w\d+|\d+\w+\d+)(/|\\|\||-|)(\d+|\w[a-zA-Z]|))(\s)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))

    if len(result)!=0:
        return(result[3])
    else:
        return None
    
def find_policy_cor_icici2(corpus):
    regex = r"\b\d{8}\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

In [1485]:
def information_extract_ICICI(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus

    provider = "ICICI PRUDENTIAL"

    policy_number = find_policy_cor_icici1(corpus)
    if policy_number is None:
        policy_number = find_policy_cor_icici2(corpus)

    
    premium_amt = find_premium_cor_icici1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_cor_icici2(corpus)


    insured_name = find_name_cor_icici(corpus)
    if insured_name is None:
        insured_name = find_name(rendered)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)   
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)
    
    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus) 

    payment_mode = pay_mode1(corpus)
    if payment_mode is None:
        payment_mode = pay_mode(corpus)
             
    #return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)
    return (provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)


### LIFE INSURANCE_KOTAK

In [1486]:
def find_premium_cor_kotak1(corpus):
    regex = r"((We acknowledge receipt of| We acknowledge with thanks(.|..|...)receipt of Rs))(INR|Inr|Rs.|RS.|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|)(.*?)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[5])
    else:
        return None
    
def find_premium_cor_kotak2(corpus):
    regex = r"((Total Instalment Premium|Instalment Premium|instalment Premium))(INR|Inr|Rs.|RS.|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|)(.*?)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def find_name_cor_kotak(corpus):
    regex=r"((Received with thanks from|Insured Name|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Name of Nominee|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Name and Address of Policyholder|Name of the Life assured|Policy Owner|To whomsoever it may concern)(.|..|...|....)|)\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Policyholder Name|Name and Address of Policyholder|Name of the Life assured|Policy Owner|To whomsoever it may concern)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    
def find_policy_cor_kotak(corpus):
    regex = r"\b(Policy|POLICY|policy|Policy|Plicy|plicy|Poticy)\b(.|..|...|....|.....)(No|Num|No\.|no\.|num|Number|number).*?(\b\d{8}\b)"

    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[3])
    else:
        return None
    
def find_date_cor_kotak(corpus):
    regex=r"((Date of Receipt|DATE OF RECEIPT|Date of Issue|DATE OF ISSUE|Date and Time|DATE AND TIME|Date...Time.|Date..Time...|Date...Time....|Date of Receipt|Receipt Date|INSURANCE Date:|Payment Date and Time :|Receipt Date)(\s|)(.{1,20})\b(((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4}\b\s))"
#     regex=r"((Date of Receipt|DATE OF ISSUE:|DATE OF ISSUE :|Date and Time|Date...Time.|Date..Time...|Date...Time....|Receipt Date|INSURANCE Date:|Payment Date and Time :|Date|DATE|TNO|Tr\. No\.)(\s|)(.|..|...|....|.....)\b((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4}\b\s)"
#     regex=r"((Date of Receipt|DATE OF ISSUE:|DATE OF ISSUE :|Date...Time.|Date..Time...|Date...Time....|Receipt Date|INSURANCE Date:|Payment Date and Time :|Date :|Date:|Date|TNO|Tr\. No\.)(\s|)(|:|)+.(\d+|\w+|\w)(-|/|\|.|,|\s)(\d+|\w+)(-|/|\|.|,|\s)(\d+|\w+))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[4])
    else:
        return None

In [1487]:
def information_extract_KOTAK(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus

    provider = "KOTAK LIFE"
    
    policy_number = find_policy_cor_kotak(corpus)
    
    premium_amt = find_premium_cor_kotak1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_cor_kotak2(corpus)


    insured_name = find_name_cor_kotak(corpus)
    if insured_name is None:
        insured_name = find_name(rendered)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_date_cor_kotak(corpus)   
    if premium_date is None:
        premium_date = date_def2(corpus)
    
    
    payment_mode = pay_mode1(corpus)
    if payment_mode is None:
        payment_mode = pay_mode(corpus)
             
    #return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)
    return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)


### LIFE INSURANCE_LIC

In [1488]:
def find_name_cor_lic1(corpus):
    regex=r"((Received with thanks from|Insured Name|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Name of Nominee|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|life insurance policy held by|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Life Assured|life insurance policies held by|life insurance policies held by Kr)(.|..|...|....)|)\b(Smt/Me /Sr iba|Smt/Ms\./Shri(|.|..|...|....) SHRI|Smit/Ms/Shri|Shri/Smt\.Smt|Smt/Ms/St|Smt/Ms/Shri|SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|Smt/Ms\./Shri|Mr\./Mrs\.|Mr/Ms|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|rt/MsJShri|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|The following premium has been received for life insurance policies of|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|life insurance policy held by|life insurance policy hel by|life insurance policy he! by|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Policyholder Name|life insurance policies held by)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Ali|ALI|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Joy|JOY|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Ali|ALI|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Joy|JOY|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
#     regex=r"(((Received with thanks from|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Name of Nominee|Life Insured..|Owner Name...|Dear Mr\.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of)(.|..|...|....)|)\b(Smt/Ms/Sr|Smt/Ms\./St|Smt/MsJ/Shri|Smt\./Ms\./Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|M/s|Mr\./Mrs\.|\bMr\b|\bMS\b|\bMR\b|Mr\.|mr\.|Ms|Ms\.|Miss|Mrs|Mrs\.|Prof|Mr\./Mrs\.|Dear MR\.|Prof\.|Smt\.|Ms\.|Shri|Sri|Sri\.|MR |insured Name|Policy Holder Name|Premium Payor Name|Name of Nominee|Policyholder's Name|Life Insured..|Owner Name...|Dear Mr\.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Received with thanks from|Proposer's)(.|..|...|....|)((\w+ \w(,|\.) \w+|\w+ \w+ \w+)|(\w+ \w+)|(\w\w+)+|(\w \w+ \w+|\w(\.|,|,\s|\s|\.\s)\w(\.|,|,\s|\s|\.\s)\w+)))"
#     regex=r"(((Received with thanks from|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Name of Nominee|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of)(.|..|...|....)|)(Smt\./Ms\./S
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

def find_name_cor_lic2(corpus):
    regex=r"\b(Smt/Me /Sr iba|Smt/Ms./Shri(|.|..|...|....)SHRI|Smt/Ms.J/Shri|Smit/Ms/Shri|Smt/Me /Sr|Smt\./Ms/Shri|Smt/Ms/Shri|SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|To Whom It May Concern|To Whomsoever It May Concern|To whomsoever it may concern)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Ali|ALI|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Joy|JOY|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Rks|RKS|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s\s|\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Ali|ALI|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Joy|JOY|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Rks|RKS|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    
def find_name_cor_lic3(corpus):
    regex=r"(Renewal Premium Receipt).{0,10}?(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,}).{0,7}?(Receipt Number)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[1])
    else:
        return None
    
def find_name_cor_lic4(corpus):
    regex=r"(Email).{1,60}?(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,}).{0,7}?(Amount)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[1])
    else:
        return None
    
def find_name_cor_lic5(corpus):
    regex=r"(ONLY|PAISE ONLY)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,}).*?This receipt is electronically generated"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[2])
    else:
        return None    

def find_premium_cor_lic0(corpus):
    regex = r"((Grand Total).*?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||).*?(\d+(,|\.|\d+)\d+(,|\.)\d+(,|\.|)(\d+|)))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[4])
    else:
        return None    
    
def find_premium_cor_lic1(corpus):
    regex = r"((Total amount said towards premium is|Total amount received towards premium for the policy number|Total amount received towards premium for the policies listed above is|amount received towards premium for the policies listed above is|We acknowledge with thanks|We acknowledge receipt of an amount|We acknowledge receipt of|Renewal Premium to be Paid|Received with thanks|received with thanks|Received withthanks|ceived with thanks|eived with thanks|RECEIVED WITH THANKS|Premium Received|Payment Amount(.|..|...)Rs.|Total amount received|Total Received|Grand|Total Premium|Total Amount|Total amount|Total|total|TOTAL|TO(.|..)TAL|TOA U\\ aRs).*?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||).*?(\d+(,|\.|\d+)\d+(,|\.)\d+(,|\.|)(\d+|)))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[6])
    else:
        return None
    
def find_premium_cor_lic2(corpus):
    regex = r"((Premium Amount|Prem|Instalment Premium|instalment premium|policies listed above is))(.|..|...|....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|)(\s|)\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_premium_cor_lic3(corpus):
# FOR AMOUNT PAID BY CASH BELOW TABLE FORMAT
    regex = r"(Paid by Cash.Card(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||).*?(\d+(,|\.)\d+(,|\.|)\d+(,|\.|)(\d+|)))(.|..|...|....|.....)(Paid by Cheque(.|..|...|....|.....)0\.00)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def find_premium_cor_lic4(corpus):
# FOR AMOUNT PAID BY CHEQUE BELOW TABLE FORMAT
    regex = r"(Paid by Cash.Card(.|..|...|....|.....)0\.00)(.|..|...|....|.....)(Paid by Cheque(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||).*?(\d+(,|\.)\d+(,|\.|)\d+(,|\.|)(\d+|)))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[7])
    else:
        return None

def find_date_cor_lic1(corpus):
## FOR LIC TABLE FORMAT TO FETCH THE RECEIVED DATE
    #(PREMIUM PAID STATEMENT FOR THE YEAR).*?(life insurance policies from the userid).*?\d{9}.*?((\d+(,|\.)\d+(,|\.|)\d+(,|\.|)(\d+|))(.|..|...)\b(((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{4})(\b\s|-))
    regex = r"(PREMIUM PAID STATEMENT FOR THE YEAR).*?(life insurance policies from the userid).*?\d{9}.*?((\d+(,|\.)\d+(,|\.|)\d+(,|\.|)(\d+|))(.|..|...)\b(((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{4})(\b\s|-))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[9])
    else:
        return None

def find_policy_cor_lic1(corpus):
    regex=r"\b(policy number|Policy|policy|PolicyNo|Proposal/Policy|Polley No|Policy|Policv|Plicy|plicy|PolicyNumber|Poticy|Palicy)\b(.|..|...|....)(No|Num|No\.|no\.|num|Number|number|).*?(\s\d{9}\b|\b\d{9}is\b)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[3])
    else:
        return None
    
def find_policy_cor_lic2(corpus):
    regex=r"\b(Towards the following).{0,10}?(\b\d{9}\b|\b\d{9}is\b)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[1])
    else:
        return None    
    
def find_cgst_cor_lic(corpus):
#     regex=r"RENEWAL PREMIUM RECEIPT.*?\b\d{9}\b.*?(\d+(,|\.)\d+(,|\.|)\d+(,|\.|)(\d+|)).*?(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)).*?(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)).*?\d{2}\/\d{2}\/\d{4}"
    regex=r"Page Total.{0,20}?(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|).{0,10}?\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)).{0,10}?(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[9])
    else:
        return None
    
def find_sgst_cor_lic(corpus):
    regex=r"Page Total.{0,20}?(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|).{0,10}?\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)).{0,10}?(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[9])
    else:
        return None
    
def find_GST_lic(corpus):
    regex = r"(Total amount said towards.{0,35}?GST)(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
#     regex = r"(\b(CGST|cGsT|CGST|cGST|GST|Goods & Services Tax|Goods & Service Tax|Service Tax)\b(.|)(\W\d\W|).(\W|)(.|)\d(.|)(\d+|\d))|(GST(.|)as(.|)applicable(.|)(\W|)\w+(\W|)(.|)(\d+|\d)(.|)(\d+|\d))|(?<=GST\(9.00%\))(.+?\d+.\d{2})|(?<=CGST\(9.00%\))(.+?\d+.\d{2})|(?<=GST @9%)(.+?\d+.\d{2})|(?<=CGST @9%)(.+?\d+.\d{2})"
    
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))        
    if len(result)!=0:
        return(result[5])
    else:
        return None  
    
def find_late_fee_lic(corpus):
    regex = r"Page Total.{0,20}?(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|).{0,10}?\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|).{0,10}?\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)).{0,10}?(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
#     regex = r"(\b(CGST|cGsT|CGST|cGST|GST|Goods & Services Tax|Goods & Service Tax|Service Tax)\b(.|)(\W\d\W|).(\W|)(.|)\d(.|)(\d+|\d))|(GST(.|)as(.|)applicable(.|)(\W|)\w+(\W|)(.|)(\d+|\d)(.|)(\d+|\d))|(?<=GST\(9.00%\))(.+?\d+.\d{2})|(?<=CGST\(9.00%\))(.+?\d+.\d{2})|(?<=GST @9%)(.+?\d+.\d{2})|(?<=CGST @9%)(.+?\d+.\d{2})"
    
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))        
    if len(result)!=0:
        return(result[13])
    else:
        return None      

In [1489]:
def information_extract_LIC(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus

    provider = "LIFE INSURANCE CORPORATION OF INDIA"
    
    policy_number = find_policy_cor_lic1(corpus)
    if policy_number is None:
        policy_number = find_policy_cor_lic2(corpus)
        
    premium_amt = find_premium_cor_lic0(corpus)
    if premium_amt is None:
        premium_amt = find_premium_cor_lic1(corpus)
        if premium_amt is None:
            premium_amt = find_premium_cor_lic2(corpus)
            if premium_amt is None:
                premium_amt = find_premium_cor_lic3(corpus)
                if premium_amt is None:
                    premium_amt = find_premium_cor_lic4(corpus)


    insured_name = find_name_cor_lic1(corpus)
    if insured_name is None:
        insured_name = find_name_cor_lic2(corpus)
        if insured_name is None:
            insured_name = find_name(rendered)   
            if insured_name is None:
                insured_name = find_name_cor_lic3(corpus)
                if insured_name is None:
                    insured_name = find_name_cor_lic4(corpus)
                    if insured_name is None:
                        insured_name = find_name_cor_lic5(corpus)

    #insured_name = natural_language(insured_name)
    
    premium_date = find_date_cor_lic1(corpus)
    if premium_date is None:
        premium_date = find_if_date_and_time(corpus)   
        if premium_date is None:
            premium_date = date_def2(corpus)


    
    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
            if CGST is None:
                CGST = find_cgst_cor_lic(corpus) 
                if CGST is None:
                    CGST = find_GST_lic(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
        if SGST is None:
            SGST = find_sgst_cor_lic(corpus)
            
    late_fee = find_late_fee_lic(corpus) 
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = pay_mode1(corpus)
    if payment_mode is None:
        payment_mode = pay_mode(corpus)
        
    remark3 = 'LATE FEE'
             
    #return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)
    return (provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,late_fee,remark1,remark2,remark3,payment_mode)

### LIFE INSURANCE_MAX LIFE

In [1490]:
def find_premium_cor_max_life1(corpus):
    regex=r"((Total Premium Received|Total Amount Received|Total Premium Receved|This is to acknowledge the receipt of|receipt of))(|.|..|...|....|.....)(INR|Inr|Rs\.|RS\.|Rs|RS|rs|eRe\.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|)(\s|).{0,50}?(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[5])
    else:
        return None
    
def find_premium_cor_max_life2(corpus):
    regex=r"((Total|Fotal))(|.|..|...|....)(INR|Inr|Rs\.|RS\.|Rs|RS|rs|eRe\.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|)(\s|).{0,50}?(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[5])
    else:
        return None
    
def find_premium_cor_max_life3(corpus):
    regex=r"((Premium Amount|Premium amount|premium amount))(.|..|...|....)(INR|Inr|Rs.|RS.|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|)(\s|)(.*?)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[6])
    else:
        return None
    
def find_policy_cor_max_life(corpus):
    regex=r"\b(POLICY|Policy|policy|Poticy|Poticy|Policy|Policy\.|Receipt)+\s(NO|No|Num|No\.|no\.|num|Number|number)(:|,|;|-|=|\s|)(.*?)\b(\d{9})\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def find_date_cor_max_life(corpus):
    regex=r"((Date of Receipt|DATE OF RECEIPT|Date of Issue|DATE OF ISSUE|Date of Issuc|Date and Time|DATE AND TIME|Date...Time.|Date..Time...|Date...Time....|Receipt Date|INSURANCE Date:|Payment Date and Time :|Date|DATE|TNO|Tr\. No\.)(\s|)(.|..|...|....|.....)\b(((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4}\b\s))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[4])
    else:
        return None

    

In [1491]:
def information_extract_MAX_LIFE(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus

    provider = "MAX LIFE"
    
    policy_number = find_policy_cor_max_life(corpus)
    
    premium_amt = find_premium_cor_max_life1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_cor_max_life2(corpus)
        if premium_amt is None:
            premium_amt = find_premium_cor_max_life3(corpus)

    insured_name = find_name2(corpus)
    if insured_name is None:
        insured_name = find_name(rendered)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_date_cor_max_life(corpus)
    if premium_date is None:
        premium_date = date_def2(corpus)
    
#     CGST = lic_CGST(corpus)
#     SGST = lic_SGST(corpus)
    
#     remark1 = lic_remark1(corpus)
#     remark2 = lic_remark2(corpus)
    
    payment_mode = pay_mode1(corpus)
    if payment_mode is None:
        payment_mode = pay_mode(corpus)
             
    #return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)
    return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)

### LIFE INSURANCE_RELIANCE

In [1492]:
def find_premium_cor_reliance(corpus):
    regex = r"(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))\s{1,}(Terms and Conditions|terms and conditions|Terms and conditions)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[0])
    else:
        return None
    
def find_policy_cor_reliance(corpus):
    regex = r"\b(Policy|POLICY|policy|Policy|Plicy|plicy|Poticy)\b(\s)(No|Num|No\.|no\.|num|Number|number|).*?(\b\d{8}\b)"

    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[3])
    else:
        return None
    
def find_name_cor_reliance(corpus):
    regex=r"((Received with thanks from|Insured Name|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Name of Nominee|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Name)(.|..|...|....)|)\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Policyholder Name|Name)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()


In [1493]:
def information_extract_RELIANCE(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus

    provider = "RELIANCE NIPPON"
    
    policy_number = find_policy_cor_reliance(corpus)
            
    premium_amt = find_premium_cor_reliance(corpus)


    insured_name = find_name_cor_reliance(corpus)
    if insured_name is None:
        insured_name = find_name3(corpus)

    
    premium_date = find_if_date_and_time(corpus)   
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)
    
    
    payment_mode = pay_mode1(corpus)
    if payment_mode is None:
        payment_mode = pay_mode(corpus)
             
    #return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)
    return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)


### LIFE INSURANCE_SBI

In [1494]:
def find_premium_cor_sbi1(corpus):
    regex = r"((Total Amount Paid|Installment premium|Installment Premium|Received with thanks|Reevived with thanks))(INR|Inr|Rs.|RS.|Re.|Bs|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|¢|%|\(USD\) \$|£|€|=|)(.|..|...|....|.....|......|.......|............)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def find_premium_cor_sbi2(corpus):
    regex = r"((Total Premium))(INR|Inr|Rs.|RS.|Re.|Bs|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|¢|%|\(USD\) \$|£|€|=|)(.|..|...|....|.....|......|.......|............)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def find_policy_cor_sbi(corpus):
    #regex = r" (Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Master olicy)+\s(No|Num|No.|no.|num|Number|number) \d+"
    regex=r"\b(POLICY|Policy|policy|Poticy|PoticyNo|Proposal/Policy|Policy|Vroposal)+\s(NO|No|Num|No\.|no\.|num|Number|number|)(\s|\s+|.|)(:|,|;|-|=|\s|)(.|..|...|....|.....|......)(\d\w\d+|\d+\w+\d+)(/|\\|\||)(\d+|\w+)"
#     regex=r"\b(POLICY|Policy|policy|P\.|p\.|P|p|Pol|pol|Pol\.|pol\.|Details|Proposal/Policy|Policy)+\s(NO|No|Num|No\.|no\.|num|Number|number|)(\s|\s+|.|)(:|,|;|-|=|\s|) (\d\w\d+|\d+\w+\d+)(/|\\|\||)(\d+|\w+)|((?<=Section 80C).*?(\d+))|((Remark(?:s)|remark(?:s)|REMARK(?:S))).*?\d+"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    
def find_name_cor_sbi(corpus):
    #regex = r" (Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Master olicy)+\s(No|Num|No.|no.|num|Number|number) \d+"
    regex=r"((Received with thanks from|Insured Name|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Name of Nominee|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Name of LifeAssured|Name of Life Resured)(.|..|...|....)|)\b(Smt/Ms/Shri|SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Policyholder Name|Name of LifeAssured|Name of Life Resured)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
#     regex=r"\b(POLICY|Policy|policy|P\.|p\.|P|p|Pol|pol|Pol\.|pol\.|Details|Proposal/Policy|Policy)+\s(NO|No|Num|No\.|no\.|num|Number|number|)(\s|\s+|.|)(:|,|;|-|=|\s|) (\d\w\d+|\d+\w+\d+)(/|\\|\||)(\d+|\w+)|((?<=Section 80C).*?(\d+))|((Remark(?:s)|remark(?:s)|REMARK(?:S))).*?\d+"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

In [1495]:
def information_extract_SBI(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus

    provider = "SBI"
    
    policy_number = find_policy_cor_sbi(corpus)

    premium_amt = find_premium_cor_sbi1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_cor_sbi2(corpus)


    insured_name = find_name_cor_sbi(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)   
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)
    
    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = pay_mode1(corpus)
    if payment_mode is None:
        payment_mode = pay_mode(corpus)
             
    #return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)
    return (provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### LIFE INSURANCE_TATA AIA

In [1496]:
def find_premium_cor_tata_aia(corpus):
    regex=r"((Your payment of Rs|Net Banking Amount|Total Modal Premium))(.|..|...|....)(INR|Inr|Rs.|RS.|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|)(\s|)(.*?)(\d+(,|\.)\d+(,|\.|)\d+(,|\.|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[6])
    else:
        return None
    
def find_policy_cor_tata_aia(corpus):
    regex=r"\b(Policy|policy|Pol|pol|Pol\.|pol\.|Proposal/Policy|Policy|Plicy|plicy|towards the following)\b(.|..|...|....|.....)(No|Num|No\.|no\.|num|Number|number).*?((\w|€|\()\d{9})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[3])
    else:
        return None

In [1497]:
def information_extract_TATA_AIA(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus

    provider = "TATA AIA"
    
    policy_number = find_policy_cor_tata_aia(corpus)

    premium_amt = find_premium_cor_tata_aia(corpus)

    insured_name = find_name2(corpus)
    if insured_name is None:
        insured_name = find_name(rendered)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)   
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)
    
    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = pay_mode1(corpus)
    if payment_mode is None:
        payment_mode = pay_mode(corpus)
             
    #return (provider,policy_number,premium_amt,insured_name,premium_date,payment_mode)
    return (provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### Mediclaim

In [1498]:
###################################################################PROVIDER

def find_provider_reg(corpus):
    regex=r"\b(STAR HEALTH AND ALLIED INSURANCE|STAR COMPREHENSIVE INSURANCE|National Insurance Company Limited|Star Health|STAR HEALTH|Apollo Munich|APOLLO MUNICH|Apollo|APOLLO|IFFCO TOKIO GENERAL INSURANCE|IFFCO TOKIO|IFFCO Tokio|IFFCO|STATE BANK OF INDIA|State Bank of India|SBI Life|SBI|sbi|EXIDE Life|Exide Life|Exide|EXIDE|ICICI PRUDENTIAL|ICICI Lombard|ICICI LOMBARD|ICICI|icici|Maxhealthcare|Max Life|MAX LIFE|max life|HDFC LIFE|HDFC Life|hdfc life|HDFC ERGO|hdfc ergo|HDFC Standard Life|HDFC ERGO General Insurance|HOFC Life|HDFC Mutual Fund|HDFC|hdfc|LIC of India|LIC OF INDIA|Life Insurance Corporation|LIFE INSURANCE CORPORATION|LIC|lic|kotak life|KOTAK LIFE|Kotak Life|Kotak Mahindra Life|KOTAK LIFE|KOTAK|kotak|NEW INDIA ASSURANCE|New India Assurance|NEW INDIA|Aegon Life|Aegon life|aegon life|aegon|AEGON|Bajaj Allianz|BAJAJ ALLIANZ|BAJAJ|bajaj|Aditya Birla Health Insurance|Birla Health Insurance|Aditya Birla|Birla Sun Life Mutual Fund|Birla Sun Life Insurance|Birla Sun Life|ABSL|BIRLA SUN LIFE|birla sun life|BIRLA SUN|Birla Sun|PUNJAB NATIONAL BANK|Punjab National Bank|Punjab National|punjab national|PNB MetLife India Insurance|PNB MetLife India|PNB MetLife|pnb metlife|PNB METLIFE|PNB|pnb|AVIVA|Aviva|aviva|Bharti axa|Bharti AXA|bharti axa|MAX BUPA|max bupa|Max Bupa|UTI|uti|Bank of Baroda|DHFL|Dhfl|IDBI|GIC|Cigna TTK|Cigna|Mumbai District Central bank|Mumbai District Central Co-op. Bank|Mumbai District Central|Mumbai.*?Co-op. Bank|LIC HFL|LIC Housing Finance Ltd|SARASWAT Bank|Saraswat Bank|SARASWAT|Saraswat|Royal Sundaram General|royal sundaram|Royal Sundaram|UNITED INDIA INSURANCE|United India|united india|ORIENTAL INSURANCE|Oriental Insurance|oriental insurance|National Mediclaim|L&T Mutual Fund|L&T|Axis Mutual Fund|Axis|axis|Motilal Oswal Mutual Fund|Motilal Oswal|RELIANCE Mutual fund|RELIANCE|Mahindra Mutual Fund|Mahindra|Franklin Templeton|Franklin India|DSP BLACKROCK|OSP BLACKROCK|DSP BlackRock|DSP|Dewan Housing Finance Corporation|Dewan Housing Finance|Dewan Housing|National Medictaim)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_provider_For_DCB(corpus):
    regex = r"\b(DCB Bank|DCB|dcb bank)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################INSURED NAME

# def med_name(corpus):
#     regex = r"((proposers(.|)Name(.|)(.|)((\w+|\w)(.|)(\w+|\w)(.|)(\w+))))"
#     test_str = corpus
#     matches = re.finditer(regex, test_str, re.MULTILINE)
    
#     for matchNum,match in enumerate(matches):
#         matchNum = matchNum + 1
#         if match:
#             return match.group()
#     return None

# def med_name1(corpus):
#     regex = r"(Name(.|insured Name|Insured Name|Insured Name|)(.|..|...|....)((Mrs|mrs|Mr|mr)|)(.|)((\w+|\w)(.|)(\w+|\w)|(\w+|\w)(.|)(\w+|\w)(.|)(\w+|\w)))"
# #     regex = r"(Name(.|)(.|)((Mrs|mrs|Mr|mr)|)(.|)((\w+|\w)(.|)(\w+|\w)|(\w+|\w)(.|)(\w+|\w)(.|)(\w+|\w)))"
#     test_str = corpus
#     matches = re.finditer(regex, test_str, re.MULTILINE)
    
#     for matchNum,match in enumerate(matches):
#         matchNum = matchNum + 1
#         if match:
#             return match.group()
#     return None

# def med_name2(corpus):
#     regex=r"(M/s(.|)(.|)(.|)(.|)\w+(.|)\w+)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()

###################################################################POLICY NUMBER

def med_policy1(corpus):
    regex=r"(policy reference|Policy|POLICY|policy|Polley|PSlicy)(.|..|...|)(Number|number|NUMBER|NO|no|No).{0,10}?((\d+\W+\d+|\w+(/)\d+|\w+\d+).*?\s)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[3])
    else:
        return None

# def med_policy1(corpus):
#     regex=r"(Policy|POLICY|policy|Polley)(.|..|...|)(Number|number|NUMBER|NO|no|No)(.|..|...|....|.....)((\d+\W+\d+|\w+(/)\d+|\w+\d+).*?\s)"
# #     regex=r"(Policy|POLICY|policy|Polley)(.|)(Number|number|NUMBER|NO|no|No)(.|..|...|....)(\w+.*?\s)"
# #     regex = r" (Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Master olicy)+\s(No|Num|No.|no.|num|Number|number) \d+"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()

def med_policy2(corpus):
    #regex = r" (Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Master olicy)+\s(No|Num|No.|no.|num|Number|number) \d+"
    regex=r"(Policy|POLICY|policy)(.|..|...|)(Number|number|NUMBER|NO|no|No)(.|..|...|....|.....|)(\w+.*?\s|(\d+|\d)(.|)(\d+|\d|\w+\w)(.|)(\w+|\w|\d+|\d)(.|)(\d+|\d))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    
    
###################################################################AMOUNT
    
def med_prem_amt(corpus):
    regex = r"\b((Total Premium|Gross Premium|has paid|Total Amount).*?)(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|¥|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|\d)\d+(,|\.)\d+(,|\.|)(\d+|)|(\d+(.|..)\d+))"
    
    test_str = corpus
    matches = re.finditer(regex, test_str, re.MULTILINE)
    
    for matchNum,match in enumerate(matches):
        matchNum = matchNum + 1
        if match:
            return match.group()
    return None

###################################################################GST

# def med_CGST0(corpus):
#     regex=r"(sGsT/UTGST|CGST|cGsT|CGST|cGST|cest|C@ST|SGST|sGsT|SGsT|S@ST|SGST|sGST|SQST_|SQST|CaST_|CaST)\b(.|..|...|....|.....)(INR|Inr|Rs\.|RS\.|Rs|RS|R8\.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
#     #regex=r"\b(CGST|cGsT|CGST|cGST|GST|IGST)\b(.....|....|...|..|.)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\|)(.|..|...|....|.....)(\d+(,|\.|)\d+(,|\.|)\d+)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     result = []
#     for matchNum, match in enumerate(matches, start=1):

#         #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

#         for groupNum in range(0, len(match.groups())):
#             groupNum = groupNum + 1

#             result.append(match.group(groupNum))
            
#     if len(result)!=0:
#         return(result[5])
#     else:
#         return None
      
    
# def med_CGST1(corpus):
#     regex=r"(CGST\(9\.00%\)|CGST @9.00% 2|CGST @9.00%|CGST @9%|GST\(9\.00%\)|GST @9%|Central Tax 9%|State Tax 9%)(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
#     #regex=r"\b(CGST|cGsT|CGST|cGST|GST|IGST)\b(.....|....|...|..|.)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\|)(.|..|...|....|.....)(\d+(,|\.|)\d+(,|\.|)\d+)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     result = []
#     for matchNum, match in enumerate(matches, start=1):
#         for groupNum in range(0, len(match.groups())):
#             groupNum = groupNum + 1
#             result.append(match.group(groupNum))           
#     if len(result)!=0:
#         return(result[5])
#     else:
#         return None
    
# def med_CGST2(corpus):
#     regex = r"(Integrated Goods and Service Tax \(18\.00 %\)|Service Tax \(18\.00 \%\)|GST\(18\.00%\)|Goods & Services Tax|Goods & Service Tax|Service Tax|GST)(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
# #     regex = r"(\b(CGST|cGsT|CGST|cGST|GST|Goods & Services Tax|Goods & Service Tax|Service Tax)\b(.|)(\W\d\W|).(\W|)(.|)\d(.|)(\d+|\d))|(GST(.|)as(.|)applicable(.|)(\W|)\w+(\W|)(.|)(\d+|\d)(.|)(\d+|\d))|(?<=GST\(9.00%\))(.+?\d+.\d{2})|(?<=CGST\(9.00%\))(.+?\d+.\d{2})|(?<=GST @9%)(.+?\d+.\d{2})|(?<=CGST @9%)(.+?\d+.\d{2})"
    
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     result = []
#     for matchNum, match in enumerate(matches, start=1):
#         for groupNum in range(0, len(match.groups())):
#             groupNum = groupNum + 1
#             result.append(match.group(groupNum))        
#     if len(result)!=0:
#         return(result[5])
#     else:
#         return None


# def med_SGST1(corpus):
#     regex=r"(sGsT/UTGST|UTGST|SGST|sGsT|SGsT|S@ST|SGST|sGST|SBC|SQST_|SQST|KKC|CGST|cGsT|CGST|cGST|cest|CaST_|CaST|Goods & Services Tax|Goods & Service Tax|Service Tax)\b(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     result = []
#     for matchNum, match in enumerate(matches, start=1):
#         for groupNum in range(0, len(match.groups())):
#             groupNum = groupNum + 1
#             result.append(match.group(groupNum))
#     if len(result)!=0:
#         return(result[5])
#     else:
#         return None

# def med_SGST2(corpus):    
#     regex = r"(SGST/UTGST @9%|SGST\(9.00%\)|SGST @9.00% 2|SGST @9.00%|SGST @9%|Service Tax \(18\.00 \%\)|CGST\(9\.00%\)|CGST @9.00% 2|CGST @9.00%|CGST @9%|GST\(9\.00%\)|GST @9%|GST\(18\.00%\)|Central Tax 9%|State Tax 9%)(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     result = []
#     for matchNum, match in enumerate(matches, start=1):
#         for groupNum in range(0, len(match.groups())):
#             groupNum = groupNum + 1
#             result.append(match.group(groupNum))    
#     if len(result)!=0:
#         return(result[5])
#     else:
#         return None    
# def med_SGST3(corpus):
    
#     regex = r"((UTGST|SGST(.|)(\W\d\W|).(\W|)(.|)\d(.|)(\d+|\d))|(SBC(.|)(\W|)(.|)KKC(.|)(\W|.|)(\w+)(\W|.|)(.|)(\d+|\d)(.|)(\d+|\d)))"
    
#     test_str = corpus
#     matches = re.finditer(regex, test_str, re.MULTILINE)
    
#     for matchNum,match in enumerate(matches):
#         matchNum = matchNum + 1
#         if match:
#             return match.group()
#     return None
    
###################################################################REMARK

# def med_remark1(corpus):
#     corpus1 = corpus.lower()
#     regex=r"(cgst|sgst|utgst|sgst/utgst|c@st|s@st|sqst(|.|..))\b"
#     matches = re.finditer(regex, corpus1, re.MULTILINE)
#     for match in matches:
#         return ("CGST")
#     return None

# def med_remark2(corpus):
#     corpus1 = corpus.lower()
#     regex=r"(GsT|GST|Integrated Goods and Service Tax|Central Goods and Service Tax|Goods & Services Tax|Goods & Service Tax|Service Tax)\b"
#     matches = re.finditer(regex, corpus1, re.MULTILINE)
#     for match in matches:
#         return ("GST")
#     return None

# def med_remark3(corpus):
#     corpus1 = corpus.lower()
#     regex=r"(cgst|sgst|utgst|sgst/utgst|c@st|s@st|sqst(|.|..)|sbc|kkc)\b"
#     matches = re.finditer(regex, corpus1, re.MULTILINE)
#     for match in matches:
#         return ("SGST")
#     return None

# def med_remark4(corpus):
#     regex=r"(UTGST|SGST|sGsT|SGST|sGST|SBC|KKC)\b"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

In [1499]:
def information_extract_MEDICLAIM_def(corpus,rendered):
    
    provider = find_provider_reg(corpus)
    if provider is None:
        provider = find_provider_lower(corpus)
        if provider is None:
            provider = find_provider_For_DCB(corpus)
    
    policy_number = med_policy1(corpus)
    if policy_number is None:
#         policy_number = med_policy2(corpus)
#         if policy_number is None:
        policy_number = find_policy(corpus)
        if policy_number is None:
            policy_number = find_policy_slash1(corpus)
            if policy_number is None:
                policy_number = find_policy_slash2(corpus)
                            
    premium_amt = amt_lic_1(corpus)
    if premium_amt is None:
        premium_amt = amt_lic_2(corpus)
        if premium_amt is None:
            premium_amt = find_premium(rendered)
            if premium_amt is None:
                premium_amt = med_prem_amt(corpus)
                premium_amt = premium_amt
                if premium_amt is None:
                    premium_amt = prem_amt(corpus)
                    premium_amt=premium_amt
                    if premium_amt is None:
                        premium_amt = prem_amt1(corpus)
                        premium_amt=premium_amt
                        if premium_amt is None:
                            premium_amt = find_premium_cor1(corpus)
                            if premium_amt is None:
                                premium_amt = find_premium_cor1_1(corpus)
                                if premium_amt is None:
                                    premium_amt = find_premium_cor2(corpus)


#     insured_name = med_name2(corpus)
#     if insured_name is None:
#         insured_name = find_name(rendered)
#         if insured_name is None:
#             insured_name = med_name(corpus)
#             insured_name = insured_name
#             if insured_name is None:
#                 insured_name = med_name1(corpus)
#                 insured_name = insured_name
#                 if insured_name is None:
    insured_name = find_name2(corpus)
    if insured_name is None:
        insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)
    
    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = 'Yearly'
             
    return(provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### MEDICLAIM_ADITYA_BIRLA

In [1500]:
def find_name_MEDICLAIM_aditya_birla(corpus):
    regex = r"\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Policyholder Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Insured Person and|This is to certify that|Thisis to certify that|is to certify that|Client Name|proposed by)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_date_MEDICLAIM_aditya_birla(corpus):
    regex=r"((Start Date of Policy & Time 00:00 hrs on|Start Date of Policy & Time  00:00 hrs on|Start Date of Policy.*?hrs on)(\s|)(.|..|...|....|.....)\b(((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4}\b\s))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[4])
    else:
        return None

    
# def find_premium_MEDICLAIM_birla0(corpus):
#     regex=r"(Total Premium Received|Total Premia Received|Total Premium).*?(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))(.|..|...)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))(.|..|...)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)).{1,30}Note(.....|....|...|..|.)Income Tax"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     result = []
#     for matchNum, match in enumerate(matches, start=1):

#         #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

#         for groupNum in range(0, len(match.groups())):
#             groupNum = groupNum + 1

#             result.append(match.group(groupNum))
            
#     if len(result)!=0:
#         return(result[13])
#     else:
#         return None
    
def find_premium_MEDICLAIM_birla0(corpus):
    regex=r"(Total Premium Received|Total Premia Received|Total Premium).*?(\d+(,|\.)\d+(,|\.|)\d+(,|\.|)(\d+|).*?\d+(,|\.)\d+(,|\.|)\d+(,|\.|)(\d+|).*?\d+(,|\.)\d+(,|\.|)\d+(,|\.|)(\d+|)).*?(\d+(,|\.)\d+(,|\.|)\d+(,|\.|)(\d+|)).*?(Note|Income Tax|Inceme Tax|Incomé Tax)"
#     regex=r"(Total Premium Received|Total Premia Received|Total Premium).*?(\d+(,|\.|\d+)\d+(,|\.|\d+)\d+(,|\.|\d+)(\d+|)|\d+\.\d+).*?(\d+(,|\.|\d+)\d+(,|\.|\d+)\d+(,|\.|\d+)(\d+|)|\d+\.\d+).*?(\d+(,|\.|\d+)\d+(,|\.|\d+)\d+(,|\.|\d+)(\d+|)|\d+\.\d+).*?(\d+(,|\.|\d+)\d+(,|\.|\d+)\d+(,|\.|\d+)(\d+|)|\d+\.\d+).*?Note(.....|....|...|..|.)(Income Tax|Inceme Tax|Incomé Tax)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[14])
    else:
        return None    

def find_premium_MEDICLAIM_birla1(corpus):
    regex=r"(Total Received|Gross Premium)(.|..|...|....|.....|......|) (INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[5])
    else:
        return None

def find_premium_MEDICLAIM_birla2(corpus):
    regex = r"(We confirm receipt of premium amount|We confirm .*? amount|receipt of premium amount|receipt of .*? amount).*?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[4])
    else:
        return None



def find_premium_MEDICLAIM_birla3(corpus):
    regex=r"(Year wise proportionate Premium).*?(\d{1,4}-\d{1,4}).*?(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[2])
    else:
        return None

def find_premium_MEDICLAIM_birla4(corpus):
    regex=r"Total Premium.*?(\d+\.\d+|NA).*?(\d+\.\d+|NA).*?(\d+\.\d+|NA).*?(\d+\.\d+|NA).*?(\d+\.\d+|NA).*?(\d+\.\d+|NA).*?(\d+\.\d+|NA).*?(\d+\.\d+|NA)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[7])
    else:
        return None
    
def find_premium_MEDICLAIM_birla5(corpus):
    regex=r"(Amount % Amount).*?\b(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)).*?\b((\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))|NA).*?(%).*?\b(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[13])
    else:
        return None
    
def find_premium_MEDICLAIM_birla6(corpus):
    regex=r"Amount.*?of first Policy.*?(%).*?(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[1])
    else:
        return None
    
def find_premium_MEDICLAIM_birla7(corpus):
    regex = r"(Received with thanks an amount of Rs|amount of Rs)\.\s(.*?)\s(Only|only)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None
    
def find_cgst_MEDICLAIM_birla0(corpus):
    regex=r"(cGsT|CGST|cgst|cGST).*?(\d+\.\d+|NA).*?(\d+\.\d+|NA).*?(\d+\.\d+|NA).*?(\d+\.\d+|NA).*?(\d+\.\d+|NA)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[5])
    else:
        return None
    
def find_cgst_MEDICLAIM_birla1(corpus):
    regex=r"(Total Premium Received|Total Premia Received|Total Premium).*?(\d+(,|\.|\d+)\d+(,|\.|\d+)\d+(,|\.|\d+)(\d+|)|\d+\.\d+).*?(\d+(,|\.|\d+)\d+(,|\.|\d+)\d+(,|\.|\d+)(\d+|)|\d+\.\d+).*?(\d+(,|\.|\d+)\d+(,|\.|\d+)\d+(,|\.|\d+)(\d+|)|\d+\.\d+).*?(\d+(,|\.|\d+)\d+(,|\.|\d+)\d+(,|\.|\d+)(\d+|)|\d+\.\d+).*?Note(.....|....|...|..|.)(Income Tax|Inceme Tax|Incomé Tax)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[11])
    else:
        return None    
    
    
def find_sgst_MEDICLAIM_birla(corpus):
    regex=r"(UTGST|SGST|sGsT|SGST|sGST|sgst|SBC|KKC|CGST|cGsT|CGST|cGST).*?(\d+\.\d+|NA).*?(\d+\.\d+|NA).*?(\d+\.\d+|NA).*?(\d+\.\d+|NA).*?(\d+\.\d+|NA).*?(\d+\.\d+|NA)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[6])
    else:
        return None

In [1501]:
def information_extract_MEDICLAIM_ADITYA_BIRLA(corpus,rendered):
    
    provider = "ADITYA BIRLA"
    
    policy_number = med_policy1(corpus)
    if policy_number is None:
#         policy_number = med_policy2(corpus)
#         if policy_number is None:
        policy_number = find_policy(corpus)
        if policy_number is None:
            policy_number = find_policy_slash1(corpus)
            if policy_number is None:
                policy_number = find_policy_slash2(corpus)
    
    premium_amt = find_premium_MEDICLAIM_birla0(corpus)
    if premium_amt is None:
        premium_amt = find_premium_MEDICLAIM_birla1(corpus)
        if premium_amt is None:
            premium_amt = find_premium_MEDICLAIM_birla2(corpus)
            if premium_amt is None:
                premium_amt = find_premium_MEDICLAIM_birla3(corpus)
                if premium_amt is None:
                    premium_amt = find_premium_MEDICLAIM_birla4(corpus)
                    if premium_amt is None:
                        premium_amt = find_premium_MEDICLAIM_birla5(corpus)
                        if premium_amt is None:
                            premium_amt = find_premium_MEDICLAIM_birla6(corpus)

#     insured_name = med_name2(corpus)
#     if insured_name is None:
#         insured_name = find_name(rendered)
#         if insured_name is None:
#             insured_name = med_name(corpus)
#             insured_name = insured_name
#             if insured_name is None:
#                 insured_name = med_name1(corpus)
#                 insured_name = insured_name
#                 if insured_name is None:
    insured_name = find_name_MEDICLAIM_aditya_birla(corpus)
    if insured_name is None:
        insured_name = find_name2(corpus)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_date_MEDICLAIM_aditya_birla(corpus)
    if premium_date is None:
        premium_date = find_if_date_and_time(corpus)
        premium_date=premium_date
        if premium_date is None:
            premium_date = date_def2(corpus)

    CGST = find_cgst_MEDICLAIM_birla0(corpus)
    if CGST is None:
        CGST = find_cgst_MEDICLAIM_birla1(corpus)
        if CGST is None:
            CGST = CGST0(corpus)
            if CGST is None:
                CGST = CGST1(corpus)
                if CGST is None:
                    CGST = CGST2(corpus)
    
    SGST = find_sgst_MEDICLAIM_birla(corpus)
    if SGST is None:
        SGST = SGST0(corpus)
        if SGST is None:
            SGST = SGST1(corpus)
#             if SGST is None:
#                 SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = 'Yearly'
             
    return(provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### MEDICLAIM_APOLLO

In [1502]:
def find_name_MEDICLAIM_apollo(corpus):
    regex = r"\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Policyholder Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Insured Person and|This is to certify that|Thisis to certify that|is to certify that|Client Name|proposed by)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_premium_MEDICLAIM_apollo1(corpus):
    regex = r"(Gross Premium|gross premium|Gross premium|has paid Rs|HAS PAID RS)(.|..|...|....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[5])
    else:
        return None

def find_premium_MEDICLAIM_apollo2(corpus):
    regex = r"\(Rs\.\)(....|...|..|.)\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)(....|...|..|.)Rupees"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_cgst_MEDICLAIM_apollo(corpus):
    regex=r"(cGsT|CGST|cgst|cGST|CCST|GST).{1,10}\s(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[1])
    else:
        return None
    
def find_sgst_MEDICLAIM_apollo(corpus):
    regex=r"(UTGST|SGST|sGsT|SGST|sGST|sgst|SBC|KKC|CGST|cGsT|CGST|cGST).{1,10}\s(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[1])
    else:
        return None
    
def find_date_MEDICLAIM_apollo(corpus):
    regex=r"((Policy issuance date|policy issuance date)(\s|).{0,10}?\b(((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4})(\b\s|-|,))"
#     regex=r"((Date of Receipt|DATE OF ISSUE:|DATE OF ISSUE :|Date and Time|Date...Time.|Date..Time...|Date...Time....|Receipt Date|INSURANCE Date:|Payment Date and Time :|Date|DATE|TNO|Tr\. No\.)(\s|)(.|..|...|....|.....)\b((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4}\b\s)"
#     regex=r"((Date of Receipt|DATE OF ISSUE:|DATE OF ISSUE :|Date...Time.|Date..Time...|Date...Time....|Receipt Date|INSURANCE Date:|Payment Date and Time :|Date :|Date:|Date|TNO|Tr\. No\.)(\s|)(|:|)+.(\d+|\w+|\w)(-|/|\|.|,|\s)(\d+|\w+)(-|/|\|.|,|\s)(\d+|\w+))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))
    if len(result)!=0:
        return(result[3])
    else:
        return None    

In [1503]:
def information_extract_MEDICLAIM_APOLLO(corpus,rendered):
    
    provider = "APOLLO MUNICH"
    
    policy_number = med_policy1(corpus)
    if policy_number is None:
#         policy_number = med_policy2(corpus)
#         if policy_number is None:
        policy_number = find_policy(corpus)
        if policy_number is None:
            policy_number = find_policy_slash1(corpus)
            if policy_number is None:
                policy_number = find_policy_slash2(corpus)
                            
    premium_amt = find_premium_MEDICLAIM_apollo1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_MEDICLAIM_apollo2(corpus)

#     insured_name = med_name2(corpus)
#     if insured_name is None:
#         insured_name = find_name(rendered)
#         if insured_name is None:
#             insured_name = med_name(corpus)
#             insured_name = insured_name
#             if insured_name is None:
#                 insured_name = med_name1(corpus)
#                 insured_name = insured_name
#                 if insured_name is None:
    insured_name = find_name_MEDICLAIM_apollo(corpus)
    if insured_name is None:
        insured_name = find_name2(corpus)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_date_MEDICLAIM_apollo(corpus)
    if premium_date is None:
        premium_date = find_if_date_and_time(corpus)
        if premium_date is None:
            premium_date = date_def2(corpus)

    CGST = find_cgst_MEDICLAIM_apollo(corpus)
    if CGST is None:
        CGST = CGST0(corpus)
        if CGST is None:
            CGST = CGST1(corpus)
            if CGST is None:
                CGST = CGST2(corpus)
    
    SGST = find_sgst_MEDICLAIM_apollo(corpus)
    if SGST is None:
        SGST = SGST0(corpus)
        if SGST is None:
            SGST = SGST1(corpus)
#             if SGST is None:
#                 SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = 'Yearly'
             
    return(provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### MEDICLAIM_BAJAJ

In [1504]:
def find_name_MEDICLAIM_bajaj(corpus):
    regex = r"\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Policyholder Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Insured Person and|This is to certify that|Thisis to certify that|is to certify that|Client Name|proposed by|Proposer Name)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_premium_MEDICLAIM_bajaj(corpus):
    regex = r"(Gross Premium|gross premium|Gross premium|has paid|HAS PAID|Total Amount|Total amount)(.|..|...|....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

In [1505]:
def information_extract_MEDICLAIM_BAJAJ(corpus,rendered):
    
    provider = "BAJAJ ALLIANZ"
    
    policy_number = med_policy1(corpus)
    if policy_number is None:
#         policy_number = med_policy2(corpus)
#         if policy_number is None:
        policy_number = find_policy(corpus)
        if policy_number is None:
            policy_number = find_policy_slash1(corpus)
            if policy_number is None:
                policy_number = find_policy_slash2(corpus)
                            
    premium_amt = find_premium_MEDICLAIM_bajaj(corpus)

#     insured_name = med_name2(corpus)
#     if insured_name is None:
#         insured_name = find_name(rendered)
#         if insured_name is None:
#             insured_name = med_name(corpus)
#             insured_name = insured_name
#             if insured_name is None:
#                 insured_name = med_name1(corpus)
#                 insured_name = insured_name
#                 if insured_name is None:
    insured_name = find_name_MEDICLAIM_bajaj(corpus)
    if insured_name is None:
        insured_name = find_name2(corpus)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)

    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = 'Yearly'
             
    return(provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### MEDICLAIM_CIGNA

In [1506]:
def find_premium_MEDICLAIM_cigna1(corpus):
    regex = r"(Receipt Amount|Receipt amount|receipt amount|has paid the premium of)(.....|....|...|..|.)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\|)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_premium_MEDICLAIM_cigna2(corpus):
    regex=r"(Receipt Amount|Receipt Amount.*?|receipt amount|has paid the premium of)(.|..|...|....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\|)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[5])
    else:
        return None

In [1507]:
def information_extract_MEDICLAIM_CIGNA(corpus,rendered):
    
    provider = "CIGNA TTK"
    
    policy_number = med_policy1(corpus)
    if policy_number is None:
#         policy_number = med_policy2(corpus)
#         if policy_number is None:
        policy_number = find_policy(corpus)
        if policy_number is None:
            policy_number = find_policy_slash1(corpus)
            if policy_number is None:
                policy_number = find_policy_slash2(corpus)
                            
    premium_amt = find_premium_MEDICLAIM_cigna1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_MEDICLAIM_cigna2(corpus)

#     insured_name = med_name2(corpus)
#     if insured_name is None:
#         insured_name = find_name(rendered)
#         if insured_name is None:
#             insured_name = med_name(corpus)
#             insured_name = insured_name
#             if insured_name is None:
#                 insured_name = med_name1(corpus)
#                 insured_name = insured_name
#                 if insured_name is None:
    insured_name = find_name2(corpus)
    if insured_name is None:
        insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)

    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = 'Yearly'
             
    return(provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### MEDICLAIM_HDFC_ERGO

In [1508]:
def find_premium_MEDICLAIM_hdfc_ergo(corpus):
    regex = r"(Total Premium \(Including Taxes\)|Total Premium|Total premium|TOTAL PREMIUM|total premium|ial Premium|we have received an amount of)(|.|..|...|....|.....|\s{0,20}?)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum)) 
    if len(result)!=0:
        return(result[5])
    else:
        return None

def find_policy_MEDICLAIM_hdfc_ergo(corpus):
    regex=r"(Policy|POLICY|policy|Polley)(\s|.|)(Number|number|NUMBER|NO|no|No)(.|..|...|....|.....)(\d+(\s)\d+(\s)\d+(\s)\d+(\s)\d+).*?\s"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def find_name_MEDICLAIM_hdfc_ergo(corpus):
    regex=r"((Received with thanks from|Insured Name|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Name of Nominee|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Life Assured)(.|..|...|....)|)\b(Smt/Ms/Shri|SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Mr/Ms|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|issued to|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Policyholder Name|Name of the Proposer|ime of the Proposer)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()    

In [1509]:
def information_extract_MEDICLAIM_HDFC_ERGO(corpus,rendered):
    
    provider = "HDFC ERGO working"

    policy_number = find_policy_MEDICLAIM_hdfc_ergo(corpus)
    if policy_number is None:            
        policy_number = med_policy1(corpus)
        if policy_number is None:
    #         policy_number = med_policy2(corpus)
    #         if policy_number is None:
            policy_number = find_policy(corpus)
            if policy_number is None:
                policy_number = find_policy_slash1(corpus)
                if policy_number is None:
                    policy_number = find_policy_slash2(corpus)
                            
    premium_amt = find_premium_MEDICLAIM_hdfc_ergo(corpus)

#     insured_name = med_name2(corpus)
#     if insured_name is None:
#         insured_name = find_name(rendered)
#         if insured_name is None:
#             insured_name = med_name(corpus)
#             insured_name = insured_name
#             if insured_name is None:
#                 insured_name = med_name1(corpus)
    insured_name = find_name2(corpus)
    if insured_name is None:
        insured_name = find_name_MEDICLAIM_hdfc_ergo(corpus)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)

    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = 'Yearly'
             
    return(provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### MEDICLAIM_ICICI

In [1510]:
def find_name_MEDICLAIM_icici(corpus):
    regex = r"(Policy Holder Name|Policyholder\'s Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Policyholder Name|Policyhalder's Name|Account Name|Life Insured|Life insured|Owner Name|Name of the Life Assured|Insured's Name|Name of the Policy Holder|Proposer Name)(.|..|...|....)\b(Sh\./Smt\./ M/s|Smt/Ms/Shri|SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Mr/Ms|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

# def find_premium_cor_icici1(corpus):
#     regex = r"(Total Premium Paid|total premium paid|Total Premium Paid \(2\))(.....|....|...|..|.)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

# def find_premium_MEDICLAIM_icici1(corpus):
#     regex=r"(Total Premium Paid|total premium paid|Total Premium Paid \(2\)|Total Premium Paid \(%\))((.{1,10})(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)))"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     result = []
#     for matchNum, match in enumerate(matches, start=1):

#         #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

#         for groupNum in range(0, len(match.groups())):
#             groupNum = groupNum + 1

#             result.append(match.group(groupNum))
            
#     if len(result)!=0:
#         return(result[1])
#     else:
#         return None

def find_premium_MEDICLAIM_icici(corpus):
    regex=r"(Total Premium.{0,10}?\d+\.\d+.*?\d+\.\d+.*?\d+\.\d+).*? (\d+) .{0,40}?(Agent Details|The product|Important)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[1])
    else:
        return None
    
def find_tax_icici(corpus):
    regex=r"Total Premium.{0,10}?\d+\.\d+.*? (\d+\.\d+.) *?\d+\.\d+.*? (\d+) .{0,40}?(Agent Details|The product|Important)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[0])
    else:
        return None    

In [1511]:
def information_extract_MEDICLAIM_ICICI(corpus,rendered):
    
    provider = "ICICI LOMBARD"
    
    policy_number = med_policy1(corpus)
    if policy_number is None:
#         policy_number = med_policy2(corpus)
#         if policy_number is None:
        policy_number = find_policy(corpus)
        if policy_number is None:
            policy_number = find_policy_slash1(corpus)
            if policy_number is None:
                policy_number = find_policy_slash2(corpus)
                            
#     premium_amt = find_premium_MEDICLAIM_icici1(corpus)
#     if premium_amt is None:
    premium_amt = find_premium_MEDICLAIM_icici(corpus)

#     insured_name = med_name2(corpus)
#     if insured_name is None:
#         insured_name = find_name(rendered)
#         if insured_name is None:
#             insured_name = med_name(corpus)
#             insured_name = insured_name
#             if insured_name is None:
#                 insured_name = med_name1(corpus)
#                 insured_name = insured_name
#                 if insured_name is None:
    insured_name = find_name_MEDICLAIM_icici(corpus)
    if insured_name is None:
        insured_name = find_name2(corpus)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)

    CGST = find_tax_icici(corpus)
    if CGST is None:
        CGST = CGST0(corpus)
        if CGST is None:
            CGST = CGST1(corpus)
            if CGST is None:
                CGST = CGST2(corpus)
        
        
    SGST = find_tax_icici(corpus)
    if SGST is None:        
        SGST = SGST0(corpus)
        if SGST is None:
            SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = 'Yearly'
             
    return(provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)


### MEDICLAIM_IFFCO

In [1512]:
def find_name_MEDICLAIM_iffco(corpus):
    regex = r"\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Name)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_premium_MEDICLAIM_iffco(corpus):
    regex = r"((Total Amount|Total amount|total amount|TOTAL AMOUNT|Total Received|total received|Installment Premium\(Total\)|Installment Premium\(Tetal\)|Instalimant Premium\(Total\)|Instalimant Premium\(Tetal\)))(.|..|...|....)(INR|Inr|Rs.|RS.|Rs|RS|rs|eRe.|\(Rs\.\)|\$|\*¥|¥|%|\(USD\) \$|£|€|=|)(\s|)\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

In [1513]:
def information_extract_MEDICLAIM_IFFCO(corpus,rendered):
    
    provider = "IFFCO"
    
    policy_number = med_policy1(corpus)
    if policy_number is None:
#         policy_number = med_policy2(corpus)
#         if policy_number is None:
        policy_number = find_policy(corpus)
        if policy_number is None:
            policy_number = find_policy_slash1(corpus)
            if policy_number is None:
                policy_number = find_policy_slash2(corpus)
                            
    premium_amt = find_premium_MEDICLAIM_iffco(corpus)

#     insured_name = med_name2(corpus)
#     if insured_name is None:
#         insured_name = find_name(rendered)
#         if insured_name is None:
#             insured_name = med_name(corpus)
#             insured_name = insured_name
#             if insured_name is None:
#                 insured_name = med_name1(corpus)
#                 insured_name = insured_name
#                 if insured_name is None:
    insured_name = find_name_MEDICLAIM_iffco(corpus)
    if insured_name is None:
        insured_name = find_name2(corpus)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)

    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = 'Yearly'
             
    return(provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### MEDICLAIM_MAX

In [1514]:
def find_date_MEDICLAIM_max(corpus):
    regex=r"(Location.*?)((Date)(\s|)(.|..|...|....|.....)\b(((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4}\b\s))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[5])
    else:
        return None

def find_premium_MEDICLAIM_max1(corpus):
    regex = r"(Gross Premium|Gross premium|GROSS PREMIUM|gross premium|Sum of|Sum OF|sum of|Sg OF)(|.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(2\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_premium_MEDICLAIM_max2(corpus):
    regex = r"\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)(.....|....|...|..|.)(Rupees|Rupee|Ruppes)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_policy_MEDICLAIM_max(corpus):
    regex=r"Policy Number.{0,15}?\b(\d{12,16}\b)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[0])
    else:
        return None

In [1515]:
def information_extract_MEDICLAIM_MAX(corpus,rendered):
    
    provider = "MAX BUPA"
    
    policy_number = find_policy_MEDICLAIM_max(corpus)
                            
    premium_amt = find_premium_MEDICLAIM_max1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_MEDICLAIM_max2(corpus)

#     insured_name = med_name2(corpus)
#     if insured_name is None:
#         insured_name = find_name(rendered)
#         if insured_name is None:
#             insured_name = med_name(corpus)
#             insured_name = insured_name
#             if insured_name is None:
#                 insured_name = med_name1(corpus)
#                 insured_name = insured_name
#                 if insured_name is None:
    insured_name = find_name2(corpus)
    if insured_name is None:
        insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_date_MEDICLAIM_max(corpus)
    if premium_date is None:
        premium_date = find_if_date_and_time(corpus)
        premium_date=premium_date
        if premium_date is None:
            premium_date = date_def2(corpus)

    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = 'Yearly'
             
    return(provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### MEDICLAIM_NATIONAL_MEDICLAIM

In [1516]:
def find_name_MEDICLAIM_NATIONAL_MEDICLAIM(corpus):
    regex = r"\b(This is to certify that|Customer Name)\b(.|..|...|....|.....)\b(Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_policy_MEDICLAIM_NATIONAL_MEDICLAIM(corpus):
    regex=r"Policy Number.{0,70}?\b(\d{16,}\b)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[0])
    else:
        return None
    
def find_premium_MEDICLAIM_NATIONAL_MEDICLAIM1(corpus):
    corpus1 = corpus.lower()
    regex=r"(total amount|motal amount)(|.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(2\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus1, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[5])
    else:
        return None
    
def find_premium_MEDICLAIM_NATIONAL_MEDICLAIM2(corpus):
#AMOUNT IN WORDS
    regex = r"(RUPEES|Rupees)(.*?)(ONLY|Only|only)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        amt_wrds = result[1]
        amt_wrds= amt_wrds.replace('.',' ')
        amt_wrds= amt_wrds.replace(',',' ')
        return(str(w2n.word_to_num(amt_wrds)))
    else:
        return None
    



In [1517]:
def information_extract_MEDICLAIM_NATIONAL_MEDICLAIM(corpus,rendered):
    
    provider = "NATIONAL INSURANCE"

    policy_number = find_policy_MEDICLAIM_NATIONAL_MEDICLAIM(corpus)
    if policy_number is None:    
        policy_number = med_policy1(corpus)
        if policy_number is None:
    #         policy_number = med_policy2(corpus)
    #         if policy_number is None:
            policy_number = find_policy(corpus)
            if policy_number is None:
                policy_number = find_policy_slash1(corpus)
                if policy_number is None:
                    policy_number = find_policy_slash2(corpus)
                            
    premium_amt = find_premium_MEDICLAIM_NATIONAL_MEDICLAIM1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_MEDICLAIM_NATIONAL_MEDICLAIM2(corpus)
#         if premium_amt is None:
#             premium_amt = find_premium(rendered)
#             if premium_amt is None:
#                 premium_amt = med_prem_amt(corpus)
#                 premium_amt = premium_amt
#                 if premium_amt is None:
#                     premium_amt = prem_amt(corpus)
#                     premium_amt=premium_amt
#                     if premium_amt is None:
#                         premium_amt = prem_amt1(corpus)
#                         premium_amt=premium_amt
#                         if premium_amt is None:
#                             premium_amt = find_premium_cor1(corpus)
#                             if premium_amt is None:
#                                 premium_amt = find_premium_cor1_1(corpus)
#                                 if premium_amt is None:
#                                     premium_amt = find_premium_cor2(corpus)


#     insured_name = med_name2(corpus)
#     if insured_name is None:
#         insured_name = find_name(rendered)
#         if insured_name is None:
#             insured_name = med_name(corpus)
#             insured_name = insured_name
#             if insured_name is None:
#                 insured_name = med_name1(corpus)
#                 insured_name = insured_name
#                 if insured_name is None:
    insured_name = find_name_MEDICLAIM_NATIONAL_MEDICLAIM(corpus)
    if insured_name is None:
        insured_name = find_name2(corpus)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)
    
    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = 'Yearly'
             
    return(provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### MEDICLAIM_NEW_INDIA

In [1518]:
def find_name_MEDICLAIM_new_india1(corpus):
    regex = r"\s(Customer Name|CUSTOMER NAME|customer name|Name of the customer|Name of the Customer|Customer's name|Customers Name|Proposer Name|ProposersName|Insured Name)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_name_MEDICLAIM_new_india2(corpus):
    regex = r"\b(Mr\./Mrs\. MISS|Mr\./Mrs\. MR\.\s|SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None


def find_premium_MEDICLAIM_new_india1(corpus):
    regex = r"(Total Amount|TOTAL AMOUNT|Total amount|total amount|has paid Rs|HAS PAID RS|has paid < RUPEES|has paid RUPEES|has paid)(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(2\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[5])
    else:
        return None

def find_premium_MEDICLAIM_new_india2(corpus):
    regex = r"(Total|total)(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|©|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(2\)|\(zy\)|\)|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)).*?(Utilization details|Your Payment)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def find_premium_MEDICLAIM_new_india3(corpus):

    regex = r"(Transaction Amount|transaction amount|Transaction amount|TRANSACTION AMOUNT|Transaction amt|Transaction Amt)(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(2\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[5])
    else:
        return None    
    
def find_premium_MEDICLAIM_new_india4(corpus):
#AMOUNT IN WORDS
    regex = r"(has paid|Has Paid|HAS PAID|Has paid).{0,10}?(RUPEES|Rupees)(.*?)(ONLY|Only)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        amt_wrds = result[2]
        return(str(w2n.word_to_num(amt_wrds)))
    else:
        return None


In [1519]:
def information_extract_MEDICLAIM_NEW_INDIA(corpus,rendered):
    
    provider = "NEW INDIA ASSURANCE"
    
    policy_number = med_policy1(corpus)
    if policy_number is None:
#         policy_number = med_policy2(corpus)
#         if policy_number is None:
        policy_number = find_policy(corpus)
        if policy_number is None:
            policy_number = find_policy_slash1(corpus)
            if policy_number is None:
                policy_number = find_policy_slash2(corpus)
                            
    premium_amt = find_premium_MEDICLAIM_new_india1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_MEDICLAIM_new_india2(corpus)
        if premium_amt is None:
            premium_amt = find_premium_MEDICLAIM_new_india3(corpus)
            if premium_amt is None:
                premium_amt = find_premium_MEDICLAIM_new_india4(corpus)

#     insured_name = med_name2(corpus)
#     if insured_name is None:
#         insured_name = find_name(rendered)
#         if insured_name is None:
#             insured_name = med_name(corpus)
#             insured_name = insured_name
#             if insured_name is None:
#                 insured_name = med_name1(corpus)
#                 insured_name = insured_name
#                 if insured_name is None:

    insured_name = find_name_MEDICLAIM_new_india1(corpus)
    if insured_name is None:
        insured_name = find_name_MEDICLAIM_new_india2(corpus)
        if insured_name is None:
            insured_name = find_name2(corpus)
            if insured_name is None:
                insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)
#     premium_date=premium_date
#     if premium_date is None:
#         premium_date = date_def2(corpus)

    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = 'Yearly'
             
    return(provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### MEDICLAIM_ORIENTAL

In [1520]:
def find_premium_MEDICLAIM_oriental1(corpus):
    regex = r"(Total|TOTAL|total)(.....|....|...|..|.)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(2\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_premium_MEDICLAIM_oriental2(corpus):
# AMOUNT IN WORDS
    regex = r"(Premium in words).{0,10}?(RUPEES|Rupees)(.*?)(ONLY|Only)"
    matches = re.finditer(regex, corpus, re.MULTILINE)

    result = []
    for matchNum, match in enumerate(matches, start=1):

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        amt_wrds = result[2]
        return(str(w2n.word_to_num(amt_wrds)))
    else:
        return None
    
def find_policy_MEDICLAIM_oriental1(corpus):
    regex=r"(policy reference|Policy|POLICY|policy|Polley|PSlicy)(.|..|...|)(Number|number|NUMBER|NO|no|No).{0,10}?((\d+\W+\d+|\w+(/)\d+|\w+\d+).*?\s)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[3])
    else:
        return None    
    
def find_policy_MEDICLAIM_oriental2(corpus):
    regex=r"\b(Policy|policy|Proposal/Policy|Policy|Plicy|plicy)\b(\s)(No|Num|No\.|no\.|num|Number|number).*?\s(\d+\/\d+)\s"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[3])
    else:
        return None    

In [1521]:
def information_extract_MEDICLAIM_ORIENTAL(corpus,rendered):
    
    provider = "ORIENTAL INSURANCE"
    
    policy_number = find_policy_MEDICLAIM_oriental1(corpus)
    if policy_number is None:
        policy_number = find_policy_MEDICLAIM_oriental2(corpus)

                            
    premium_amt = find_premium_MEDICLAIM_oriental1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_MEDICLAIM_oriental2(corpus)

#     insured_name = med_name2(corpus)
#     if insured_name is None:
#         insured_name = find_name(rendered)
#         if insured_name is None:
#             insured_name = med_name(corpus)
#             insured_name = insured_name
#             if insured_name is None:
#                 insured_name = med_name1(corpus)
#                 insured_name = insured_name
#                 if insured_name is None:
    insured_name = find_name2(corpus)
    if insured_name is None:
        insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)
#     premium_date=premium_date
#     if premium_date is None:
#         premium_date = date_def2(corpus)

    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = 'Yearly'
             
    return(provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### MEDICLAIM_RELIGARE

In [1522]:
def find_premium_MEDICLAIM_religare1(corpus):
    regex = r"(Total|TOTAL|total|Premium Paid|Premium paid|PREMIUM PAID|premium paid)(.....|....|...|..|.)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(2\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_premium_MEDICLAIM_religare2(corpus):
    regex = r"(Midnight|midnight).*?((INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(2\)|\(zy\)|\|)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[5])
    else:
        return None

def find_date_MEDICLAIM_religare(corpus):
    regex=r"\b(00:00 hrs|CARE|SARE|Floater|Individual).*?(\b((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4}\b).*?(Midnight|midnight)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[1])
    else:
        return None
    
def find_policy_MEDICLAIM_religare(corpus):
    regex=r"\b(Policy|policy|Proposal/Policy|Policy|Plicy|plicy)\b(\s)(No|Num|No\.|no\.|num|Number|number|).*?(\b\d{8,9}\b)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        return(result[3])
    else:
        return None

In [1523]:
def information_extract_MEDICLAIM_RELIGARE(corpus,rendered):
    
    provider = "RELIGARE"
    
    
    policy_number = find_policy_MEDICLAIM_religare(corpus)
    if policy_number is None:
        policy_number = med_policy1(corpus)
        if policy_number is None:
    #         policy_number = med_policy2(corpus)
    #         if policy_number is None:
            policy_number = find_policy(corpus)
            if policy_number is None:
                policy_number = find_policy_slash1(corpus)
                if policy_number is None:
                    policy_number = find_policy_slash2(corpus)
                            
    premium_amt = find_premium_MEDICLAIM_religare1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_MEDICLAIM_religare2(corpus)

#     insured_name = med_name2(corpus)
#     if insured_name is None:
#         insured_name = find_name(rendered)
#         if insured_name is None:
#             insured_name = med_name(corpus)
#             insured_name = insured_name
#             if insured_name is None:
#                 insured_name = med_name1(corpus)
#                 insured_name = insured_name
#                 if insured_name is None:
    insured_name = find_name2(corpus)
    if insured_name is None:
        insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date =find_date_MEDICLAIM_religare(corpus)
    if premium_date is None:
        premium_date = find_if_date_and_time(corpus)

    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = 'Yearly'
             
    return(provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### MEDICLAIM_ROYAL_SUNDARAM

In [1524]:
def find_name_MEDICLAIM_royal_sundaram(corpus):
    regex = r"\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Policyholder Name|Policy Holder|Proposer Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Insured Person and|Client Name|proposed by)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_premium_MEDICLAIM_royal_sundaram1(corpus):
    regex=r"(\d+(,|\.|)\d+(,|\.|)\d+)\s{1,}This is to certify that"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[0])
    else:
        return None

def find_premium_MEDICLAIM_royal_sundaram2(corpus):
    regex = r"(Gross Premium|Gross premium|GROSS PREMIUM|gross premium|has received an amount of|received an amount of|received an amount)(.....|....|...|..|.)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|¥|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(2\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_gst_MEDICLAIM_royal_sundaram(corpus):
    regex=r"(\d+(,|\.|)\d+(,|\.|)\d+) (\d+(,|\.|)\d+(,|\.|)\d+)\s{1,}This is to certify that"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[0])
    else:
        return None

In [1525]:
def information_extract_MEDICLAIM_ROYAL_SUNDARAM(corpus,rendered):
    
    provider = "ROYAL SUNDARAM GENERAL"
    
    policy_number = med_policy1(corpus)
    if policy_number is None:
#         policy_number = med_policy2(corpus)
#         if policy_number is None:
        policy_number = find_policy(corpus)
        if policy_number is None:
            policy_number = find_policy_slash1(corpus)
            if policy_number is None:
                policy_number = find_policy_slash2(corpus)
                            
    premium_amt = find_premium_MEDICLAIM_royal_sundaram1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_MEDICLAIM_royal_sundaram2(corpus)

#     insured_name = med_name2(corpus)
#     if insured_name is None:
#         insured_name = find_name(rendered)
#         if insured_name is None:
#             insured_name = med_name(corpus)
#             insured_name = insured_name
#             if insured_name is None:
#                 insured_name = med_name1(corpus)
#                 insured_name = insured_name
#                 if insured_name is None:
    
    insured_name = find_name_MEDICLAIM_royal_sundaram(corpus)
    if insured_name is None:
        insured_name = find_name2(corpus)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)

    CGST = find_gst_MEDICLAIM_royal_sundaram(corpus)
    if CGST is None:
        CGST = CGST0(corpus)
        if CGST is None:
            CGST = CGST1(corpus)
            if CGST is None:
                CGST = CGST2(corpus)            
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = 'Yearly'
             
    return(provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### MEDICLAIM_STAR_HEALTH

In [1526]:
def find_name_MEDICLAIM_star_health(corpus):
    regex = r"\s(This is to certify that|THIS IS TO CERTIFY THAT|Customer Name|CUSTOMER NAME|customer name|Name of the customer|Name of the Customer|Customer's name|Customers Name|Proposer Name|ProposersName)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_date_MEDICLAIM_star_health(corpus):
    regex=r"(Collection Date|Date|issued on|Receipt No & Date(.|..|...)\d{0,10}|ReceiptNo &Date(.|..|...)\d{0,10}|ReceiptDste)(\s|)(.|..|...|....|.....)\b(((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4}\b\s)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[5])
    else:
        return None

def find_premium_MEDICLAIM_star_health1(corpus):
    regex=r"(Amount Collected|Amount collected|amount collected|AMOUNT COLLECTED)(...|..|.)(Mode of Pay|Mode of pay|MODE OF PAY)(.|..|...|)(Bank Name|Bank name|BANK NAME) .*?(\d+).*? (\W\w+|\w+)(   |  | )(\W\w+|\w+)(   |  | )(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))(   |  | )(\W\w+|\w+)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[10])
    else:
        return None

def find_premium_MEDICLAIM_star_health2(corpus):
    regex = r"(Total Premium|Total premium|TOTAL PREMIUM|total premium|has paid Rs|HAS PAID RS|Amount Collected|Amount collected|Amount collected|AMOUNT COLLECTED|Total Invoice Value \(in Figures\)|Total Invoice Value)(.|..|...|....|.....|)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|¥|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(2\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[5])
    else:
        return None
    
def find_policy_MEDICLAIM_star_health(corpus):
    regex=r"(policy reference|Policy|POLICY|policy|Polley|PSlicy)(.|..|...|)(Number|number|NUMBER|NO|no|No|).{0,10}?((\d+\W+\d+|\w+(/)\d+|\w+\d+).*?\s)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[3])
    else:
        return None    

In [1527]:
def information_extract_MEDICLAIM_STAR_HEALTH(corpus,rendered):
    
    provider = "STAR HEALTH & ALLIED"
    
    policy_number = find_policy_MEDICLAIM_star_health(corpus)
    if policy_number is None:
        policy_number = med_policy1(corpus)
        if policy_number is None:
    #         policy_number = med_policy2(corpus)
    #         if policy_number is None:
            policy_number = find_policy(corpus)
            if policy_number is None:
                policy_number = find_policy_slash1(corpus)
                if policy_number is None:
                    policy_number = find_policy_slash2(corpus)
                            
    premium_amt = find_premium_MEDICLAIM_star_health1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_MEDICLAIM_star_health2(corpus)

#     insured_name = med_name2(corpus)
#     if insured_name is None:
#         insured_name = find_name(rendered)
#         if insured_name is None:
#             insured_name = med_name(corpus)
#             insured_name = insured_name
#             if insured_name is None:
#                 insured_name = med_name1(corpus)
#                 insured_name = insured_name
#                 if insured_name is None:


    insured_name = find_name_MEDICLAIM_star_health(corpus)
    if insured_name is None:
        insured_name = find_name2(corpus)
        if insured_name is None:
            insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_date_MEDICLAIM_star_health(corpus)
    if premium_date is None:
        premium_date = find_if_date_and_time(corpus)
        premium_date=premium_date
        if premium_date is None:
            premium_date = date_def2(corpus)

    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
            
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    payment_mode = 'Yearly'
             
    return(provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### MEDICLAIM_UNITED_INDIA

In [1528]:
def find_premium_MEDICLAIM_united_india1(corpus):
    regex = r"(Total Amount|Total amount|TOTAL AMOUNT|total amount|Total|TOTAL|total)(.....|....|...|..|.)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|¥|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(2\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(\.)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_premium_MEDICLAIM_united_india2(corpus):
    regex = r"(Total \(Rounded Off\)|TOTAL \(Rounded Off\)|total \(Rounded Off\))(.....|....|...|..|.)(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|¥|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(2\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(\.)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

In [1529]:
def information_extract_MEDICLAIM_UNITED_INDIA(corpus,rendered):
    
    provider = "UNITED INDIA INSURANCE"
    
    policy_number = med_policy1(corpus)
    if policy_number is None:
#         policy_number = med_policy2(corpus)
#         if policy_number is None:
        policy_number = find_policy(corpus)
        if policy_number is None:
            policy_number = find_policy_slash1(corpus)
            if policy_number is None:
                policy_number = find_policy_slash2(corpus)
                            
    premium_amt = find_premium_MEDICLAIM_united_india1(corpus)
    if premium_amt is None:
        premium_amt = find_premium_MEDICLAIM_united_india2(corpus)

#     insured_name = med_name2(corpus)
#     if insured_name is None:
#         insured_name = find_name(rendered)
#         if insured_name is None:
#             insured_name = med_name(corpus)
#             insured_name = insured_name
#             if insured_name is None:
#                 insured_name = med_name1(corpus)
#                 insured_name = insured_name
#                 if insured_name is None:
    insured_name = find_name2(corpus)
    if insured_name is None:
        insured_name = find_name3(corpus)
    #insured_name = natural_language(insured_name)
    
    premium_date = find_if_date_and_time(corpus)
    premium_date=premium_date
    if premium_date is None:
        premium_date = date_def2(corpus)

    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)

    
    payment_mode = 'Yearly'
             
    return(provider,policy_number,premium_amt,insured_name,premium_date,CGST,SGST,remark1,remark2,payment_mode)

### Mutual Funds

In [1530]:
###################################################################PROVIDER

def mutual_provider(corpus):
    regex=r"\b(STAR HEALTH AND ALLIED INSURANCE|STAR COMPREHENSIVE INSURANCE|Star Health|STAR HEALTH|Apollo Munich|APOLLO MUNICH|Apollo|APOLLO|STATE BANK OF INDIA|State Bank of India|SBI Life|SBI|sbi|EXIDE Life|Exide Life|Exide|EXIDE|ICICI PRUDENTIAL|ICICI Lombard|ICICI LOMBARD|ICICI|icici|Max Life|MAX LIFE|max life|HDFC LIFE|HDFC Life|hdfc life|HDFC ERGO|hdfc ergo|HDFC Standard Life|HDFC ERGO General Insurance|HOFC Life|HDFC Mutual Fund|HDFC|hdfc|LIC of India|LIC OF INDIA|Life Insurance Corporation|LIFE INSURANCE CORPORATION|LIC|lic|kotak life|KOTAK LIFE|Kotak Life|Kotak Mahindra Life|KOTAK LIFE|KOTAK|kotak|NEW INDIA ASSURANCE|New India Assurance|NEW INDIA|Aegon Life|Aegon life|aegon life|aegon|AEGON|Bajaj Allianz|BAJAJ ALLIANZ|BAJAJ|bajaj|Birla Sun Life Mutual Fund|Birla Sun Life Insurance|Birla Sun Life|ABSL|BIRLA SUN LIFE|birla sun life|BIRLA SUN|Birla Sun|PUNJAB NATIONAL BANK|Punjab National Bank|Punjab National|punjab national|PNB MetLife India Insurance|PNB MetLife India|PNB MetLife|pnb metlife|PNB METLIFE|PNB|pnb|AVIVA|Aviva|aviva|Bharti axa|Bharti AXA|bharti axa|MAX BUPA|max bupa|Max Bupa|UTI|uti|Bank of Baroda|DHFL|Dhfl|IDBI|GIC|Mumbai District Central bank|Mumbai District Central Co-op. Bank|Mumbai District Central|Mumbai.*?Co-op. Bank|LIC HFL|LIC Housing Finance Ltd|SARASWAT Bank|Saraswat Bank|SARASWAT|Saraswat|Royal Sundaram General|royal sundaram|Royal Sundaram|UNITED INDIA INSURANCE|United India|united india|ORIENTAL INSURANCE|Oriental Insurance|oriental insurance|National Mediclaim|L&T Mutual Fund|L&T|Axis Mutual Fund|Axis|axis|Motilal Oswal Mutual Fund|Motilal Oswal|RELIANCE Mutual fund|RELIANCE|Mahindra Mutual Fund|Mahindra|Franklin Templeton|Franklin India|DSP BLACKROCK|OSP BLACKROCK|DSP BlackRock|DSP|Dewan Housing Finance Corporation|Dewan Housing Finance|Dewan Housing|fisdom|MIRAE ASSET|mirae asset|Mirae Asset|IDFC MUTUAL FUND|IDFC|BNP PARIBAS)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()


def find_provider_For_DCB(corpus):
    regex = r"\b(DCB Bank|DCB|dcb bank)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################INSURED NAME

def mutual_payee_name1(corpus):
    regex = r"\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Investor Name|This is to certify that|THIS IS TO CERTIFY THAT|this is to certify that|To Whom It May Concern|To Whomsoever It May Concern|To whomsoever it may concern|Dear Mr|Dear|DEAR|First Unit Holder|First Holder|Holder Name)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\{ |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\{ |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\{ |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    
# def mutual_payee_name2(corpus):
#     regex = r"(\w+.\w+.\w+|\w+.\w+)(.|..|...|....)(Mode (of|Of|0f) Holding|Status|Tax Status).*?(\d+|Flat|House|Chawl|Chaw!)"
# #     regex = r"(Mode of Holding|Status|Folio No(.|..|...|....)\d+(.|..|...|....)\d+(.|..|...|....)Statement Date(.|..|...|....)\d+.\w+.\d+|Date(.|..|...|....)\d+.\w+.\d+|Folio No(.|..|...|....)\d+(.|..|...|....)\d+).*?(\d+|Flat|House|Chawl|Chaw!)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()

def mutual_payee_name2(corpus):
    #AXIS 1
    regex = r"(Statement Date).*?(Folio No).{0,20}?(For the period).{0,30}?(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,}).{0,5}?(Mode of Holding)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[3])
    else:
        return None  
    
def mutual_payee_name3(corpus):
    #AXIS 2
    regex = r"(FolioNO).{0,40}?(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,}).{0,5}?(Statement Date).{0,20}(Axis)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[1])
    else:
        return None  
    
def mutual_payee_name4(corpus):
    #Reliance
    regex = r"(Not Transferable).{0,50}?(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,}).{0,100}?(Mode of Holding|Mode)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[1])
    else:
        return None  
    
def mutual_payee_name5(corpus):
    #DSP
    regex = r"(service@dspim).{0,110}?(Folio No).{0,50}?(\d+( |)(/|\\| |)( |)\d+( |)(/|\\| |)( |)\d+).{0,10}?(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,}).{0,10}?(Page 2 of 2|Statement Date)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[8])
    else:
        return None  
    
def mutual_payee_name6(corpus):
    regex = r"(BNP PARIBAS).{0,50}?(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,}).{0,10}?(Statement Period)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[1])
    else:
        return None  

###################################################################FOLIO NUMBER
    
def mutual_folio_number(corpus):
    corpus1 = corpus.lower()
    regex = r"(folio no \(mandatory\)|follo no 2|follo no 1|follo no|folio no|folio number|foliono|folionumber|io number).{0,25}?(\d+( |)(/|\\|\d)( |)\d+( |)(/|\\|\d|\))( |)\d+)"
#     regex = r"(Folio No|Folio no|Folio Number|FOLIO NO|FOLIO NUMBER|folio no|folio number|Folio number|FolioNo).*?((\d+(.|)(/|\\| )(.|)\d+)|\d+)"
#     regex = r"(Folio No|Folio no|Folio Number|FOLIO NO|FOLIO NUMBER|folio no|folio number|Folio number).*?((\d+(.|)(/|\\)(.|)\d+)|\d+)"
    matches = re.finditer(regex, corpus1, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[1])
    else:
        return None  
    
###################################################################DATE
    
def mutual_statement_date(corpus):
    regex = r"(Statement Date|STATEMENT DATE|tatement Date|statement date|Statement date|tatement date).{0,20}?\b(((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4})(\b\s|-)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[1])
    else:
        return None  

###################################################################SCHEME

def mutual_scheme(corpus):
    regex = r"(TaxSaver-Growth|Tax Saver Fund - Reg-G|Tax Saver Fund - Reg -G|\(Tax Saving\) - Growth|Long Term Equity Fund Growth|Long Term Equity Fund - Growth|Long Term Equity Fund|TAX SAVER|Tax Relief|Tax Saver Fund|Tax saver fund)"
#     regex = r"(HDFC TaxSaver-Growth|Tax Saver Fund - Reg-G|ICICI Prudential Long Term Equsy Fund \(Tax Saving\) - Growth|Axis Long Term Equity Fund Growth|RELIANCE TAX SAVER (ELSS) FUND - GROWTH PLAN GROWTH OPTION (TSGPG)|RELIANCE TAX SAVER|ABSL Tax Relief '96 Fund-ELSS|ABSL Tax Relief '96 Fund-ELSS - Dividend|ABSL Tax Relief '96 Fund-ELSS - Dividend -Payout|ABSL Tax Relief '96 Fund-ELSS - Growth|ABSL Tax Relief)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

###################################################################AMOUNT
    
def mutual_amount1(corpus):
    regex=r"(Grand Total).{0,30}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||\(|\)|\[|\]|\{|\}|)(|.|..|...|....|.....)\s(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def mutual_amount2(corpus):
    regex=r"(Total|Tota).{0,30}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||\(|\)|\[|\]|\{|\}|)(|.|..|...|....|.....)\s(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
    
def check_date(a):
    try:
        TODAY_CHECK = datetime.datetime.strptime(a, "%d/%m/%Y").strftime('%d/%m/%Y')
    except ValueError as e:
        return None
    return TODAY_CHECK

def check_date1(a):
    try:
        TODAY_CHECK = datetime.datetime.strptime(a, "%d%m/%Y").strftime('%d/%m/%Y')
    except ValueError as e:
        return None
    return TODAY_CHECK

def check_date2(a):
    try:
        TODAY_CHECK = datetime.datetime.strptime(a, "%d/%m%Y").strftime('%d/%m/%Y')
    except ValueError as e:
        return None
    return TODAY_CHECK
    
def MF_DATE(a):
    date = check_date(a)
    if date is None:
        date = check_date1(a)
        if date is None:
            date = check_date2(a)
            if date is None:
                date = TODAY_CHECK = datetime.datetime.now().strftime('%d/%m/%Y')
    return date

def mutual_amount3(rendered_text):
    regex = r"(Opening Balance|NAV Date |Date Transaction|OPENING BALANCE|‘Opening Balance|Transaction Type)"
    regex1 = r"(Available Units|Your Broker|Bank|Market|Bank Name|Live Systematic|Market Value|Balance Units|Summary of Holdings|CLOSING BALANCE)"
    render_c = []
    for i in range(0,len(rendered_text)):
            matches = re.match(regex, rendered_text[i])
            if matches:
                render_c = []

                for j in range(0,100):
                    matches = re.match(regex1, rendered_text[i+j])
                    if matches:
                        break
                    elif rendered_text[i+j]==rendered_text[-1]:
                        break
                    else:
                        render_c.append(rendered_text[i+j+1])


    #return render_c
    render_table = render_c
    render_table_string = str(render_table)
    regex = r"(\d+\/\d+\/\d+|\d{2} \w{3} \d{4}|\d{4}\/\d+[2018|2019]|\d{2}\/\d{2}\d+[2018|2019]).+?.\b((\d+,\d+\.\d{2}|(\d+,\d+)|(\d+\.\d{2})))\b"
#     regex = r"(\d+\/\d+\/\d+).+?.\b((\d+,\d+\.\d{2}|(\d+,\d+)|(\d+\.\d{2})))\b"
    test_str = render_table_string
    matches = re.finditer(regex, test_str, re.MULTILINE)

    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))        
    if len(result)!=0:
        MF_date = []
        MF_amount = []
        for i in range(0,len(result),5):
            MF_date.append(result[i])

        for j in range(1,len(result),5):
            MF_amount.append(result[j])
        Transaction_table = pd.DataFrame()

        Transaction_table['Date'] = MF_date
        Transaction_table['Amount'] = MF_amount

        #return Transaction_table
        current_year_only = pd.DataFrame(columns=['Amount'])
        for i in range(0,len(Transaction_table)):
            TODAY_CHECK = MF_DATE(Transaction_table.loc[i,'Date'])
            TODAY_CHECK = datetime.datetime.strptime(TODAY_CHECK, "%d/%m/%Y")
                      
#             try:
#                 TODAY_CHECK = datetime.datetime.strptime(Transaction_table.loc[i,'Date'], "%d/%m/%Y")
#             except ValueError as e:         
#                 TODAY_CHECK = datetime.datetime.now()
#                 pass
            start = datetime.datetime.strptime("01/04/2018", "%d/%m/%Y")
            end = datetime.datetime.strptime("31/03/2019", "%d/%m/%Y")
            
            if start <= TODAY_CHECK <= end:
                current_year_only.loc[i] = Transaction_table.loc[i,'Amount']

        current_year_only['Amount']=current_year_only['Amount'].str.replace(r"[\,]","")
        current_year_only = current_year_only.astype(float)
        Total = current_year_only['Amount'].sum()
        Total = str(Total)
        return Total
    else:
        return None

In [1531]:
def information_extract_MUTUAL_FUND(corpus,rendered):
    
    provider = mutual_provider(corpus)
    if provider is None:
        provider = find_provider_lower(corpus)
        if provider is None:
            provider = find_provider_For_DCB(corpus)
        
    payee_name = mutual_payee_name1(corpus)
    if payee_name is None:
        payee_name = mutual_payee_name2(corpus)
        if payee_name is None:
            payee_name = mutual_payee_name3(corpus)
            if payee_name is None:
                payee_name = mutual_payee_name4(corpus)
                if payee_name is None:        
                    payee_name = mutual_payee_name5(corpus)
                    if payee_name is None:
                        payee_name = mutual_payee_name6(corpus)        
        
    statement_date = mutual_statement_date(corpus)
    if statement_date is None:
        statement_date = date_def2(corpus)
        
    folio_number = mutual_folio_number(corpus)
    
    scheme = mutual_scheme(corpus)
    
    amount = mutual_amount1(corpus)
    if amount is None:
        amount = mutual_amount2(corpus)
        if amount is None:
            amount = mutual_amount3(rendered)        
    
    return(provider,folio_number,amount,payee_name,statement_date,scheme)


### NSC

In [1532]:
###################################################################INSURED NAME

def nsc_name(corpus):
    regex=r"((Name|name|NAME)(|.)of(|.)(Investor|investor|INVESTOR)|\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|This is to certify that|THIS IS TO CERTIFY THAT|this is to certify that|To Whom It May Concern|To Whomsoever It May Concern|To whomsoever it may concern|Dear|DEAR|Feaployex Name)\b)(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
#     regex=r"(((Name|name|NAME)(|.)of(|.)(Investor|investor|INVESTOR)|\b(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Mr./Mrs.|Name:|Dear MR.|Smt.|Ms.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri|Sri|Sri.|Shri/Smt\.|Shri/Si|MR\s|This is to certify that|THIS IS TO CERTIFY THAT|this is to certify that|To Whom It May Concern|To Whomsoever It May Concern|To whomsoever it may concern|Dear|DEAR|Feaployex Name)\b)((|.)(1|l)|.|..|...|....)(.|)(|.)(\w+(\s|\s\s|.\s|.)\w+(\s|\s\s|.\s|.)\w{3,}|\w+ \w+))"
#     regex=r"(((Name|name|NAME)(|.)of(|.)(Investor|investor|INVESTOR)|Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Mr./Mrs.|Name:|Dear MR.|Smt.|Ms.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri|Sri|Sri.|Shri/Smt\.|Shri/Si|MR\s|This is to certify that|THIS IS TO CERTIFY THAT|this is to certify that|To Whom It May Concern|To Whomsoever It May Concern|To whomsoever it may concern|Dear|DEAR|Feaployex Name)((|.)(1|l)|.|..|...|....)(.|)(|.)(\w+|\w)(|.)(\w+|\w)(|.)(\w+|\w))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

###################################################################ACCOUNT NUMBER

def nsc_account_no(corpus):
    regex=r"((Account|account|ACCOUNT)(|.)(No|no|NO)(|.)(\d+|\d))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

###################################################################DATE
    
def nsc_date(corpus):
    regex=r"(Date(|.)of(|.)Investment(|.)(|.)value(|.)date\W(.|)(.|)(.|)(\d+|\d)(.|)(\d+|\d)(.|)(\d+|\d))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

###################################################################AMOUNT
    
def nsc_amount(corpus):
    regex=r"((Amount|amount|amt|Amount paid|Total Premium Paid|Amount Deposited)(.|)(.|)\w+(.|)\w+(.|)(Rs|rs).*?((\d+|\w+)|(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|)(\s|)\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()


In [1533]:
def information_extract_NSC(corpus,rendered):
#     rendered = row.rendered_text
#     corpus = row.corpus
    
    name = nsc_name(corpus)
    account_no = nsc_account_no(corpus)
    amount = nsc_amount(corpus)
    date = nsc_date(corpus)
    if date is None:
        date = date_def2(corpus)
             
    return (name,account_no,amount,date)


### NPS Tier 1

In [1534]:
###################################################################INSURED NAME
def find_payee_name_with_NAME_Mr(corpus):
    regex = r"((insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Life Insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Subscriber|NAME OF THE SUBSCRIBER|Name of Subscriber|NAME OF SUBSCRIBER|PERSONAL DETAILS|Personal details|personal details|Life Assured|life assured|Name of the Life Assured|Name|Name of Life Assured)(.|..|...|....)|)\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Life Insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Account Name|Name of the Life Assured|Name of the Subscriber|Name of Life Assured)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
#     regex = r"((insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Subscriber|NAME OF THE SUBSCRIBER|Name of Subscriber|NAME OF SUBSCRIBER|PERSONAL DETAILS|Personal details|personal details|Life Assured|life assured|Name)(.|..|...|....|)(Smt\./Ms/Sri\. Sree WATT|Smt./Ms/Sri. Sree|M/s|Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|MR |insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of)(.|..|...|....|)((\w+ \w+ \w+)|(\w+ \w+)|(\w\w+)+|(\w \w+ \w+)))"
#     regex = r"(\b(Name of the Subscriber|NAME OF THE SUBSCRIBER|Name of Subscriber|NAME OF SUBSCRIBER|PERSONAL DETAILS|Personal details|personal details)\b.*?\d{5,} (\w+(\||)( |)\w+(\||)( |)(\w+|)(\||)( |)))|(\b(Name of the Subscriber|NAME OF THE SUBSCRIBER|Name of Subscriber|NAME OF SUBSCRIBER|Name|NAME|PERSONAL DETAILS|Personal details|personal details)\b(:|-|.|)(\w+ \w+ \w+))"
#     regex = r"(Name of the Subscriber|NAME OF THE SUBSCRIBER|Name of Subscriber|NAME OF SUBSCRIBER|Name|NAME|PERSONAL DETAILS|Personal details|personal details)(:|-|.|)(\w+ \w+ \w+)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

# def find_payee_name(corpus):
#     regex = r"(((insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Subscriber|NAME OF THE SUBSCRIBER|Name of Subscriber|NAME OF SUBSCRIBER|PERSONAL DETAILS|Personal details|personal details|Life Assured|life assured|\s)(.|..|...|....)|)(Smt\./Ms/Sri\. Sree WATT|Smt./Ms/Sri. Sree|M/s|Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|MR |insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of)(.|..|...|....|)((\w+ \w+ \w+)|(\w+ \w+)|(\w\w+)+|(\w \w+ \w+)))|\b(Dear)\b(.|..)(\w+ \w+ \w{2,}|\w+ \w+)|(\w[a-zA-Z]+ \w[a-zA-Z]+ \w[a-zA-Z]+|\w+ \w \w+|\w+ \w\. \w+|\w\. \w\. \w+|\w \w \w+)+(.|..|...)(Address|Flat|Room|Road)|(Name(.|insured Name|Insured Name|Insured Name|)(.|..|...|....)((Mrs|mrs|Mr|mr|Smt\./Ms/Sri\. Sree WATT|Smt./Ms/Sri. Sree|M/s|Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|MR )|)(.|)((\w+|\w)(.|)(\w+|\w)|(\w+|\w)(.|)(\w+|\w)(.|)(\w+|\w)))"
# #     regex = r"(\b(Name of the Subscriber|NAME OF THE SUBSCRIBER|Name of Subscriber|NAME OF SUBSCRIBER|PERSONAL DETAILS|Personal details|personal details)\b.*?\d{5,} (\w+(\||)( |)\w+(\||)( |)(\w+|)(\||)( |)))|(\b(Name of the Subscriber|NAME OF THE SUBSCRIBER|Name of Subscriber|NAME OF SUBSCRIBER|Name|NAME|PERSONAL DETAILS|Personal details|personal details)\b(:|-|.|)(\w+ \w+ \w+))"
# #     regex = r"(Name of the Subscriber|NAME OF THE SUBSCRIBER|Name of Subscriber|NAME OF SUBSCRIBER|Name|NAME|PERSONAL DETAILS|Personal details|personal details)(:|-|.|)(\w+ \w+ \w+)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

# def find_payee_name_after_PRAN(corpus):
#     regex = r"(\w+[a-zA-Z]|)(\||)( |)\d{5,} ((\w+[a-zA-Z]\s\w+[a-zA-Z]\s\w+[a-zA-Z]{2,})|(\w+[a-zA-Z](\s|)\w+[a-zA-Z])|(\w{1}(\s|\.|\.\s)\w{1}(\s|\.|\.\s)\w{3,}))"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

# def find_payee_name_before_PRAN(corpus):
#     regex = r"\w+[a-zA-Z](\||)( |)\w+[a-zA-Z](\||)( |)(\w+[a-zA-Z]|)(\||)( |)\d{5,}"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

# def find_payee_name_from_email(corpus):
#     regex = r"(Email|email|Fer 1D|emailid|email id|EMAILID|EMAIL ID|Email Id)(.|..|...)\w+(@|gmail|GMAIL|yahoo|YAHOO|\.com|\.COM)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

###################################################################PRAN

def find_PRAN(corpus):
    regex = r"\b(permanent retirement account number|Permanent Retirement Account Number|National Pension System|NPS|Welcome Subscriber|Welcome Subsenter|Policy No|Policy No\.|POLICY NUMBER|policy number|Policy Number|PRAN|pran|Pran).*?(\d+)\b|\b(PRAN|pran|Pran|Permanent Retirement Account Number|National Pension System|NPS|Welcome Subscriber|Welcome Subsenter|Policy No\.|POLICY NUMBER|policy number|Policy Number).*?(\d+|\w[a-zA-Z0-9])\b"
#     regex = r"(PRAN|pran|Pran|pram|PRAM|Pram|Permanent Retirement Account Number|National Pension System|NPS).*?\d+"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################DATE

def find_date_of_investment1(corpus):
    regex = r"((Statement Date|statement date|Statement Oate|Transaction Date|Date of Receipt|DATE OF ISSUE:|DATE OF ISSUE :|Date and Time|Date...Time.|Date..Time...|Date...Time....|Receipt Date|INSURANCE Date:|Payment Date and Time :|Date|DATE|TNO|Tr\. No\.)(\s|).{0,15}?\b(((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4}\b\s))"
#     regex = r"(Statement Date|statement date|Statement Oate|Transaction Date|Date).*?((\d+(-|/)\d+(-|/)\d+)|[a-zA-z] \d+, \d{4})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[3])
    else:
        return None

def find_date_of_investment2(corpus):
    regex = r"\b((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s)\d{1,4}\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################AMOUNT

def find_amount1(corpus):
#     regex = r"((Amount.\(INR\))|Investment Cost|Cost of Investment).*?((\d+(,|\.)\d+(,|\.)\d+(,|\.)\d+)|(\d+(,|\.)\d+(,|\.)\d+))"
    regex = r"(The total contribution to your pension account till \w+ \d{1,2}, \d{4}|Renewal Premium to be Paid|Premium Recieved \(in figure\)|receipt of an amount|Contribution Amount|Premium Amount|Amount|amount|AMOUNT|total contribution|contribution amount|CONTRIBUTION AMOUNT|Premium.A.|Premium Amount|Premium amount|PREMIUM AMOUNT|Premium|Total Premium Amount|Amount paid|Total Premium Paid|Gross Premium .Rs.|Gross|Total Received).{0,20}?(\s|)(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
#     regex = r"((Contribution Amount|Premium Amount|Amount|amount|AMOUNT|total contribution|contribution amount|CONTRIBUTION AMOUNT)(.|..|...|....|.....)(INR|\(INR\)|Rs.|RS.|Rs|RS|) \d+(,|.)\d+(,|.)\d+)|(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€)(\s|)\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)"
#     regex = r"((Contribution Amount|Amount|amount|AMOUNT|total contribution|contribution amount|CONTRIBUTION AMOUNT)(.|..|...|....|.....)(INR|\(INR\)|Rs.|RS.|Rs|RS) \d+(,|.)\d+(,|.)\d+)|(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€)(\s|)\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_amount2(corpus):
#AMOUNT IN WORDS
    regex = r"(Premium Recieved \(in words\)|Premium Recteved \(in wards\)).{0,10}?(RUPEES|Rupees|rupees)(.*?)(ONLY|Only|only|Onty|onty)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))      
    if len(result)!=0:
        amt_wrds = result[2]
        return(str(w2n.word_to_num(amt_wrds)))
    else:
        return None
    

In [1535]:
def information_extract_NPS_tier1(corpus,rendered):
    
    payee_name = find_payee_name_with_NAME_Mr(corpus)
#     if payee_name is None:
#         payee_name = find_payee_name(corpus)
#         if payee_name is None:
#             payee_name = find_payee_name_after_PRAN(corpus)
#             if payee_name is None:
#                 payee_name = find_payee_name_before_PRAN(corpus)
#                 payee_name=payee_name
#                 if payee_name is None:
#                     payee_name = find_payee_name_from_email(corpus)
    
    date_of_investment = find_date_of_investment1(corpus)
    if date_of_investment is None:
        date_of_investment = find_date_of_investment2(corpus)
        if date_of_investment is None:
            date_of_investment = date_def2(corpus)
            
    PRAN = find_PRAN(corpus)
    amount = find_amount1(corpus)
    if amount is None:
        amount = find_amount2(corpus)
    
    return (payee_name,PRAN,date_of_investment,amount)


### NPS Tier 2

In [1536]:
def information_extract_NPS_tier2(corpus,rendered):
    
    payee_name = find_payee_name_with_NAME_Mr(corpus)
#     if payee_name is None:
#         payee_name = find_payee_name(corpus)
#         if payee_name is None:
#             payee_name = find_payee_name_after_PRAN(corpus)
#             if payee_name is None:
#                 payee_name = find_payee_name_before_PRAN(corpus)
#                 payee_name=payee_name
#                 if payee_name is None:
#                     payee_name = find_payee_name_from_email(corpus)
    
    date_of_investment = find_date_of_investment1(corpus)
    if date_of_investment is None:
        date_of_investment = find_date_of_investment2(corpus)
        if date_of_investment is None:
            date_of_investment = date_def2(corpus)
        
    PRAN = find_PRAN(corpus)
    amount = find_amount1(corpus)
    if amount is None:
        amount = find_amount2(corpus)    
    
    return (payee_name,PRAN,date_of_investment,amount)


### FD

In [1537]:
###################################################################PROVIDER

def FD_provider(corpus):
    regex=r"\b(STAR HEALTH AND ALLIED INSURANCE|STAR COMPREHENSIVE INSURANCE|Star Health|STAR HEALTH|Apollo Munich|APOLLO MUNICH|Apollo|APOLLO|STATE BANK OF INDIA|State Bank of India|SBI Life|SBI|sbi|EXIDE Life|Exide Life|Exide|EXIDE|ICICI PRUDENTIAL|ICICI Lombard|ICICI LOMBARD|ICICI|icici|Max Life|MAX LIFE|max life|HDFC LIFE|HDFC Life|hdfc life|HDFC ERGO|hdfc ergo|HDFC Standard Life|HDFC ERGO General Insurance|HOFC Life|HDFC Mutual Fund|HDFC|hdfc|LIC of India|LIC OF INDIA|Life Insurance Corporation|LIFE INSURANCE CORPORATION|LIC|lic|kotak life|KOTAK LIFE|Kotak Life|Kotak Mahindra Life|KOTAK LIFE|KOTAK|kotak|NEW INDIA ASSURANCE|New India Assurance|NEW INDIA|Aegon Life|Aegon life|aegon life|aegon|AEGON|Bajaj Allianz|BAJAJ ALLIANZ|BAJAJ|bajaj|Birla Sun Life Mutual Fund|Birla Sun Life Insurance|Birla Sun Life|ABSL|BIRLA SUN LIFE|birla sun life|BIRLA SUN|Birla Sun|PUNJAB NATIONAL BANK|Punjab National Bank|Punjab National|punjab national|PNB MetLife India Insurance|PNB MetLife India|PNB MetLife|pnb metlife|PNB METLIFE|PNB|pnb|AVIVA|Aviva|aviva|Bharti axa|Bharti AXA|bharti axa|MAX BUPA|max bupa|Max Bupa|UTI|uti|Bank of Baroda|DHFL|Dhfl|IDBI|GIC|Mumbai District Central bank|Mumbai District Central Co-op. Bank|Mumbai District Central|Mumbai.*?Co-op. Bank|LIC HFL|LIC Housing Finance Ltd|SARASWAT Bank|Saraswat Bank|SARASWAT|Saraswat|Royal Sundaram General|royal sundaram|Royal Sundaram|UNITED INDIA INSURANCE|United India|united india|ORIENTAL INSURANCE|Oriental Insurance|oriental insurance|National Mediclaim|L&T Mutual Fund|L&T|Axis Mutual Fund|Axis|axis|Motilal Oswal Mutual Fund|Motilal Oswal|RELIANCE Mutual fund|RELIANCE|Mahindra Mutual Fund|Mahindra|Franklin Templeton|Franklin India|DSP BLACKROCK|OSP BLACKROCK|DSP BlackRock|DSP|Dewan Housing Finance Corporation|Dewan Housing Finance|Dewan Housing)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_provider_For_DCB(corpus):
    regex = r"\b(DCB Bank|DCB|dcb bank|DCR RANK|DCB BANK|Deposit Confirmation Advice)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return ('DCB BANK')
    return None

###################################################################INSURED NAME

def FD_payee_name(corpus):
    regex = r"((Received with thanks from|Received From|Received from|Account Name|Policyholder's Name|Policy Holder Name|Premium Payor Name|Owner Name)(.|..|...|....)|)\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
#     regex = r"(Received From|Received from|Account Name).*?\b(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|MR|MS |)\b(.|..)(\w+ \w+ \w+|\w+ \w+|.*?(Address|flat|Flat|floor|Floor|Flr|flr|\w\d+))"
#     regex = r"(Received From|Account Name).*?\b(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|MR|MS )\b(.|..)(\w+ \w+ \w+|\w+ \w+|.*?Address)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################DATE

def FD_date1(corpus):
    regex = r"\b(Date|Date:|DATE|date|Date of Receipt|Date of issue|DATE OF ISSUE|Date of Issue|Date of Issuc|Date and Time :|Receipt Date|INTEREST CERTIFICATE|DATE OF RECEIPT|Date and Time|DATE AND TIME|INSURANCE Date|Date of deposit|date of deposit)(\s|)(|.|..|...|....|.....)\b(((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4})(\b\s|-)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[3])
    else:
        return None

def FD_date2(corpus):
    regex = r"(Deposit Confirmation Advice).*?\b(Start Date|Start Bate).*?(Value Date|Value Bate)(\s|)(|.|..|...|....|.....|.*?)\b(((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4})(\b\s|-).*?(Date of Maturity)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[5])
    else:
        return None
    
###################################################################ACCOUNT NUMBER

def FD_accno_lower(corpus):
    corpus_split = corpus.lower()
    regex=r"\b(deposit account number|account number|acc(\.|\s|\.\s)no|a/c(\.|\s|\.\s)no|aooount neu)\b(|.|..|...|....|.....)(\d{7,17})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[4])
    else:
        return None

###################################################################AMOUNT

def FD_amount1(corpus):
    regex = r"(This is to certify that an aggregate sum of|Current Deposit Amount|Deposit Amount|rincipal Amount|PRINCIPAL AMOUNT|principal amount|Balance|balance|BALANCE|Grand Total)(INR|Inr|Rs\.|RS\.|Rs|RS|R8\.|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|¥|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||).{1,100}?(\d+(,|\.|\d+)\d+(,|\.)\d+(,|\.|)(\d+|))"
#     regex = r"((Amount.\(INR\))|Investment Cost|Cost of Investment|Cost % Investment).*?((\d+(,|\.|\d)\d+(,|\.|\d)\d+(,|\.|\d)\d+)|(\d+(,|\.|\d)\d+(,|\.|\d)\d+))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[3])
    else:
        return None
    
def FD_amount2(corpus):
#     regex = r"(\s)(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|)(\s|)\d+(,|\.|\d)\d+(,|\.)\d+(,|\.|)(\d+|)"
    regex = r"(\s)(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)\d+(,|\.|\d)\d+(,|\.)\d+(,|\.|)(\d+|)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None


In [1538]:
def information_extract_FD(corpus,rendered):
    provider = FD_provider(corpus)
    if provider is None:
        provider = find_provider_lower(corpus)
        if provider is None:
            provider = find_provider_For_DCB(corpus)
        
    payee_name = FD_payee_name(corpus)
    
    date = FD_date1(corpus)
    if date is None:
        date = FD_date2(corpus)
        if date is None:
            date = date_def2(corpus)

    account_no = FD_accno_lower(corpus)
    
    amount = FD_amount1(corpus)
    if amount is None:
        amount = FD_amount2(corpus)
    
    return (provider,payee_name,date,account_no,amount)

### Education Loan

In [1539]:
###################################################################PROVIDER

def EduLoan_provider(corpus):
    regex=r"\b(STAR HEALTH AND ALLIED INSURANCE|STAR COMPREHENSIVE INSURANCE|Star Health|STAR HEALTH|Apollo Munich|APOLLO MUNICH|Apollo|APOLLO|STATE BANK OF INDIA|State Bank of India|SBI Life|SBI|sbi|EXIDE Life|Exide Life|Exide|EXIDE|ICICI PRUDENTIAL|ICICI Lombard|ICICI LOMBARD|ICICI|icici|Max Life|MAX LIFE|max life|HDFC LIFE|HDFC Life|hdfc life|HDFC ERGO|hdfc ergo|HDFC Standard Life|HDFC ERGO General Insurance|HOFC Life|HDFC Mutual Fund|HDFC|hdfc|LIC of India|LIC OF INDIA|Life Insurance Corporation|LIFE INSURANCE CORPORATION|LIC|lic|kotak life|KOTAK LIFE|Kotak Life|Kotak Mahindra Life|KOTAK LIFE|KOTAK|kotak|NEW INDIA ASSURANCE|New India Assurance|NEW INDIA|Aegon Life|Aegon life|aegon life|aegon|AEGON|Bajaj Allianz|BAJAJ ALLIANZ|BAJAJ|bajaj|Birla Sun Life Mutual Fund|Birla Sun Life Insurance|Birla Sun Life|ABSL|BIRLA SUN LIFE|birla sun life|BIRLA SUN|Birla Sun|PUNJAB NATIONAL BANK|Punjab National Bank|Punjab National|punjab national|PNB MetLife India Insurance|PNB MetLife India|PNB MetLife|pnb metlife|PNB METLIFE|PNB|pnb|AVIVA|Aviva|aviva|Bharti axa|Bharti AXA|bharti axa|MAX BUPA|max bupa|Max Bupa|UTI|uti|Bank of Baroda|DHFL|Dhfl|IDBI|GIC|Mumbai District Central bank|Mumbai District Central Co-op. Bank|Mumbai District Central|Mumbai.*?Co-op. Bank|LIC HFL|LIC Housing Finance Ltd|SARASWAT Bank|Saraswat Bank|SARASWAT|Saraswat|Royal Sundaram General|royal sundaram|Royal Sundaram|UNITED INDIA INSURANCE|United India|united india|ORIENTAL INSURANCE|Oriental Insurance|oriental insurance|National Mediclaim|L&T Mutual Fund|L&T|Axis Mutual Fund|Axis|axis|Motilal Oswal Mutual Fund|Motilal Oswal|RELIANCE Mutual fund|RELIANCE|Mahindra Mutual Fund|Mahindra|Franklin Templeton|Franklin India|DSP BLACKROCK|OSP BLACKROCK|DSP BlackRock|DSP|Dewan Housing Finance Corporation|Dewan Housing Finance|Dewan Housing)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_provider_For_DCB(corpus):
    regex = r"\b(DCB Bank|DCB|dcb bank)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################INSURED NAME

def EduLoan_payee_name(corpus):
    regex = r"\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Name & Father Name of Investor|Name & Father Name of Investor 1)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################DATE

def EduLoan_date(corpus):
    regex = r"((Date|Date:|DATE|date|Date of Receipt|Date of issue|DATE OF ISSUE|Date and Time :|Receipt Date|INTEREST CERTIFICATE))( {0,1})( |:|)( {0,1})(\d+|\w{3,})(-|/|\||\.|,|\s)(\d+|\w{3,})(-|/|\||\.|,|\s)(\d+|\w{3,})"
#     regex = r"((Date|Date:|DATE|date|Date of Receipt|Date of issue|DATE OF ISSUE|Date and Time :|Receipt Date|INTEREST CERTIFICATE)).*?((\d+(.|)(/|\\)(.|)\d+)|\d+)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################ACCOUNT NUMBER

def EduLoan_accno_lower(corpus):
    corpus_split = corpus.lower()
    regex=r"(your dcb educational loan account|dcb educational loan account|educational loan account|your dcb education loan account|dcb education loan account|education loan account|account number|account no(\.|)|acc(\.|\s|\.\s)no(\.|)|a/c(\.|\s|\.\s)no(\.|))(.|..|...|....)\w+[a-zA-Z0-9]"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################INTEREST

def EduLoan_interest(corpus):
    regex = r"(Principal Amount|Balance|Total Interest debited during the period|Interest Component).*?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||) .*?(\d+(,|.|)\d+(,|.|)\d+(,|.)(\d+|))\s"
#     regex = r"(Principal Amount|Balance|Total Interest debited during the period|Interest Component).*?(INR|Rs.|RS.|Rs|RS) \d+(,|\.|)\d+(,|\.|)\d+"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

In [1540]:
def information_extract_EduLoan(corpus,rendered):
    provider = EduLoan_provider(corpus)
    if provider is None:
        provider = find_provider_lower(corpus)
        if provider is None:
            provider = find_provider_For_DCB(corpus)
        
    payee_name = EduLoan_payee_name(corpus)
    date = EduLoan_date(corpus)
    if date is None:
        date = date_def2(corpus)
        
    account_no = EduLoan_accno_lower(corpus)
    
    amount = EduLoan_interest(corpus)    
    return (provider,payee_name,date,account_no,amount)


### Child Education

In [1541]:
###################################################################SCHOOL NAME

def ChildEdu_school_name(corpus):
    regex = r"(aga khan education service|hindu public school|utkarsha vidyalaya|k j somaiya college|rita memorial school|eurokids preschool|catalyser|st. xavier's high school|st. xavier's|vsg international|vsginternational|vedatya institute|st. peters school|st. peter's school|st joseph's convent sen sec school|aaditya vidya vihar international school|aaditya vidya wihal nternational school|oakridge educational society|oakridge international school|calorx education & research foundation|orchids the international school|sri chaitanya educational institutions|global edge school|shalom’s music zone llp| sky international school|samved school|shalimar garden holy angels school|smt. sulochanadevi singhania school|narayana (e.m.) high school|st. agnes’ convent school|guru shree shantivijai jain vidyalaya|satnav preschools private limited|k.c.m. public high school|holy cross school|st.john the baptist|st. aloysius english high school|sri sri academy|founding years learning solutions|swami vivekananda saraswati play school|sri chaitanya school|jain toddlers an international play school|jain toddlers an ternational play school|b.j.e.m school|st. joseph anglo.indian hr.sec. school|society of the sisters of the presentation of the blessed virgin mary|st. mary's convent senior secondary school|madhavrao scindia public school|maharashtra technical education society|st.xavier high school|n.g. vartak english med. high school & junior college|presentation nursery and primary school|sundatta high school|kites international play school|podar international school|success school|indira national school|chinmaya vidyalaya|young horizons school|ann's high school|st. ann's high school|hume mc henry memorial higher secondary school & junior college|dav public school|billabong high international school|sheth n.k.t.t. english medium school & jr. college of commerce & science|sheth t.j. education society|amulya kids|choithram school|sacred heart convent senior secondary school|narayana vidyalayam|v a v international school|v av international school|rbk education solutions private limited|s.k.v.v. sangha's  english medium high school|mother nursery & primary school|delhi public school|amity international school|bvm sr. sec. school|sagar international school|shaw public school|sri aurobindo's school|ravindra bharathi e.m school|ave maria nursery school|mount litera zee school|ims educational society|rashtrotthana vidya kendra|a.k. children academy|a.k. children play school|deepti convent high school|brilliants play school|achievers foundation school|sri guru nanak public school|apple play school|galaxy english school|bal bharati public school|nc jindal public school|s.a.v. public school|new horizon scholars  school|senthil public school|holy cross convent high school|thakur vidya mandir high school & junior college|thakor vidya mandir high school & junior gollege|tree house high school|bharat national public school|ryan shalom montessori|achievers education academy|pollocks school|zagdu singh charitable|sheth dhanjisha rustamji umrigar memorial school|kana thakur vidyalaya|maheshwari public school|vadanta international school|st. anselm’s school|kc global school|national public school|tmt. ponnammal jothipragasam education|smt. ponnammal jothipragasam education|ponnammal jothipragasam education|mother's public school |hari om english pre-primary school|st. joseph's boys' high school|slate the school|slate1819|muljibhai mehta international school|kalka public school|thakur international school|reliance foundation school|vedatya institute|lilavatibai podar senior secondary school|d.a.v public school|st.little flower matriculation hr. sec. school|vidyasagar school|chrysalis high|ravindra bharathi school|st.lawrence high school|sadhu vaswani international school|sardar vallabhbhai patel vidyalaya|sri sankara vidyalayaa|vkids|sanskruti an english medium school|sboa matriculation school|smt. bhagubai changu thakur vidyalaya|aster public school|empyrean school|n. l. dalmia high school|montessori english medium school|shiv darshan education and charitable trust|rhiv darshan education and charitable tust|sagar public school|carmel convent school|little angels centre|st. anthony high school|rbk global school|modern english school|kantabanji lions public school|lr dav public school|modern senior secondary school|sai international school|sai international schoo!|saplings english medium school|chinmaya vidyalaya|rustomjee cambridge international school|takshashila international school|baldwin girls’ high school|shri vasantrao chougule english medium school|garodia's academy a preschool|garodia's academy|gurukul international kindergarten|st. peter's high school|st. peter's high schoo!|pallavi model school|budding blossoms|saai memorial girls school|expert's international high school|rms international school|siddhartha public school|siddhartha public schoo|sboa matric|velammal institute of technology|csi bain school|carmel international school|city montessori school|alpha cbse school|mount litera zee school|sri chaitanya educational institutions|royal college of arts. science & commerce|royal college|adarsh vidya mandir|johnson grammar school|sardar vallabhbhai national institute of technology|sardar vallabhbhai national institute|central academy|varsity education management|st mary of the angels|cal public school|podar jumbo kids plus|alpha cbse school|sri ravishankar vidya mandir|saai memorial girls school|gaal memorial girls school|vedant international|st. joseph's convent|oxford public school|sacred heart school|st(.|..|...)paul academy|model engineering college|gokhale education society|bhausaheb vartak.{0,27}?college|heart school|opg world school|world school|st.augustine high school|augustine high school)"
    corpus1 = corpus.lower()
    matches = re.finditer(regex, corpus1, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################STUDENT NAME

def ChildEdu_student_name0(corpus):
    regex = r"(BOSCO PUBLIC SCHOOL|Father's Name).*?(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Ali|ALI|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Joy|JOY|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Rks|RKS|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s\s|\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Ali|ALI|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Joy|JOY|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Rks|RKS|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,}).{0,10}?(Father's Name|BOSCO PUBLIC SCHOOL)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[1])
    else:
        return None

def ChildEdu_student_name1(corpus):
    regex=r"\s(This is to certify that|THIS IS TO CERTIFY THAT|Student Name|STUDENT NAME|student name|Name of the student|Name of the Student|Student's name|Students Name|Student Nane|Student.{0,30}Name|Child's Name|is paid by|being received from|Being Received From|Received from)\b(|.|..|...|....|.....|......)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})|(\w+(\s|\.|\,|\: |] |\.\s|\|\s)\w+(\s|\.|\,|\: |] |\.\s|\|\s)\w{4,}|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s)\w{4,}|\w{4,})(.|..|...|....|.....|.....|.is a.)Bonafide student"
#     regex=r"\s(This is to certify that|THIS IS TO CERTIFY THAT|Student Name|STUDENT NAME|student name|Name of the student|Name of the Student|Student's name|Students Name|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Smt|SMT)\b(.....|....|...|..|.)(\w+(\s|\.|\,|\: |] |\.\s|\|\s)\w+(\s|\.|\,|\: |] |\.\s|\|\s)\w{4,}|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s)\w{4,}|\w{4,})"
#     regex=r"(\b(This is to certify that|THIS IS TO CERTIFY THAT|Student Name|STUDENT NAME|student name|Name of the student|Name of the Student|Name|name|NAME)\b(.|..|...|....)(\w+ \w+ \w{3,}|\w+ \w+|\w+( |\| )\w+))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def ChildEdu_student_name2(corpus):
    regex=r"\b(Employee Name:|Father's Nane|Name|name|NAME|Name:|NAME:|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Smt|SMT|Sri|Shri|Master|master)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
#     regex=r"((Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Employee Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|Shri/Smt\.|MR|Dear Mr.|This is to certify that|Student Name|student name|Name of the student|Name of the Student|Name|name|NAME)(.|..|...|....)(\w+ \w+ \w{2,}|\w+ \w+))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################DATE

def ChildEdu_statement_date(corpus):
    regex = r"((Date of Receipt|Date of issue|DATE OF ISSUE|Date and Time :|Receipt Date|receipt date|Date|Date:|DATE|date|INTEREST CERTIFICATE|Authorized Signatory))(.|..|...|....|.....)( {0,1})( |:|)( {0,1})(\d+|\w{3,})(-|/|\||\.|,|,\s|\s)(\d+|\w{3,})(-|/|\||\.|,|,\s|\s)(\d+|\w{3,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[2])
    else:
        return None

##################################################################Receipt NO

def ChildEdu_doc_no(corpus):
    regex = r"\b(Receipt No|receipt no|RECEIPT NO|Receipt no|Document no|Document No|document no|DOCUMENT NO)\b(.|..|...|....|.....)((\d+\W+\d+|\w+(/)\d+|\w+\d+).*?\s)"
#     regex = r"\b(Receipt No|receipt no|RECEIPT NO|Receipt no|Document no|Document No|document no|DOCUMENT NO)\b.*?(\w{3,}.{0,5}?\s)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[2])
    else:
        return None

###################################################################TUITION

def ChildEdu_tuition1(corpus):
    regex = r"\b(PUTION FEE|TUITION FEE|Tution Fees|Tution fees|tution fees|tution fee|Tution Fee|tuition fees|Tuition Fee|Class Fee|Class fee|class fee|TunonFees|dutionFees|TutonFees|YotionFees|Tuition|tuition|Tution|tution|TUTION|TUITION|Tution ee|tution ee)\b.{0,30}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\|)(|.|..|...|....|.....)\s(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))(\s|/)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def ChildEdu_tuition2(corpus):
    regex = r"\b(PUTION FEE|TUITION FEE|Tution Fees|Tution fees|tution fees|tution fee|Tution Fee|Tuition Fee|Class Fee|Class fee|class fee|tuition fae|TunonFees|dutionFees|TutonFees|YotionFees|Tuition Fae|Tution ee|tution ee|Tuition|tuition|Tution|tution|TUTION|TUITION)\b.{0,30}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(|.|..|...|....|.....)\s(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))(\s|/)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[4])
    else:
        return None

###################################################################TERM

def ChildEdu_term1(corpus):
    regex = r"(TERM - 1|Term - 1|term - 2|TERM - 2|Term - 2|term - 2|TERM FEE|Term Fee|Term fee|term fee|Term|term|TERM|Annual Fee|Annual Fees|Annual fee|Annual fees|annual fees|annual fee|Annual|annual).{0,25}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\|)(|.|..|...|....|.....)\s(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))(\s|/)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[4])
    else:
        return None

def ChildEdu_term2(corpus):
    regex = r"(TERM - 1|Term - 1|term - 2|TERM - 2|Term - 2|term - 2|TERM FEE|Term Fee|Term fee|term fee|Term|term|TERM|Annual Fee|Annual Fees|Annual fee|Annual fees|annual fees|annual fee|Annual|annual).{0,30}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(|.|..|...|....|.....)\s(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))(\s|/)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
###################################################################COMPUTER

def ChildEdu_computer1(corpus):
    regex = r"(COMPUTER FEES|Computer Fees|Computer fees|COMPUTER FEE|Computer Fee|Computer fee|computer fee|COMPUTER|Computer|computer|COMPUTES|Coniputer|Somputer|Comp, Lab).{0,50}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\|)(|.|..|...|....|.....)\s(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
                                                                                             
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def ChildEdu_computer2(corpus):
    regex = r"(Computer Amenities|computer amenities|COMPUTER FEES|Computer Fees|Computer fees|COMPUTER FEE|Computer Fee|Computer fee|computer fee|COMPUTER|Computer|computer|COMPUTES|Coniputer|Somputer|Comp, Lab).{0,50}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(|.|..|...|....|.....)\s(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
                                                                                             
    if len(result)!=0:
        return(result[4])
    else:
        return None

###################################################################TOTAL AMOUNT

def ChildEdu_total0(corpus):
    corpus1 = corpus.lower()
    regex = r"\b(total paid amount|total amount|total received|total fees|total fee|received the sum of|total sum of|tora amount)\b.{0,50}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(|.|..|...|....|.....)(\s|\.|\"|\'|:|\||/)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))(\s|/)"
    matches = re.finditer(regex, corpus1, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[5])
    else:
        return None
    
def ChildEdu_total1(corpus):
    regex = r"(GRAND TOTAL|Grand Total|Grand total).{0,50}?(INR|Inr|Rs\.|RS\.|Rs|RS|R8\.|rs|eRe\.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(|.|..|...|....|.....)\s(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[4])
    else:
        return None


####
# RS and Proper AMT
def ChildEdu_total2(corpus):
    regex = r"(Total|total|TOTAL|Jota!|Paid Amount|Pald Amount|paid Rs|Fees - Gross|fees - gross).{0,50}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\|)(|.|..|...|....|.....)(\s|:|\||/|\.|\"|\'|)(\d+(,|\.|)\d+(,|\.)\d+(,|\.|)(\d+|))(\s|/)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[5])
    else:
        return None

# Not RS and Proper AMT    
def ChildEdu_total3(corpus):
    regex = r"(Total|total|TOTAL|Jota!|Paid Amount|Pald Amount|paid Rs|Fees - Gross|fees - gross).{0,50}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(|.|..|...|....|.....)(\s|:|\||/|\.|\"|\'|)(\d+(,|\.|)\d+(,|\.)\d+(,|\.|)(\d+|))(\s|/)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[5])
    else:
        return None    
    
# RS and Any AMT    
def ChildEdu_total4(corpus):
    regex = r"(Total|total|TOTAL|Jota!|Paid Amount|Pald Amount|paid Rs|Fees - Gross|fees - gross).{0,50}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\|)(|.|..|...|....|.....)(\s|:|\||/|\.|\"|\'|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))(\s|/)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[5])
    else:
        return None     

# Not RS and Any AMT      
def ChildEdu_total5(corpus):
    regex = r"(Total|total|TOTAL|Jota!|Paid Amount|Pald Amount|paid Rs|Fees - Gross|fees - gross).{0,50}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(|.|..|...|....|.....)(\s|:|\||/|\.|\"|\'|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))(\s|/)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[5])
    else:
        return None    
    
    
def ChildEdu_total6(corpus):
    regex = r"(Amount|amount|AMOUNT|Balance|balance|BALANCE|BIANCE|SUB TOTAL|Sub Total|Sub total|sub total).{0,50}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(|.|..|...|....|.....)(\s|:|\|)(\d+(,|\.)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[5])
    else:
        return None    

def ChildEdu_total7(corpus):
    regex = r"(\d+(,|\.|)\d+(,|\.|)\d+(,|\.)(\d+|))(.|..|...|....|.....)(Total|total|TOTAL)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[0])
    else:
        return None

In [1542]:
def information_extract_ChildEdu(corpus,rendered):
    School_Name = ChildEdu_school_name(corpus)
    
    Doc_no = ChildEdu_doc_no(corpus)
    
    Student_Name = ChildEdu_student_name0(corpus)
    if Student_Name is None:
        Student_Name = ChildEdu_student_name1(corpus)
        if Student_Name is None:
            Student_Name = ChildEdu_student_name2(corpus)
    
    Statement_Date = ChildEdu_statement_date(corpus)
    if Statement_Date is None:
        Statement_Date = date_def2(corpus)
        
    Tuition = ChildEdu_tuition1(corpus)
    if Tuition is None:
        Tuition = ChildEdu_tuition2(corpus)
    
    Term = ChildEdu_term1(corpus)
    if Term is None:
        Term = ChildEdu_term2(corpus) 
    
    Computer = ChildEdu_computer1(corpus)
    if Computer is None:
        Computer = ChildEdu_computer2(corpus) 
    
    Total = ChildEdu_total0(corpus)
    if Total is None:
        Total = ChildEdu_total1(corpus)
        if Total is None:
            Total = ChildEdu_total2(corpus)
            if Total is None:
                Total = ChildEdu_total3(corpus)
                if Total is None:
                    Total = ChildEdu_total4(corpus)
                    if Total is None:
                        Total = ChildEdu_total5(corpus)
                        if Total is None:
                            Total = ChildEdu_total6(corpus)
                            if Total is None:
                                Total = ChildEdu_total7(corpus)
        
    return(School_Name,Doc_no,Student_Name,Statement_Date,Tuition,Term,Computer,Total)


### Housing Loan

In [1543]:
###################################################################PROVIDER
def find_provider_HomeLoan(corpus):
    regex=r"\b(BANK OF INDIA|BOI|INDIABULLS HOUSING FINANCE LIMITED|INDIABULLS HOUSING|INDIABULLS|homeloans@indiabulls.com|www.indiabulls.com|https//retail.oninesbi.com/retail/homeloancertificatepdt.html|https//retail.oninesbi.com/retail/homeloancertificatepdt.ntm|https://retail.onlinesbi.com/retai/homeloancertificatepdt.htm|www.hdfes.com|www.hdfescom|STAR HEALTH AND ALLIED INSURANCE|STAR COMPREHENSIVE INSURANCE|Star Health|STAR HEALTH|Apollo Munich|APOLLO MUNICH|Apollo|APOLLO|STATE BANK OF INDIA|State Bank of India|SBI Life|SBI|sbi|EXIDE Life|Exide Life|Exide|EXIDE|ICICI PRUDENTIAL|ICICI Lombard|ICICI LOMBARD|ICICI|icici|Max Life|MAX LIFE|max life|HDFC LIFE|HDFC Life|hdfc life|HDFC ERGO|hdfc ergo|HDFC Standard Life|HDFC ERGO General Insurance|HOFC Life|HDFC Mutual Fund|HDFC|hdfc|LIC of India|LIC OF INDIA|Life Insurance Corporation|LIFE INSURANCE CORPORATION|LIC|lic|kotak life|KOTAK LIFE|Kotak Life|Kotak Mahindra Life|KOTAK LIFE|KOTAK|kotak|NEW INDIA ASSURANCE|New India Assurance|NEW INDIA|Aegon Life|Aegon life|aegon life|aegon|AEGON|Bajaj Allianz|BAJAJ ALLIANZ|BAJAJ|bajaj|Birla Sun Life Mutual Fund|Birla Sun Life Insurance|Birla Sun Life|ABSL|BIRLA SUN LIFE|birla sun life|BIRLA SUN|Birla Sun|PUNJAB NATIONAL BANK|Punjab National Bank|Punjab National|punjab national|PNB MetLife India Insurance|PNB MetLife India|PNB MetLife|pnb metlife|PNB METLIFE|PNB|pnb|AVIVA|Aviva|aviva|Bharti axa|Bharti AXA|bharti axa|MAX BUPA|max bupa|Max Bupa|UTI|uti|Bank of Baroda|DHFL|Dhfl|IDBI|GIC|Mumbai District Central bank|Mumbai District Central Co-op. Bank|Mumbai District Central|Mumbai.*?Co-op. Bank|LIC HFL|LIC Housing Finance Ltd|SARASWAT Bank|Saraswat Bank|SARASWAT|Saraswat|Royal Sundaram General|royal sundaram|Royal Sundaram|UNITED INDIA INSURANCE|United India|united india|ORIENTAL INSURANCE|Oriental Insurance|oriental insurance|National Mediclaim|L&T Mutual Fund|L&T|Axis Mutual Fund|Axis|axis|Motilal Oswal Mutual Fund|Motilal Oswal|RELIANCE Mutual fund|RELIANCE|Mahindra Mutual Fund|Mahindra|Franklin Templeton|Franklin India|DSP BLACKROCK|OSP BLACKROCK|DSP BlackRock|DSP|Dewan Housing Finance Corporation|Dewan Housing Finance|Dewan Housing|Allahabad Bank|Standard Chartered Bank|Standard Chartered|HLT\/MBHI\/P-H)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_provider_For_DCB(corpus):
    regex = r"\b(DCB Bank|DCB|dcb bank)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

################################################################### INSURED NAME

def find_payee_name_HomeLoan0(corpus):
    regex = r"\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})((.....|....|...|..|.| )(&  |& |and |, |/|\s)\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,}))"
#     regex = r"(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Employee Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|Shri/Smt\.|SHRI/SMT|MR|Dear Mr.|This is to certify that|THIS IS TO CERTIFY THAT|This is to state that|BORROWER NAME|borrower)(.|..|...|....)((Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS|SHRI/SMT|)\w+(.*?( &  | & |, |/))(Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS) \w+ \w+ (\w{2,}|)|(\w+(\W|) \w+(\W|) \w{2,}(\W|)|\w+ \w+ \w{2,}|\w+ \w+)|.*?has/have)|\b(Dear)\b(.|..)(\w+ \w+ \w{2,}|\w+ \w+)|(\w[a-zA-Z]+ \w[a-zA-Z]+ \w[a-zA-Z]+|\w+ \w \w+|\w+ \w\. \w+|\w\. \w\. \w+|\w \w \w+)+(.|..|...)(Address|Flat|Room|Road)"
#     regex = r"(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Employee Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|Shri/Smt\.|SHRI/SMT|MR|Dear Mr.|This is to certify that|THIS IS TO CERTIFY THAT|BORROWER NAME|borrower|TO WHOMSOEVER IT MAY CONCERN Provisional Home Loan Interest Certificate |TO WHOMSOEVER IT MAY CONCERN|Employee Name)( |  )((Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS|SHRI/SMT|)\w+(.*?( &  | & |, |/))(Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS) \w+ \w+ (\w{2,}|)|(\w+(\W|) \w+(\W|) \w{2,}(\W|)|\w+ \w+ \w{2,}|\w+ \w+)|.*?has/have)|(\d+(/|-|\.|\\|\|)\d+(/|-|\.|\\|\|)\d+)(.|..|...)(\w+ \w+ (\w{2,}|)|(\w+(\W|) \w+(\W|) \w{2,}(\W|)|\w+ \w+ \w{2,}|\w+ \w+))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_payee_name_HomeLoan1(corpus):
    regex = r"((This is to certify that|THIS IS TO CERTIFY THAT|This is to state that|BORROWER NAME|borrower|Name\(s\) of Borrower\(s\))(.|..|...|....)|)\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})((.....|....|...|..|.| )(&  |& |and |, |/|\s)\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,}))"
#     regex = r"(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Employee Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|Shri/Smt\.|SHRI/SMT|MR|Dear Mr.|This is to certify that|THIS IS TO CERTIFY THAT|This is to state that|BORROWER NAME|borrower)(.|..|...|....)((Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS|SHRI/SMT|)\w+(.*?( &  | & |, |/))(Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS) \w+ \w+ (\w{2,}|)|(\w+(\W|) \w+(\W|) \w{2,}(\W|)|\w+ \w+ \w{2,}|\w+ \w+)|.*?has/have)|\b(Dear)\b(.|..)(\w+ \w+ \w{2,}|\w+ \w+)|(\w[a-zA-Z]+ \w[a-zA-Z]+ \w[a-zA-Z]+|\w+ \w \w+|\w+ \w\. \w+|\w\. \w\. \w+|\w \w \w+)+(.|..|...)(Address|Flat|Room|Road)"
#     regex = r"(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Employee Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|Shri/Smt\.|SHRI/SMT|MR|Dear Mr.|This is to certify that|THIS IS TO CERTIFY THAT|BORROWER NAME|borrower|TO WHOMSOEVER IT MAY CONCERN Provisional Home Loan Interest Certificate |TO WHOMSOEVER IT MAY CONCERN|Employee Name)( |  )((Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS|SHRI/SMT|)\w+(.*?( &  | & |, |/))(Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS) \w+ \w+ (\w{2,}|)|(\w+(\W|) \w+(\W|) \w{2,}(\W|)|\w+ \w+ \w{2,}|\w+ \w+)|.*?has/have)|(\d+(/|-|\.|\\|\|)\d+(/|-|\.|\\|\|)\d+)(.|..|...)(\w+ \w+ (\w{2,}|)|(\w+(\W|) \w+(\W|) \w{2,}(\W|)|\w+ \w+ \w{2,}|\w+ \w+))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_payee_name_HomeLoan2(corpus):
    regex = r"(((This is to certify that|THIS IS TO CERTIFY THAT|This is to state that|BORROWER NAME|borrower |Name\(s\) of Borrower\(s\))(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})(&  |& |and |, |/)(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})(....|...|..|.)(has been|has/have been))|((This is to certify that|THIS IS TO CERTIFY THAT|This is to state that|BORROWER NAME|borrower |Name\(s\) of Borrower\(s\))(.....|....|...|..|.)(\w+(\s|\.|\,|\: |] |\.\s|\|\s)\w+(\s|\.|\,|\: |] |\.\s|\|\s)\w{4,}|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s)\w{4,}|\w{4,})))|((This is to certify that|THIS IS TO CERTIFY THAT|This is to state that|BORROWER NAME|borrower |Name\(s\) of Borrower\(s\))(.|..|...|....)|)\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
#     regex = r"((This is to certify that|THIS IS TO CERTIFY THAT|This is to state that|BORROWER NAME|borrower)(.....|....|...|..|.)(\w+(\s|\.|\,|\: |] |\.\s|\|\s)\w+(\s|\.|\,|\: |] |\.\s|\|\s)\w{4,}|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s)\w{4,}|\w{4,})(&  |& |and |, |/)(.....|....|...|..|.)(\w+(\s|\.|\,|\: |] |\.\s|\|\s)\w+(\s|\.|\,|\: |] |\.\s|\|\s)\w{4,}|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s)\w{4,}|\w{4,})(....|...|..|.)(has been|has/have been))|((This is to certify that|THIS IS TO CERTIFY THAT|This is to state that|BORROWER NAME|borrower)(.....|....|...|..|.)(\w+(\s|\.|\,|\: |] |\.\s|\|\s)\w+(\s|\.|\,|\: |] |\.\s|\|\s)\w{4,}|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s)\w{4,}|\w{4,}))"
#     regex = r"(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Employee Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|Shri/Smt\.|SHRI/SMT|MR|Dear Mr.|This is to certify that|THIS IS TO CERTIFY THAT|This is to state that|BORROWER NAME|borrower)(.|..|...|....)((Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS|SHRI/SMT|)\w+(.*?( &  | & |, |/))(Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS) \w+ \w+ (\w{2,}|)|(\w+(\W|) \w+(\W|) \w{2,}(\W|)|\w+ \w+ \w{2,}|\w+ \w+)|.*?has/have)|\b(Dear)\b(.|..)(\w+ \w+ \w{2,}|\w+ \w+)|(\w[a-zA-Z]+ \w[a-zA-Z]+ \w[a-zA-Z]+|\w+ \w \w+|\w+ \w\. \w+|\w\. \w\. \w+|\w \w \w+)+(.|..|...)(Address|Flat|Room|Road)"
#     regex = r"(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Employee Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|Shri/Smt\.|SHRI/SMT|MR|Dear Mr.|This is to certify that|THIS IS TO CERTIFY THAT|BORROWER NAME|borrower|TO WHOMSOEVER IT MAY CONCERN Provisional Home Loan Interest Certificate |TO WHOMSOEVER IT MAY CONCERN|Employee Name)( |  )((Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS|SHRI/SMT|)\w+(.*?( &  | & |, |/))(Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS) \w+ \w+ (\w{2,}|)|(\w+(\W|) \w+(\W|) \w{2,}(\W|)|\w+ \w+ \w{2,}|\w+ \w+)|.*?has/have)|(\d+(/|-|\.|\\|\|)\d+(/|-|\.|\\|\|)\d+)(.|..|...)(\w+ \w+ (\w{2,}|)|(\w+(\W|) \w+(\W|) \w{2,}(\W|)|\w+ \w+ \w{2,}|\w+ \w+))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_payee_name_HomeLoan3(corpus):
    regex = r"((This is to certify that|THIS IS TO CERTIFY THAT|This is to state that|BORROWER NAME|borrower)(.|..|...|....)|)\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
#     regex = r"(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Employee Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|Shri/Smt\.|SHRI/SMT|MR|Dear Mr.|This is to certify that|THIS IS TO CERTIFY THAT|This is to state that|BORROWER NAME|borrower)(.|..|...|....)((Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS|SHRI/SMT|)\w+(.*?( &  | & |, |/))(Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS) \w+ \w+ (\w{2,}|)|(\w+(\W|) \w+(\W|) \w{2,}(\W|)|\w+ \w+ \w{2,}|\w+ \w+)|.*?has/have)|\b(Dear)\b(.|..)(\w+ \w+ \w{2,}|\w+ \w+)|(\w[a-zA-Z]+ \w[a-zA-Z]+ \w[a-zA-Z]+|\w+ \w \w+|\w+ \w\. \w+|\w\. \w\. \w+|\w \w \w+)+(.|..|...)(Address|Flat|Room|Road)"
#     regex = r"(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Employee Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|Shri/Smt\.|SHRI/SMT|MR|Dear Mr.|This is to certify that|THIS IS TO CERTIFY THAT|BORROWER NAME|borrower|TO WHOMSOEVER IT MAY CONCERN Provisional Home Loan Interest Certificate |TO WHOMSOEVER IT MAY CONCERN|Employee Name)( |  )((Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS|SHRI/SMT|)\w+(.*?( &  | & |, |/))(Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS) \w+ \w+ (\w{2,}|)|(\w+(\W|) \w+(\W|) \w{2,}(\W|)|\w+ \w+ \w{2,}|\w+ \w+)|.*?has/have)|(\d+(/|-|\.|\\|\|)\d+(/|-|\.|\\|\|)\d+)(.|..|...)(\w+ \w+ (\w{2,}|)|(\w+(\W|) \w+(\W|) \w{2,}(\W|)|\w+ \w+ \w{2,}|\w+ \w+))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_payee_name_HomeLoan4(corpus):
    regex = r"\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Main Applicant|Account Name|account name)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
#     regex = r"(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Employee Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|Shri/Smt\.|SHRI/SMT|MR|Dear Mr.|This is to certify that|THIS IS TO CERTIFY THAT|This is to state that|BORROWER NAME|borrower)(.|..|...|....)((Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS|SHRI/SMT|)\w+(.*?( &  | & |, |/))(Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS) \w+ \w+ (\w{2,}|)|(\w+(\W|) \w+(\W|) \w{2,}(\W|)|\w+ \w+ \w{2,}|\w+ \w+)|.*?has/have)|\b(Dear)\b(.|..)(\w+ \w+ \w{2,}|\w+ \w+)|(\w[a-zA-Z]+ \w[a-zA-Z]+ \w[a-zA-Z]+|\w+ \w \w+|\w+ \w\. \w+|\w\. \w\. \w+|\w \w \w+)+(.|..|...)(Address|Flat|Room|Road)"
#     regex = r"(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Employee Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|Shri/Smt\.|SHRI/SMT|MR|Dear Mr.|This is to certify that|THIS IS TO CERTIFY THAT|BORROWER NAME|borrower|TO WHOMSOEVER IT MAY CONCERN Provisional Home Loan Interest Certificate |TO WHOMSOEVER IT MAY CONCERN|Employee Name)( |  )((Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS|SHRI/SMT|)\w+(.*?( &  | & |, |/))(Mr.|Mr|MR.|MR|Mrs.|Mrs|MRS.|MRS) \w+ \w+ (\w{2,}|)|(\w+(\W|) \w+(\W|) \w{2,}(\W|)|\w+ \w+ \w{2,}|\w+ \w+)|.*?has/have)|(\d+(/|-|\.|\\|\|)\d+(/|-|\.|\\|\|)\d+)(.|..|...)(\w+ \w+ (\w{2,}|)|(\w+(\W|) \w+(\W|) \w{2,}(\W|)|\w+ \w+ \w{2,}|\w+ \w+))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

################################################################### DATE

def find_statement_date_HomeLoan1(corpus):
    regex = r"(Date|Date:|DATE|date|Date of Receipt|Date of issue|DATE OF ISSUE|Date of Issue|Date and Time :|Receipt Date|INTEREST CERTIFICATE|Date of Receipt|DATE OF RECEIPT|Date and Time|DATE AND TIME|INSURANCE Date|for the period from)(\s|)(.|..|...|....|.....)\b(((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4})(\b\s|-)"
    matches = re.finditer(regex, corpus, re.MULTILINE)

    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[3])
    else:
        return None

def find_statement_date_HomeLoan2(corpus):
    regex = r"\b(((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4})(\b\s|-)"
    matches = re.finditer(regex, corpus, re.MULTILINE)

    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[0])
    else:
        return None

################################################################### LOAN CODE

# def find_loan_code_HomeLoan(corpus):
#     regex = r"(Loan Code|loan code|LOAN CODE|LOAN NO|Loan No|loan no|Loan Account Number|Loan A/c\.No|Ref\. LA|Ref: A/c no).*?\d+( |)(/ \d+|- \d+| \d+|)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

def find_loan_code_HomeLoan1(corpus):
    corpus1 = corpus.lower()
    regex = r"(loan account number|loan account no|loan code|loan no|loan account|loan a/c no|loan(.|..|...|)no|loan acc no|loan(.|..|...|)account mo|loan a/c\.no|account no|account number|avc no|a\/c no).*?((\d+\W+\d+|\w+(/)\d+|\w+\d+).*?\s)"
    matches = re.finditer(regex, corpus1, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[3])
    else:
        return None
    
def find_loan_code_HomeLoan2(corpus):
    corpus1 = corpus.lower()
    regex = r"(ref\. la|ref.{0,10}?no).*?((\d+\W+\d+|\w+(/)\d+|\w+\d+).*?\s)"
    matches = re.finditer(regex, corpus1, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[2])
    else:
        return None    

################################################################### AMOUNT

def prem_for_IDBI(corpus):   
    regex = r"(A Govt. of India Owned Bank|India Owned Bank).*?(Total for the Year).{0,5}?\s{1,5}(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))\s{1,5}(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"

    matches = re.finditer(regex, corpus, re.MULTILINE)

    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[7])
    else:
        return None

def principal_amt_HomeLoan1(corpus):
    regex = r"(PRINCIPAL COMPONENT|Principal Component|Principal component|principal component|Prinéipal Componenti|Prinéipal componenti|prinéipal componenti|Principal Amount|Principal amount|principal amount|towards Principal|towards principal).{0,25}?(INR|Inr|Rs\.|RS\.|Rs|RS|R8\.|rs|eRe\.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(|.|..|...|....|.....)(\d+(,|,,|, |\.|\. )\d+(,|, |\.|\. |)\d+(,|, |\.|\. |)(\d+|))(\s|/|)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def principal_amt_HomeLoan2(corpus):
    regex = r"(PRINCIPAL COMPONENT|Principal Component|principal component).{0,50}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(|.|..|...|....|.....)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))(\s|/)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def principal_amt_HomeLoan3(corpus):
    regex = r"(Principal|PRINCIPAL|principal|Paincipal|paincipal|Peinsipal|peinsipal).{0,50}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(|.|..|...|....|.....)(\s|\.|:)(\d+(,|\.)\d+(,|\.|)\d+(,|\.|)(\d+|))(\s|/)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[5])
    else:
        return None
    
def principal_amt_HomeLoan4(corpus):
    regex = r"(Principal|PRINCIPAL|principal|Peinsipal|peinsipal).{0,140}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(|.|..|...|....|.....)(\s|\.|:)(\d+(,|\.)\d+(,|\.|)\d+(,|\.|)(\d+|))(\s|/)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[5])
    else:
        return None
    
def principal_amt_HomeLoan5(corpus):
    regex = r"(Principal|PRINCIPAL|principal|Peinsipal|peinsipal).{0,50}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(|.|..|...|....|.....)(\s|\.|:)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))(\s|/)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[5])
    else:
        return None


def principle_amt_between_HomeLoan(corpus):
#     regex = r"((Amount.\(INR\))|Investment Cost|Cost of Investment).*?((\d+(,|\.)\d+(,|\.)\d+(,|\.)\d+)|(\d+(,|\.)\d+(,|\.)\d+))"
    regex = r"(TOTAL AMOUNT|Total Amount|Total amount|total amount|Total Amo).*?(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))|(?<=Principal).{1,20}?(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)).{1,20}?(?=Interest)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def principle_amt_before_interest_HomeLoan(corpus):
    regex = r"\d+(,|\.|\s|)\d+(,|\.|\d|\s|)\d+(,|\.|\s|)\d+(,|\.|\s)(\d+|)(.|..|...|....)(Toward Interest|Towards Interest|toward interest)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def prem_from_date_to_date(corpus):   
    regex = r"(Principal from|PRINCIPAL FROM|Principal From|principal from) \d{1,4}(-|/|\\|\||\.)\d{1,4}(-|/|\\|\||\.)\d{1,4} (TO|To|to) \d{1,4}(-|/|\\|\||\.)\d{1,4}(-|/|\\|\||\.)\d{1,4}.{0,10}?(:|\s)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[7])
    else:
        return None

################################################################### INTEREST

def interest_for_IDBI(corpus):   
    regex = r"(A Govt. of India Owned Bank|India Owned Bank).*?(Total for the Year).{0,5}?\s{1,5}(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))\s{1,5}(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))\s{1,5}(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"

    matches = re.finditer(regex, corpus, re.MULTILINE)

    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[12])
    else:
        return None

def interest_amt_HomeLoan1(corpus):
    corpus1 = corpus.lower()    
    regex = r"(interest component|towards interest|linierest component|interest compor|interest amount).{0,40}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||\(|\)|\[|\]|\{|\}|)(|.|..|...|....|.....)(\d+(,|,,|, |\.|\. )\d+(,|, |\.|\. |)\d+(,|, |\.|\. |)(\d+|))(\s|/|)"
    matches = re.finditer(regex, corpus1, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):    
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def interest_amt_HomeLoan2(corpus):
    corpus1 = corpus.lower()    
    regex = r"(interest component|towards interest|linierest component|interest compor|interest inr|total interest).{0,50}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||\(|\)|\[|\]|\{|\}|)(|.|..|...|....|.....)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))(\s|/|)"
    matches = re.finditer(regex, corpus1, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):    
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def interest_amt_HomeLoan3(corpus):
    corpus1 = corpus.lower()    
    regex = r"(interest|linierest|dnterest|terest).{0,40}?(INR|inr|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||\(|\)|\[|\]|\{|\}|)(|.|..|...|....|.....)(\d+(,|,,|, |\.|\. )\d+(,|, |\.|\. |)\d+(,|, |\.|\. |)(\d+|))(\s|/|)"
    matches = re.finditer(regex, corpus1, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):    
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def interest_amt_HomeLoan4(corpus):
    corpus1 = corpus.lower()    
    regex = r"(interest|dnterest|terest|linierest).{0,50}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||\(|\)|\[|\]|\{|\}|)(|.|..|...|....|.....)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))(/)"
    matches = re.finditer(regex, corpus1, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):    
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[4])
    else:
        return None
    
def interest_amt_HomeLoan5(corpus):
    corpus1 = corpus.lower()    
    regex = r"(interest|dnterest|terest|linierest).{0,30}?(INR|Inr|Rs.|RS.|Rs|RS|R8.|rs|eRe.|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||\(|\)|\[|\]|\{|\}|)(|.|..|...|....|.....)(\s|:|\.)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))(\s|/|)"
    matches = re.finditer(regex, corpus1, re.MULTILINE)
    result = []
    for matchNum, match in enumerate(matches, start=1):    
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            result.append(match.group(groupNum))            
    if len(result)!=0:
        return(result[5])
    else:
        return None    

def int_from_date_to_date(corpus):   
    regex = r"(Interest from|INTEREST FROM|Interest From|interest from) \d{1,4}(-|/|\\|\||\.)\d{1,4}(-|/|\\|\||\.)\d{1,4} (TO|To|to) \d{1,4}(-|/|\\|\||\.)\d{1,4}(-|/|\\|\||\.)\d{1,4}.{0,10}?(:|\s)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"

    matches = re.finditer(regex, corpus, re.MULTILINE)

    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[7])
    else:
        return None

In [1544]:
def information_extract_HomeLoan(corpus,rendered):
    provider = find_provider_HomeLoan(corpus)
    if provider is None:
        provider = find_provider_lower(corpus)
        if provider is None:
            provider = find_provider_For_DCB(corpus)
    
    insured_name = find_payee_name_HomeLoan0(corpus)
    if insured_name is None:
        insured_name = find_payee_name_HomeLoan1(corpus)
        if insured_name is None:
            insured_name = find_payee_name_HomeLoan2(corpus)
            if insured_name is None:
                insured_name = find_payee_name_HomeLoan3(corpus)
                if insured_name is None:
                    insured_name = find_payee_name_HomeLoan4(corpus)            
    
    date = find_statement_date_HomeLoan1(corpus)
    if date is None:
        date = find_statement_date_HomeLoan2(corpus)
        if date is None:
            date = date_def2(corpus)
            
    loan_code = find_loan_code_HomeLoan1(corpus)
    if loan_code is None:
        loan_code = find_loan_code_HomeLoan2(corpus)
    
    principal = prem_for_IDBI(corpus)
    if principal is None:
        principal = prem_from_date_to_date(corpus)
        if principal is None:
            principal = principal_amt_HomeLoan1(corpus)
            if principal is None:
                principal = principal_amt_HomeLoan2(corpus)
                if principal is None:
                    principal = principal_amt_HomeLoan3(corpus)                
                    if principal is None:
                        principal = principal_amt_HomeLoan4(corpus)                    
                        if principal is None:
                            principal = principal_amt_HomeLoan5(corpus)
                            if principal is None:
                                principal = principle_amt_between_HomeLoan(corpus)

    interest = interest_for_IDBI(corpus)
    if interest is None:
        interest = int_from_date_to_date(corpus)
        if interest is None:
            interest = interest_amt_HomeLoan1(corpus)
            if interest is None:
                interest = interest_amt_HomeLoan2(corpus)
                if interest is None:
                    interest = interest_amt_HomeLoan3(corpus)
                    if interest is None:
                        interest = interest_amt_HomeLoan4(corpus)
                        if interest is None:
                            interest = interest_amt_HomeLoan5(corpus)            

    return(provider,insured_name,date,loan_code,principal,interest)


### Handicap

In [1545]:
###################################################################INSURED NAME

def find_payee_name_handy1(corpus):
    regex = r"(\b(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Mr./Mrs.|Name:|Dear MR.|Smt.|Ms.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri|Sri|Sri.|Shri/Smt\.|Shri/Si|MR\s)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,}))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_payee_name_handy2(corpus):
    regex = r"(\b(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Mr./Mrs.|Name:|Dear MR.|Smt.|Ms.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri|Sri|Sri.|Shri/Smt\.|Shri/Si|MR\s|This is to certify that|THIS IS TO CERTIFY THAT|this is to certify that|To Whom It May Concern|To Whomsoever It May Concern|To whomsoever it may concern|Dear|DEAR)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,}))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################DATE

def find_date_handy(corpus):
    regex = r"((Date|Date:|DATE|date|Date of Receipt|Date of issue|DATE OF ISSUE|Date and Time :|Receipt Date|INTEREST CERTIFICATE))( {0,1})( |:|)( {0,1})(\d+|\w{3,})(-|/|\||\.|,|\s)(\d+|\w{3,})(-|/|\||\.|,|\s)(\d+|\w{3,})"
#     regex = r"((Date|Date:|DATE|date|Date of Receipt|Date of issue|DATE OF ISSUE|Date and Time :|Receipt Date|INTEREST CERTIFICATE)).*?((\d+(.|)(/|\\)(.|)\d+)|\d+)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################DISABILITY PERCENTAGE

def find_disability_pctg_handy(corpus):
    regex = r"(Percentage Of Disability|Disability|Disability Percentage|Causes of loss of functional capacity).*?\d+(\s|)(%|percent|PERCENT|Percent)|(IQ|iq|1Q|1q).*?\d+(\s%|\s)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return("0%")


In [1546]:
def information_extract_handy(corpus):
    Patient_Name = find_payee_name_handy1(corpus)
    if Patient_Name is None:
        Patient_Name = find_payee_name_handy2(corpus)
    
    Date = find_date_handy(corpus)
    if Date is None:
        Date = date_def2(corpus)
    
    Disability_Pctg = find_disability_pctg_handy(corpus)
    return(Patient_Name,Date,Disability_Pctg)


### Special Disease

In [1547]:
def find_amount_handy(corpus):
    regex = r"(Total Amount|Paid Amount|Amount Paid|amount paid|Amount paid|Amount|amount)(\s|)(INR|Inr|Rs.|RS.|Rs|RS|rs|\$|\(USD\) \$|£|€|)(\s|)\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)"

    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def information_extract_special_disease(corpus,rendered):
    
    Patient_Name = find_payee_name_handy1(corpus)
    if Patient_Name is None:
        Patient_Name = find_payee_name_handy2(corpus)
    
    Date = find_date_handy(corpus)
    if Date is None:
        Date = date_def2(corpus)
    #Disability_Pctg = find_disability_pctg_handy(corpus)
    amt = find_amount_handy(corpus)
    return(Patient_Name,Date,amt)


### ULIP

In [1548]:
###################################################################PROVIDER

def find_provider_ulip(corpus):
    regex=r"\b(STAR HEALTH AND ALLIED INSURANCE|STAR COMPREHENSIVE INSURANCE|Star Health|STAR HEALTH|Apollo Munich|APOLLO MUNICH|Apollo|APOLLO|STATE BANK OF INDIA|State Bank of India|SBI Life|SBI|sbi|EXIDE Life|Exide Life|Exide|EXIDE|ICICI PRUDENTIAL|ICICI Lombard|ICICI LOMBARD|ICICI|icici|Max Life|MAX LIFE|max life|HDFC LIFE|HDFC Life|hdfc life|HDFC ERGO|hdfc ergo|HDFC Standard Life|HDFC ERGO General Insurance|HOFC Life|HDFC Mutual Fund|HDFC|hdfc|LIC of India|LIC OF INDIA|Life Insurance Corporation|LIFE INSURANCE CORPORATION|LIC|lic|kotak life|KOTAK LIFE|Kotak Life|Kotak Mahindra Life|KOTAK LIFE|KOTAK|kotak|NEW INDIA ASSURANCE|New India Assurance|NEW INDIA|Aegon Life|Aegon life|aegon life|aegon|AEGON|Bajaj Allianz|BAJAJ ALLIANZ|BAJAJ|bajaj|Birla Sun Life Mutual Fund|Birla Sun Life Insurance|Birla Sun Life|ABSL|BIRLA SUN LIFE|birla sun life|BIRLA SUN|Birla Sun|PUNJAB NATIONAL BANK|Punjab National Bank|Punjab National|punjab national|PNB MetLife India Insurance|PNB MetLife India|PNB MetLife|pnb metlife|PNB METLIFE|PNB|pnb|AVIVA|Aviva|aviva|Bharti axa|Bharti AXA|bharti axa|MAX BUPA|max bupa|Max Bupa|UTI|uti|Bank of Baroda|DHFL|Dhfl|IDBI|GIC|Mumbai District Central bank|Mumbai District Central Co-op. Bank|Mumbai District Central|Mumbai.*?Co-op. Bank|LIC HFL|LIC Housing Finance Ltd|SARASWAT Bank|Saraswat Bank|SARASWAT|Saraswat|Royal Sundaram General|royal sundaram|Royal Sundaram|UNITED INDIA INSURANCE|United India|united india|ORIENTAL INSURANCE|Oriental Insurance|oriental insurance|National Mediclaim|L&T Mutual Fund|L&T|Axis Mutual Fund|Axis|axis|Motilal Oswal Mutual Fund|Motilal Oswal|RELIANCE Mutual fund|RELIANCE|Mahindra Mutual Fund|Mahindra|Franklin Templeton|Franklin India|DSP BLACKROCK|OSP BLACKROCK|DSP BlackRock|DSP|Dewan Housing Finance Corporation|Dewan Housing Finance|Dewan Housing|TATA AIA|oSBI Life)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_provider_For_DCB(corpus):
    regex = r"\b(DCB Bank|DCB|dcb bank)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################INSURED NAME

def find_payee_name_ulip(corpus):
    regex = r"\b(SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|This is to certify that|THIS IS TO CERTIFY THAT|this is to certify that|To Whom It May Concern|To Whomsoever It May Concern|To whomsoever it may concern|This is to state that|Policy Holder Name|Policyowner|Dear)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
#     regex = r"(\b(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Mr./Mrs.|Name:|ocaed Name|Dear MR.|Smt.|Ms.|Shri/Si|Shri/gme|Shri|Sri|Sri.|Shri/Smt\.|Shri/Si|MR\s|Dear|To whomsoever it may concern|Policyowner)\b(.|..|...|....|.....)(\w+(\s|\s\s|.\s|.)\w+(\s|\s\s|.\s|.)\w{2,}|\w+ \w+))"
#     regex = r"(\b(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Mr./Mrs.|Name:|Dear MR.|Smt.|Ms.|Shri/Si|Shri/gme|Shri|Sri|Sri.|Shri/Smt\.|Shri/Si|MR\s|Dear|To whomsoever it may concern)\b(.|..|...|....|.....)(\w+(\s|\s\s|.\s|.)\w+(\s|\s\s|.\s|.)\w{2,}|\w+ \w+))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

# def find_payee_name_before_client_code_ulip(corpus):
#     regex = r"(\w+(\s|\s\s|.\s|.)\w+(\s|\s\s|.\s|.)\w{2,}|\w+ \w+)(.|..)(Unique Client Code|unique client code|unique code|client code|Client Code|Client code)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

###################################################################POLICY NUMBER

def find_policy_no_with_section_ulip(corpus):
    regex = r"(Policy No\.|Policy No|policy no\.|policy no|Folio No\.|Folio No|Folio no|folio no\.|folio no|)((Section|section)(\s|)(80C|)(.*?\s|)(Unit Linked|unit linked|unit Linked|linked|Linked)).*?((\d+(.|)(/|\\| )(.|)\d+)|\d+)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_policy_no_ulip(corpus):
    regex = r"(Folio No|Folio no|Folio Number|FOLIO NO|FOLIO NUMBER|folio no|folio number|Folio number|FolioNo|Policy Number|policy number|Policy No:|Policy No|policy no).*?((\d+(.|)(/|\\| )(.|)\d+)|\d+)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################DATE

def find_date_ulip(corpus):
    regex = r"((Payment Date and Time|Date of Receipt|DATE OF ISSUE:|DATE OF ISSUE :|Date and Time|Date...Time.|Date..Time...|Date...Time....|Receipt Date|INSURANCE Date:|Payment Date and Time :|TNO|Tr\. No\.)(\s|)(.|..|...|....|.....)\b(.*?)((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4}\b\s)"
#     regex = r"((Date|Date:|DATE|date|Date of Receipt|Date of issue|DATE OF ISSUE|Date and Time :|Receipt Date|INTEREST CERTIFICATE)).*?((\d+(.|)(/|\\)(.|)\d+)|\d+)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        dt =  match.group()
        return date_def2(dt)
    return None

def find_date_date_month_ulip(corpus):
    regex = r"\d{1,2}(/|-|\||\.|\\|\s|,\s)\w{3,9}(/|-|\||\.|\\|\s|,\s)\d{2,4}"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_date_only_number_ulip(corpus):
    regex = r"\d{1,2}(/|-|\||\.|\\)\d{1,2}(/|-|\||\.|\\)\d{2,4}"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def find_date_month_date_ulip(corpus):
    regex = r"\w{3,9}(/|-|\||\.|\\|\s|,\s)\d{1,2}(/|-|\||\.|\\|\s|,\s)\d{2,4}"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

###################################################################AMOUNT

# def find_amount_ulip(corpus):
#     regex = r"(Premium Paid|Premium amount|Premium \(A\)|Target Amount|Amount)(\s|).*?(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|)(\s|)\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)"
# #     regex = r"(Principal Amount|Balance|Total Interest debited during the period|Interest Component).*?(INR|Rs.|RS.|Rs|RS) \d+(,|\.|)\d+(,|\.|)\d+"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

def find_amount_ulip(corpus):
    regex = r"(Contibution Amount|Contribution|Premium Paid|Premium amount|Premium \(A\)|Amount Received|Gross Amount|Received Amount|Received amount|Amount Paid)(\s|).*?(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|)(\s|)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"
    matches = re.finditer(regex, corpus, re.MULTILINE)

    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[4])
    else:
        return None

In [1549]:
def information_extract_ulip(corpus,rendered):
    Provider = find_provider_ulip(corpus)
    if Provider is None:
        provider = find_provider_lower(corpus)
        if provider is None:
            Provider = find_provider_For_DCB(corpus)
        
    Payee_Name = find_payee_name_ulip(corpus)
#     if Payee_Name is None:
#         Payee_Name = find_payee_name_before_client_code_ulip(corpus)
        
    Date = find_date_ulip(corpus)
    if Date is None:
        Date = find_date_date_month_ulip(corpus)
        if Date is None:
            Date = find_date_only_number_ulip(corpus)
            if Date is None:
                Date = find_date_month_date_ulip(corpus)
                if Date is None:
                    Date = date_def2(corpus)
    
    Policy_No = find_policy_no_with_section_ulip(corpus)
    if Policy_No is None:
        Policy_No = find_policy_no_ulip(corpus)
    
    premium_amt = find_amount_ulip(corpus)
    if premium_amt is None:
        premium_amt = amt_lic_1(corpus)
        if premium_amt is None:
            premium_amt = amt_lic_2(corpus)
            if premium_amt is None:
                premium_amt = find_premium_cor1(corpus)
                premium_amt = premium_amt
                if premium_amt is None:
                    premium_amt = find_premium_cor1_1(corpus)
                    if premium_amt is None:
                        premium_amt = find_premium_cor2(corpus)
                        premium_amt = premium_amt
                        if premium_amt is None:
                            premium_amt = med_prem_amt(corpus)
                            premium_amt = premium_amt
                            if premium_amt is None:
                    #             premium_amt = find_premium_ex(corpus)
                    #             premium_amt=premium_amt
                    #             if premium_amt is None:
                                premium_amt = prem_amt(corpus)
                                premium_amt=premium_amt
                                if premium_amt is None:
                                    premium_amt = prem_amt1(corpus)
                                    premium_amt=premium_amt
                                    if premium_amt is None:
                    #                     premium_amt=find_premium_back(rendered)
                    #                     premium_amt=premium_amt
                    #                     if premium_amt is None:
                                        premium_amt = lic_amount1(corpus)
    
    CGST = CGST0(corpus)
    if CGST is None:
        CGST = CGST1(corpus)
        if CGST is None:
            CGST = CGST2(corpus)
        
    SGST = SGST0(corpus)
    if SGST is None:
        SGST = SGST1(corpus)
#         if SGST is None:
#             SGST = SGST2(corpus)
    
    remark1 = cgst_remark1(corpus)
    if remark1 is None:
        remark1 = cgst_remark2(corpus)
        
    remark2 = sgst_remark1(corpus)
    if remark2 is None:
        remark2 = sgst_remark2(corpus)
    
    return(Provider,Payee_Name,Date,Policy_No,premium_amt,CGST,SGST,remark1,remark2)


### Rental Agreement

In [1550]:

################################################################### INSURED NAME

def name_rent(corpus):
#     regex=r"((M/s|Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|MR |Policy Holder Name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|Name & Father Name of Investor|Name & Father Name of Investor 1)(.|..|...|....)((\w+ \w+ \w{3,})|(\w \w+)|(\w+ \w+)))|\b(Dear)\b(.|..)(\w+ \w+ \w{3,}|\w+ \w+)"
    regex=r"((Received with thanks from|Insured Name|insured Name|Policy Holder Name|Premium Payor Name|Policyholder's Name|Name of Nominee|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|received for life insurance policies from the userid of|life insurance policies from the userid of|from the userid of|Name of the Life Assured)(.|..|...|....)|)\b(Sh\./Smt\./ M/s|Smt/Ms/Shri|SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Mr/Ms|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Policyholder Name|Name of the Proposer|ime of the Proposer|Name & Father Name of Investor|Name & Father Name of Investor 1|licensee name and address|licensee name|Received from|Licensor Name and Address 1\) Name|Licensor Name and Address|Licensor Name)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})|\b(Dear)\b(.|..)(\w+ \w+ \w{3,}|\w+ \w+)|RENT RECEIPT(.|..|...|....|.....)(\w+ \w+ \w{3,}|\w+ \w+)(.|..|...|....|.....)Received from"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

################################################################### DATE
    
def date_rent1(corpus):
    regex = r"((Date|Date:|DATE|date|Date of Receipt|Date of issue|DATE OF ISSUE|Date and Time :|Receipt Date|INTEREST CERTIFICATE))( {0,1})( |:|)( {0,1})(\d+|\w{3,})(-|/|\||\.|,|\s)(\d+|\w{3,})(-|/|\||\.|,|\s)(\d+|\w{3,})|(\d+(-)((?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?))(-)\d+)|\d{1,2}-\d{1,2}-\d{2,4}|((?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?))( |)\d{1,2}(, |,| ,| , )\d{2,4}"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

def date_rent2(corpus):
    regex = r"\b((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{1,4}\b(\s|\.)"
#     regex = r"(\d{1,4}|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||,|,\s|\s)(\d{1,4}|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||,|,\s|\s)\d{1,4}"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None


###################################################################AMOUNT

def amt_rent(corpus):
    regex = r"\b(Principal Amt|Principal Amount|Principal amount|sum of|asum of|Amount Paid|Amount paid|amount paid|AMOUNT PAID|Total Repayment Received|total repayment received|Total Amount Rs|Total Amount|total amount|amount of Rs|amount of|Amt. Collected|Amount Deposited|Rent of Rs|Total|AMOUNT|Amount|amount|amt|agreed rate of rent of the premises to be let out is fixed at|rent of the premises to be let out is fixed at|monthly rent of the above said premises is revised/decided|rent of the above said premises is revised/decided)\b(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(|.|..|...|....|.....)(\d+(,|,,|, |\.|\. |)\d+(,|, |\.|\. |)\d+(,|, |\.|\. |)(\d+|))(\s|/|)"

    matches = re.finditer(regex, corpus, re.MULTILINE)

    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[5])
    else:
        return None
    
# def amt_rent1(corpus):
#     regex = r"(\b(Premium.A.|Principal Amt|Premium Amount|Premium amount|Total Premium Amount|Amount Paid|Total Premium Paid|Total|Total Amount Rs. |amount of|Amount paid|premium amount of Rs.|Total Premium :Rs|Amt. Collected|Total Premium :Rs.)\b...((\d+.\d+)|(: \d+.\d\d)|(\d+,\d+.\d\d)))"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

# def amt_rent2(corpus):
#     regex = r"((Amount|amount|amt|Amount paid|Total Premium Paid|Amount Deposited)(.|)(.|)\w+(.|)\w+(.|)(Rs|rs).*?((\d+|\w+)|(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|)(\s|)\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)))"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

# def amt_rent3(corpus):
#     regex = r"(Total Repayment Received|total repayment received).*?(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)\d+(,|\.|\d)\d+(,|\.)\d+(,|\.|)(\d+|)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

# def amt_rent4(corpus):
#     regex = r"(Amount paid|amount paid|Amount Deposited|AMOUNT|amount|Amount|Total Amount|total amount)(.|..|...|....)(\d+\,\d+\.\d+|\d+\.\d+|\d+)"
# #     regex = r"(Amount paid|amount paid|Amount Deposited|Total Amount|total amount)(.|..|...|....)\d+"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

In [1551]:
def information_extract_rent(corpus,rendered):
        
    insured_name = name_rent(corpus)
    
    date = date_rent1(corpus)
    if date is None:
        date = date_rent2(corpus)

    amount = amt_rent(corpus)
#     if amount is None:
#         amount = amt_rent2(corpus)
#         if amount is None:
#             amount = amt_rent3(corpus)
#             if amount is None:
#                 amount = amt_rent4(corpus)
    return(insured_name,date,amount)


### PPF

In [1552]:
###################################################################PROVIDER

def provider_ppf(corpus):
    regex=r"\b(STAR HEALTH AND ALLIED INSURANCE|STAR COMPREHENSIVE INSURANCE|Star Health|STAR HEALTH|Apollo Munich|APOLLO MUNICH|Apollo|APOLLO|STATE BANK OF INDIA|State Bank of India|SBI Life|SBI|sbi|EXIDE Life|Exide Life|Exide|EXIDE|ICICI PRUDENTIAL|ICICI Lombard|ICICI LOMBARD|ICICI|icici|Max Life|MAX LIFE|max life|HDFC LIFE|HDFC Life|hdfc life|HDFC ERGO|hdfc ergo|HDFC Standard Life|HDFC ERGO General Insurance|HOFC Life|HDFC Mutual Fund|HDFC|hdfc|LIC of India|LIC OF INDIA|Life Insurance Corporation|LIFE INSURANCE CORPORATION|LIC|lic|kotak life|KOTAK LIFE|Kotak Life|Kotak Mahindra Life|KOTAK LIFE|KOTAK|kotak|NEW INDIA ASSURANCE|New India Assurance|NEW INDIA|Aegon Life|Aegon life|aegon life|aegon|AEGON|Bajaj Allianz|BAJAJ ALLIANZ|BAJAJ|bajaj|Birla Sun Life Mutual Fund|Birla Sun Life Insurance|Birla Sun Life|ABSL|BIRLA SUN LIFE|birla sun life|BIRLA SUN|Birla Sun|PUNJAB NATIONAL BANK|Punjab National Bank|Punjab National|punjab national|PNB MetLife India Insurance|PNB MetLife India|PNB MetLife|pnb metlife|PNB METLIFE|PNB|pnb|AVIVA|Aviva|aviva|Bharti axa|Bharti AXA|bharti axa|MAX BUPA|max bupa|Max Bupa|UTI|uti|Bank of Baroda|DHFL|Dhfl|IDBI|GIC|Mumbai District Central bank|Mumbai District Central Co-op. Bank|Mumbai District Central|Mumbai.*?Co-op. Bank|LIC HFL|LIC Housing Finance Ltd|SARASWAT Bank|Saraswat Bank|SARASWAT|Saraswat|Royal Sundaram General|royal sundaram|Royal Sundaram|UNITED INDIA INSURANCE|United India|united india|ORIENTAL INSURANCE|Oriental Insurance|oriental insurance|National Mediclaim|L&T Mutual Fund|L&T|Axis Mutual Fund|Axis|axis|Motilal Oswal Mutual Fund|Motilal Oswal|RELIANCE Mutual fund|RELIANCE|Mahindra Mutual Fund|Mahindra|Franklin Templeton|Franklin India|DSP BLACKROCK|OSP BLACKROCK|DSP BlackRock|DSP|Dewan Housing Finance Corporation|Dewan Housing Finance|Dewan Housing)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

################################################################### INSURED NAME

def name_ppf(corpus):
#     regex=r"((M/s|Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|MR |Policy Holder Name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|Name & Father Name of Investor|Name & Father Name of Investor 1)(.|..|...|....)((\w+ \w+ \w{3,})|(\w \w+)|(\w+ \w+)))|\b(Dear)\b(.|..)(\w+ \w+ \w{3,}|\w+ \w+)"
    regex=r"\b(Smt\./Ms\./Shri|SmtiMsiShri|Smt/Ms\./SK|Smt/Ms_/SKi|Smt/Ms\./Sri|Smt\./Ms\./Shri: MAST|Smt/MsJ/Shrit|smt/Ms/Shri|SmtJ/Ms\./Shri|Smt/ivis/Sri|Shri/Ss|ShrilSmt|Smt/MsJ/Shri|Smt/MsJShri|SmtJMsJ/Shri|SmtiMs/Shri|Smt\./Ms\./Shri|Smit/MsJ/Shri|Shri/Smt\.Smt\.|Smt\./Ms/Sri\. Sree WATT|Smt\./Ms/Sri\. Sree|Smt/Ms/Sr|Smt/Ms\./St|M/s|Smt/Ms\./Shri|Mr\./Mrs\.|Shri/Smt/Kum|Shri/Si|Shri/gme|Shri/Smt\.|Mrs|MRS|Mr|MR|Ms|MS|Miss|MISS|Shri|SHRI|Sri|SRI|Smt|SMT|Name:|NAME:|Policy Holder Name|insured Name|Insured Name|insured name|Insured name|Premium Payor Name|Policyholder's Name|Account Name|Life Insured|Life insured|Owner Name|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of|The following premium has been received for life insurance policies|The following premia have been received under the following life insurance policies of|received for life insurance policies from the userid of|Received with thanks from|life insurance policies from the userid of|from the userid of|Name of the Life Assured|Policyholder Insurance Details|Insured's Name|Insured/Recipient Name and Address|Tnsured/Recipient Name and Address|Name of the Policy Holder|Name & Father Name of Investor|Name & Father Name of Investor 1|Customer Name|Name of Subscriber)\b(.|..|...|....|.....)(\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )\w+(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{1,}(\s|\.|\,|\: |] |\.\s|\|\s|! |\) |\} |\\. )(\w{4,}|Bal|BAL|Ben|BEN|Bir|BIR|Dar|DAR|Das|DAS|Dey|DEY|Din|DIN|Deo|DEO|Jai|JAI|Jha|JHA|Kar|KAR|Lad|LAD|Lal|LAL|Oak|OAK|Pai|PAI|Pal|PAL|Rai|RAI|Rao|RAO|Rau|RAU|Ray|RAY|Rey|REY|Roy|ROY|Sai|SAI|Sha|SHA|Sen|SEN)|\w{4,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

################################################################### DATE
    
def date_ppf1(corpus):
    regex = r"(Date|Date:|DATE|date|Date of Receipt|Date of issue|DATE OF ISSUE|Date and Time :|Receipt Date|INTEREST CERTIFICATE)(\s|)(.|..|...|....|.....)\b(((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{4})(\b\s|-)"
    matches = re.finditer(regex, corpus, re.MULTILINE)

    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[3])
    else:
        return None

def date_ppf2(corpus):
    regex = r"\b(((?:(0|O|o|)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|(?:19(?:[0-9][0-9]))|(?:(20|2O|2o)(?:[0-9][0-9]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)((?:(0|O|o)(?:[1-9]))|(?:1(?:[0-9]))|(?:2(?:[0-9]))|(?:3(?:[0-1]))|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||~|,|,\s|\s|\"\s|\'\s|.\s|..\s)\d{4})(\b\s|-)"
#     regex = r"(\d{1,4}|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||,|,\s|\s)(\d{1,4}|((?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sept(?:ember)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sept(?:ember)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|(?:JAN(?:UARY)?|FEB(?:RUARY)?|MAR(?:CH)?|APR(?:IL)?|MAY|JUN(?:E)?|JUL(?:Y)?|AUG(?:UST)?|SEPT(?:EMBER)?|SEP(?:TEMBER)?|OCT(?:OBER)?|NOV(?:EMBER)?|DEC(?:EMBER)?)))(/|-|\.|\\|\||,|,\s|\s)\d{1,4}"
    matches = re.finditer(regex, corpus, re.MULTILINE)

    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[0])
    else:
        return None


###################################################################AMOUNT

def amt_ppf(corpus):
    regex = r"\b(Principal Amt|Principal Amount|Principal amount|sum of|Amount Paid|Amount paid|amount paid|AMOUNT PAID|Total Repayment Received|total repayment received|Total Amount Rs|Total Amount|total amount|amount of Rs|amount of|Amt. Collected|Amount Deposited|Rent of Rs|Total|AMOUNT|Amount|amount|amt)\b(.|..|...|....|.....)(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(|.|..|...|....|.....)(\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|))"

    matches = re.finditer(regex, corpus, re.MULTILINE)

    result = []
    for matchNum, match in enumerate(matches, start=1):

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            result.append(match.group(groupNum))
            
    if len(result)!=0:
        return(result[5])
    else:
        return None
    
# def amt_ppf1(corpus):   
#     regex = r"(\b(Amount Paid|Total|Total Amount Rs. |amount of|Amount paidTotal Premium :Rs|Amt. Collected)\b...((\d+.\d+)|(: \d+.\d\d)|(\d+,\d+.\d\d)))"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

# def amt_ppf2(corpus):
#     regex = r"((Amount|amount|amt|Amount paid|Total Premium Paid|Amount Deposited)(.|)(.|)\w+(.|)\w+(.|)(Rs|rs).*?((\d+|\w+)|(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|)(\s|)\d+(,|\.|)\d+(,|\.|)\d+(,|\.|)(\d+|)))"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

# def amt_ppf3(corpus):
#     regex = r"(Total Repayment Received|total repayment received).*?(INR|Inr|Rs.|RS.|Rs|RS|rs|\(Rs\.\)|\$|\(USD\) \$|£|€|\*{1,}|&|f|%|=|=(\s|)\*{1,}|\(z\)|\(zy\)|\||)(\s|)\d+(,|\.|\d)\d+(,|\.)\d+(,|\.|)(\d+|)"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

# def amt_ppf4(corpus):
#     regex = r"(Amount paid|amount paid|Amount Deposited|AMOUNT|amount|Amount|Total Amount|total amount)(.|..|...|....)(\d+\,\d+\.\d+|\d+\.\d+|\d+)"
# #     regex = r"(Amount paid|amount paid|Amount Deposited|Total Amount|total amount)(.|..|...|....)\d+"
#     matches = re.finditer(regex, corpus, re.MULTILINE)
#     for match in matches:
#         return match.group()
#     return None

In [1553]:
def information_extract_ppf(corpus,rendered):
    provider = provider_ppf(corpus)
    if provider is None:
        provider = find_provider_lower(corpus)
        if provider is None:
            provider = find_provider_For_DCB(corpus)
        
    insured_name = name_ppf(corpus)
    
    date = date_ppf1(corpus)
    if date is None:
        date = date_ppf2(corpus)


#     loan_code = find_loan_code_HomeLoan(corpus)
    amount = amt_ppf(corpus)
#     if amount is None:
#         amount = amt_ppf2(corpus)
#         if amount is None:
#             amount = amt_ppf3(corpus)
#             if amount is None:
#                 amount = amt_ppf4(corpus)
    return(provider,insured_name,date,amount)


### Default Function Call

In [1554]:
def information_extract_default(corpus,rendered):
    provider = provider_def(corpus)
    if provider is None:
        provider = find_provider_lower(corpus)
        if provider is None:
            provider = find_provider_For_DCB(corpus)
        
    insured_name = name_def(corpus)
    date = date_def1(corpus)
    if date is None:
        date = date_def2(corpus)
    
    policy_number = policy_def1(corpus)
    if policy_number is None:   
        policy_number = policy_def2(corpus)
        if policy_number is None:   
            policy_number = policy_def3(corpus)

#     loan_code = find_loan_code_HomeLoan(corpus)
    amount = amt_def(corpus)
#     if amount is None:
#         amount = amt_def2(corpus)
#         if amount is None:
#             amount = amt_def3(corpus)
#             if amount is None:
#                 amount = amt_def4(corpus)
    return(provider,insured_name,date,policy_number,amount)

## Main Function

In [1555]:
def extraction():
    path = []
    provider = []
    policy_number = []
    premium_amt = []
    insured_name = []
    premium_date = []
    investment_type = []
    payment_frequency = []
    amount1 = []
    amount2 = []
    amount3 = []
    remark1 = []
    remark2 = []
    remark3 = []
    scheme = []
    filename = []
    for i in range(0,len(df_result)):
        print(i)
        corpus = df_result.loc[i,'corpus']
        rendered = df_result.loc[i,'rendered_text']


        if df_result.loc[i,'INVESTMENT_TYPE']=='LIFE INSURANCE_DEFAULT':
            investment = 'LIFE INSURANCE'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_LIC = 'NA'
            amount3_LIC = 'NA'
            remark3_LIC = 'NA'
            #provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)

            provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,amount1_LIC,amount2_LIC,remark1_LIC,remark2_LIC,pay_mode_LIC = information_extract_LIC_def(corpus,rendered)
            provider.append(provider_LIC)
            
            if (policy_number_LIC != None) and (len(policy_number_LIC)>1):
                policy_number_LIC = re.findall('\d{2,}',policy_number_LIC)
                if len(policy_number_LIC) != 0:
                    policy_number_LIC = policy_number_LIC[0]
            else:
                pass

            
            policy_number.append(policy_number_LIC)
            premium_amt.append(premium_amt_LIC)
            
            insured_name.append(insured_name_LIC)
            premium_date.append(premium_date_LIC)
            investment_type.append(investment)
            amount1.append(amount1_LIC)
            amount2.append(amount2_LIC)
            amount3.append(amount3_LIC)
            remark1.append(remark1_LIC)
            remark2.append(remark2_LIC)
            remark3.append(remark3_LIC)
            payment_frequency.append(pay_mode_LIC)
            scheme.append(scheme_LIC)
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='LIFE INSURANCE_LIC':
            investment = 'LIFE INSURANCE'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_LIC = 'NA'

            #provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)


            provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,amount1_LIC,amount2_LIC,amount3_LIC,remark1_LIC,remark2_LIC,remark3_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)
            provider.append(provider_LIC)
            
            if (policy_number_LIC != None) and (len(policy_number_LIC)>1):
                policy_number_LIC = re.findall('\d{2,}',policy_number_LIC)
                if len(policy_number_LIC) != 0:
                    policy_number_LIC = policy_number_LIC[0]
            else:
                pass

            policy_number.append(policy_number_LIC)
            premium_amt.append(premium_amt_LIC)
            
            insured_name.append(insured_name_LIC)
            premium_date.append(premium_date_LIC)
            investment_type.append(investment)
            amount1.append(amount1_LIC)
            amount2.append(amount2_LIC)
            amount3.append(amount3_LIC)
            remark1.append(remark1_LIC)
            remark2.append(remark2_LIC)
            remark3.append(remark3_LIC)
            payment_frequency.append(pay_mode_LIC)
            scheme.append(scheme_LIC)
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='LIFE INSURANCE_BIRLA':
            investment = 'LIFE INSURANCE'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_LIC = 'NA'
            amount3_LIC = 'NA'
            remark3_LIC = 'NA'
            #provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)

            provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,amount1_LIC,amount2_LIC,remark1_LIC,remark2_LIC,pay_mode_LIC = information_extract_BIRLA(corpus,rendered)
            provider.append(provider_LIC)
            
            if (policy_number_LIC != None) and (len(policy_number_LIC)>1):
                policy_number_LIC = re.findall('\d{2,}',policy_number_LIC)
                if len(policy_number_LIC) != 0:
                    policy_number_LIC = policy_number_LIC[0]
            else:
                pass

            policy_number.append(policy_number_LIC)
            
            premium_amt.append(premium_amt_LIC)
            
            insured_name.append(insured_name_LIC)
            premium_date.append(premium_date_LIC)
            investment_type.append(investment)
            amount1.append(amount1_LIC)
            amount2.append(amount2_LIC)
            amount3.append(amount3_LIC)
            remark1.append(remark1_LIC)
            remark2.append(remark2_LIC)
            remark3.append(remark3_LIC)
            payment_frequency.append(pay_mode_LIC)
            scheme.append(scheme_LIC)            


            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='LIFE INSURANCE_EXIDE':
            investment = 'LIFE INSURANCE'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_LIC = 'NA'
            amount3_LIC = 'NA'
            remark3_LIC = 'NA'
            #provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)

            provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,amount1_LIC,amount2_LIC,remark1_LIC,remark2_LIC,pay_mode_LIC = information_extract_EXIDE(corpus,rendered)
            provider.append(provider_LIC)
            
            if (policy_number_LIC != None) and (len(policy_number_LIC)>1):
                policy_number_LIC = re.findall('\d{2,}',policy_number_LIC)
                if len(policy_number_LIC) != 0:
                    policy_number_LIC = policy_number_LIC[0]
            else:
                pass

            policy_number.append(policy_number_LIC)
            premium_amt.append(premium_amt_LIC)
            
            insured_name.append(insured_name_LIC)
            premium_date.append(premium_date_LIC)
            investment_type.append(investment)
            amount1.append(amount1_LIC)
            amount2.append(amount2_LIC)
            amount3.append(amount3_LIC)
            remark1.append(remark1_LIC)
            remark2.append(remark2_LIC)
            remark3.append(remark3_LIC)
            payment_frequency.append(pay_mode_LIC)
            scheme.append(scheme_LIC)
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='LIFE INSURANCE_SBI':
            investment = 'LIFE INSURANCE'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_LIC = 'NA'
            amount3_LIC = 'NA'
            remark3_LIC = 'NA'
            #provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)

            provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,amount1_LIC,amount2_LIC,remark1_LIC,remark2_LIC,pay_mode_LIC = information_extract_SBI(corpus,rendered)
            provider.append(provider_LIC)

            policy_number.append(policy_number_LIC)
            premium_amt.append(premium_amt_LIC)
            
            insured_name.append(insured_name_LIC)
            premium_date.append(premium_date_LIC)
            investment_type.append(investment)
            amount1.append(amount1_LIC)
            amount2.append(amount2_LIC)
            amount3.append(amount3_LIC)
            remark1.append(remark1_LIC)
            remark2.append(remark2_LIC)
            remark3.append(remark3_LIC)
            payment_frequency.append(pay_mode_LIC)
            scheme.append(scheme_LIC)
            
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='LIFE INSURANCE_AVIVA':
            investment = 'LIFE INSURANCE'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_LIC = 'NA'
            amount3_LIC = 'NA'
            remark3_LIC = 'NA'
            #provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)

            provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,amount1_LIC,amount2_LIC,remark1_LIC,remark2_LIC,pay_mode_LIC = information_extract_AVIVA(corpus,rendered)
            provider.append(provider_LIC)
            
            if (policy_number_LIC != None) and (len(policy_number_LIC)>1):
                policy_number_LIC = re.findall('\d{2,}',policy_number_LIC)
                if len(policy_number_LIC) != 0:
                    policy_number_LIC = policy_number_LIC[0]
            else:
                pass

            policy_number.append(policy_number_LIC)
            premium_amt.append(premium_amt_LIC)
            
            insured_name.append(insured_name_LIC)
            premium_date.append(premium_date_LIC)
            investment_type.append(investment)
            amount1.append(amount1_LIC)
            amount2.append(amount2_LIC)
            amount3.append(amount3_LIC)
            remark1.append(remark1_LIC)
            remark2.append(remark2_LIC)
            remark3.append(remark3_LIC)
            payment_frequency.append(pay_mode_LIC)
            scheme.append(scheme_LIC)
            

        elif df_result.loc[i,'INVESTMENT_TYPE']=='LIFE INSURANCE_MAX LIFE':
            investment = 'LIFE INSURANCE'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_LIC = 'NA'

            #provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)

            provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_MAX_LIFE(corpus,rendered)
            provider.append(provider_LIC)
            
            if (policy_number_LIC != None) and (len(policy_number_LIC)>1):
                policy_number_LIC = re.findall('\d{2,}',policy_number_LIC)
                if len(policy_number_LIC) != 0:
                    policy_number_LIC = policy_number_LIC[0]
            else:
                pass

            policy_number.append(policy_number_LIC)
            premium_amt.append(premium_amt_LIC)
            
            insured_name.append(insured_name_LIC)
            premium_date.append(premium_date_LIC)
            investment_type.append(investment)
            amount1.append('NA')
            amount2.append('NA')
            amount3.append('NA')
            remark1.append('NA')
            remark2.append('NA')
            remark3.append('NA')
            payment_frequency.append(pay_mode_LIC)
            scheme.append(scheme_LIC)
            
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='LIFE INSURANCE_AEGON':
            investment = 'LIFE INSURANCE'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_LIC = 'NA'
            amount3_LIC = 'NA'
            remark3_LIC = 'NA'
            #provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)

            provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,amount1_LIC,amount2_LIC,remark1_LIC,remark2_LIC,pay_mode_LIC = information_extract_AEGON(corpus,rendered)
            provider.append(provider_LIC)
            
            if (policy_number_LIC != None) and (len(policy_number_LIC)>1):
                policy_number_LIC = re.findall('\d{2,}',policy_number_LIC)
                if len(policy_number_LIC) != 0:
                    policy_number_LIC = policy_number_LIC[0]
            else:
                pass

            policy_number.append(policy_number_LIC)
            premium_amt.append(premium_amt_LIC)
            
            insured_name.append(insured_name_LIC)
            premium_date.append(premium_date_LIC)
            investment_type.append(investment)
            amount1.append(amount1_LIC)
            amount2.append(amount2_LIC)
            amount3.append(amount3_LIC)
            remark1.append(remark1_LIC)
            remark2.append(remark2_LIC)
            remark3.append(remark3_LIC)
            payment_frequency.append(pay_mode_LIC)
            scheme.append(scheme_LIC)
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='LIFE INSURANCE_TATA AIA':
            investment = 'LIFE INSURANCE'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_LIC = 'NA'
            amount3_LIC = 'NA'
            remark3_LIC = 'NA'
            #provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)

            provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,amount1_LIC,amount2_LIC,remark1_LIC,remark2_LIC,pay_mode_LIC = information_extract_TATA_AIA(corpus,rendered)
            provider.append(provider_LIC)

            policy_number.append(policy_number_LIC)
            premium_amt.append(premium_amt_LIC)
            
            insured_name.append(insured_name_LIC)
            premium_date.append(premium_date_LIC)
            investment_type.append(investment)
            amount1.append(amount1_LIC)
            amount2.append(amount2_LIC)
            amount3.append(amount3_LIC)
            remark1.append(remark1_LIC)
            remark2.append(remark2_LIC)
            remark3.append(remark3_LIC)
            payment_frequency.append(pay_mode_LIC)
            scheme.append(scheme_LIC)
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='LIFE INSURANCE_BAJAJ':
            investment = 'LIFE INSURANCE'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_LIC = 'NA'
            amount3_LIC = 'NA'
            remark3_LIC = 'NA'
            #provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)

            provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,amount1_LIC,amount2_LIC,remark1_LIC,remark2_LIC,pay_mode_LIC = information_extract_BAJAJ(corpus,rendered)
            provider.append(provider_LIC)
            
            if (policy_number_LIC != None) and (len(policy_number_LIC)>1):
                policy_number_LIC = re.findall('\d{2,}',policy_number_LIC)
                if len(policy_number_LIC) != 0:
                    policy_number_LIC = policy_number_LIC[0]
            else:
                pass

            policy_number.append(policy_number_LIC)
            premium_amt.append(premium_amt_LIC)
            
            insured_name.append(insured_name_LIC)
            premium_date.append(premium_date_LIC)
            investment_type.append(investment)
            amount1.append(amount1_LIC)
            amount2.append(amount2_LIC)
            amount3.append(amount3_LIC)
            remark1.append(remark1_LIC)
            remark2.append(remark2_LIC)
            remark3.append(remark3_LIC)
            payment_frequency.append(pay_mode_LIC)
            scheme.append(scheme_LIC)
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='LIFE INSURANCE_ICICI':
            investment = 'LIFE INSURANCE'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_LIC = 'NA'
            amount3_LIC = 'NA'
            remark3_LIC = 'NA'
            #provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)

            provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,amount1_LIC,amount2_LIC,remark1_LIC,remark2_LIC,pay_mode_LIC = information_extract_ICICI(corpus,rendered)
            provider.append(provider_LIC)
            
            if (policy_number_LIC != None) and (len(policy_number_LIC)>1):
                policy_number_LIC = re.findall('\d{2,}',policy_number_LIC)
                if len(policy_number_LIC) != 0:
                    policy_number_LIC = policy_number_LIC[0]
            else:
                pass

            policy_number.append(policy_number_LIC)
            premium_amt.append(premium_amt_LIC)
            
            insured_name.append(insured_name_LIC)
            premium_date.append(premium_date_LIC)
            investment_type.append(investment)
            amount1.append(amount1_LIC)
            amount2.append(amount2_LIC)
            amount3.append(amount3_LIC)
            remark1.append(remark1_LIC)
            remark2.append(remark2_LIC)
            remark3.append(remark3_LIC)
            payment_frequency.append(pay_mode_LIC)
            scheme.append(scheme_LIC)
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='LIFE INSURANCE_HDFC':
            investment = 'LIFE INSURANCE'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_LIC = 'NA'
            amount3_LIC = 'NA'
            remark3_LIC = 'NA'
            #provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)

            provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,amount1_LIC,amount2_LIC,remark1_LIC,remark2_LIC,pay_mode_LIC = information_extract_HDFC(corpus,rendered)
            provider.append(provider_LIC)
            
            if (policy_number_LIC != None) and (len(policy_number_LIC)>1):
                policy_number_LIC = re.findall('\d{2,}',policy_number_LIC)
                if len(policy_number_LIC) != 0:
                    policy_number_LIC = policy_number_LIC[0]
            else:
                pass

            policy_number.append(policy_number_LIC)
            premium_amt.append(premium_amt_LIC)
            
            insured_name.append(insured_name_LIC)
            premium_date.append(premium_date_LIC)
            investment_type.append(investment)
            amount1.append(amount1_LIC)
            amount2.append(amount2_LIC)
            amount3.append(amount3_LIC)
            remark1.append(remark1_LIC)
            remark2.append(remark2_LIC)
            remark3.append(remark3_LIC)
            payment_frequency.append(pay_mode_LIC)
            scheme.append(scheme_LIC)

        elif df_result.loc[i,'INVESTMENT_TYPE']=='LIFE INSURANCE_KOTAK':
            investment = 'LIFE INSURANCE'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_LIC = 'NA'

            #provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)

            provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_KOTAK(corpus,rendered)
            provider.append(provider_LIC)
            
            if (policy_number_LIC != None) and (len(policy_number_LIC)>1):
                policy_number_LIC = re.findall('\d{2,}',policy_number_LIC)
                if len(policy_number_LIC) != 0:
                    policy_number_LIC = policy_number_LIC[0]
            else:
                pass

            policy_number.append(policy_number_LIC)
            premium_amt.append(premium_amt_LIC)
            
            insured_name.append(insured_name_LIC)
            premium_date.append(premium_date_LIC)
            investment_type.append(investment)
            amount1.append('NA')
            amount2.append('NA')
            amount3.append('NA')
            remark1.append('NA')
            remark2.append('NA')
            remark3.append('NA')
            payment_frequency.append(pay_mode_LIC)
            scheme.append(scheme_LIC)
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='LIFE INSURANCE_RELIANCE':
            investment = 'LIFE INSURANCE'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_LIC = 'NA'

            #provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)

            provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_RELIANCE(corpus,rendered)
            provider.append(provider_LIC)
            
            if (policy_number_LIC != None) and (len(policy_number_LIC)>1):
                policy_number_LIC = re.findall('\d{2,}',policy_number_LIC)
                if len(policy_number_LIC) != 0:
                    policy_number_LIC = policy_number_LIC[0]
            else:
                pass

            policy_number.append(policy_number_LIC)
            premium_amt.append(premium_amt_LIC)
            
            insured_name.append(insured_name_LIC)
            premium_date.append(premium_date_LIC)
            investment_type.append(investment)
            amount1.append('NA')
            amount2.append('NA')
            amount3.append('NA')
            remark1.append('NA')
            remark2.append('NA')
            remark3.append('NA')
            payment_frequency.append(pay_mode_LIC)
            scheme.append(scheme_LIC)
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='LIFE INSURANCE_FUTURE GENERALI':
            investment = 'LIFE INSURANCE'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_LIC = 'NA'

            #provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)

            provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_FUTURE_GENERALI(corpus,rendered)
            provider.append(provider_LIC)
            
            if (policy_number_LIC != None) and (len(policy_number_LIC)>1):
                policy_number_LIC = re.findall('\d{2,}',policy_number_LIC)
                if len(policy_number_LIC) != 0:
                    policy_number_LIC = policy_number_LIC[0]
            else:
                pass

            policy_number.append(policy_number_LIC)
            premium_amt.append(premium_amt_LIC)
            
            insured_name.append(insured_name_LIC)
            premium_date.append(premium_date_LIC)
            investment_type.append(investment)
            amount1.append('NA')
            amount2.append('NA')
            amount3.append('NA')
            remark1.append('NA')
            remark2.append('NA')
            remark3.append('NA')
            payment_frequency.append(pay_mode_LIC)
            scheme.append(scheme_LIC)
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='LIFE INSURANCE_BHARTI AXA':
            investment = 'LIFE INSURANCE'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_LIC = 'NA'
            amount3_LIC = 'NA'
            remark3_LIC = 'NA'
            #provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,pay_mode_LIC = information_extract_LIC(corpus,rendered)

            provider_LIC,policy_number_LIC,premium_amt_LIC,insured_name_LIC,premium_date_LIC,amount1_LIC,amount2_LIC,remark1_LIC,remark2_LIC,pay_mode_LIC = information_extract_BHARTI_AXA(corpus,rendered)
            provider.append(provider_LIC)

            policy_number.append(policy_number_LIC)
            premium_amt.append(premium_amt_LIC)
            
            insured_name.append(insured_name_LIC)
            premium_date.append(premium_date_LIC)
            investment_type.append(investment)
            amount1.append(amount1_LIC)
            amount2.append(amount2_LIC)
            amount3.append(amount3_LIC)
            remark1.append(remark1_LIC)
            remark2.append(remark2_LIC)
            remark3.append(remark3_LIC)
            payment_frequency.append(pay_mode_LIC)
            scheme.append(scheme_LIC)


        elif df_result.loc[i,'INVESTMENT_TYPE']=='MEDICLAIM_DEFAULT':
            img_path = df_result.loc[i,'path']
            path.append(img_path)            
            investment = 'MEDICLAIM'
            scheme_MEDICLAIM = 'NA'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            amount3_MEDICLAIM = 'NA'
            remark3_MEDICLAIM = 'NA'
            provider_MEDICLAIM,policy_number_MEDICLAIM,premium_amt_MEDICLAIM,insured_name_MEDICLAIM,premium_date_MEDICLAIM,amount1_MEDICLAIM,amount2_MEDICLAIM,remark1_MEDICLAIM,remark2_MEDICLAIM,pay_mode_MEDICLAIM = information_extract_MEDICLAIM_def(corpus,rendered)

            provider.append(provider_MEDICLAIM)
            policy_number.append(policy_number_MEDICLAIM)
            premium_amt.append(premium_amt_MEDICLAIM)
            insured_name.append(insured_name_MEDICLAIM)
            premium_date.append(premium_date_MEDICLAIM)
            investment_type.append(investment)
            amount1.append(amount1_MEDICLAIM)
            amount2.append(amount2_MEDICLAIM)
            remark1.append(remark1_MEDICLAIM)
            remark2.append(remark2_MEDICLAIM)
            amount3.append(amount3_MEDICLAIM)
            remark3.append(remark3_MEDICLAIM)
            payment_frequency.append(pay_mode_MEDICLAIM)
            scheme.append(scheme_MEDICLAIM)
            
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='MEDICLAIM_ADITYA_BIRLA':
            img_path = df_result.loc[i,'path']
            path.append(img_path)            
            investment = 'MEDICLAIM'
            scheme_MEDICLAIM = 'NA'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            amount3_MEDICLAIM = 'NA'
            remark3_MEDICLAIM = 'NA'
            provider_MEDICLAIM,policy_number_MEDICLAIM,premium_amt_MEDICLAIM,insured_name_MEDICLAIM,premium_date_MEDICLAIM,amount1_MEDICLAIM,amount2_MEDICLAIM,remark1_MEDICLAIM,remark2_MEDICLAIM,pay_mode_MEDICLAIM = information_extract_MEDICLAIM_ADITYA_BIRLA(corpus,rendered)

            provider.append(provider_MEDICLAIM)
            policy_number.append(policy_number_MEDICLAIM)
            premium_amt.append(premium_amt_MEDICLAIM)
            insured_name.append(insured_name_MEDICLAIM)
            premium_date.append(premium_date_MEDICLAIM)
            investment_type.append(investment)
            amount1.append(amount1_MEDICLAIM)
            amount2.append(amount2_MEDICLAIM)
            remark1.append(remark1_MEDICLAIM)
            remark2.append(remark2_MEDICLAIM)
            amount3.append(amount3_MEDICLAIM)
            remark3.append(remark3_MEDICLAIM)
            payment_frequency.append(pay_mode_MEDICLAIM)
            scheme.append(scheme_MEDICLAIM)
            
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='MEDICLAIM_APOLLO':
            img_path = df_result.loc[i,'path']
            path.append(img_path)            
            investment = 'MEDICLAIM'
            scheme_MEDICLAIM = 'NA'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            amount3_MEDICLAIM = 'NA'
            remark3_MEDICLAIM = 'NA'
            provider_MEDICLAIM,policy_number_MEDICLAIM,premium_amt_MEDICLAIM,insured_name_MEDICLAIM,premium_date_MEDICLAIM,amount1_MEDICLAIM,amount2_MEDICLAIM,remark1_MEDICLAIM,remark2_MEDICLAIM,pay_mode_MEDICLAIM = information_extract_MEDICLAIM_APOLLO(corpus,rendered)

            provider.append(provider_MEDICLAIM)
            policy_number.append(policy_number_MEDICLAIM)
            premium_amt.append(premium_amt_MEDICLAIM)
            insured_name.append(insured_name_MEDICLAIM)
            premium_date.append(premium_date_MEDICLAIM)
            investment_type.append(investment)
            amount1.append(amount1_MEDICLAIM)
            amount2.append(amount2_MEDICLAIM)
            remark1.append(remark1_MEDICLAIM)
            remark2.append(remark2_MEDICLAIM)
            amount3.append(amount3_MEDICLAIM)
            remark3.append(remark3_MEDICLAIM)
            payment_frequency.append(pay_mode_MEDICLAIM)
            scheme.append(scheme_MEDICLAIM)
            
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='MEDICLAIM_BAJAJ':
            img_path = df_result.loc[i,'path']
            path.append(img_path)            
            investment = 'MEDICLAIM'
            scheme_MEDICLAIM = 'NA'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            amount3_MEDICLAIM = 'NA'
            remark3_MEDICLAIM = 'NA'
            provider_MEDICLAIM,policy_number_MEDICLAIM,premium_amt_MEDICLAIM,insured_name_MEDICLAIM,premium_date_MEDICLAIM,amount1_MEDICLAIM,amount2_MEDICLAIM,remark1_MEDICLAIM,remark2_MEDICLAIM,pay_mode_MEDICLAIM = information_extract_MEDICLAIM_BAJAJ(corpus,rendered)

            provider.append(provider_MEDICLAIM)
            policy_number.append(policy_number_MEDICLAIM)
            premium_amt.append(premium_amt_MEDICLAIM)
            insured_name.append(insured_name_MEDICLAIM)
            premium_date.append(premium_date_MEDICLAIM)
            investment_type.append(investment)
            amount1.append(amount1_MEDICLAIM)
            amount2.append(amount2_MEDICLAIM)
            remark1.append(remark1_MEDICLAIM)
            remark2.append(remark2_MEDICLAIM)
            amount3.append(amount3_MEDICLAIM)
            remark3.append(remark3_MEDICLAIM)
            payment_frequency.append(pay_mode_MEDICLAIM)
            scheme.append(scheme_MEDICLAIM)
            
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='MEDICLAIM_CIGNA':
            img_path = df_result.loc[i,'path']
            path.append(img_path)            
            investment = 'MEDICLAIM'
            scheme_MEDICLAIM = 'NA'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            amount3_MEDICLAIM = 'NA'
            remark3_MEDICLAIM = 'NA'
            provider_MEDICLAIM,policy_number_MEDICLAIM,premium_amt_MEDICLAIM,insured_name_MEDICLAIM,premium_date_MEDICLAIM,amount1_MEDICLAIM,amount2_MEDICLAIM,remark1_MEDICLAIM,remark2_MEDICLAIM,pay_mode_MEDICLAIM = information_extract_MEDICLAIM_CIGNA(corpus,rendered)

            provider.append(provider_MEDICLAIM)
            policy_number.append(policy_number_MEDICLAIM)
            premium_amt.append(premium_amt_MEDICLAIM)
            insured_name.append(insured_name_MEDICLAIM)
            premium_date.append(premium_date_MEDICLAIM)
            investment_type.append(investment)
            amount1.append(amount1_MEDICLAIM)
            amount2.append(amount2_MEDICLAIM)
            remark1.append(remark1_MEDICLAIM)
            remark2.append(remark2_MEDICLAIM)
            amount3.append(amount3_MEDICLAIM)
            remark3.append(remark3_MEDICLAIM)
            payment_frequency.append(pay_mode_MEDICLAIM)
            scheme.append(scheme_MEDICLAIM)
            
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='MEDICLAIM_HDFC_ERGO':
            img_path = df_result.loc[i,'path']
            path.append(img_path)            
            investment = 'MEDICLAIM'
            scheme_MEDICLAIM = 'NA'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            amount3_MEDICLAIM = 'NA'
            remark3_MEDICLAIM = 'NA'
            provider_MEDICLAIM,policy_number_MEDICLAIM,premium_amt_MEDICLAIM,insured_name_MEDICLAIM,premium_date_MEDICLAIM,amount1_MEDICLAIM,amount2_MEDICLAIM,remark1_MEDICLAIM,remark2_MEDICLAIM,pay_mode_MEDICLAIM = information_extract_MEDICLAIM_HDFC_ERGO(corpus,rendered)

            provider.append(provider_MEDICLAIM)
            policy_number.append(policy_number_MEDICLAIM)
            premium_amt.append(premium_amt_MEDICLAIM)
            insured_name.append(insured_name_MEDICLAIM)
            premium_date.append(premium_date_MEDICLAIM)
            investment_type.append(investment)
            amount1.append(amount1_MEDICLAIM)
            amount2.append(amount2_MEDICLAIM)
            remark1.append(remark1_MEDICLAIM)
            remark2.append(remark2_MEDICLAIM)
            amount3.append(amount3_MEDICLAIM)
            remark3.append(remark3_MEDICLAIM)
            payment_frequency.append(pay_mode_MEDICLAIM)
            scheme.append(scheme_MEDICLAIM)
            
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='MEDICLAIM_ICICI':
            img_path = df_result.loc[i,'path']
            path.append(img_path)            
            investment = 'MEDICLAIM'
            scheme_MEDICLAIM = 'NA'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            amount3_MEDICLAIM = 'NA'
            remark3_MEDICLAIM = 'NA'
            provider_MEDICLAIM,policy_number_MEDICLAIM,premium_amt_MEDICLAIM,insured_name_MEDICLAIM,premium_date_MEDICLAIM,amount1_MEDICLAIM,amount2_MEDICLAIM,remark1_MEDICLAIM,remark2_MEDICLAIM,pay_mode_MEDICLAIM = information_extract_MEDICLAIM_ICICI(corpus,rendered)

            provider.append(provider_MEDICLAIM)
            policy_number.append(policy_number_MEDICLAIM)
            premium_amt.append(premium_amt_MEDICLAIM)
            insured_name.append(insured_name_MEDICLAIM)
            premium_date.append(premium_date_MEDICLAIM)
            investment_type.append(investment)
            amount1.append(amount1_MEDICLAIM)
            amount2.append(amount2_MEDICLAIM)
            remark1.append(remark1_MEDICLAIM)
            remark2.append(remark2_MEDICLAIM)
            amount3.append(amount3_MEDICLAIM)
            remark3.append(remark3_MEDICLAIM)
            payment_frequency.append(pay_mode_MEDICLAIM)
            scheme.append(scheme_MEDICLAIM)
            
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='MEDICLAIM_IFFCO':
            img_path = df_result.loc[i,'path']
            path.append(img_path)            
            investment = 'MEDICLAIM'
            scheme_MEDICLAIM = 'NA'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            amount3_MEDICLAIM = 'NA'
            remark3_MEDICLAIM = 'NA'
            provider_MEDICLAIM,policy_number_MEDICLAIM,premium_amt_MEDICLAIM,insured_name_MEDICLAIM,premium_date_MEDICLAIM,amount1_MEDICLAIM,amount2_MEDICLAIM,remark1_MEDICLAIM,remark2_MEDICLAIM,pay_mode_MEDICLAIM = information_extract_MEDICLAIM_IFFCO(corpus,rendered)

            provider.append(provider_MEDICLAIM)
            policy_number.append(policy_number_MEDICLAIM)
            premium_amt.append(premium_amt_MEDICLAIM)
            insured_name.append(insured_name_MEDICLAIM)
            premium_date.append(premium_date_MEDICLAIM)
            investment_type.append(investment)
            amount1.append(amount1_MEDICLAIM)
            amount2.append(amount2_MEDICLAIM)
            remark1.append(remark1_MEDICLAIM)
            remark2.append(remark2_MEDICLAIM)
            amount3.append(amount3_MEDICLAIM)
            remark3.append(remark3_MEDICLAIM)
            payment_frequency.append(pay_mode_MEDICLAIM)
            scheme.append(scheme_MEDICLAIM)
            
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='MEDICLAIM_MAX':
            img_path = df_result.loc[i,'path']
            path.append(img_path)            
            investment = 'MEDICLAIM'
            scheme_MEDICLAIM = 'NA'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            amount3_MEDICLAIM = 'NA'
            remark3_MEDICLAIM = 'NA'
            provider_MEDICLAIM,policy_number_MEDICLAIM,premium_amt_MEDICLAIM,insured_name_MEDICLAIM,premium_date_MEDICLAIM,amount1_MEDICLAIM,amount2_MEDICLAIM,remark1_MEDICLAIM,remark2_MEDICLAIM,pay_mode_MEDICLAIM = information_extract_MEDICLAIM_MAX(corpus,rendered)

            provider.append(provider_MEDICLAIM)
            policy_number.append(policy_number_MEDICLAIM)
            premium_amt.append(premium_amt_MEDICLAIM)
            insured_name.append(insured_name_MEDICLAIM)
            premium_date.append(premium_date_MEDICLAIM)
            investment_type.append(investment)
            amount1.append(amount1_MEDICLAIM)
            amount2.append(amount2_MEDICLAIM)
            remark1.append(remark1_MEDICLAIM)
            remark2.append(remark2_MEDICLAIM)
            amount3.append(amount3_MEDICLAIM)
            remark3.append(remark3_MEDICLAIM)
            payment_frequency.append(pay_mode_MEDICLAIM)
            scheme.append(scheme_MEDICLAIM)
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='MEDICLAIM_NATIONAL_MEDICLAIM':
            img_path = df_result.loc[i,'path']
            path.append(img_path)            
            investment = 'MEDICLAIM'
            scheme_MEDICLAIM = 'NA'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            amount3_MEDICLAIM = 'NA'
            remark3_MEDICLAIM = 'NA'
            provider_MEDICLAIM,policy_number_MEDICLAIM,premium_amt_MEDICLAIM,insured_name_MEDICLAIM,premium_date_MEDICLAIM,amount1_MEDICLAIM,amount2_MEDICLAIM,remark1_MEDICLAIM,remark2_MEDICLAIM,pay_mode_MEDICLAIM = information_extract_MEDICLAIM_NATIONAL_MEDICLAIM(corpus,rendered)

            provider.append(provider_MEDICLAIM)
            policy_number.append(policy_number_MEDICLAIM)
            premium_amt.append(premium_amt_MEDICLAIM)
            insured_name.append(insured_name_MEDICLAIM)
            premium_date.append(premium_date_MEDICLAIM)
            investment_type.append(investment)
            amount1.append(amount1_MEDICLAIM)
            amount2.append(amount2_MEDICLAIM)
            remark1.append(remark1_MEDICLAIM)
            remark2.append(remark2_MEDICLAIM)
            amount3.append(amount3_MEDICLAIM)
            remark3.append(remark3_MEDICLAIM)
            payment_frequency.append(pay_mode_MEDICLAIM)
            scheme.append(scheme_MEDICLAIM)            
            
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='MEDICLAIM_NEW_INDIA':
            img_path = df_result.loc[i,'path']
            path.append(img_path)            
            investment = 'MEDICLAIM'
            scheme_MEDICLAIM = 'NA'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            amount3_MEDICLAIM = 'NA'
            remark3_MEDICLAIM = 'NA'
            provider_MEDICLAIM,policy_number_MEDICLAIM,premium_amt_MEDICLAIM,insured_name_MEDICLAIM,premium_date_MEDICLAIM,amount1_MEDICLAIM,amount2_MEDICLAIM,remark1_MEDICLAIM,remark2_MEDICLAIM,pay_mode_MEDICLAIM = information_extract_MEDICLAIM_NEW_INDIA(corpus,rendered)

            provider.append(provider_MEDICLAIM)
            policy_number.append(policy_number_MEDICLAIM)
            premium_amt.append(premium_amt_MEDICLAIM)
            insured_name.append(insured_name_MEDICLAIM)
            premium_date.append(premium_date_MEDICLAIM)
            investment_type.append(investment)
            amount1.append(amount1_MEDICLAIM)
            amount2.append(amount2_MEDICLAIM)
            remark1.append(remark1_MEDICLAIM)
            remark2.append(remark2_MEDICLAIM)
            amount3.append(amount3_MEDICLAIM)
            remark3.append(remark3_MEDICLAIM)
            payment_frequency.append(pay_mode_MEDICLAIM)
            scheme.append(scheme_MEDICLAIM)
            
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='MEDICLAIM_ORIENTAL':
            img_path = df_result.loc[i,'path']
            path.append(img_path)            
            investment = 'MEDICLAIM'
            scheme_MEDICLAIM = 'NA'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            amount3_MEDICLAIM = 'NA'
            remark3_MEDICLAIM = 'NA'
            provider_MEDICLAIM,policy_number_MEDICLAIM,premium_amt_MEDICLAIM,insured_name_MEDICLAIM,premium_date_MEDICLAIM,amount1_MEDICLAIM,amount2_MEDICLAIM,remark1_MEDICLAIM,remark2_MEDICLAIM,pay_mode_MEDICLAIM = information_extract_MEDICLAIM_ORIENTAL(corpus,rendered)

            provider.append(provider_MEDICLAIM)
            policy_number.append(policy_number_MEDICLAIM)
            premium_amt.append(premium_amt_MEDICLAIM)
            insured_name.append(insured_name_MEDICLAIM)
            premium_date.append(premium_date_MEDICLAIM)
            investment_type.append(investment)
            amount1.append(amount1_MEDICLAIM)
            amount2.append(amount2_MEDICLAIM)
            remark1.append(remark1_MEDICLAIM)
            remark2.append(remark2_MEDICLAIM)
            amount3.append(amount3_MEDICLAIM)
            remark3.append(remark3_MEDICLAIM)
            payment_frequency.append(pay_mode_MEDICLAIM)
            scheme.append(scheme_MEDICLAIM)
            
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='MEDICLAIM_RELIGARE':
            img_path = df_result.loc[i,'path']
            path.append(img_path)            
            investment = 'MEDICLAIM'
            scheme_MEDICLAIM = 'NA'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            amount3_MEDICLAIM = 'NA'
            remark3_MEDICLAIM = 'NA'
            provider_MEDICLAIM,policy_number_MEDICLAIM,premium_amt_MEDICLAIM,insured_name_MEDICLAIM,premium_date_MEDICLAIM,amount1_MEDICLAIM,amount2_MEDICLAIM,remark1_MEDICLAIM,remark2_MEDICLAIM,pay_mode_MEDICLAIM = information_extract_MEDICLAIM_RELIGARE(corpus,rendered)

            provider.append(provider_MEDICLAIM)
            policy_number.append(policy_number_MEDICLAIM)
            premium_amt.append(premium_amt_MEDICLAIM)
            insured_name.append(insured_name_MEDICLAIM)
            premium_date.append(premium_date_MEDICLAIM)
            investment_type.append(investment)
            amount1.append(amount1_MEDICLAIM)
            amount2.append(amount2_MEDICLAIM)
            remark1.append(remark1_MEDICLAIM)
            remark2.append(remark2_MEDICLAIM)
            amount3.append(amount3_MEDICLAIM)
            remark3.append(remark3_MEDICLAIM)
            payment_frequency.append(pay_mode_MEDICLAIM)
            scheme.append(scheme_MEDICLAIM)
            
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='MEDICLAIM_ROYAL_SUNDARAM':
            img_path = df_result.loc[i,'path']
            path.append(img_path)            
            investment = 'MEDICLAIM'
            scheme_MEDICLAIM = 'NA'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            amount3_MEDICLAIM = 'NA'
            remark3_MEDICLAIM = 'NA'
            provider_MEDICLAIM,policy_number_MEDICLAIM,premium_amt_MEDICLAIM,insured_name_MEDICLAIM,premium_date_MEDICLAIM,amount1_MEDICLAIM,amount2_MEDICLAIM,remark1_MEDICLAIM,remark2_MEDICLAIM,pay_mode_MEDICLAIM = information_extract_MEDICLAIM_ROYAL_SUNDARAM(corpus,rendered)

            provider.append(provider_MEDICLAIM)
            policy_number.append(policy_number_MEDICLAIM)
            premium_amt.append(premium_amt_MEDICLAIM)
            insured_name.append(insured_name_MEDICLAIM)
            premium_date.append(premium_date_MEDICLAIM)
            investment_type.append(investment)
            amount1.append(amount1_MEDICLAIM)
            amount2.append(amount2_MEDICLAIM)
            remark1.append(remark1_MEDICLAIM)
            remark2.append(remark2_MEDICLAIM)
            amount3.append(amount3_MEDICLAIM)
            remark3.append(remark3_MEDICLAIM)
            payment_frequency.append(pay_mode_MEDICLAIM)
            scheme.append(scheme_MEDICLAIM)
            
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='MEDICLAIM_STAR_HEALTH':
            img_path = df_result.loc[i,'path']
            path.append(img_path)            
            investment = 'MEDICLAIM'
            scheme_MEDICLAIM = 'NA'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            amount3_MEDICLAIM = 'NA'
            remark3_MEDICLAIM = 'NA'
            provider_MEDICLAIM,policy_number_MEDICLAIM,premium_amt_MEDICLAIM,insured_name_MEDICLAIM,premium_date_MEDICLAIM,amount1_MEDICLAIM,amount2_MEDICLAIM,remark1_MEDICLAIM,remark2_MEDICLAIM,pay_mode_MEDICLAIM = information_extract_MEDICLAIM_STAR_HEALTH(corpus,rendered)

            provider.append(provider_MEDICLAIM)
            policy_number.append(policy_number_MEDICLAIM)
            premium_amt.append(premium_amt_MEDICLAIM)
            insured_name.append(insured_name_MEDICLAIM)
            premium_date.append(premium_date_MEDICLAIM)
            investment_type.append(investment)
            amount1.append(amount1_MEDICLAIM)
            amount2.append(amount2_MEDICLAIM)
            remark1.append(remark1_MEDICLAIM)
            remark2.append(remark2_MEDICLAIM)
            amount3.append(amount3_MEDICLAIM)
            remark3.append(remark3_MEDICLAIM)
            payment_frequency.append(pay_mode_MEDICLAIM)
            scheme.append(scheme_MEDICLAIM)
            
            
        elif df_result.loc[i,'INVESTMENT_TYPE']=='MEDICLAIM_UNITED_INDIA':
            img_path = df_result.loc[i,'path']
            path.append(img_path)            
            investment = 'MEDICLAIM'
            scheme_MEDICLAIM = 'NA'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            amount3_MEDICLAIM = 'NA'
            remark3_MEDICLAIM = 'NA'
            provider_MEDICLAIM,policy_number_MEDICLAIM,premium_amt_MEDICLAIM,insured_name_MEDICLAIM,premium_date_MEDICLAIM,amount1_MEDICLAIM,amount2_MEDICLAIM,remark1_MEDICLAIM,remark2_MEDICLAIM,pay_mode_MEDICLAIM = information_extract_MEDICLAIM_UNITED_INDIA(corpus,rendered)

            provider.append(provider_MEDICLAIM)
            policy_number.append(policy_number_MEDICLAIM)
            premium_amt.append(premium_amt_MEDICLAIM)
            insured_name.append(insured_name_MEDICLAIM)
            premium_date.append(premium_date_MEDICLAIM)
            investment_type.append(investment)
            amount1.append(amount1_MEDICLAIM)
            amount2.append(amount2_MEDICLAIM)
            remark1.append(remark1_MEDICLAIM)
            remark2.append(remark2_MEDICLAIM)
            amount3.append(amount3_MEDICLAIM)
            remark3.append(remark3_MEDICLAIM)
            payment_frequency.append(pay_mode_MEDICLAIM)
            scheme.append(scheme_MEDICLAIM)

        elif df_result.loc[i,'INVESTMENT_TYPE']=='MUTUAL FUND':
            investment = 'MUTUAL FUND'
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            pay_mode_MUTUAL_FUND = 'Monthly'
            file = df_result.loc[i,'file_name']
            filename.append(file)

            provider_MUTUAL_FUND,policy_number_MUTUAL_FUND,premium_amt_MUTUAL_FUND,insured_name_MUTUAL_FUND,premium_date_MUTUAL_FUND,scheme_MUTUAL_FUND = information_extract_MUTUAL_FUND(corpus,rendered)
            
            if policy_number_MUTUAL_FUND != None:
                policy_number_MUTUAL_FUND = re.findall('(\d+\W+\d+\W+\d+|\d+\W+\d+|\d+)',policy_number_MUTUAL_FUND)        
                if len(policy_number_MUTUAL_FUND) != 0:
                    policy_number_MUTUAL_FUND = policy_number_MUTUAL_FUND[0]
            else:
                pass
            
            amount1_MUTUAL_FUND = 'NA'
            amount2_MUTUAL_FUND = 'NA'
            remark1_MUTUAL_FUND = 'NA'
            remark2_MUTUAL_FUND = 'NA'
            amount3_MUTUAL_FUND = 'NA'
            remark3_MUTUAL_FUND = 'NA'
            provider.append(provider_MUTUAL_FUND)
            policy_number.append(policy_number_MUTUAL_FUND)
            premium_amt.append(premium_amt_MUTUAL_FUND)
            insured_name.append(insured_name_MUTUAL_FUND)
            premium_date.append(premium_date_MUTUAL_FUND)
            investment_type.append(investment)
            payment_frequency.append(pay_mode_MUTUAL_FUND)
            scheme.append(scheme_MUTUAL_FUND)
            amount1.append(amount1_MUTUAL_FUND)
            amount2.append(amount2_MUTUAL_FUND)
            amount3.append(amount3_MUTUAL_FUND)
            remark1.append(remark1_MUTUAL_FUND)
            remark2.append(remark2_MUTUAL_FUND)
            remark3.append(remark3_MUTUAL_FUND)

        elif df_result.loc[i,'INVESTMENT_TYPE']=='NSC':
            investment = 'NSC'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_NSC = 'NA'
            pay_mode_nsc = 'One Time'
            provider_nsc = 'NA'
            amount1_nsc = 'NA'
            amount2_nsc = 'NA'
            amount3_nsc = 'NA'
            remark1_nsc = 'NA'
            remark2_nsc = 'NA'
            remark3_nsc = 'NA'
            name_nsc,account_no_nsc,amount_nsc,date_nsc = information_extract_NSC(corpus,rendered)

            policy_number.append(account_no_nsc)
            premium_amt.append(amount_nsc)
            insured_name.append(name_nsc)
            premium_date.append(date_nsc)
            investment_type.append(investment)
            payment_frequency.append(pay_mode_nsc)
            provider.append(provider_nsc)
            amount1.append(amount1_nsc)
            amount2.append(amount2_nsc)
            remark1.append(remark1_nsc)
            remark2.append(remark2_nsc)
            amount3.append(amount3_nsc)
            remark3.append(remark3_nsc)
            scheme.append(scheme_NSC)


        elif df_result.loc[i,'INVESTMENT_TYPE']=='NPSE_Tier1':
            investment = 'NPSE_Tier1'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_nps_tier1 = 'NA'
            pay_mode_nps_tier1 = 'Yearly'
            provider_nps_tier1 = 'NPS'
            amount1_nps_tier1 = 'NA'
            amount2_nps_tier1 = 'NA'
            remark1_nps_tier1 = 'NA'
            remark2_nps_tier1 = 'NA'
            amount3_nps_tier1 = 'NA'
            remark3_nps_tier1 = 'NA'

            name_nps_tier1,account_no_nps_tier1,date_nps_tier1,amount_nps_tier1 = information_extract_NPS_tier1(corpus,rendered)

            if account_no_nps_tier1 != None:
                account_no_nps_tier1 = re.findall('\d+|\d',account_no_nps_tier1)
                if len(account_no_nps_tier1) != 0:
                    account_no_nps_tier1 = account_no_nps_tier1[0]
            else:
                pass
            
            insured_name.append(name_nps_tier1)
            policy_number.append(account_no_nps_tier1)
            premium_date.append(date_nps_tier1)
            premium_amt.append(amount_nps_tier1)
            investment_type.append(investment)
            payment_frequency.append(pay_mode_nps_tier1)
            provider.append(provider_nps_tier1)
            amount1.append(amount1_nps_tier1)
            amount2.append(amount2_nps_tier1)
            remark1.append(remark1_nps_tier1)
            remark2.append(remark2_nps_tier1)
            amount3.append(amount3_nps_tier1)
            remark3.append(remark3_nps_tier1)
            scheme.append(scheme_nps_tier1)


        elif df_result.loc[i,'INVESTMENT_TYPE']=='NPSE_Tier2':
            investment = 'NPSE_Tier2'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_nps_tier2 = 'NA'
            pay_mode_nps_tier2 = 'Monthly'
            provider_nps_tier2 = 'NPS'
            amount1_nps_tier2 = 'NA'
            amount2_nps_tier2 = 'NA'
            remark1_nps_tier2 = 'NA'
            remark2_nps_tier2 = 'NA'
            amount3_nps_tier2 = 'NA'
            remark3_nps_tier2 = 'NA'
            name_nps_tier2,account_no_nps_tier2,date_nps_tier2,amount_nps_tier2 = information_extract_NPS_tier2(corpus,rendered)
            
            if account_no_nps_tier2 != None:
                account_no_nps_tier2 = re.findall('\d+|\d',account_no_nps_tier2)
                if len(account_no_nps_tier2) != 0:
                    account_no_nps_tier2 = account_no_nps_tier2[0]
            else:
                pass
            
            insured_name.append(name_nps_tier2)
            policy_number.append(account_no_nps_tier2)
            premium_date.append(date_nps_tier2)
            premium_amt.append(amount_nps_tier2)
            investment_type.append(investment)
            payment_frequency.append(pay_mode_nps_tier2)
            provider.append(provider_nps_tier2)
            amount1.append(amount1_nps_tier2)
            amount2.append(amount2_nps_tier2)
            remark1.append(remark1_nps_tier2)
            remark2.append(remark2_nps_tier2)
            amount3.append(amount3_nps_tier2)
            remark3.append(remark3_nps_tier2)
            scheme.append(scheme_nps_tier2)


        elif df_result.loc[i,'INVESTMENT_TYPE']=='FD':
            investment = 'Fixed Deposit'
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            scheme_FD = 'NA'
            pay_mode_FD = 'One Time'
            amount1_FD = 'NA'
            amount2_FD = 'NA'
            remark1_FD = 'NA'
            remark2_FD = 'NA'
            amount3_FD = 'NA'
            remark3_FD = 'NA'
#             policy_number_FD = 'NA'

            provider_FD,insured_name_FD,date_FD,account_no_FD,amount_FD = information_extract_FD(corpus,rendered)

            provider.append(provider_FD)
            insured_name.append(insured_name_FD)
            premium_date.append(date_FD)
            premium_amt.append(amount_FD)
            payment_frequency.append(pay_mode_FD)
            policy_number.append(account_no_FD)
            investment_type.append(investment)        
            amount1.append(amount1_FD)
            amount2.append(amount2_FD)
            remark1.append(remark1_FD)
            remark2.append(remark2_FD)
            amount3.append(amount3_FD)
            remark3.append(remark3_FD)
            scheme.append(scheme_FD)


        elif df_result.loc[i,'INVESTMENT_TYPE']=='CHILDREN EDUCATION':
            investment = 'CHILDREN EDUCATION'
            scheme_ChildEdu = 'NA'
            pay_mode_ChildEdu = 'Yearly'
            remark1_ChildEdu = 'Tution'
            remark2_ChildEdu = 'Term'
            remark3_ChildEdu = 'Computer'
            
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            provider_ChildEdu,policy_number_ChildEdu,isured_name_ChildEdu,date_ChildEdu,amount1_ChildEdu,amount2_ChildEdu,amount3_ChildEdu,Total_ChildEdu = information_extract_ChildEdu(corpus,rendered)

            provider.append(provider_ChildEdu)
            insured_name.append(isured_name_ChildEdu)
            premium_date.append(date_ChildEdu)
            premium_amt.append(Total_ChildEdu)
            payment_frequency.append(pay_mode_ChildEdu)
            policy_number.append(policy_number_ChildEdu)
            investment_type.append(investment)        
            amount1.append(amount1_ChildEdu)
            amount2.append(amount2_ChildEdu)
            amount3.append(amount3_ChildEdu)
            remark1.append(remark1_ChildEdu)
            remark2.append(remark2_ChildEdu)
            remark3.append(remark3_ChildEdu)
            scheme.append(scheme_ChildEdu)

        elif df_result.loc[i,'INVESTMENT_TYPE']=='EDUCATION LOAN':
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            provider_EduLoan,insured_name_EduLoan,date_EduLoan,account_no_EduLoan,amount_EduLoan = information_extract_EduLoan(corpus,rendered)
            provider.append(provider_EduLoan)
            insured_name.append(insured_name_EduLoan)
            premium_date.append(date_EduLoan)
            policy_number.append(account_no_EduLoan)
            premium_amt.append(amount_EduLoan)
            file = df_result.loc[i,'file_name']
            filename.append(file)

            payment_frequency.append("Yearly")

            investment_type.append("EDUCATION LOAN")        
            amount1.append("NA")
            amount2.append("NA")
            amount3.append("NA")
            remark1.append("NA")
            remark2.append("NA")
            remark3.append("NA")
            scheme.append("NA")

        elif df_result.loc[i,'INVESTMENT_TYPE'] == 'RENTAL AGREEMENT':
            investment = 'RENTAL AGREEMENT'
            rent_insured_name,rent_date,rent_amount = information_extract_rent(corpus,rendered)
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            provider.append("NA")
            investment_type.append(investment)
            policy_number.append(None)
            insured_name.append(rent_insured_name)
            premium_date.append(rent_date)
            payment_frequency.append(None)
            scheme.append(None)
            premium_amt.append(rent_amount)
            amount1.append("NA")
            amount2.append("NA")
            remark1.append("NA")
            remark2.append("NA")
            amount3.append("NA")
            remark3.append("NA")
            
        elif df_result.loc[i,'INVESTMENT_TYPE'] == 'PPF':
            investment = 'PPF'
            ppf_provider,ppf_insured_name,ppf_date,ppf_amount = information_extract_ppf(corpus,rendered)
            provider.append(ppf_provider)
            investment_type.append(investment)
            policy_number.append(None)
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            insured_name.append(ppf_insured_name)
            premium_date.append(ppf_date)
            payment_frequency.append(None)
            scheme.append(None)
            premium_amt.append(ppf_amount)
            amount1.append("NA")
            amount2.append("NA")
            remark1.append("NA")
            remark2.append("NA")
            amount3.append("NA")
            remark3.append("NA")
        elif df_result.loc[i,'INVESTMENT_TYPE'] == 'HOUSING LOAN':
            provider_housing_loan,insured_name_housing_loan,date_housing_loan,loan_code_housing_loan,amount1_houing_loan,amount2_housing_loan = information_extract_HomeLoan(corpus,rendered)
            investment = 'HOUSING LOAN'
            
            
            provider.append(provider_housing_loan)
            investment_type.append(investment)
            policy_number.append(loan_code_housing_loan)
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            insured_name.append(insured_name_housing_loan)
            premium_date.append(date_housing_loan)
            payment_frequency.append("Yearly")
            scheme.append(None)
            premium_amt.append(None)
            amount1.append(amount1_houing_loan)
            amount2.append(amount2_housing_loan)
            remark1.append("Principal")
            remark2.append("Interest")
            amount3.append("NA")
            remark3.append("NA")
        elif df_result.loc[i,'INVESTMENT_TYPE'] == 'ULIP':
            provider_ulip,name_ulip,date_ulip,policy_no_ulip,amt_ulip,amount1_ulip,amount2_ulip,remark1_ulip,remark2_ulip = information_extract_ulip(corpus,rendered)
            investment = 'ULIP'
            
            if policy_no_ulip != None:
                policy_no_ulip = re.findall('\d+|\d',policy_no_ulip)
                if len(policy_no_ulip) != 0:
                    policy_no_ulip = policy_no_ulip[0]
            else:
                pass
            
            provider.append(provider_ulip)
            investment_type.append(investment)
            policy_number.append(policy_no_ulip)
            insured_name.append(name_ulip)
            premium_date.append(date_ulip)
            payment_frequency.append(None)
            scheme.append(None)
            premium_amt.append(amt_ulip)
            file = df_result.loc[i,'file_name']
            filename.append(file)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            
            amount1.append(amount1_ulip)
            remark1.append(remark1_ulip)
            amount2.append(amount2_ulip)
            remark2.append(remark2_ulip)
            amount3.append("NA")
            remark3.append("NA")
        elif df_result.loc[i,'INVESTMENT_TYPE'] == 'Special disease':
            investment = 'Special disease'
            provider.append("NA")
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            name_spec,date_spec,amt_spec = information_extract_special_disease(corpus,rendered)

            investment_type.append(investment)
            policy_number.append(None)
            insured_name.append(name_spec)
            premium_date.append(date_spec)
            payment_frequency.append(None)
            scheme.append(None)
            premium_amt.append(amt_spec)
            amount1.append("NA")
            file = df_result.loc[i,'file_name']
            filename.append(file)

            amount2.append("NA")
            remark1.append("NA")
            remark2.append("NA")
            amount3.append("NA")
            remark3.append("NA")
        elif df_result.loc[i,'INVESTMENT_TYPE'] == 'Handicap':

            name_handy,date_handy,disability_handy = information_extract_handy(corpus)
            investment = 'Handicap'
            provider.append("NA")
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            investment_type.append(investment)
            policy_number.append("NA")
            insured_name.append(name_handy)
            premium_date.append(date_handy)
            payment_frequency.append("NA")
            scheme.append("NA")
            file = df_result.loc[i,'file_name']
            filename.append(file)
            #if patient's disability is grater than 80% so benefit amount should be 125000/- and if patient's diability is grater than 40% and less than 80% so benefit amount should be 75000/-
            disability_handy = re.findall('\d+|\d',disability_handy)
            disability_handy = int(disability_handy[0])

            if disability_handy > 80 and disability_handy < 100:
                amt_handy = '125000'
            elif disability_handy > 40 and disability_handy < 80:
                amt_handy = '75000'
            else:
                amt_handy = '0'  

            premium_amt.append(amt_handy)
            amount1.append("NA")
            amount2.append("NA")
            remark1.append("NA")
            remark2.append("NA")
            amount3.append("NA")
            remark3.append("NA")
        elif df_result.loc[i,'INVESTMENT_TYPE'] == 'Invalid Document':
            investment = 'Invalid Document'
            inv_provider,inv_insured_name,inv_date,inv_policy_number,inv_amount = information_extract_default(corpus,rendered)
            provider.append(inv_provider)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            investment_type.append(investment)
            
            if inv_policy_number != None:
                inv_policy_number = re.findall('\d+|\d',inv_policy_number)
                if len(inv_policy_number) != 0:
                    inv_policy_number = inv_policy_number[0]
            else:
                pass
            
            policy_number.append(inv_policy_number)
            insured_name.append(inv_insured_name)
            premium_date.append(inv_date)
            payment_frequency.append("NA")
            scheme.append("NA")
            premium_amt.append(inv_amount)
            amount1.append("NA")
            amount2.append("NA")
            remark1.append("NA")
            file = df_result.loc[i,'file_name']
            filename.append(file)
            remark2.append("NA")
            amount3.append("NA")
            remark3.append("NA")
        else:
            investment = ''
            def_provider,def_insured_name,def_date,def_policy_number,def_amount = information_extract_default(corpus,rendered)
            provider.append(def_provider)
            img_path = df_result.loc[i,'path']
            path.append(img_path)
            investment_type.append(investment)
            
            if def_policy_number != None:
                def_policy_number = re.findall('\d+|\d',def_policy_number)
                if len(def_policy_number) != 0:
                    def_policy_number = def_policy_number[0]
            else:
                pass
            
            
            policy_number.append(def_policy_number)
            insured_name.append(def_insured_name)
            premium_date.append(def_date)
            payment_frequency.append("NA")
            scheme.append("NA")
            premium_amt.append(def_amount)
            amount1.append("NA")
            amount2.append("NA")
            remark1.append("NA")
            file = df_result.loc[i,'file_name']
            filename.append(file)
            remark2.append("NA")
            amount3.append("NA")
            remark3.append("NA")

        print('done')
    df_final_result = pd.DataFrame()
    df_final_result['path'] = path
    df_final_result['File_name'] = filename
    df_final_result['provider'] = provider
    df_final_result['investment_type'] = investment_type
    df_final_result['policy_number'] = policy_number
    df_final_result['insured_name'] = insured_name
    df_final_result['premium_date'] = premium_date
    df_final_result['Pay_mode'] = payment_frequency
    df_final_result['Scheme'] = scheme
    df_final_result['premium_amt'] = premium_amt
    df_final_result['Amount1'] = amount1
    df_final_result['remark1'] = remark1
    df_final_result['Amount2'] = amount2
    df_final_result['remark2'] = remark2
    df_final_result['Amount3'] = amount3
    df_final_result['remark3'] = remark3
    
    return df_final_result

In [1556]:
df_final_result2 = extraction()

0
done
1
done
2
done
3
done
4
done
5
done
6
done
7
done
8
done
9
done
10
done
11
done
12
done
13
done
14
done
15
done
16
done
17
done
18
done
19
done
20
done
21
done
22
done
23
done
24
done
25
done
26
done
27
done
28
done
29
done
30
done
31
done
32
done
33
done
34
done
35
done
36
done
37
done
38
done
39
done
40
done
41
done
42
done
43
done
44
done
45
done
46
done
47
done
48
done
49
done
50
done
51
done
52
done
53
done
54
done
55
done
56
done
57
done
58
done
59
done
60
done
61
done
62
done
63
done
64
done
65
done
66
done
67
done
68
done
69
done
70
done
71
done
72
done
73
done
74
done
75
done
76
done
77
done
78
done
79
done
80
done
81
done
82
done
83
done
84
done
85
done
86
done
87
done
88
done
89
done
90
done
91
done
92
done
93
done
94
done
95
done
96
done
97
done
98
done
99
done
100
done
101
done
102
done
103
done
104
done
105
done
106
done
107
done
108
done
109
done
110
done
111
done
112
done
113
done
114
done
115
done
116
done
117
done
118
done
119
done
120
done
121
done
122
done
123

done
976
done
977
done
978
done
979
done
980
done
981
done
982
done
983
done
984
done
985
done
986
done
987
done
988
done
989
done
990
done
991
done
992
done
993
done
994
done
995
done
996
done
997
done
998
done
999
done
1000
done
1001
done
1002
done
1003
done
1004
done
1005
done
1006
done
1007
done
1008
done
1009
done
1010
done
1011
done
1012
done
1013
done
1014
done
1015
done
1016
done
1017
done
1018
done
1019
done
1020
done
1021
done
1022
done
1023
done
1024
done
1025
done
1026
done
1027
done
1028
done
1029
done
1030
done
1031
done
1032
done
1033
done
1034
done
1035
done
1036
done
1037
done
1038
done
1039
done
1040
done
1041
done
1042
done
1043
done
1044
done
1045
done
1046
done
1047
done
1048
done
1049
done
1050
done
1051
done
1052
done
1053
done
1054
done
1055
done
1056
done
1057
done
1058
done
1059
done
1060
done
1061
done
1062
done
1063
done
1064
done
1065
done
1066
done
1067
done
1068
done
1069
done
1070
done
1071
done
1072
done
1073
done
1074
done
1075
done
1076
done
1077
done

1908
done
1909
done
1910
done
1911
done
1912
done
1913
done
1914
done
1915
done
1916
done
1917
done
1918
done
1919
done
1920
done
1921
done
1922
done
1923
done
1924
done
1925
done
1926
done
1927
done
1928
done
1929
done
1930
done
1931
done
1932
done
1933
done
1934
done
1935
done
1936
done
1937
done
1938
done
1939
done
1940
done
1941
done
1942
done
1943
done
1944
done
1945
done
1946
done
1947
done
1948
done
1949
done
1950
done
1951
done
1952
done
1953
done
1954
done
1955
done
1956
done
1957
done
1958
done
1959
done
1960
done
1961
done
1962
done
1963
done
1964
done
1965
done
1966
done
1967
done
1968
done
1969
done
1970
done
1971
done
1972
done
1973
done
1974
done
1975
done
1976
done
1977
done
1978
done
1979
done
1980
done
1981
done
1982
done
1983
done
1984
done
1985
done
1986
done
1987
done
1988
done
1989
done
1990
done
1991
done
1992
done
1993
done
1994
done
1995
done
1996
done
1997
done
1998
done
1999
done
2000
done
2001
done
2002
done
2003
done
2004
done
2005
done
2006
done
2007
done


done
2755
done
2756
done
2757
done
2758
done
2759
done
2760
done
2761
done
2762
done
2763
done
2764
done
2765
done
2766
done
2767
done
2768
done
2769
done
2770
done
2771
done
2772
done
2773
done
2774
done
2775
done
2776
done
2777
done
2778
done
2779
done
2780
done
2781
done
2782
done
2783
done
2784
done
2785
done
2786
done
2787
done
2788
done
2789
done
2790
done
2791
done
2792
done
2793
done
2794
done
2795
done
2796
done
2797
done
2798
done
2799
done
2800
done
2801
done
2802
done
2803
done
2804
done
2805
done
2806
done
2807
done
2808
done
2809
done
2810
done
2811
done
2812
done
2813
done
2814
done
2815
done
2816
done
2817
done
2818
done
2819
done
2820
done
2821
done
2822
done
2823
done
2824
done
2825
done
2826
done
2827
done
2828
done
2829
done
2830
done
2831
done
2832
done
2833
done
2834
done
2835
done
2836
done
2837
done
2838
done
2839
done
2840
done
2841
done
2842
done
2843
done
2844
done
2845
done
2846
done
2847
done
2848
done
2849
done
2850
done
2851
done
2852
done
2853
done
2854


In [1562]:
df_final_result2.to_excel('Before_WR_04.xlsx',index = False)

In [1563]:
df_final_result = df_final_result2

In [1564]:
df_final_result

,path,File_name,provider,investment_type,policy_number,insured_name,premium_date,Pay_mode,Scheme,premium_amt,Amount1,remark1,Amount2,remark2,Amount3,remark3
0,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2184_Rent_Agreement...,HDFC,,None,None,"15.06.2016,",NA,NA,"50,000",NA,NA,NA,NA,NA,NA
1,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2185_Rent_Agreement...,NA,RENTAL AGREEMENT,None,None,12. 13. 14.,None,None,None,NA,NA,NA,NA,NA,NA
2,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2186_Rent_Agreement...,NA,RENTAL AGREEMENT,None,None,18. 19. 20.,None,None,None,NA,NA,NA,NA,NA,NA
3,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2187_Rent_Agreement...,None,,None,None,None,NA,NA,None,NA,NA,NA,NA,NA,NA
4,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2188_Rent_Agreement...,None,,None,None,None,NA,NA,None,NA,NA,NA,NA,NA,NA
5,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2189_Rent_Agreement...,None,,None,None,None,NA,NA,None,NA,NA,NA,NA,NA,NA
6,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2190_Rent_Agreement...,None,Invalid Document,None,Mr.G.Chidanandappa,None,NA,NA,None,NA,NA,NA,NA,NA,NA
7,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2191_Rent_Agreement...,NA,RENTAL AGREEMENT,None,None,None,None,None,"8,600",NA,NA,NA,NA,NA,NA
8,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2192_Rent Receipt-D...,None,,None,None,None,NA,NA,None,NA,NA,NA,NA,NA,NA
9,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2193_Rent Receipt_B...,NA,RENTAL AGREEMENT,None,"MR, RAVINDRA LAXMAN JOGDAND",12-Jan-2019,None,None,None,NA,NA,NA,NA,NA,NA


In [1565]:
runstats = pd.DataFrame()

In [1566]:
df_final_result['insured_name']=df_final_result['insured_name'].str.replace(r"[\(\)\{\}\#\+<>,']","")
words = ['Name of Subscriber Boba mm: _thomdax','This is to certify that tne roliowina oavments','Student Name Class Amount Discount','Name : Std: VB RollNo','Smt/Ms./Sri. eee AT Towards','Licensor Name and Address','Name Student No Inst','Licensee Name and Address','licensee name and address','Licensor Name and Address','SRI SRI RAVISHANKAR VIDYA MANDIR','Shri iL f rparistieTelowngpaieyies Policy','Mr/ts. RAE Holder','Loan Account No. PHR','Name of Investor 1 : Cla','Sri. “~  Towards the following','Student Name > ARSH Date','Smt/Ms./Shri:  Sds  towards the following','Smt/MsJ/Shri: JR  towards the following','Student Name : Receipt No: Section','Name : ATTA Name','Name ATTA Name','Name: Ist Quarter','Life Insured ir','Student ID: 17EC99 Receipt No.: 218 Name','Ms Shei','Ms\./Srl','Smt/Ms/St. we','life insurance policy he! by','life insurance policy hel by','Smt/Ms/Shri :  towards the following','Amount of \(in figures\)','Name of Student Class','Ms, __ ti','The following premium has been received for life insurance policies fram the userid','M/s,  ‘The sum','Smt/Ms./Sri  Towards the followin','This is to certify that the umrari','This is to certify that the borrower','This is to certify that the repayment occurred','This is to certify that the following payments have','This is to certify that the repayment expected','borrower:  This','This is to certify that the above amount','M/s,  ‘The sum','This is to certify that the following premium have','This is to certify that the follo','This is to certify that the following life insurance','This is to certify that th following life insurance','This is to certify that the following amounts have','This is to certify that we would have','This is to certify that we have sanctioned','This is to certify that we have.sanctioned','This is to certify that for: Loan','This is to certify that the','This is to certify that','THIS IS TO CERTIFY THAT','this is to certify that','Thisis to certify that','THIS IS TO STATE THAT','This is to state that','this is to state that','Mrs. the Month','M/s.  the sum sibs','MR Thay','Shri : towards the following','this is to certify that the following payme life','Name  towards the fees','Ms, __ ti','M/s.  The sum','Mrs.  thly Quarterly','Smt/Ms./Sri. : towards the following','Shri .  Towards the following','Ms/ Shri Towards the','Shri . Towards the following','Smt/Ms./Sri. Towards the following','M/s. The sum','M/s.  the sumol Rupees','Shri the sum','Mr towards the House','IN CAPITAL LETTERS','Scanned  CamScanner','Life Insured Address of Life','Name of the Life assured','Name and Address of Policyholder','Life insured or on behalf','Life Insured Name of Policyholder','Payment received from','Payment Received from','Acknowledgement Number','Contribution Receipt Number','Transaction Reference Number','Transaction Date','Transaction Status','PROVISIONAL CERTIFICATE FOR THE FINANCIAL YEAR','CERTIFICATE FOR THE FINANCIAL YEAR','Provisional Home Loan Interest Certificate','FOR CLAIMING DEDUCTIONS UNDER SECTIONS','BORROWER NAME','Borrower Name','borrower name','EMPLOYEE NAME','Employee Name','employee name','Student Name','STUDENT NAME','student name','Name of the student','Name of the Student','Name ofthe student','Student\'s name','Students name','Students Name','Name & Father Name of Investor','Name & Father Name of Investor 1','Name of the Subscriber','NAME OF THE SUBSCRIBER','Name of Subscriber|','NAME OF SUBSCRIBER','PERSONAL DETAILS','Personal details','personal details','Feaployex Name','NAME OF INVESTOR','Name of Investor','Name Of Investor','Name of investor','name of investor','To Whom It May Concern','To Whomsoever It May Concern','To whomsoever it may concern','Name of Nominee','Policyholder Name','Policy Holder Name','Insured Name i','insured Name','Insured Name','insured name','Insured name','Premium Payor Name','Policyholder\'s Name','Policyholders Name','Account Name','Life Insured','Life insured','Owner Name','following life insurance policy held by Sri','following life insurance policy held by','The following premium has been received for life insurance policies from the userid of Shri','The following premium has been received for life insurance policies from the userid of','The following premium has been received for life insurance policies from the userid','The following premium has been received for life insurance policies Sri','The following premium has been received for life insurance policies of','The following premium has been received for life insurance policies','The following premia have been received under the following life insurance policies of','received for life insurance policies from the userid of','Received with thanks from  Dated','Received with thanks from','RECEIVED WITH THANKS FROM','RECEIVED FROM','Received From','Received from','received from','life insurance policies from the userid of','life insurance policies held by Mr/Ha:','life insurance policies held by','from the userid of','Name of the Life Assured','Life Assured','Life assured','life assured','Policyholder Insurance Details','Insured\'s Name','Insureds Name','Insured/Recipient Name and Address','Tnsured/Recipient Name and Address','Name of LifeAssured','Name of the Policy Holder','Group Activ','Unique Client Code','Contribution Amount','Contribution Tier Type','NON-TRANSFERABLE','PL NO','Educational Institutions','International School','Name Branch Name','proposed by','Client Name','Residential Address','ay number','in figures','Off Pokhran','Presentor Name','Name of Proposer','Amount of','Customer Name Mis','Customer Name','the follawing amounts have','NAME OF STUDENT','Joint Venture','np warata weary','has paid','have paid','Policyholder Insurance Details ae','towards thesfollowing','following life insurance policy held by M  Name','towards the following','Name of Student Master','Name of Student','EXTN RAMA PARK','Sri, \* Towards the following','Paisabazaar Marketing','Name of Life Insured','Loan Account No','Not Opted','Ms Sr.  Towards the following','Sri. Towards the following','Towards the foliowing','Towards the following','Wo confinm receipt','Shri ¢: towards','Ms./Sri.  Tete','Smt./Ms./Sri. Gala Wart','Policy Holder ‘S','Smt./Ms./Sri. ere wart','Insured Person and  \[Residential Address','Insured Person and','Insured Person','MS  S/O','MISS Aginne','Smt\) IMsshri','Sri ~ ThWatds INS Wehlodnhg','MS Mota DF tokdiog','Smt./Ms/Si. AORIES','Name ss /Section','Signature ofDepositor Signature','Signature ofDepositor','Signature of Depositor','J agQu 8 inad','Name Admission No Class','Name Student No eeandond','Name Student No sina','Name Student No Standard','Name Student No nner','Name studentno','Name studentNo','vena SARE PALE','Name of Pupil','RECEIWED AMOUNT','Re-admission Charges','Name Empiores','pasa yond','Drs nS Gowduls','Quick Links','Name\(s\) of Borrower\(s\)','licensee name and address','licensee name','Ms. A sum','RENT RECEIPT','FROND SARA','D  BLOCK','MRS OMe SOAR','STI FRE FRIERA','age Total','Blue Bell Vasant Valley','w noted','Date Of Birth','Bonafide student','Licensee Name','licensee name','Licensor Name','H NO MBH COLONY','was sanctioned housing loan','Financial Year','PAN ATIPPE','Name\(s\) of Borrower\(s\)','Sri KURNOOL DIST','First Holder E-mail','TO WHOMSOEVER','To Whomsoever','Sri  © owards the lotowing','Sri, oO Towards','Collecting Branch','Dateof Birth','Insured\'s Name\(s\) \|Dateof Birth','Insureds Name\(s\) \|Dateof Birth','Age Resident','Detailed Premium Payment',' Student Name \| ISTUDENT','MS AIS HAS TASS',' Licensee Name','MRAO ADIK INSTITUTE','M/s R.S.R. Infra','Ms% ‘Lob Cunawr','Lob Cunawr','MR  KHAPA NAGPUR','Ms \[Stly','POLICY','Policy','policy','HOLDER','Holder','holder','Namet','Names','NAME','Name','name','DEAR','Dear','dear','INVESTOR','Investor','investor','Insured\'s','insured\'s','Insureds','insureds','INSURED','Insured','insured','PREMIUM','Premium','premium','PAYOR','Payor','payor','POLICYHOLDERS','Policyholders','policyholders','Policyhalders','POLICYHOLDER\'S','Policyholder\'s','policyholder\'s','Policyhalder\'s','POLICYHOLDER','Policyholder','policyholder','ACCOUNT','Account','account','LifeAssured','LIFE','Life','life','OWNER','Owner','owner','FOLLOWING','Following','following','INSURANCE','Insurance','insurance','HELD','Held','held','BY','By','by','\bThe\b','\bthe\b','\bHas\b','\bhas\b','\bHave\b','\bhave\b','\bBeen\b','\bbeen\b','Received','received','\bFor\b','\bfor\b','POLICIES','Policies','policies','FROM','From','from','Userid','userid','\bOf\b','\bof\b','Premia','premia','Under','under','With','with','Assured','assured','Details','details','Insured/Recipient','insured/recipient','Recipient','recipient','And','Address','address','Addreas','addreas','Tnsured/Recipient','Tnsured','ii','Far','far','IRMS','Grade','grade','STUDENT','Student','student','Student\'s','Students','Father\'s','father\'s','Fathers','fathers','Father','father','Parent','parent','Course','course','GSTIN','Health','health','\bNo\b','\bno\b','\bards\b','Commencement','commencement','Sommencement','sommencement','Towards','towards','Consolidated','consolidated','Head','head','Office','office','Ddhhh','MASTER','MAST','Product','product','warata','weary','\biss\b','| ” e','\bie\b','CADE','Career','career','DATE','Date','date','Issuing','issuing','Borrower','borrower','Office','office','PERSONAL','Personal','personal','DETAILS','Details','details','SUBSCRIBER','Subscriber','subscriber','Feaployex','INVESTOR','Investor','investor','CERTIFY','Certify','certify','WHOMSOEVER','Whomsoever','whomsoever','CONCERN','Concern','concern','RECEIVED','Received','received','THANKS','Thanks','thanks','THANK','Thank','thank','EMPLOYEE','Employee','employee','STATE','State','state','&','BORROWER','Borrower','borrower','has been','has/have been','hashave been','have been','POLICYOWNER','Policyowner','policyowner','PAID','Paid','paid','Active','MUMBAI','Mumbai','mumbai','India','BANK','Bank','bank','SELF','Self','RELATION','Relation','relation','SPOUSE','Spouse','spouse','Aged','BRANCH','Branch','branch','CONTRIBUTION','Contribution','contribution','SERVICES','Services','services','TAX','Tax','tax','PAYMENT','Payment','payment','GATEWAY','Gateway','gateway','CHARGE','Charge','charge','GST','EMAIL','Email','email','PERSONAL','Personal','personal','INFORMATION','Information','information','SINGLE','Single','single','NOMINEE','Nominee','nominee','HOUSING','Housing','housing','FINANCE','Finance','finance','LIMITED','Limited','limited','FLOOR','Floor','floor','CARDIAC','Cardiac','cardiac','REPORT','Report','report','discrepancy','EasyCall','ACCOUNT','Account','account','STATEMENT','Statement','statement','Mode','HOLDING','Holding','holding','STATUS','Status','status','INDIVIDUAL','Individual','individual','FOLIO','Folio','folio','PAGE','Page','page','MUTUAL','Mutual','mutual','FUND','Fund','fund','FUHIC','WHOSE','Whose','whose','APPEAR','Appear','appear','CERTIFICATE','Certificate','certificate','singly','jointly','GRANTED','Granted','granted','ALLOTMENT','Allotment','allotment','UNITS','Units','units','SUBJECT','Subject','subject','REALIZATION','Realization','realization','APPLICABLE','Applicable','applicable','LOAD','Load','load','STRUCTURE','Structure','structure','REFERENCE','Reference','reference','REFER','Refer','refer','FURTHER','Further','further','FIND','Find','find','PERIOD','Period','period','YOUR','Your','your','STAMP','Stamp','stamp','DUTY','Duty','duty','FLAT','Flat','flat','FIRST','First','first','SECOND','Second','second','THIRD','Third','third','GUARDIAN','Guardian','guardian','EDUCATIONAL','Educational','educational','INSTITUTIONS','Institutions','institutions','TRANSFERABLE','Transferable','transferable','Issuing','Portfolio','SUMMARY','Summary','summary','CODE','Code','code','DEVELOPMENT','Development','development','HEALTH','Health','health','STANDARD','Standard','standard','ptandard','INSURER','Insurer','insurer','TERMS','Terms','terms','INTERNATIONAL','International','international','SCHOOL','School','school','RollNo','ROLL','Roll','roll','TERM','Term','term','ROAD','Road','road','MASTER','Master','master','MAST','Mast','mast','CHEQUE','Cheque','cheque','INSTITUTE','Institute','institute','CLASSVI','CLASSI','Classen','CLASS','Class','class','ciass','ENROLLMENT','Enrollment','enrollment','QUARTER','Quarter','quarter','QUARTERLY','Quarterly','quarterly','SESSION','Session','session','STUDYING','Studying','studying','ACADEMY','Academy','academy','AdmissionNat','ADMISSION','Admission','admission','Admissian','FREQUENCY','Frequency','frequency','PLUS','Plus','plus','DEATH','Death','death','SECTION','Section','section','PERCENTAGE','Percentage','percentage','DEDUCTIONS','Deductions','deductions','NUMBER','Number','number','MOBILE','Mobile','mobile','SECTOR','Sector','sector','SURVIVIOR','Survivor','survivor','HOUSE','House','house','COMMENCE','Commence','commence','ReceiptNo','RECEIPT','Receipt','receipt','BORROWERS','Borrowers','borrowers','PRESENTOR','Presentor','presentor','TRANSACTION','Transaction','transaction','COLLECTION','Collection','collection','PERSON','Person','person','CREDIT','Credit','credit','DEBIT','Debit','debit','PROPOSED','Proposed','proposed','CLIENT','Client','client','RESIDENTIAL','Residential','residential','PROPOSER','Proposer','proposer','FIGURES','Figures','figures','POKHRAN','Pokhran','pokhran','DOCUMENT','Document','document','NOTEPAD','Notepad','notepad','COMPLETE','Complete','complete','CORPORATION','Amount','thefolowing','Company','fram','Assessment','Year','Dies','ensure','Mother','Hospital','Issue','stile','FATHER','Institule','Five','Fees','High','Group','SOTCPr','Citizencredlt','Scholar','Enment','Union','Ward','This','Rents','Applicant','SOSKe','Interest','aged','December','Igave','Daughter','CUSTOMER','Customer','customer','SERIAL','Serial','serial','INSTRUMENT','Instrument','instrument','HRMS','COMPONENT','Component','component','CONCESSION','Concession','concession','working','PLOT','Greetings','Plan','SirMadam','Sir/Madam','Change','Plot','Hno2','WARD','AGENT','Agent','agent','AdmNo','encascteouse','Signature','QUFEL','KCUVee','hreghayy','S0ch9','S2naees','areal 4','Drreuaabcs','Sabcoayieies','XOMAR','MICR','WIFE','Property','residing','cowards','PARTICULARS','asum','F2113604','Ms #pmOR20','Clemens','Aadhaar','Since','Occupier','Lite','Bill','E-mail','Collecting','Opted','Detailed','Avenue','College','ADDRESS','address','Address','Smt/Ms/Sri. z Towards','Ms./Sri. o Towards','SmtJ/Ms./Shri: Sh','Smt.\) Ms./ Sahib','Smt/Ms./Sri wae','Mrs./Shrimati/Miss','Smt./Ms/Sri. Sree WATT','Smt./Ms/Sri. Sree','SmtiMsiShri','Smt/Ms./SK','Smt/Ms_/SKi','Smt/Ms./Sri','Smt./Ms./Shri: MAST','Smt/MsJ/Shri.','Smt/MsJ/Shri i','smt/Ms/Shri','SmtJ/Ms./Shri','Smt/ivis/Sri','Shri/Ss','ShrilSmt','Smt/MsJ/Shri','Smt/Ms/Shri','Smt/Ms/Sri','Smt/MsJShri','SmtJMsJ/Shri','Smt/JMs./Shri','SmtiMs/Shri','Smt./Ms./Shri','Smt./Ms./Sri.','Smt./Ms./Sri','Smit/MsJ/Shri','Smt./Ms/Shri','Shri/Smt.Smt.','Smt/Ms./Shrl','Smt/MsJSHi','Smt/Ms/Sr','Smt/Ms./St','M/s','Smt/Ms./Shri','Shri/Smt.Sri','Smt/Ms./Stiri','Shri/Smt/Kum','Smt/Ms/SH.','Smt/Me/Shri','Smt/Ms.JShri','Smt/Ms./Shti','ShrilSmt.Sri','Sh./Smt./ M/s','Mr/Ms. SHRI','Mr./Mrs.','Shri/Si','Shri/gme','Shri/Smt.','Shri/Smt','Sri/Smt','Shri.Sint','Mr./Mrs. fnki','Mr./Mrs. Het Colaba','Mrs-fhokit','Mrs-—/9 Kit Duran','Mr./Mrs taki','Mr./Mrs. fink','Ms/Sri','Ms./Sri.','Ms./Shri','Ms/Shri','Ms,/Shrl','Mr/Ms.','Ms./Si.','Mr/Mea','Mrs./Shrimati','Ms/Shriz','Mr./Mirs','Mrs','MRS','Mr','MR','Ms','MS','Miss','MISS','Smt','SMT','we have d','we have','the Company has','gfvards ine GHOWING','the   have','the have','Sri  the','Shi S','BrhesOt','the  amounts have','bile solf','Agein Years','Sri ere wart','is to  that','the  has','LICHELP   to','a  a Maas','Sri  the','the  payme','ards the','of halder','aatimns aan Toward','of Permanent','arate Tart','iSri anette sari','Sri   erctior SATaT','the  s have','REGENT COURT IRAHEJA','the re occurred','DCB','will be shuck','Te SARE','handGalass Narel','CONTACT No ADMN','ttf eee CONTACT','V Fhe ee KILPAUK','Namie','Woolies sie eros','of the Pupil','eee  No','of pf','Wa meotan','of the Ward','OF THE','of Child','Child','of candi','Component Amount Concession','the fees','Me pyyy','jat ne','of the Author','Sri Ram Nagar','of The','of the','of Almighty','Particulars','w Rees','the re occurred','you  sanctioned','Non  s','ON RAR BONN','for Loan','we havesanctione','Loan  No','comprising Principal','the re expected','we have sanctioned','and the s','I has taken','Ba ale Hari','I Rabe Shivtl','you hav been sanctioned','is free','the re expected','for the purpose','s  ls','Loan  No PHR','Loan  No PH','s  ls','the re expected','will repay','FISD','REGENT COURT IRAHEJA','TECHNOPARK  CHSL','Kumar Park','Sri reoraost','NEW HEM MOTORS','Pragati','NEW HEM MOTORS','Ram Beash Pavan Nara','aw iANDIANI','Sea breeze','SAI KRISHNA','Mahindra plendour','NAtESH Duge','Alergy','J AGESH PDvaeky','Oberoi Splendor splendorcompiex','Oberoi Splendor rande','MV Mantsh Mehl Vinohg','Hindustan Service Station','Shree Siddhi Vinayak','Sati Suveshr','Sati Sevepr','Debi Prasad Dash','SETA LOGISTICS','aflage auaal','mati','BHARAT NAGAR','Phalke Jaya eepak','MTR Form','JTahomspry Arlo','Lopes Julie illiam','Decpove Soly','Decpave Soly','SRI SIVADURGA MEDICALS NEAR','Such Sued','Quad','Quduech Suave','MTR Form','JTahomspry Arlo','Dig ambav','L igesite','Sqamib av GanPat','Lopes Julie illiam','Magnum Tower','Shel','Shel','Accessories agete','LALGI','Ommieh','Trikuta Tower','Kaji Parinda anish','NEW RAJ SAGAR','vs Reavt dive','ar Shobha','TATA CAPITAL FINANCIAL','in alee Bienuine','Uma Radhakrishnan','Dai ARDO','DAs ABppiaoauin','PiaDar ARDOLADwIn SHALL','SHOIVADA GAON TRANSIST','oY ae thesum','Shethae','Shekhoy','RSGANES','SEVA','Mileage','pe Taran noes','HIRANANDANI E','Chandiwala Complex','Saly mela Prtetafe','VICEROY PARK','pSfedapuagnya','Hleaned','VGisian lane','Seca','ards','Sri\. \. ils','arte wat','arta wat','wards','Salt lake','erates TART','iss a me','eretter wart','SLUATHA','Greta wart']
for i in words:
    df_final_result['insured_name'] = df_final_result['insured_name'].replace(to_replace=i,value="",regex=True)

In [1567]:
# df_final_result['insured_name']=df_final_result['insured_name'].str.replace(r"[\(\)\{\}\#\+<>,']","")
# words = ['Life Insured Address of Life','Name of the Life assured','Name and Address of Policyholder','Life insured or on behalf','Life Insured Name of Policyholder','Payment received from','Payment Received from','Acknowledgement Number','Contribution Receipt Number','Transaction Reference Number','Transaction Date','Transaction Status','PROVISIONAL CERTIFICATE FOR THE FINANCIAL YEAR','CERTIFICATE FOR THE FINANCIAL YEAR','Provisional Home Loan Interest Certificate','FOR CLAIMING DEDUCTIONS UNDER SECTIONS','BORROWER NAME','Borrower Name','borrower name','EMPLOYEE NAME','Employee Name','employee name','Student Name','STUDENT NAME','student name','Name of the student','Name of the Student','Student\'s name','Students Name','Name & Father Name of Investor','Name & Father Name of Investor 1','Name of the Subscriber','NAME OF THE SUBSCRIBER','Name of Subscriber|','NAME OF SUBSCRIBER','PERSONAL DETAILS','Personal details','personal details','Feaployex Name','NAME OF INVESTOR','Name of Investor','Name Of Investor','Name of investor','name of investor','This is to certify that we would have','This is to certify that','THIS IS TO CERTIFY THAT','this is to certify that','Thisis to certify that','THIS IS TO STATE THAT','This is to state that','this is to state that','To Whom It May Concern','To Whomsoever It May Concern','To whomsoever it may concern','Insured Name','insured Name','Name of Nominee','Policyholder Name','Policy Holder Name','insured Name','Insured Name','insured name','Insured name','Premium Payor Name','Policyholder\'s Name','Account Name','Life Insured','Life insured','Owner Name','following life insurance policy held by','The following premium has been received for life insurance policies from the userid of','The following premium has been received for life insurance policies','The following premia have been received under the following life insurance policies of','received for life insurance policies from the userid of','Received with thanks from','RECEIVED WITH THANKS FROM','RECEIVED FROM','Received From','Received from','received from','life insurance policies from the userid of','from the userid of','Name of the Life Assured','Life Assured','Life assured','life assured','Policyholder Insurance Details','Insured\'s Name','Insured/Recipient Name and Address','Tnsured/Recipient Name and Address','Name of LifeAssured','Name of the Policy Holder','Group Activ','Received from','Unique Client Code','Contribution Amount','Contribution Tier Type','NON-TRANSFERABLE','PL NO','Educational Institutions','International School','Name Branch Name','proposed by','Client Name','Residential Address','ay number','in figures','Off Pokhran','Presentor Name','POLICY','Policy','policy','HOLDER','Holder','holder','NAME','Name','name','DEAR','Dear','dear','INVESTOR','Investor','investor','INSURED','Insured','insured','PREMIUM','Premium','premium','PAYOR','Payor','payor','POLICYHOLDER\'S','policyholder\'s','policyholder\'s','POLICYHOLDER','Policyholder','policyholder','ACCOUNT','Account','account','LifeAssured','LIFE','Life','life','OWNER','Owner','owner','FOLLOWING','Following','following','INSURANCE','Insurance','insurance','HELD','Held','held','BY','By','by','\bThe\b','\bthe\b','\bHas\b','\bhas\b','\bHave\b','\bhave\b','\bBeen\b','\bbeen\b','Received','received','\bFor\b','\bfor\b','POLICIES','Policies','policies','FROM','From','from','Userid','userid','\bOf\b','\bof\b','Premia','premia','Under','under','With','with','Thanks','thanks','Assured','assured','Details','details','Insured\'s','insured\'s','Insured/Recipient','insured/recipient','Recipient','recipient','And','Address','address','Addreas','addreas','Tnsured/Recipient','Tnsured','ii','Far','far','IRMS','Grade','grade','Class','class','STUDENT','Student','student','Student\'s','Students','Father\'s','father\'s','Fathers','fathers','Father','father','Parent','parent','Course','course','GSTIN','Health','health','\bNo\b','\bno\b','\bards\b','Commencement','commencement','Sommencement','sommencement','Towards','towards','Consolidated','consolidated','Head','head','Office','office','Ddhhh','MASTER','MAST','Product','product','warata','weary','\biss\b','| ” e','\bie\b','CADE','Career','career','Date','date','Issuing','issuing','Borrower','borrower','Office','office','PERSONAL','Personal','personal','DETAILS','Details','details','SUBSCRIBER','Subscriber','subscriber','Feaployex','INVESTOR','Investor','investor','CERTIFY','Certify','certify','WHOMSOEVER','Whomsoever','whomsoever','CONCERN','Concern','concern','RECEIVED','Received','received','THANKS','Thanks','thanks','EMPLOYEE','Employee','employee','STATE','State','state','&','BORROWER','Borrower','borrower','has been','has/have been','hashave been','have been','POLICYOWNER','Policyowner','policyowner','PAID','Paid','paid','Active','MUMBAI','Mumbai','mumbai','India','BANK','Bank','bank','SELF','Self','RELATION','Relation','relation','SPOUSE','Spouse','spouse','Aged','BRANCH','Branch','branch','CONTRIBUTION','Contribution','contribution','SERVICES','Services','services','TAX','Tax','tax','PAYMENT','Payment','payment','GATEWAY','Gateway','gateway','CHARGE','Charge','charge','GST','EMAIL','Email','email','PERSONAL','Personal','personal','INFORMATION','Information','information','SINGLE','Single','single','NOMINEE','Nominee','nominee','HOUSING','Housing','housing','FINANCE','Finance','finance','LIMITED','Limited','limited','FLOOR','Floor','floor','CARDIAC','Cardiac','cardiac','REPORT','Report','report','discrepancy','EasyCall','ACCOUNT','Account','account','STATEMENT','Statement','statement','Mode','HOLDING','Holding','holding','STATUS','Status','status','INDIVIDUAL','Individual','individual','FOLIO','Folio','folio','PAGE','Page','page','MUTUAL','Mutual','mutual','FUND','Fund','fund','FUHIC','WHOSE','Whose','whose','APPEAR','Appear','appear','CERTIFICATE','Certificate','certificate','singly','jointly','GRANTED','Granted','granted','ALLOTMENT','Allotment','allotment','UNITS','Units','units','SUBJECT','Subject','subject','REALIZATION','Realization','realization','APPLICABLE','Applicable','applicable','LOAD','Load','load','STRUCTURE','Structure','structure','REFER','Refer','refer','FURTHER','Further','further','FIND','Find','find','PERIOD','Period','period','YOUR','Your','your','STAMP','Stamp','stamp','DUTY','Duty','duty','FLAT','Flat','flat','FIRST','First','first','SECOND','Second','second','THIRD','Third','third','GUARDIAN','Guardian','guardian','EDUCATIONAL','Educational','educational','INSTITUTIONS','Institutions','institutions','TRANSFERABLE','Transferable','transferable','Issuing','Portfolio','SUMMARY','Summary','summary','CODE','Code','code','DEVELOPMENT','Development','development','HEALTH','Health','health','STANDARD','Standard','standard','INSURER','Insurer','insurer','TERMS','Terms','terms','INTERNATIONAL','International','international','SCHOOL','School','school','ROLL','Roll','roll','TERM','Term','term','ROAD','Road','road','MASTER','Master','master','MAST','Mast','mast','CHEQUE','Cheque','cheque','INSTITUTE','Institute','institute','CLASS','Class','class','ENROLLMENT','Enrollment','enrollment','QUARTER','Quarter','quarter','QUARTERLY','Quarterly','quarterly','SESSION','Session','session','STUDYING','Studying','studying','ACADEMY','Academy','academy','ADMISSION','Admission','admission','FREQUENCY','Frequency','frequency','PLUS','Plus','plus','DEATH','Death','death','SECTION','Section','section','PERCENTAGE','Percentage','percentage','DEDUCTIONS','Deductions','deductions','NUMBER','Number','number','MOBILE','Mobile','mobile','SECTOR','Sector','sector','SURVIVIOR','Survivor','survivor','HOUSE','House','house','COMMENCE','Commence','commence','RECEIPT','Receipt','receipt','BORROWERS','Borrowers','borrowers','PRESENTOR','Presentor','presentor','TRANSACTION','Transaction','transaction','COLLECTION','Collection','collection','PERSON','Person','person','CREDIT','Credit','credit','DEBIT','Debit','debit','PROPOSED','Proposed','proposed','CLIENT','Client','client','RESIDENTIAL','Residential','residential','PROPOSER','Proposer','proposer','FIGURES','Figures','figures','POKHRAN','Pokhran','pokhran','DOCUMENT','Document','document','NOTEPAD','Notepad','notepad','COMPLETE','Complete','complete','SmtiMsiShri','Smt/Ms\./SK','Smt/Ms_/SKi','Smt/Ms\./Sri','Smt\./Ms\./Shri: MAST','Smt/MsJ/Shrit','smt/Ms/Shri','SmtJ/Ms\./Shri','Smt/ivis/Sri','Shri/Ss','ShrilSmt','Smt/MsJ/Shri','Smt/MsJShri','SmtJMsJ/Shri','SmtiMs/Shri','Smt\./Ms\./Shri','Smit/MsJ/Shri','Shri/Smt\.Smt\.','Smt\./Ms/Sri\. Sree WATT','Smt\./Ms/Sri\. Sree','Smt/Ms/Sr','Smt/Ms\./St','M/s','Smt/Ms\./Shri','Mr\./Mrs\.','Shri/Smt/Kum','Shri/Si','Shri/gme','Shri/Smt\.','Mrs','MRS','Mr','MR','Ms','MS','Miss','MISS','Shri','SHRI','Smt','SMT']
# for i in words:
#     df_final_result['insured_name'] = df_final_result['insured_name'].replace(to_replace=i,value="",regex=True)

In [1568]:
words = ['your dcb educational loan account','dcb educational loan account','educational loan account','your dcb education loan account','dcb education loan account','education loan account','P policy andsend it to','Creation Date Investor PAN Holding Invested Amount Market Value','Welcome Subscriber','Welcome Subsenter','HOFC Unit Linked','\(s\) as per details given below','as per details given below','National Pension System \(NPS\)  Central Recordkeeping Agency \(CRA\) sae eee\}                       NASEEM NIZARAL\| SUTARWALA','Transaction Status','Transaction Date','Transaction Reference Number','Contribution Receipt Number','Acknowledgement Number','GST','Payment Gateway Charge','Goods & Services Tax Payment Gateway','Goods & Services Tax','POP trail commission','Contribution Tier Type','Contribution Amount','\]','Name of the Subscriber','Total Amt','CD Charges','Sum Assured','DOC','D.O.C.','Agency Code','Next Due','Edu Cess','Late Fee','Due','Term Mode','Branch Name','Service Tax','Of inst','Total Premium','Prem\(Rs\)','Proposal Number','Vroposal Number','Proposal/Policy No','Life','Assured','Name Status','Loan Code','LOAN NO','loan code','LOAN CODE','Loan No','loan no','Loan A/c\.No','Ref\. LA','Ref: A/c no','Loan Account Number','Permanent','Retirement','PRAN','# Contribution Amount  Contribution Tier Type  POP trail commission  Goods & Services Tax Payment Gateway  Payment Gateway Charge  Goods & Services Tax GST  Acknowledgement  Contribution Receipt  Transaction Reference  Transaction Date  Transaction Status  110042676646','account number','account no','Policy','No','Plan','Inst','Number','Proposal','Vroposal','Previaus','Account','Folio','folio','#2','Permanent Retirement Account Number\(PRAN\) Name of the Subscriber  # Contribution Amount  Contribution Tier Type  POP trail commission \+ Goods & Services Tax Payment Gateway  Payment Gateway Charge \+ Goods & Services Tax \(GST\)  Acknowledgement Number Contribution Receipt Number Transaction Reference Number Transaction Date  Transaction Status  ','Fr','To','pt No','pt','No','no','with','the','following','details','Subscriber','Welcome','educational','your','dcb','education','loan','Subsenter','Creation','Date','Investor','PAN','Holding','Invested','Amount','Market','Value','Policy','policy','and','send','it','to','account','number','Document','POLICY','Polley','notepad','Details','POLICY NO','acc','a/c','no','is','The','\$','"','\'']
for i in words:
    df_final_result['policy_number'] = df_final_result['policy_number'].replace(to_replace=i,value="",regex=True)

In [1569]:
words = ['Date of Issue','DATE OF ISSUE','Authorized Signatory','INSURANCE Date','Authorized','Signatory','Date of deposit','INTEREST CERTIFICATE','Statement Date','tatement Date','Status Mode of Holding PAN : 4834776 / 82','Date','Receipt','of','Inception','Time','Investment','value','date','110042676646 SUBHRANSU KUMAR SABAT  INR 4000.00 Tier \|  INR 11.80 SBI PAYMENT GATEWAY  INR 0.71 12001343  1263969501 7999327  9710359615109','PN Name PRATIK BHUPENORA PANCHOL! From veo, 2017 Te ee 31.2097 Address 1.403 VARTA Pane APT BAH Trt Shae ACTIVE Gevwralon Date December 20.2010  RAVI DARSHAN SOC BHATAR CHAR TheR Sates Not Ata  RASTA Scheme choles ACTIVE CHOICE  surat POP Rego sonoot POP-SP RegNo — e012615,  ‘cuuanar.ansor7','08.45 PL \| \+ 2 97 ANANT VAY APARTMENT HAVGHAR ROAD GaN Genoration Gap. 2016 \| Address” SAIBABA NAGAR OFF SALASAR NAGAR BHIAYANDAR, EAST Bate OFBIth see 25, 198s \| THANE Saving BNKAIC ary p4000" \| MAHARASHTRA - 601705 Ne CH OLAS ‘ \(NOUA \| APY-SP Bank \| SN Pateseeon \| IRA Stands IRA correlont Rey. no. \+ Foeade ee Mabie genrapapo6','Statement Oate \>','Statement Oate','Transaction Date  Transaction Status','DATE','Date','DATE OF ISSUE :','DATE OF ISSUE','TNO','Tr\. No\.','Tr','"',","]
for i in words:
    df_final_result['premium_date'] = df_final_result['premium_date'].replace(to_replace=i,value="",regex=True)

In [1570]:
words = ['Modal','Premium','Mode','Payment','payment','Frequency','frequency','and','Rs'] 
for i in words:
    df_final_result['Pay_mode'] = df_final_result['Pay_mode'].replace(to_replace=i,value="",regex=True)

In [1571]:
words = ['We acknowledge with thanks','03-JAN-2019','Premium Received / Top Up#/ indexed Premium Received# \(2\)','fax Benefit','Tax Benefit','tax Benefit','Premium Paid \(Rs\.\)','Premium Paid','Target Amount \(2\)','Target Amount','Rs.','RS.','Total Amount','Paid Amount','Amount Paid','amount paid','Amount paid','Amount','amount','Contribution Amount  Contribution Tier Type  POP trail commission \+ Goods & Services Tax Payment Gateway  Payment Gateway Charge \+ Goods & Services Tax \(GST\)  Acknowledgement Number Contribution Receipt Number Transaction Reference Number Transaction Date  Transaction Status  110042676646 SUBHRANSU KUMAR SABAT  INR','total contribution to your pension account till Dec 31, 2017 was','premium','Contribution','amt','of','Rs','Grand','Total','amount','has','paid','Gross','gross','Premium','premium','Principal','Amount','INR','Toward Interest','Towards Interest','toward interest','7\*\*','receipt']
for i in words:
    df_final_result['premium_amt'] = df_final_result['premium_amt'].replace(to_replace=i,value="",regex=True)

In [1572]:
words = ['CGST @9%','UTGST @9%','Amount','Amount (INR)','INR','RS.','Rs.','RS','Tuition Fee','Tuition','tuition','Tution','tution','TUTION','TUITION','tution','fee','Tution','Fee','Term - 1','term - 1','TERM - 1','Term','term']
for i in words:
    df_final_result['Amount1'] = df_final_result['Amount1'].replace(to_replace=i,value="",regex=True)

In [1573]:
words = ['UTGST @9%','Amount','Amount (INR)','INR','RS.','Rs.','RS','Tuition Fee','Tuition','tuition','Tution','tution','TUTION','TUITION','tution','fee','Tution','Fee','Term - 1','term - 1','TERM - 1','TERMI 7','Term','term'] 
for i in words:
    df_final_result['Amount2'] = df_final_result['Amount2'].replace(to_replace=i,value="",regex=True)

In [1574]:
words = ['Status Mode of Holding PAN :']
for i in words:
    df_final_result['Scheme'] = df_final_result['Scheme'].replace(to_replace=i,value="",regex=True)

In [1575]:
df_final_result['provider']=df_final_result['provider'].str.replace(r"[\(\)\{\}<>/]","")
df_final_result['policy_number']=df_final_result['policy_number'].str.replace(r"[\(\)\{\}\:\.,\»\+\#<>|]","")
df_final_result['premium_date']=df_final_result['premium_date'].str.replace(r"[\(\)\{\}\:=<>|]","")
df_final_result['premium_date']=df_final_result['premium_date'].str.replace(r"[,]"," ")
df_final_result['premium_date']=df_final_result['premium_date'].str.replace(r"."," ")
df_final_result['Pay_mode']=df_final_result['Pay_mode'].str.replace(r"[\(\)\{\}<>,']","")
df_final_result['premium_amt']=df_final_result['premium_amt'].str.replace(r"[\(\)\{\}\~\=\:\/]","")

In [1576]:
k=0
for l in df_final_result['insured_name']:
    if l != None:
        df_final_result.loc[k,'insured_name']=re.sub('[^A-Za-z., ]', "", df_final_result.loc[k,'insured_name'])
    k=k+1

In [1577]:
e=0
for f in df_final_result['Pay_mode']:
    if f != None:
        df_final_result.loc[e,'Pay_mode']=re.sub('[^A-Za-z, ]', "", df_final_result.loc[e,'Pay_mode'])
    e=e+1

In [1578]:
a=0
for b in df_final_result['Amount1']:
    if b != None:
        df_final_result.loc[a,'Amount1']=re.sub('[^0-9.]', "", df_final_result.loc[a,'Amount1'])
    a=a+1

In [1579]:
c=0
for d in df_final_result['Amount2']:
    if d != None:
        df_final_result.loc[c,'Amount2']=re.sub('[^0-9.]', "", df_final_result.loc[c,'Amount2'])
    c=c+1

In [1580]:
a=0
for b in df_final_result['premium_amt']:
    if b != None:
        df_final_result.loc[a,'premium_amt']=re.sub('[^0-9.]', "", df_final_result.loc[a,'premium_amt'])
    a=a+1

In [1581]:
def trimAllColumns(df_final_result):
    trimStrings = lambda x: x.strip() if type(x) is str else x
    return df_final_result.applymap(trimStrings)

df_final_result = trimAllColumns(df_final_result)

In [1582]:
for i in range(0,len(df_final_result)):
    try:
        df_final_result.loc[i,'premium_date'] = pd.to_datetime(df_final_result.loc[i,'premium_date'],dayfirst=True)
    except (ValueError,OverflowError):
        df_final_result.loc[i,'premium_date'] = None

In [1583]:
df_final_result['premium_date'] = pd.to_datetime(df_final_result['premium_date'],dayfirst=True).dt.date

In [1584]:
for i in range(0,len(df_final_result)):
    df_final_result.loc[i,'insured_name'] = str(df_final_result.loc[i,'insured_name'])
df_final_result['insured_name'] = df_final_result.insured_name.apply(lambda x: x.strip('.'))

In [1585]:
def unique_id():
    r = random.randint(1, 1000000)
    t = datetime.datetime.now()
    t = str(t).replace(r" ", "_")
    unique_ID = 'VScan_'+str(t)+'_'+str(r)
    # unique_ID_json = '[{'+ str("\"unique_ID\"") + ':' + '"' + unique_ID + '"}]'
    # unique_ID_json = json.loads(unique_ID_json)
    return unique_ID

In [1586]:
df_final_result

,path,File_name,provider,investment_type,policy_number,insured_name,premium_date,Pay_mode,Scheme,premium_amt,Amount1,remark1,Amount2,remark2,Amount3,remark3
0,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2184_Rent_Agreement...,HDFC,,None,None,2016-06-15,NA,NA,50000,,NA,,NA,NA,NA
1,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2185_Rent_Agreement...,NA,RENTAL AGREEMENT,None,None,2014-12-13,None,None,None,,NA,,NA,NA,NA
2,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2186_Rent_Agreement...,NA,RENTAL AGREEMENT,None,None,NaT,None,None,None,,NA,,NA,NA,NA
3,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2187_Rent_Agreement...,None,,None,None,NaT,NA,NA,None,,NA,,NA,NA,NA
4,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2188_Rent_Agreement...,None,,None,None,NaT,NA,NA,None,,NA,,NA,NA,NA
5,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2189_Rent_Agreement...,None,,None,None,NaT,NA,NA,None,,NA,,NA,NA,NA
6,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2190_Rent_Agreement...,None,Invalid Document,None,G.Chidanandappa,NaT,NA,NA,None,,NA,,NA,NA,NA
7,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2191_Rent_Agreement...,NA,RENTAL AGREEMENT,None,None,NaT,None,None,8600,,NA,,NA,NA,NA
8,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2192_Rent Receipt-D...,None,,None,None,NaT,NA,NA,None,,NA,,NA,NA,NA
9,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2193_Rent Receipt_B...,NA,RENTAL AGREEMENT,None,RAVINDRA LAXMAN JOGDAND,2019-01-12,None,None,None,,NA,,NA,NA,NA


In [1587]:
for i in range(0,len(df_final_result)):
    if df_final_result.loc[i,'provider'] == '':
        df_final_result.loc[i,'provider'] = None
    if df_final_result.loc[i,'Amount1'] == '':
        df_final_result.loc[i,'Amount1'] = None
    if df_final_result.loc[i,'Amount2'] == '':
        df_final_result.loc[i,'Amount2'] = None
    if df_final_result.loc[i,'Amount3'] == '':
        df_final_result.loc[i,'Amount3'] = None
    if df_final_result.loc[i,'insured_name'] == '':
        df_final_result.loc[i,'insured_name'] = None
    if df_final_result.loc[i,'policy_number'] == '':
        df_final_result.loc[i,'policy_number'] = None
    if df_final_result.loc[i,'premium_amt'] == '':
        df_final_result.loc[i,'premium_amt'] = None
    if df_final_result.loc[i,'premium_date'] == '':
        df_final_result.loc[i,'premium_date'] = None 

In [1588]:
for i in range(0,len(df_final_result)):
    
    if df_final_result.loc[i,'policy_number']!= None:
        df_final_result.loc[i,'policy_number'] = str(df_final_result.loc[i,'policy_number'])
        if len(df_final_result.loc[i,'policy_number']) < 4:
            df_final_result.loc[i,'policy_number'] = None
        elif len(df_final_result.loc[i,'policy_number']) > 40:
            df_final_result.loc[i,'policy_number'] = None
    else:
        pass

In [1589]:
for i in range(0,len(df_final_result)):
    if df_final_result.loc[i,'premium_amt']!= None:
        if len(df_final_result.loc[i,'premium_amt']) > 12:
            df_final_result.loc[i,'premium_amt'] = None
    else:
        pass
    if df_final_result.loc[i,'Amount1']!= None:
        if len(df_final_result.loc[i,'Amount1']) > 10:
            df_final_result.loc[i,'Amount1'] = None
    else:
        pass
    if df_final_result.loc[i,'Amount2']!= None:
        if len(df_final_result.loc[i,'Amount2']) > 10:
            df_final_result.loc[i,'Amount2'] = None
    else:
        pass
    if df_final_result.loc[i,'Amount3']!= None:
        if len(df_final_result.loc[i,'Amount3']) > 10:
            df_final_result.loc[i,'Amount3'] = None
    else:
        pass

In [1590]:
for i in range(0,len(df_final_result)):
    df_final_result.loc[i,'premium_amt'] = str(df_final_result.loc[i,'premium_amt'])
    
    df_final_result.loc[i,'Amount1'] = str(df_final_result.loc[i,'Amount1'])
    df_final_result.loc[i,'Amount2'] = str(df_final_result.loc[i,'Amount2'])
    df_final_result.loc[i,'Amount3'] = str(df_final_result.loc[i,'Amount3'])

def rectify_amt(corpus):
    regex=r"(\.{2,})"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        s = re.sub("\.{2,}", "", corpus, 1)
        return(s)

def rectify_amt1(corpus):
    regex=r"\b(\d+\.\d+\.\d+\.\d+)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        s = re.sub("\.", "", corpus, 1)
        s = re.sub("\.", "", s, 1)
        return(s)

def rectify_amt2(corpus):
    regex=r"\b(\d+\.\d+\.\d+)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        s = re.sub("\.", "", corpus, 1)
        return(s)

def rectify_amt3(corpus):
    regex=r"\b(\.\d+\.\d+\.\d+)\b"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        s = re.sub("\.", "", corpus, 1)
        return(s)

def amount_rectification(row):
    corpus = row.premium_amt
    Amount = rectify_amt(corpus)
    if Amount is None:
        Amount = rectify_amt1(corpus)
        if Amount is None:
            Amount = rectify_amt2(corpus)
            if Amount is None:
                Amount = rectify_amt3(corpus)
                if Amount is None:
                    Amount = corpus
    return Amount

def amount_rectification1(row):
    corpus = row.Amount1
    Amount = rectify_amt(corpus)
    if Amount is None:
        Amount = rectify_amt1(corpus)
        if Amount is None:
            Amount = rectify_amt2(corpus)
            if Amount is None:
                Amount = rectify_amt3(corpus)
                if Amount is None:
                    Amount = corpus
    return Amount

def amount_rectification2(row):
    corpus = row.Amount2
    Amount = rectify_amt(corpus)
    if Amount is None:
        Amount = rectify_amt1(corpus)
        if Amount is None:
            Amount = rectify_amt2(corpus)
            if Amount is None:
                Amount = rectify_amt3(corpus)
                if Amount is None:
                    Amount = corpus
    return Amount

def amount_rectification3(row):
    corpus = row.Amount3
    Amount = rectify_amt(corpus)
    if Amount is None:
        Amount = rectify_amt1(corpus)
        if Amount is None:
            Amount = rectify_amt2(corpus)
            if Amount is None:
                Amount = rectify_amt3(corpus)
                if Amount is None:
                    Amount = corpus
    return Amount

In [1591]:
df_final_result['premium_amt'] = df_final_result.premium_amt.apply(lambda x: x.strip('.'))

In [1592]:
df_final_result['premium_amt'] = df_final_result.apply(amount_rectification,axis = 1)

In [1593]:
df_final_result['Amount1'] = df_final_result.apply(amount_rectification1,axis = 1)

In [1594]:
df_final_result['Amount2'] = df_final_result.apply(amount_rectification2,axis = 1)

In [1595]:
df_final_result['Amount3'] = df_final_result.apply(amount_rectification3,axis = 1)

In [1596]:
df_final_result['premium_amt'] = df_final_result.premium_amt.apply(lambda x: x.strip('.'))

In [1597]:
df_final_result['Amount1'] = df_final_result.Amount1.apply(lambda x: x.strip('.'))

In [1598]:
df_final_result['Amount2'] = df_final_result.Amount2.apply(lambda x: x.strip('.'))

In [1599]:
df_final_result['Amount3'] = df_final_result.Amount3.apply(lambda x: x.strip('.'))

In [1600]:
def amt_strip(corpus):
    regex = r"(\d+\.\d\d|\d+\.\d|\d+)"
#     regex = r"(Amount paid|amount paid|Amount Deposited|Total Amount|total amount)(.|..|...|....)\d+"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None

In [1601]:
def prem_amount_strip(row):
    corpus = row.premium_amt
    Amount = amt_strip(corpus)
    return Amount

def amount1_strip(row):
    corpus = row.Amount1
    Amount = amt_strip(corpus)
    return Amount

def amount2_strip(row):
    corpus = row.Amount2
    Amount = amt_strip(corpus)
    return Amount

def amount3_strip(row):
    corpus = row.Amount3
    Amount = amt_strip(corpus)
    return Amount

In [1602]:
df_final_result['premium_amt'] = df_final_result.apply(prem_amount_strip,axis = 1)

In [1603]:
df_final_result['Amount1'] = df_final_result.apply(amount1_strip,axis = 1)

In [1604]:
df_final_result['Amount2'] = df_final_result.apply(amount2_strip,axis = 1)

In [1605]:
df_final_result['Amount3'] = df_final_result.apply(amount3_strip,axis = 1)

In [1606]:
for i in range(0,len(df_final_result)):
    if df_final_result.loc[i,'premium_amt']== 'None':
        df_final_result.loc[i,'premium_amt'] = None
    if df_final_result.loc[i,'Amount1']== 'None':
        df_final_result.loc[i,'Amount1'] = None
    if df_final_result.loc[i,'Amount2']== 'None':
        df_final_result.loc[i,'Amount2'] = None
    if df_final_result.loc[i,'Amount3']== 'None':
        df_final_result.loc[i,'Amount3'] = None   
    if df_final_result.loc[i,'Amount1'] == "NA":
        df_final_result.loc[i,'Amount1'] = '0'
    if df_final_result.loc[i,'Amount2'] == "NA":
        df_final_result.loc[i,'Amount2'] = '0'
    if df_final_result.loc[i,'Amount3'] == "NA":
        df_final_result.loc[i,'Amount3'] = '0'
    if df_final_result.loc[i,'premium_amt']== '':
        df_final_result.loc[i,'premium_amt'] = None
    if df_final_result.loc[i,'Amount1']== '':
        df_final_result.loc[i,'Amount1'] = None
    if df_final_result.loc[i,'Amount2']== '':
        df_final_result.loc[i,'Amount2'] = None
    if df_final_result.loc[i,'Amount3']== '':
        df_final_result.loc[i,'Amount3'] = None   

In [1607]:
df_final_result

,path,File_name,provider,investment_type,policy_number,insured_name,premium_date,Pay_mode,Scheme,premium_amt,Amount1,remark1,Amount2,remark2,Amount3,remark3
0,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2184_Rent_Agreement...,HDFC,,None,None,2016-06-15,NA,NA,50000,None,NA,None,NA,None,NA
1,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2185_Rent_Agreement...,NA,RENTAL AGREEMENT,None,None,2014-12-13,None,None,None,None,NA,None,NA,None,NA
2,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2186_Rent_Agreement...,NA,RENTAL AGREEMENT,None,None,NaT,None,None,None,None,NA,None,NA,None,NA
3,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2187_Rent_Agreement...,None,,None,None,NaT,NA,NA,None,None,NA,None,NA,None,NA
4,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2188_Rent_Agreement...,None,,None,None,NaT,NA,NA,None,None,NA,None,NA,None,NA
5,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2189_Rent_Agreement...,None,,None,None,NaT,NA,NA,None,None,NA,None,NA,None,NA
6,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2190_Rent_Agreement...,None,Invalid Document,None,G.Chidanandappa,NaT,NA,NA,None,None,NA,None,NA,None,NA
7,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2191_Rent_Agreement...,NA,RENTAL AGREEMENT,None,None,NaT,None,None,8600,None,NA,None,NA,None,NA
8,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2192_Rent Receipt-D...,None,,None,None,NaT,NA,NA,None,None,NA,None,NA,None,NA
9,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2193_Rent Receipt_B...,NA,RENTAL AGREEMENT,None,RAVINDRA LAXMAN JOGDAND,2019-01-12,None,None,None,None,NA,None,NA,None,NA


In [1608]:
for i in range(0,len(df_final_result)):

    if df_final_result.loc[i,'premium_amt']!= None:
        try:
            a = float(df_final_result.loc[i,'premium_amt'])
            if a == 0.00:
                df_final_result.loc[i,'premium_amt'] = df_final_result.loc[i,'premium_amt']
            elif a < 350.00:
                df_final_result.loc[i,'premium_amt'] = None
            elif a > 300000.00:
                df_final_result.loc[i,'premium_amt'] = None
        except ValueError:
            df_final_result.loc[i,'premium_amt'] = None
    else: 
        pass

In [1609]:
for i in range(0,len(df_final_result)):
    if df_final_result.loc[i,'investment_type'] == 'LIFE INSURANCE':  
        if df_final_result.loc[i,'Amount1']!= None:
            try:
                b = float(df_final_result.loc[i,'Amount1'])
                if b < 0.00:
                    df_final_result.loc[i,'Amount1'] = None
                elif b > 6000.00:
                    df_final_result.loc[i,'Amount1'] = None
            except ValueError:
                df_final_result.loc[i,'Amount1'] = None
        else:
            pass
    elif df_final_result.loc[i,'investment_type'] == 'MEDICLAIM':
        if df_final_result.loc[i,'Amount1']!= None:
            try:
                b = float(df_final_result.loc[i,'Amount1'])
                if b < 0.00:
                    df_final_result.loc[i,'Amount1'] = None
                elif b > 6000.00:
                    df_final_result.loc[i,'Amount1'] = None
            except ValueError:
                df_final_result.loc[i,'Amount1'] = None
        else:
            pass
    elif df_final_result.loc[i,'investment_type'] == 'CHILDREN EDUCATION':
        if df_final_result.loc[i,'Amount1']!= None:
            try:
                b = float(df_final_result.loc[i,'Amount1'])
                if b < 0.00:
                    df_final_result.loc[i,'Amount1'] = None
                elif b > 160000.00:
                    df_final_result.loc[i,'Amount1'] = None
            except ValueError:
                df_final_result.loc[i,'Amount1'] = None
        else:
            pass
    elif df_final_result.loc[i,'investment_type'] == 'HOUSING LOAN':
        if df_final_result.loc[i,'Amount1']!= None:
            try:
                b = float(df_final_result.loc[i,'Amount1'])
                if b < 1000.00:
                    df_final_result.loc[i,'Amount1'] = None
                elif b > 5000000.00:
                    df_final_result.loc[i,'Amount1'] = None
            except ValueError:
                df_final_result.loc[i,'Amount1'] = None
        else:
            pass
    else:
        if df_final_result.loc[i,'Amount1']!= None:
            try:
                b = float(df_final_result.loc[i,'Amount1'])
                if b < 0.00:
                    df_final_result.loc[i,'Amount1'] = None
                elif b > 200000.00:
                    df_final_result.loc[i,'Amount1'] = None
            except ValueError:
                df_final_result.loc[i,'Amount1'] = None
        else:
            pass

In [1610]:
for i in range(0,len(df_final_result)):
    if df_final_result.loc[i,'investment_type'] == 'LIFE INSURANCE':  
        if df_final_result.loc[i,'Amount2']!= None:
            try:
                b = float(df_final_result.loc[i,'Amount2'])
                if b < 0.00:
                    df_final_result.loc[i,'Amount2'] = None
                elif b > 6000.00:
                    df_final_result.loc[i,'Amount2'] = None
            except ValueError:
                df_final_result.loc[i,'Amount2'] = None
        else:
            pass
    elif df_final_result.loc[i,'investment_type'] == 'MEDICLAIM':
        if df_final_result.loc[i,'Amount2']!= None:
            try:
                b = float(df_final_result.loc[i,'Amount2'])
                if b < 0.00:
                    df_final_result.loc[i,'Amount2'] = None
                elif b > 6000.00:
                    df_final_result.loc[i,'Amount2'] = None
            except ValueError:
                df_final_result.loc[i,'Amount2'] = None
        else:
            pass
    elif df_final_result.loc[i,'investment_type'] == 'CHILDREN EDUCATION':
        if df_final_result.loc[i,'Amount2']!= None:
            try:
                b = float(df_final_result.loc[i,'Amount2'])
                if b < 0.00:
                    df_final_result.loc[i,'Amount2'] = None
                elif b > 160000.00:
                    df_final_result.loc[i,'Amount2'] = None
            except ValueError:
                df_final_result.loc[i,'Amount2'] = None
        else:
            pass
    elif df_final_result.loc[i,'investment_type'] == 'HOUSING LOAN':
        if df_final_result.loc[i,'Amount2']!= None:
            try:
                b = float(df_final_result.loc[i,'Amount2'])
                if b < 1000.00:
                    df_final_result.loc[i,'Amount2'] = None
                elif b > 5000000.00:
                    df_final_result.loc[i,'Amount2'] = None
            except ValueError:
                df_final_result.loc[i,'Amount2'] = None
        else:
            pass
    else:
        if df_final_result.loc[i,'Amount2']!= None:
            try:
                b = float(df_final_result.loc[i,'Amount2'])
                if b < 0.00:
                    df_final_result.loc[i,'Amount2'] = None
                elif b > 200000.00:
                    df_final_result.loc[i,'Amount2'] = None
            except ValueError:
                df_final_result.loc[i,'Amount2'] = None
        else:
            pass

In [1611]:
# for i in range(0,len(df_final_result)):
#     if df_final_result.loc[i,'Amount1']!= None:
#         try:
#             b = float(df_final_result.loc[i,'Amount1'])
#             if b < 0.00:
#                 df_final_result.loc[i,'Amount1'] = None
#             elif b > 300000.00:
#                 df_final_result.loc[i,'Amount1'] = None
#         except ValueError:
#             df_final_result.loc[i,'Amount1'] = None
#     else:
#         pass

In [1612]:
# for i in range(0,len(df_final_result)):
#     if df_final_result.loc[i,'Amount2']!= None:
#         try:
#             c = float(df_final_result.loc[i,'Amount2'])
#             if c < 0.00:
#                 df_final_result.loc[i,'Amount2'] = None
#             elif c > 200000.00:
#                 df_final_result.loc[i,'Amount2'] = None
#         except ValueError:
#             df_final_result.loc[i,'Amount2'] = None
#     else:
#         pass

In [1613]:
for i in range(0,len(df_final_result)):
    if df_final_result.loc[i,'Amount3']!= None:
        try:
            d = float(df_final_result.loc[i,'Amount3'])
            if d < 0.00:
                df_final_result.loc[i,'Amount3'] = None
            elif d > 200000.00:
                df_final_result.loc[i,'Amount3'] = None
        except ValueError:
            df_final_result.loc[i,'Amount3'] = None
    else:
        pass

In [1614]:
for i in range(0,len(df_final_result)):
    
    if pd.isnull(df_final_result.loc[i,'premium_date'])==False:
        TODAY_CHECK = str(df_final_result.loc[i,'premium_date'])
        TODAY_CHECK = datetime.datetime.strptime(TODAY_CHECK, "%Y-%m-%d")
        
        start = datetime.datetime.strptime("2016-04-01", "%Y-%m-%d")
        end = datetime.datetime.strptime("2022-03-31", "%Y-%m-%d")

        if start <= TODAY_CHECK <= end:
            pass
        else:
            df_final_result.loc[i,'premium_date'] = pd.NaT
    else:
        pass

In [1615]:
for i in range(0,len(df_final_result)):
    if (df_final_result.loc[i,'investment_type']== 'LIFE INSURANCE') or (df_final_result.loc[i,'investment_type']== 'MEDICLAIM'): 
        if (df_final_result.loc[i,'premium_amt']!=None) and (df_final_result.loc[i,'Amount1']!=None):
            if float(df_final_result.loc[i,'premium_amt']) < float(df_final_result.loc[i,'Amount1']):
                df_final_result.loc[i,'premium_amt'] = None
                df_final_result.loc[i,'Amount1'] = None
                df_final_result.loc[i,'Amount2'] = None

In [1616]:
for i in range(0,len(df_final_result)):
    if df_final_result.loc[i,'insured_name'] == 'None':
        df_final_result.loc[i,'insured_name'] = None

In [1617]:
df_final_result

,path,File_name,provider,investment_type,policy_number,insured_name,premium_date,Pay_mode,Scheme,premium_amt,Amount1,remark1,Amount2,remark2,Amount3,remark3
0,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2184_Rent_Agreement...,HDFC,,None,None,2016-06-15,NA,NA,50000,None,NA,None,NA,None,NA
1,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2185_Rent_Agreement...,NA,RENTAL AGREEMENT,None,None,NaT,None,None,None,None,NA,None,NA,None,NA
2,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2186_Rent_Agreement...,NA,RENTAL AGREEMENT,None,None,NaT,None,None,None,None,NA,None,NA,None,NA
3,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2187_Rent_Agreement...,None,,None,None,NaT,NA,NA,None,None,NA,None,NA,None,NA
4,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2188_Rent_Agreement...,None,,None,None,NaT,NA,NA,None,None,NA,None,NA,None,NA
5,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2189_Rent_Agreement...,None,,None,None,NaT,NA,NA,None,None,NA,None,NA,None,NA
6,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2190_Rent_Agreement...,None,Invalid Document,None,G.Chidanandappa,NaT,NA,NA,None,None,NA,None,NA,None,NA
7,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2191_Rent_Agreement...,NA,RENTAL AGREEMENT,None,None,NaT,None,None,8600,None,NA,None,NA,None,NA
8,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2192_Rent Receipt-D...,None,,None,None,NaT,NA,NA,None,None,NA,None,NA,None,NA
9,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2193_Rent Receipt_B...,NA,RENTAL AGREEMENT,None,RAVINDRA LAXMAN JOGDAND,2019-01-12,None,None,None,None,NA,None,NA,None,NA


In [1618]:
def scoring(df):

    for i in range(0,len(df)):
        provider = ''
        insured_name = ''
        premium_date = ''
        premium_amt = ''
        policy_number = ''
        amount1 = ''
        amount2 = ''
        if df.loc[i,'investment_type'] == 'LIFE INSURANCE':
            count = 0
            if df.loc[i,'provider'] != None:
                count = 1
            else:
                provider = 'Provider '
                
            if df.loc[i,'policy_number'] != None:
                count += 1
            else:
                policy_number = 'Policy Number '
                
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'premium_amt'] != None:
                count +=1
            else:
                premium_amt = 'premium amount '
            
            score = str(int(count/5*100))+'%'
            
            df.loc[i,'Values_Count'] = str(int(count)) +'/5'
            df.loc[i,'score'] = score
            if score != '100%':
                df.loc[i,'Remark'] = provider + insured_name + premium_date + premium_amt + policy_number + 'not recognized'
            else:
                df.loc[i,'Remark'] = 'OK'
                
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>  

        if df.loc[i,'investment_type'] == 'MEDICLAIM':
            count = 0
            if df.loc[i,'provider'] != None:
                count = 1
            else:
                provider = 'Provider '
                
            if df.loc[i,'policy_number'] != None:
                count += 1
            else:
                policy_number = 'Policy Number '
                
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'premium_amt'] != None:
                count +=1
            else:
                premium_amt = 'premium amount '
            
            score = str(int(count/5*100))+'%'
            df.loc[i,'score'] = score
            
            if score != '100%':
                df.loc[i,'Remark'] = provider + insured_name + premium_date + premium_amt + policy_number + 'not recognized'
            else:
                df.loc[i,'Remark'] = 'OK'
            df.loc[i,'Values_Count'] = str(int(count)) +'/5'
            
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

        if df.loc[i,'investment_type'] == 'MUTUAL FUND':
            count = 0
            if df.loc[i,'provider'] != None:
                count = 1
            else:
                provider = 'Provider '
                
            if df.loc[i,'policy_number'] != None:
                count += 1
            else:
                policy_number = 'Policy Number '
                
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'premium_amt'] != None:
                count +=1
            else:
                premium_amt = 'premium amount '
                
            score = str(int(count/5*100))+'%'
            df.loc[i,'Values_Count'] = str(int(count)) +'/5'
            df.loc[i,'score'] = score
            if score != '100%':
                df.loc[i,'Remark'] = provider + insured_name + premium_date + premium_amt + policy_number + 'not recognized'
            else:
                df.loc[i,'Remark'] = 'OK'
            
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

        if df.loc[i,'investment_type'] == 'NSC':
            count = 0
            if df.loc[i,'policy_number'] != None:
                count += 1
            else:
                policy_number = 'Policy Number '
                
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'premium_amt'] != None:
                count +=1
            else:
                premium_amt = 'premium amount '
                
            score = str(int(count/4*100))+'%'
            df.loc[i,'Values_Count'] = str(int(count)) +'/4'
            df.loc[i,'score'] = score
            if score != '100%':
                df.loc[i,'Remark'] = policy_number + insured_name + premium_date + premium_amt + 'not recognized'            
            else:
                df.loc[i,'Remark'] = 'OK'           
                
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

        if df.loc[i,'investment_type'] == 'NPSE_Tier1':
            count = 0
            if df.loc[i,'policy_number'] != None:
                count += 1
            else:
                policy_number = 'Policy Number '
                
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'premium_amt'] != None:
                count +=1
            else:
                premium_amt = 'premium amount '
                
            score = str(int(count/4*100))+'%'
            df.loc[i,'score'] = score
            if score != '100%':
                df.loc[i,'Remark'] = insured_name + premium_date + premium_amt + policy_number + 'not recognized'            
            else:
                df.loc[i,'Remark'] = 'OK'    
            df.loc[i,'Values_Count'] = str(int(count)) +'/4'

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

        if df.loc[i,'investment_type'] == 'NPSE_Tier2':
            count = 0
            if df.loc[i,'policy_number'] != None:
                count += 1
            else:
                policy_number = 'Policy Number '
                
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'premium_amt'] != None:
                count +=1
            else:
                premium_amt = 'premium amount '
                
            score = str(int(count/4*100))+'%'
            df.loc[i,'score'] = score
            if score != '100%':
                df.loc[i,'Remark'] = insured_name + premium_date + premium_amt + policy_number + 'not recognized'            
            else:
                df.loc[i,'Remark'] = 'OK'
            df.loc[i,'Values_Count'] = str(int(count)) +'/4'

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

        if df.loc[i,'investment_type'] == 'Fixed Deposit':

            count = 0
            if df.loc[i,'provider'] != None:
                count = 1
            else:
                provider = 'Provider '
                
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'premium_amt'] != None:
                count +=1
            else:
                premium_amt = 'premium amount '

            score = str(int(count/4*100))+'%'
            df.loc[i,'score'] = score
            if score != '100%':
                df.loc[i,'Remark'] = provider + insured_name + premium_date + premium_amt + 'not recognized'
            else:
                df.loc[i,'Remark'] = 'OK'
            df.loc[i,'Values_Count'] = str(int(count)) +'/4'

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

        if df.loc[i,'investment_type'] == 'CHILDREN EDUCATION':
            count = 0
            
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'premium_amt'] != None:
                count +=1
            else:
                premium_amt = 'premium amount '
                
            score = str(int(count/3*100))+'%'
            df.loc[i,'score'] = score
            if score != '100%':
                df.loc[i,'Remark'] = insured_name + premium_date + premium_amt + 'not recognized'
            else:
                df.loc[i,'Remark'] = 'OK'
            df.loc[i,'Values_Count'] = str(int(count)) +'/3'

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

        if df.loc[i,'investment_type'] == 'EDUCATION LOAN':
            count = 0
            if df.loc[i,'provider'] != None:
                count = 1
            else:
                provider = 'Provider '
                
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'premium_amt'] != None:
                count +=1
            else:
                premium_amt = 'premium amount '
                
            score = str(int(count/4*100))+'%'
            df.loc[i,'score'] = score
            if score != '100%':
                df.loc[i,'Remark'] = provider + insured_name + premium_date + premium_amt + 'not recognized'            
            else:
                df.loc[i,'Remark'] = 'OK'
            df.loc[i,'Values_Count'] = str(int(count)) +'/4'

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

        if df.loc[i,'investment_type'] == 'RENTAL AGREEMENT':
            count = 0
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'premium_amt'] != None:
                count +=1
            else:
                premium_amt = 'premium amount '
                
            score = str(int(count/3*100))+'%'
            df.loc[i,'score'] = score
            if score != '100%':
                df.loc[i,'Remark'] = insured_name + premium_date + premium_amt + 'not recognized'            
            else:
                df.loc[i,'Remark'] = 'OK'
            df.loc[i,'Values_Count'] = str(int(count)) +'/3'

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

        if df.loc[i,'investment_type'] == 'PPF':
            count = 0
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'premium_amt'] != None:
                count +=1
            else:
                premium_amt = 'premium amount '
                
            score = str(int(count/3*100))+'%'
            df.loc[i,'score'] = score
            if score != '100%':
                df.loc[i,'Remark'] = insured_name + premium_date + premium_amt + 'not recognized'            
            else:
                df.loc[i,'Remark'] = 'OK'
            df.loc[i,'Values_Count'] = str(int(count)) +'/3'

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>            
            
        if df.loc[i,'investment_type'] == 'HOUSING LOAN':
            count = 0
            if df.loc[i,'provider'] != None:
                count = 1
            else:
                provider = 'Provider '
                
            if df.loc[i,'policy_number'] != None:
                count += 1
            else:
                policy_number = 'Policy Number '
                
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'Amount1'] != None:
                count +=1
            else:
                amount1 = 'Principal '
                
            if df.loc[i,'Amount2'] != None:
                count +=1
            else:
                amount2 = 'Interest '
                
            score = str(int(count/6.0*100))+'%'
            df.loc[i,'score'] = score
            if score != '100%':
                df.loc[i,'Remark'] = provider + insured_name + premium_date + amount1 + amount2 + policy_number + 'not recognized'            
            else:
                df.loc[i,'Remark'] = 'OK'
            df.loc[i,'Values_Count'] = str(int(count)) +'/6'

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>            
            
        if df.loc[i,'investment_type'] == 'ULIP':
            count = 0
            if df.loc[i,'provider'] != None:
                count = 1
            else:
                provider = 'Provider '
                
            if df.loc[i,'policy_number'] != None:
                count += 1
            else:
                policy_number = 'Policy Number '
                
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'premium_amt'] != None:
                count +=1
            else:
                premium_amt = 'premium amount '
                
            score = str(int(count/5*100))+'%'
            df.loc[i,'score'] = score
            if score != '100%':
                df.loc[i,'Remark'] = provider + insured_name + premium_date + premium_amt + policy_number + 'not recognized'
            else:
                df.loc[i,'Remark'] = 'OK'
            df.loc[i,'Values_Count'] = str(int(count)) +'/5'

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>            
            
        if df.loc[i,'investment_type'] == 'Special disease':
            count = 0
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'premium_amt'] != None:
                count +=1
            else:
                premium_amt = 'premium amount '
                
            score = str(int(count/3*100))+'%'
            df.loc[i,'score'] = score
            if score != '100%':
                df.loc[i,'Remark'] = insured_name + premium_date + premium_amt + 'not recognized'            
            else:
                df.loc[i,'Remark'] = 'OK'
            df.loc[i,'Values_Count'] = str(int(count)) +'/3'

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>            
            
        if df.loc[i,'investment_type'] == 'Handicap':
            count = 0
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'premium_amt'] != None:
                count +=1
            else:
                premium_amt = 'premium amount '
                
            score = str(int(count/3*100))+'%'
            df.loc[i,'score'] = score
            if score != '100%':
                df.loc[i,'Remark'] = insured_name + premium_date + premium_amt + 'not recognized'
            else:
                df.loc[i,'Remark'] = 'OK'
            df.loc[i,'Values_Count'] = str(int(count)) +'/3'
            
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

        if df.loc[i,'investment_type'] == 'Invalid Document':
            count = 0
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'premium_amt'] != None:
                count +=1
            else:
                premium_amt = 'premium amount '
                
            score = str(int(count/3*100))+'%'
            df.loc[i,'score'] = score
            if score != '100%':
                df.loc[i,'Remark'] = insured_name + premium_date + premium_amt + 'not recognized'            
            else:
                df.loc[i,'Remark'] = 'OK'
            df.loc[i,'Values_Count'] = str(int(count)) +'/3'
        
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

        if df.loc[i,'investment_type'] == '':
            count = 0
            if df.loc[i,'insured_name'] != None:
                count += 1
            else:    
                insured_name = 'Insured Name '
                
            if pd.isnull(df.loc[i,'premium_date'])== False:   
                count +=1
            else:
                premium_date = 'premium date '
                
            if df.loc[i,'premium_amt'] != None:
                count +=1
            else:
                premium_amt = 'premium amount '
                
            score = str(int(count/3*100))+'%'
            df.loc[i,'score'] = score
            if score != '100%':
                df.loc[i,'Remark'] = insured_name + premium_date + premium_amt + 'not recognized'            
            else:
                df.loc[i,'Remark'] = 'OK'
            df.loc[i,'Values_Count'] = str(int(count)) +'/3'
            
    return df

In [1619]:
scoring(df_final_result)

,path,File_name,provider,investment_type,policy_number,insured_name,premium_date,Pay_mode,Scheme,premium_amt,Amount1,remark1,Amount2,remark2,Amount3,remark3,score,Remark,Values_Count
0,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2184_Rent_Agreement...,HDFC,,None,None,2016-06-15,NA,NA,50000,None,NA,None,NA,None,NA,66%,Insured Name not recognized,2/3
1,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2185_Rent_Agreement...,NA,RENTAL AGREEMENT,None,None,NaT,None,None,None,None,NA,None,NA,None,NA,0%,Insured Name premium date premium amount not r...,0/3
2,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2186_Rent_Agreement...,NA,RENTAL AGREEMENT,None,None,NaT,None,None,None,None,NA,None,NA,None,NA,0%,Insured Name premium date premium amount not r...,0/3
3,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2187_Rent_Agreement...,None,,None,None,NaT,NA,NA,None,None,NA,None,NA,None,NA,0%,Insured Name premium date premium amount not r...,0/3
4,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2188_Rent_Agreement...,None,,None,None,NaT,NA,NA,None,None,NA,None,NA,None,NA,0%,Insured Name premium date premium amount not r...,0/3
5,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2189_Rent_Agreement...,None,,None,None,NaT,NA,NA,None,None,NA,None,NA,None,NA,0%,Insured Name premium date premium amount not r...,0/3
6,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2190_Rent_Agreement...,None,Invalid Document,None,G.Chidanandappa,NaT,NA,NA,None,None,NA,None,NA,None,NA,33%,premium date premium amount not recognized,1/3
7,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2191_Rent_Agreement...,NA,RENTAL AGREEMENT,None,None,NaT,None,None,8600,None,NA,None,NA,None,NA,33%,Insured Name premium date not recognized,1/3
8,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2192_Rent Receipt-D...,None,,None,None,NaT,NA,NA,None,None,NA,None,NA,None,NA,0%,Insured Name premium date premium amount not r...,0/3
9,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2193_Rent Receipt_B...,NA,RENTAL AGREEMENT,None,RAVINDRA LAXMAN JOGDAND,2019-01-12,None,None,None,None,NA,None,NA,None,NA,66%,premium amount not recognized,2/3


In [1620]:
for i in range(0, len(df_final_result)):
    if df_final_result.loc[i, 'policy_number'] == None:
        df_final_result.loc[i, 'policy_number'] = 'None'

df_final_result['policy_number'] = df_final_result.policy_number.apply(lambda x: x.strip('/'))
df_final_result['policy_number'] = df_final_result.policy_number.apply(lambda x: x.strip('-'))

In [1621]:
for i in range(0,len(df_final_result)):    
    if df_final_result.loc[i,'investment_type'] == '':
        df_final_result.loc[i,'investment_type'] = None

In [1622]:
def bad_image_cam(corpus):
    corpus_split = corpus.lower()
    regex=r"^scanned by camscanner"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("Image Scanned by CamScanner")

def poor_image_cam(corpus):
    corpus_split = corpus.lower()
    regex=r"scanned by camscanner$"
    matches = re.finditer(regex, corpus_split, re.MULTILINE)
    for match in matches:
        return("Image taken via Phone")

In [1623]:
def img_quality(row):
    corpus = row.corpus
    quality = bad_image_cam(corpus)
    if quality is None:
        quality = poor_image_cam(corpus)
    return quality

In [1624]:
df_final_result['Image_Quality'] = df_result.apply(img_quality,axis = 1)

In [1625]:
for i in range(0,len(df_final_result)):
    if df_final_result.loc[i,'Image_Quality'] == None:
        if (df_final_result.loc[i,'investment_type'] == 'RENTAL AGREEMENT') or (df_final_result.loc[i,'investment_type'] == 'CHILDREN EDUCATION') :
            if df_final_result.loc[i,'score'] == '0%':
                df_final_result.loc[i,'Image_Quality'] = 'Partially Handwritten or Low Quality Image'
    
        if df_final_result.loc[i,'investment_type'] != None:
            if df_final_result.loc[i,'score'] == '0%':
                df_final_result.loc[i,'Image_Quality'] = 'Possibly Handwritten or Low Quality Image'
                
        if df_final_result.loc[i,'investment_type'] == None:
            if df_final_result.loc[i,'score'] == '0%':
                df_final_result.loc[i,'Image_Quality'] = 'Blurred/Poor Quality Image'
                
        if df_final_result.loc[i,'investment_type'] == None:
            if df_final_result.loc[i,'score'] == '20%':
                df_final_result.loc[i,'Image_Quality'] = 'Irrelevant Text Detected'
                
        if df_final_result.loc[i,'investment_type'] != None:
            if df_final_result.loc[i,'score'] == '20%':
                df_final_result.loc[i,'Image_Quality'] = 'Possibly Handwritten Image'

In [1626]:
for i in range(0,len(df_final_result)):    
    if df_final_result.loc[i,'investment_type'] == None:
        df_final_result.loc[i,'investment_type'] = 'Unrecognized Category'

In [1627]:
for i in range(0,len(df_final_result)):
    
    if pd.isnull(df_final_result.loc[i,'premium_date'])==False:
        TODAY_CHECK = str(df_final_result.loc[i,'premium_date'])
        TODAY_CHECK = datetime.datetime.strptime(TODAY_CHECK, "%Y-%m-%d")
        
        start = datetime.datetime.strptime("2018-04-01", "%Y-%m-%d")
        end = datetime.datetime.strptime("2019-03-31", "%Y-%m-%d")

        if start <= TODAY_CHECK <= end:
            pass
        else:
            df_final_result.loc[i,'Remark'] = df_final_result.loc[i,'Remark']+',NOT FROM CURRENT FINANCIAL YEAR' 
    else:
        pass

In [1628]:
for i in range(0,len(df_final_result)):
    
    if pd.isnull(df_final_result.loc[i,'premium_date'])==False:
        TODAY_CHECK = str(df_final_result.loc[i,'premium_date'])
        TODAY_CHECK = datetime.datetime.strptime(TODAY_CHECK, "%Y-%m-%d")
        
        start_2016 = datetime.datetime.strptime("2016-04-01", "%Y-%m-%d")
        end_2017 = datetime.datetime.strptime("2017-03-31", "%Y-%m-%d")
        start_2017 = datetime.datetime.strptime("2017-04-01", "%Y-%m-%d")
        end_2018 = datetime.datetime.strptime("2018-03-31", "%Y-%m-%d")
        start_2018 = datetime.datetime.strptime("2018-04-01", "%Y-%m-%d")
        end_2019 = datetime.datetime.strptime("2019-03-31", "%Y-%m-%d")
        start_2019 = datetime.datetime.strptime("2019-04-01", "%Y-%m-%d")
        end_2020 = datetime.datetime.strptime("2020-03-31", "%Y-%m-%d")
        start_2020 = datetime.datetime.strptime("2020-04-01", "%Y-%m-%d")
        end_2021 = datetime.datetime.strptime("2021-03-31", "%Y-%m-%d")        
        start_2021 = datetime.datetime.strptime("2021-04-01", "%Y-%m-%d")
        end_2022 = datetime.datetime.strptime("2022-03-31", "%Y-%m-%d")        

        if start_2016 <= TODAY_CHECK <= end_2017:
            df_final_result.loc[i,'Financial_Year'] = '2016-17'            
        elif start_2017 <= TODAY_CHECK <= end_2018:
            df_final_result.loc[i,'Financial_Year'] = '2017-18'            
        elif start_2018 <= TODAY_CHECK <= end_2019:
            df_final_result.loc[i,'Financial_Year'] = '2018-19'            
        elif start_2019 <= TODAY_CHECK <= end_2020:
            df_final_result.loc[i,'Financial_Year'] = '2019-20'            
        elif start_2020 <= TODAY_CHECK <= end_2021:
            df_final_result.loc[i,'Financial_Year'] = '2020-21'  
        elif start_2021 <= TODAY_CHECK <= end_2022:
            df_final_result.loc[i,'Financial_Year'] = '2021-22'             
    else:
        pass

In [1629]:
for i in range(0,len(df_final_result)):
    if df_final_result.loc[i,'Amount1'] == None:
        df_final_result.loc[i,'remark1'] = None
    else:
        pass
    if df_final_result.loc[i,'Amount2'] == None:
        df_final_result.loc[i,'remark2'] = None
    else:
        pass
    if df_final_result.loc[i,'Amount3'] == None:
        df_final_result.loc[i,'remark3'] = None
    else:
        pass

In [1630]:
for i in range(0,len(df_final_result)):
    if df_final_result.loc[i,'insured_name']!= None:
        name = df_final_result.loc[i,'insured_name']
        df_final_result.loc[i,'insured_name'] = name.upper()
    else:
        pass
    if df_final_result.loc[i,'provider']!= None:
        provider = df_final_result.loc[i,'provider']
        df_final_result.loc[i,'provider'] = provider.upper()
    else:
        pass
    if df_final_result.loc[i,'policy_number']!=None:
        policy_number = df_final_result.loc[i,'policy_number']
        df_final_result.loc[i,'policy_number'] = policy_number.upper()
    else:
        pass
    if df_final_result.loc[i,'remark1']!=None:
        remark1 = df_final_result.loc[i,'remark1']
        df_final_result.loc[i,'remark1'] = remark1.upper()
    else:
        pass
    if df_final_result.loc[i,'remark2']!=None:
        remark2 = df_final_result.loc[i,'remark2']
        df_final_result.loc[i,'remark2'] = remark2.upper()
    else:
        pass
    if df_final_result.loc[i,'remark3']!=None:
        remark3 = df_final_result.loc[i,'remark3']
        df_final_result.loc[i,'remark3'] = remark3.upper()
    else:
        pass

In [1631]:
df_final_result

,path,File_name,provider,investment_type,policy_number,insured_name,premium_date,Pay_mode,Scheme,premium_amt,...,remark1,Amount2,remark2,Amount3,remark3,score,Remark,Values_Count,Image_Quality,Financial_Year
0,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2184_Rent_Agreement...,HDFC,Unrecognized Category,NONE,None,2016-06-15,NA,NA,50000,...,None,None,None,None,None,66%,"Insured Name not recognized,NOT FROM CURRENT F...",2/3,None,2016-17
1,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2185_Rent_Agreement...,NA,RENTAL AGREEMENT,NONE,None,NaT,None,None,None,...,None,None,None,None,None,0%,Insured Name premium date premium amount not r...,0/3,Possibly Handwritten or Low Quality Image,NaN
2,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2186_Rent_Agreement...,NA,RENTAL AGREEMENT,NONE,None,NaT,None,None,None,...,None,None,None,None,None,0%,Insured Name premium date premium amount not r...,0/3,Possibly Handwritten or Low Quality Image,NaN
3,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2187_Rent_Agreement...,None,Unrecognized Category,NONE,None,NaT,NA,NA,None,...,None,None,None,None,None,0%,Insured Name premium date premium amount not r...,0/3,Blurred/Poor Quality Image,NaN
4,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2188_Rent_Agreement...,None,Unrecognized Category,NONE,None,NaT,NA,NA,None,...,None,None,None,None,None,0%,Insured Name premium date premium amount not r...,0/3,Blurred/Poor Quality Image,NaN
5,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2189_Rent_Agreement...,None,Unrecognized Category,NONE,None,NaT,NA,NA,None,...,None,None,None,None,None,0%,Insured Name premium date premium amount not r...,0/3,Blurred/Poor Quality Image,NaN
6,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2190_Rent_Agreement...,None,Invalid Document,NONE,G.CHIDANANDAPPA,NaT,NA,NA,None,...,None,None,None,None,None,33%,premium date premium amount not recognized,1/3,None,NaN
7,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2191_Rent_Agreement...,NA,RENTAL AGREEMENT,NONE,None,NaT,None,None,8600,...,None,None,None,None,None,33%,Insured Name premium date not recognized,1/3,None,NaN
8,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2192_Rent Receipt-D...,None,Unrecognized Category,NONE,None,NaT,NA,NA,None,...,None,None,None,None,None,0%,Insured Name premium date premium amount not r...,0/3,Blurred/Poor Quality Image,NaN
9,D:\VScan\17th June\Data\EM000002_RAVINDRA JOGD...,EM000002_RAVINDRA JOGDAND _2193_Rent Receipt_B...,NA,RENTAL AGREEMENT,NONE,RAVINDRA LAXMAN JOGDAND,2019-01-12,None,None,None,...,None,None,None,None,None,66%,premium amount not recognized,2/3,None,2018-19


In [1633]:
df_final_result = df_final_result[['path', 'File_name', 'provider', 'investment_type', 'policy_number',
       'insured_name', 'premium_date', 'Financial_Year', 'Pay_mode', 'Scheme', 'premium_amt',
       'Amount1', 'remark1', 'Amount2', 'remark2', 'Amount3', 'remark3',
       'score', 'Remark', 'Values_Count', 'Image_Quality']]

In [1634]:
df_final_result.to_excel('Result_TATA1.xlsx',index = False)